In [6]:
import pandas as pd
import numpy as np
import datetime


In [7]:
df = pd.read_csv("hero_ban_final.csv")

In [8]:
df.head()

,match_id,Result,index_h1,index_h2,index_h3,index_h4
0,1928898739,1,1/73,2/69,3/71,4/7
1,1928932285,1,1/69,2/102,3/32,4/61
2,1928903165,1,1/85,2/68,3/75,4/2
3,1928907204,1,1/71,2/73,3/85,4/37
4,1928865084,1,1/12,2/42,3/7,4/4


In [9]:
matrix1 =np.array(df.loc[:,['match_id', 'Result','index_h1','index_h2','index_h3','index_h4']])
matrix1.shape

(461040, 6)

In [10]:
df1 = pd.read_csv("hero_names.csv")
df1.head()

,name,hero_id,localized_name,Strength,Agility,Intelligence
0,npc_dota_hero_antimage,1,Anti-Mage,0,1,0
1,npc_dota_hero_axe,2,Axe,1,0,0
2,npc_dota_hero_bane,3,Bane,0,0,1
3,npc_dota_hero_bloodseeker,4,Bloodseeker,0,1,0
4,npc_dota_hero_crystal_maiden,5,Crystal Maiden,0,0,1


In [89]:
matrix2 =np.array(df1.loc[:,['hero_id', 'Strength','Agility','Intelligence']])

In [90]:
matrix2.shape

(113, 4)

In [91]:
matrix2[112]

array([113,   0,   1,   0], dtype=int64)

In [5]:
from itertools import chain

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve, GridSearchCV




In [157]:
def tagging(match, hero_matrix):
    final_list=[]
    middle_value=""
    for element in range(2,5):
        #print("Match is: ",match[element][1])
        middle_tag = ""
        hero_entry = 0
        index_value = 0
        #length = len(match[element])
        temp_list = hero_matrix[int(match[element][2:])-1]
        if temp_list[1]==1:
            middle_tag = "St"
        elif temp_list[2]==1:
            middle_tag = "Ag"
        elif temp_list[3]==1:
            middle_tag = "It"
        if element==5:
            index_value = 1000 
        else:
            index_value = match[element+1][2:]
        hero_entry = match[element][2:]
        middle_value +=  middle_tag + "/"
        final_list.append((hero_entry, middle_value, index_value))
             
    return final_list
   
    
            

dataset = []
for i in range(len(matrix1)):
    dataset.append(tagging(matrix1[i], matrix2))
len(dataset)
#train_sent = dataset[0:200000]
train_sent = dataset[0:2000]
test_sent = dataset[2000:2300]
#test_sent = dataset[200000:233735]
#dataset_numpy = np.array(dataset)
#dataset_numpy

    #hero_info[int(token[2:])-1]

In [122]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    #print("feature",word, postag, i)
    #print(postag[0:2])
    features = {
        'bias': 1.0,
        'first_2_heros_int': (i==1 and postag[0:2]=='It' and postag[3:5]=='It'),
        'first_2_heros_st': (i==1 and postag[0:2]=='It' and postag[3:5]=='St'),
        'first_2_heros_ag': (i==1 and postag[0:2]=='It' and postag[3:5]=='Ag'),
        'alternate_condition_itst': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='St')or(postag[0:2]=='St' and postag[3:5]=='It'))),
        'alternate_condition_stag': (i==1 and ((postag[0:2]=='St' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='St'))),
        'alternate_condition_itag': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='It'))),
        
    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    #print("feature",word, postag, i)
    #print(postag[0:2])
    features = {
        'bias': 1.0,
        'first_2_heros_int': (i==1 and postag[0:2]=='It' and postag[3:5]=='It'),
        'first_2_heros_st': (i==1 and postag[0:2]=='It' and postag[3:5]=='St'),
        'first_2_heros_ag': (i==1 and postag[0:2]=='It' and postag[3:5]=='Ag'),
        'alternate_condition_itst': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='St')or(postag[0:2]=='St' and postag[3:5]=='It'))),
        'alternate_condition_stag': (i==1 and ((postag[0:2]=='St' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='St'))),
        'alternate_condition_itag': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='It'))),
        
    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [131]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    #print("feature",word, postag, i)
    #print(postag[0:2])
    features = {
        'bias': 1.0,
        'first_2_heros_int': (i==1 and postag[0:2]=='It' and postag[3:5]=='It'),
        'first_2_heros_st': (i==1 and postag[0:2]=='It' and postag[3:5]=='St'),
        'first_2_heros_ag': (i==1 and postag[0:2]=='It' and postag[3:5]=='Ag'),
        'alternate_condition_itst': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='St')or(postag[0:2]=='St' and postag[3:5]=='It'))),
        'alternate_condition_stag': (i==1 and ((postag[0:2]=='St' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='St'))),
        'alternate_condition_itag': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='It'))),
        'third_hero_st': (i==2 and postag[6:8]=='St'),
        'third_hero_it': (i==2 and postag[6:8]=='It'),
        'third_hero_ag': (i==2 and postag[6:8]=='Ag'),
        'second_pick_int': (i==1 and postag[3:5]=='It'),
        'second_pick_ag': (i==1 and postag[3:5]=='Ag'),
        'second_pick_st': (i==1 and postag[3:5]=='St'),
        'first_pick_int': (i==0 and postag[0:2]=='It'),
        'first_pick_st': (i==0 and postag[0:2]=='St'),
        'first_pick_ag': (i==0 and postag[0:2]=='Ag'),
        'fourth_pick_ag': (i==3 and postag[9:11]=='Ag'),
        'fourth_pick_it': (i==3 and postag[9:11]=='It'),
        'fourth_pick_st': (i==3 and postag[9:11]=='St'),
        'fifth_pick_ag_or_it': (i==4 and ((postag[12:14]=='Ag') or (postag[12:14]=='It'))),  
        'fifth_pick_ag': (i==4 and ((postag[12:14]=='Ag'))), 
        'fifth_pick_it': (i==4 and ((postag[12:14]=='It'))), 
        'fifth_pick_st': (i==4 and ((postag[12:14]=='St'))), 
    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [167]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    my_lab = sent[i] [2]
    #print('Sent is: ', i)
   # print("")
    #print("feature",word, postag, i)
    #print(postag[0:2])
    features = {
        'bias' : 1.0,
        'Hero_0_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_0_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_0_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_0_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_0_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_0_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_0_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_0_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_0_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_0_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_0_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_0_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_0_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_0_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_0_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_0_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_0_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_0_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_0_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_0_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_0_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_0_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_0_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_0_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_0_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_0_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_0_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_0_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_0_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_0_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_0_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_0_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_0_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_0_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_0_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_0_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_0_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_0_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_0_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_0_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_0_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_0_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_0_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_0_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_0_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_0_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_0_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_0_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_0_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_0_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_0_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_0_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_0_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_0_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_0_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_0_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_0_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_0_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_0_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_0_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_0_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_0_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_0_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_0_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_0_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_0_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_0_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_0_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_0_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_0_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_0_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_0_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_0_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_0_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_0_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_0_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_0_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_0_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_0_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_0_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_0_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_0_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_0_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_0_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_0_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_0_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_0_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_0_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_0_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_0_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_0_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_0_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_0_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_0_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_0_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_0_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_0_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_0_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_0_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_0_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_0_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_0_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_0_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_0_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_0_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_0_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_0_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_0_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_0_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_0_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_0_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_0_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_0_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_0_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_0_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_0_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_0_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_0_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_0_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_0_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_0_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_0_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_0_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_0_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_0_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_0_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_0_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_0_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_0_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_0_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_0_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_0_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_0_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_0_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_0_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_0_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_0_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_0_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_0_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_0_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_0_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_0_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_0_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_0_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_0_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_0_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_0_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_0_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_0_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_0_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_0_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_0_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_0_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_0_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_0_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_0_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_0_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_0_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_0_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_0_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_0_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_0_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_0_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_0_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_0_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_0_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_0_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_0_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_0_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_0_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_0_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_0_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_0_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_0_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_0_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_0_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_0_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_0_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_0_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_0_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_0_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_0_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_0_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_0_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_0_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_0_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_0_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_0_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_0_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_0_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_0_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_0_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_0_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_0_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_0_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_0_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_0_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_0_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_0_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_0_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_0_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_0_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_0_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_0_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_0_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_0_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_0_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_0_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_0_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_0_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_0_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_0_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_0_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_0_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_0_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_0_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_0_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_0_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_0_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_0_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_0_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_0_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_0_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_0_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_0_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_0_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_0_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_0_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_0_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_0_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_0_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_0_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_0_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_0_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_0_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_0_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_0_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_0_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_0_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_0_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_0_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_0_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_0_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_0_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_0_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_0_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_0_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_0_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_0_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_0_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_0_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_0_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_0_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_0_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_0_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_0_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_0_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_0_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_0_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_0_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_0_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_0_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_0_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_0_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_0_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_0_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_0_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_0_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_0_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_0_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_0_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_0_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_0_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_0_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_0_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_0_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_0_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_0_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_0_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_0_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_0_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_0_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_0_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_0_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_0_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_0_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_0_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_0_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_0_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_0_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_0_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_0_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_0_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_0_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_0_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_0_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_0_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_0_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_0_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_0_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_0_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_0_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_0_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_0_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_0_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_0_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_0_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_0_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_0_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_0_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_0_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_0_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_0_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_0_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_0_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_0_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_0_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_0_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_0_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_0_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_0_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_0_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_0_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_0_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_0_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_0_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_0_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_0_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_0_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_0_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_0_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_0_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_0_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_0_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_0_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_0_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_0_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_0_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_0_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_0_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_0_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_0_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_1_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_1_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_1_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_1_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_1_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_1_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_1_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_1_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_1_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_1_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_1_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_1_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_1_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_1_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_1_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_1_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_1_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_1_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_1_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_1_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_1_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_1_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_1_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_1_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_1_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_1_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_1_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_1_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_1_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_1_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_1_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_1_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_1_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_1_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_1_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_1_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_1_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_1_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_1_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_1_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_1_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_1_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_1_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_1_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_1_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_1_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_1_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_1_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_1_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_1_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_1_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_1_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_1_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_1_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_1_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_1_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_1_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_1_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_1_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_1_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_1_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_1_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_1_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_1_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_1_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_1_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_1_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_1_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_1_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_1_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_1_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_1_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_1_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_1_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_1_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_1_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_1_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_1_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_1_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_1_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_1_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_1_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_1_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_1_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_1_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_1_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_1_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_1_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_1_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_1_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_1_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_1_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_1_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_1_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_1_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_1_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_1_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_1_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_1_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_1_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_1_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_1_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_1_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_1_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_1_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_1_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_1_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_1_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_1_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_1_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_1_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_1_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_1_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_1_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_1_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_1_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_1_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_1_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_1_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_1_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_1_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_1_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_1_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_1_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_1_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_1_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_1_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_1_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_1_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_1_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_1_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_1_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_1_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_1_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_1_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_1_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_1_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_1_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_1_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_1_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_1_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_1_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_1_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_1_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_1_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_1_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_1_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_1_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_1_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_1_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_1_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_1_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_1_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_1_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_1_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_1_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_1_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_1_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_1_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_1_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_1_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_1_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_1_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_1_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_1_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_1_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_1_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_1_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_1_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_1_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_1_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_1_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_1_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_1_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_1_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_1_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_1_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_1_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_1_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_1_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_1_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_1_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_1_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_1_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_1_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_1_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_1_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_1_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_1_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_1_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_1_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_1_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_1_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_1_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_1_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_1_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_1_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_1_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_1_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_1_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_1_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_1_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_1_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_1_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_1_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_1_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_1_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_1_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_1_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_1_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_1_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_1_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_1_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_1_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_1_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_1_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_1_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_1_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_1_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_1_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_1_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_1_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_1_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_1_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_1_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_1_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_1_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_1_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_1_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_1_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_1_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_1_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_1_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_1_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_1_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_1_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_1_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_1_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_1_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_1_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_1_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_1_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_1_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_1_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_1_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_1_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_1_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_1_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_1_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_1_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_1_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_1_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_1_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_1_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_1_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_1_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_1_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_1_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_1_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_1_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_1_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_1_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_1_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_1_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_1_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_1_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_1_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_1_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_1_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_1_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_1_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_1_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_1_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_1_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_1_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_1_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_1_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_1_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_1_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_1_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_1_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_1_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_1_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_1_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_1_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_1_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_1_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_1_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_1_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_1_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_1_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_1_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_1_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_1_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_1_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_1_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_1_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_1_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_1_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_1_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_1_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_1_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_1_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_1_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_1_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_1_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_1_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_1_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_1_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_1_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_1_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_1_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_1_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_1_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_1_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_1_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_1_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_1_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_1_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_1_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_1_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_1_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_1_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_1_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_1_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_1_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_1_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_1_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_1_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_1_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_1_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_1_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_1_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_1_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_1_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_1_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_1_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_1_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_1_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_1_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_1_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_1_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_2_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_2_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_2_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_2_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_2_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_2_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_2_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_2_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_2_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_2_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_2_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_2_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_2_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_2_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_2_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_2_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_2_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_2_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_2_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_2_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_2_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_2_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_2_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_2_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_2_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_2_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_2_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_2_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_2_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_2_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_2_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_2_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_2_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_2_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_2_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_2_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_2_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_2_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_2_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_2_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_2_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_2_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_2_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_2_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_2_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_2_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_2_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_2_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_2_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_2_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_2_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_2_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_2_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_2_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_2_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_2_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_2_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_2_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_2_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_2_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_2_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_2_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_2_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_2_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_2_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_2_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_2_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_2_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_2_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_2_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_2_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_2_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_2_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_2_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_2_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_2_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_2_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_2_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_2_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_2_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_2_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_2_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_2_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_2_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_2_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_2_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_2_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_2_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_2_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_2_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_2_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_2_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_2_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_2_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_2_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_2_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_2_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_2_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_2_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_2_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_2_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_2_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_2_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_2_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_2_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_2_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_2_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_2_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_2_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_2_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_2_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_2_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_2_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_2_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_2_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_2_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_2_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_2_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_2_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_2_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_2_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_2_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_2_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_2_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_2_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_2_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_2_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_2_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_2_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_2_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_2_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_2_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_2_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_2_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_2_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_2_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_2_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_2_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_2_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_2_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_2_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_2_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_2_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_2_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_2_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_2_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_2_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_2_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_2_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_2_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_2_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_2_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_2_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_2_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_2_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_2_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_2_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_2_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_2_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_2_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_2_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_2_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_2_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_2_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_2_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_2_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_2_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_2_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_2_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_2_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_2_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_2_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_2_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_2_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_2_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_2_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_2_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_2_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_2_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_2_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_2_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_2_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_2_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_2_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_2_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_2_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_2_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_2_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_2_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_2_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_2_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_2_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_2_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_2_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_2_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_2_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_2_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_2_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_2_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_2_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_2_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_2_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_2_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_2_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_2_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_2_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_2_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_2_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_2_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_2_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_2_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_2_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_2_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_2_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_2_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_2_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_2_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_2_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_2_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_2_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_2_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_2_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_2_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_2_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_2_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_2_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_2_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_2_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_2_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_2_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_2_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_2_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_2_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_2_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_2_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_2_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_2_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_2_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_2_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_2_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_2_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_2_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_2_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_2_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_2_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_2_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_2_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_2_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_2_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_2_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_2_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_2_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_2_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_2_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_2_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_2_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_2_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_2_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_2_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_2_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_2_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_2_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_2_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_2_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_2_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_2_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_2_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_2_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_2_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_2_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_2_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_2_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_2_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_2_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_2_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_2_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_2_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_2_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_2_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_2_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_2_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_2_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_2_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_2_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_2_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_2_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_2_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_2_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_2_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_2_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_2_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_2_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_2_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_2_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_2_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_2_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_2_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_2_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_2_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_2_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_2_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_2_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_2_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_2_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_2_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_2_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_2_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_2_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_2_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_2_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_2_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_2_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_2_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_2_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_2_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_2_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_2_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_2_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_2_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_2_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_2_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_2_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_2_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_2_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_2_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_2_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_2_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_2_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_2_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_2_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_2_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_2_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_2_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_2_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_2_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_2_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_2_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_2_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_2_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_2_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_2_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_2_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_3_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_3_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_3_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_3_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_3_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_3_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_3_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_3_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_3_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_3_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_3_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_3_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_3_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_3_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_3_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_3_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_3_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_3_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_3_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_3_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_3_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_3_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_3_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_3_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_3_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_3_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_3_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_3_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_3_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_3_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_3_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_3_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_3_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_3_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_3_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_3_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_3_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_3_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_3_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_3_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_3_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_3_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_3_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_3_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_3_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_3_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_3_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_3_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_3_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_3_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_3_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_3_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_3_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_3_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_3_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_3_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_3_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_3_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_3_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_3_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_3_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_3_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_3_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_3_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_3_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_3_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_3_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_3_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_3_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_3_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_3_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_3_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_3_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_3_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_3_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_3_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_3_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_3_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_3_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_3_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_3_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_3_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_3_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_3_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_3_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_3_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_3_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_3_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_3_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_3_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_3_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_3_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_3_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_3_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_3_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_3_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_3_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_3_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_3_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_3_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_3_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_3_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_3_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_3_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_3_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_3_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_3_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_3_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_3_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_3_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_3_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_3_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_3_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_3_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_3_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_3_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_3_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_3_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_3_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_3_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_3_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_3_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_3_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_3_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_3_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_3_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_3_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_3_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_3_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_3_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_3_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_3_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_3_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_3_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_3_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_3_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_3_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_3_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_3_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_3_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_3_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_3_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_3_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_3_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_3_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_3_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_3_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_3_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_3_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_3_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_3_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_3_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_3_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_3_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_3_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_3_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_3_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_3_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_3_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_3_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_3_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_3_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_3_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_3_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_3_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_3_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_3_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_3_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_3_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_3_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_3_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_3_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_3_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_3_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_3_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_3_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_3_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_3_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_3_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_3_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_3_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_3_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_3_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_3_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_3_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_3_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_3_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_3_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_3_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_3_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_3_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_3_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_3_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_3_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_3_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_3_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_3_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_3_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_3_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_3_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_3_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_3_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_3_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_3_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_3_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_3_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_3_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_3_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_3_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_3_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_3_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_3_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_3_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_3_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_3_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_3_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_3_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_3_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_3_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_3_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_3_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_3_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_3_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_3_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_3_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_3_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_3_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_3_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_3_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_3_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_3_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_3_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_3_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_3_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_3_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_3_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_3_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_3_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_3_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_3_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_3_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_3_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_3_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_3_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_3_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_3_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_3_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_3_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_3_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_3_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_3_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_3_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_3_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_3_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_3_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_3_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_3_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_3_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_3_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_3_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_3_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_3_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_3_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_3_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_3_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_3_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_3_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_3_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_3_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_3_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_3_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_3_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_3_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_3_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_3_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_3_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_3_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_3_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_3_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_3_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_3_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_3_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_3_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_3_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_3_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_3_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_3_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_3_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_3_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_3_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_3_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_3_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_3_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_3_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_3_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_3_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_3_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_3_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_3_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_3_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_3_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_3_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_3_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_3_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_3_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_3_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_3_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_3_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_3_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_3_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_3_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_3_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_3_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_3_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_3_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_3_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_3_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_3_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_3_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_3_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_3_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_3_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_3_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_3_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_3_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_3_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_3_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_3_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_3_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_3_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_3_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_3_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_3_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_3_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_3_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_3_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_3_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_3_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_3_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_3_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_3_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_3_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_4_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_4_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_4_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_4_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_4_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_4_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_4_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_4_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_4_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_4_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_4_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_4_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_4_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_4_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_4_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_4_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_4_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_4_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_4_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_4_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_4_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_4_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_4_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_4_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_4_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_4_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_4_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_4_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_4_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_4_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_4_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_4_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_4_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_4_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_4_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_4_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_4_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_4_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_4_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_4_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_4_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_4_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_4_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_4_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_4_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_4_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_4_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_4_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_4_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_4_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_4_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_4_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_4_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_4_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_4_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_4_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_4_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_4_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_4_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_4_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_4_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_4_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_4_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_4_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_4_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_4_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_4_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_4_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_4_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_4_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_4_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_4_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_4_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_4_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_4_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_4_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_4_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_4_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_4_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_4_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_4_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_4_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_4_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_4_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_4_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_4_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_4_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_4_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_4_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_4_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_4_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_4_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_4_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_4_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_4_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_4_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_4_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_4_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_4_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_4_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_4_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_4_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_4_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_4_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_4_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_4_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_4_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_4_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_4_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_4_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_4_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_4_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_4_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_4_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_4_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_4_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_4_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_4_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_4_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_4_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_4_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_4_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_4_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_4_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_4_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_4_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_4_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_4_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_4_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_4_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_4_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_4_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_4_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_4_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_4_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_4_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_4_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_4_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_4_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_4_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_4_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_4_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_4_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_4_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_4_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_4_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_4_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_4_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_4_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_4_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_4_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_4_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_4_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_4_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_4_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_4_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_4_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_4_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_4_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_4_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_4_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_4_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_4_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_4_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_4_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_4_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_4_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_4_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_4_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_4_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_4_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_4_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_4_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_4_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_4_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_4_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_4_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_4_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_4_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_4_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_4_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_4_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_4_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_4_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_4_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_4_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_4_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_4_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_4_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_4_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_4_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_4_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_4_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_4_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_4_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_4_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_4_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_4_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_4_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_4_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_4_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_4_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_4_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_4_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_4_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_4_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_4_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_4_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_4_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_4_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_4_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_4_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_4_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_4_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_4_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_4_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_4_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_4_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_4_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_4_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_4_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_4_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_4_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_4_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_4_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_4_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_4_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_4_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_4_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_4_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_4_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_4_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_4_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_4_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_4_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_4_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_4_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_4_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_4_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_4_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_4_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_4_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_4_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_4_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_4_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_4_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_4_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_4_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_4_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_4_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_4_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_4_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_4_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_4_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_4_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_4_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_4_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_4_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_4_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_4_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_4_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_4_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_4_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_4_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_4_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_4_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_4_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_4_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_4_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_4_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_4_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_4_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_4_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_4_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_4_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_4_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_4_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_4_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_4_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_4_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_4_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_4_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_4_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_4_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_4_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_4_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_4_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_4_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_4_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_4_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_4_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_4_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_4_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_4_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_4_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_4_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_4_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_4_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_4_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_4_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_4_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_4_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_4_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_4_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_4_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_4_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_4_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_4_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_4_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_4_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_4_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_4_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_4_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_4_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_4_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_4_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_4_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_4_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_4_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_4_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_4_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_4_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_4_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_4_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_4_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_4_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_4_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_4_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_4_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_4_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_4_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_4_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_4_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_4_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_4_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_4_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_4_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_4_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_4_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_4_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_4_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_4_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_5_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_5_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_5_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_5_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_5_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_5_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_5_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_5_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_5_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_5_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_5_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_5_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_5_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_5_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_5_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_5_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_5_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_5_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_5_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_5_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_5_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_5_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_5_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_5_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_5_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_5_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_5_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_5_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_5_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_5_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_5_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_5_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_5_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_5_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_5_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_5_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_5_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_5_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_5_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_5_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_5_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_5_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_5_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_5_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_5_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_5_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_5_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_5_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_5_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_5_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_5_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_5_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_5_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_5_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_5_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_5_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_5_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_5_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_5_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_5_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_5_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_5_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_5_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_5_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_5_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_5_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_5_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_5_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_5_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_5_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_5_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_5_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_5_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_5_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_5_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_5_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_5_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_5_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_5_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_5_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_5_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_5_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_5_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_5_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_5_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_5_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_5_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_5_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_5_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_5_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_5_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_5_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_5_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_5_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_5_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_5_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_5_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_5_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_5_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_5_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_5_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_5_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_5_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_5_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_5_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_5_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_5_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_5_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_5_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_5_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_5_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_5_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_5_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_5_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_5_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_5_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_5_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_5_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_5_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_5_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_5_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_5_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_5_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_5_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_5_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_5_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_5_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_5_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_5_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_5_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_5_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_5_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_5_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_5_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_5_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_5_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_5_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_5_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_5_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_5_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_5_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_5_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_5_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_5_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_5_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_5_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_5_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_5_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_5_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_5_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_5_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_5_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_5_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_5_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_5_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_5_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_5_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_5_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_5_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_5_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_5_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_5_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_5_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_5_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_5_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_5_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_5_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_5_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_5_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_5_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_5_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_5_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_5_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_5_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_5_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_5_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_5_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_5_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_5_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_5_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_5_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_5_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_5_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_5_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_5_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_5_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_5_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_5_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_5_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_5_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_5_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_5_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_5_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_5_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_5_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_5_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_5_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_5_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_5_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_5_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_5_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_5_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_5_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_5_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_5_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_5_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_5_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_5_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_5_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_5_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_5_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_5_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_5_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_5_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_5_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_5_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_5_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_5_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_5_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_5_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_5_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_5_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_5_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_5_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_5_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_5_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_5_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_5_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_5_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_5_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_5_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_5_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_5_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_5_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_5_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_5_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_5_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_5_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_5_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_5_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_5_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_5_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_5_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_5_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_5_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_5_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_5_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_5_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_5_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_5_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_5_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_5_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_5_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_5_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_5_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_5_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_5_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_5_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_5_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_5_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_5_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_5_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_5_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_5_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_5_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_5_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_5_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_5_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_5_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_5_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_5_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_5_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_5_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_5_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_5_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_5_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_5_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_5_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_5_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_5_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_5_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_5_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_5_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_5_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_5_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_5_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_5_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_5_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_5_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_5_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_5_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_5_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_5_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_5_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_5_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_5_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_5_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_5_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_5_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_5_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_5_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_5_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_5_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_5_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_5_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_5_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_5_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_5_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_5_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_5_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_5_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_5_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_5_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_5_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_5_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_5_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_5_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_5_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_5_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_5_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_5_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_5_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_5_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_5_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_5_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_5_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_5_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_5_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_5_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_5_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_5_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_5_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_5_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_5_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_5_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_5_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_5_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_5_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_5_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_5_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_5_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_5_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_6_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_6_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_6_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_6_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_6_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_6_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_6_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_6_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_6_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_6_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_6_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_6_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_6_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_6_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_6_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_6_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_6_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_6_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_6_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_6_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_6_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_6_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_6_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_6_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_6_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_6_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_6_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_6_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_6_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_6_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_6_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_6_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_6_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_6_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_6_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_6_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_6_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_6_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_6_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_6_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_6_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_6_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_6_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_6_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_6_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_6_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_6_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_6_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_6_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_6_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_6_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_6_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_6_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_6_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_6_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_6_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_6_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_6_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_6_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_6_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_6_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_6_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_6_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_6_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_6_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_6_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_6_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_6_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_6_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_6_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_6_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_6_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_6_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_6_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_6_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_6_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_6_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_6_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_6_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_6_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_6_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_6_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_6_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_6_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_6_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_6_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_6_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_6_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_6_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_6_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_6_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_6_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_6_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_6_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_6_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_6_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_6_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_6_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_6_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_6_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_6_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_6_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_6_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_6_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_6_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_6_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_6_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_6_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_6_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_6_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_6_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_6_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_6_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_6_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_6_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_6_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_6_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_6_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_6_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_6_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_6_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_6_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_6_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_6_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_6_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_6_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_6_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_6_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_6_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_6_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_6_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_6_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_6_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_6_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_6_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_6_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_6_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_6_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_6_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_6_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_6_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_6_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_6_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_6_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_6_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_6_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_6_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_6_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_6_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_6_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_6_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_6_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_6_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_6_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_6_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_6_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_6_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_6_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_6_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_6_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_6_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_6_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_6_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_6_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_6_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_6_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_6_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_6_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_6_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_6_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_6_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_6_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_6_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_6_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_6_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_6_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_6_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_6_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_6_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_6_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_6_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_6_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_6_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_6_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_6_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_6_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_6_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_6_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_6_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_6_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_6_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_6_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_6_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_6_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_6_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_6_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_6_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_6_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_6_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_6_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_6_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_6_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_6_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_6_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_6_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_6_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_6_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_6_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_6_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_6_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_6_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_6_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_6_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_6_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_6_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_6_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_6_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_6_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_6_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_6_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_6_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_6_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_6_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_6_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_6_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_6_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_6_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_6_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_6_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_6_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_6_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_6_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_6_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_6_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_6_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_6_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_6_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_6_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_6_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_6_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_6_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_6_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_6_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_6_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_6_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_6_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_6_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_6_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_6_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_6_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_6_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_6_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_6_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_6_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_6_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_6_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_6_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_6_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_6_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_6_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_6_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_6_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_6_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_6_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_6_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_6_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_6_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_6_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_6_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_6_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_6_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_6_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_6_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_6_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_6_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_6_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_6_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_6_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_6_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_6_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_6_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_6_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_6_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_6_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_6_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_6_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_6_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_6_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_6_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_6_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_6_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_6_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_6_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_6_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_6_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_6_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_6_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_6_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_6_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_6_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_6_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_6_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_6_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_6_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_6_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_6_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_6_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_6_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_6_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_6_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_6_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_6_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_6_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_6_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_6_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_6_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_6_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_6_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_6_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_6_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_6_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_6_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_6_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_6_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_6_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_6_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_6_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_6_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_6_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_6_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_6_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_6_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_6_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_6_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_6_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_6_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_6_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_6_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_6_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_6_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_6_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_6_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_7_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_7_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_7_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_7_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_7_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_7_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_7_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_7_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_7_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_7_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_7_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_7_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_7_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_7_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_7_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_7_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_7_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_7_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_7_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_7_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_7_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_7_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_7_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_7_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_7_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_7_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_7_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_7_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_7_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_7_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_7_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_7_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_7_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_7_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_7_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_7_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_7_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_7_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_7_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_7_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_7_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_7_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_7_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_7_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_7_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_7_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_7_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_7_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_7_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_7_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_7_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_7_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_7_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_7_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_7_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_7_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_7_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_7_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_7_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_7_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_7_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_7_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_7_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_7_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_7_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_7_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_7_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_7_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_7_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_7_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_7_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_7_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_7_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_7_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_7_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_7_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_7_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_7_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_7_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_7_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_7_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_7_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_7_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_7_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_7_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_7_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_7_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_7_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_7_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_7_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_7_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_7_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_7_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_7_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_7_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_7_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_7_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_7_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_7_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_7_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_7_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_7_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_7_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_7_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_7_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_7_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_7_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_7_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_7_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_7_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_7_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_7_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_7_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_7_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_7_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_7_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_7_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_7_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_7_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_7_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_7_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_7_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_7_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_7_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_7_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_7_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_7_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_7_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_7_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_7_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_7_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_7_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_7_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_7_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_7_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_7_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_7_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_7_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_7_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_7_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_7_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_7_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_7_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_7_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_7_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_7_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_7_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_7_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_7_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_7_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_7_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_7_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_7_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_7_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_7_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_7_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_7_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_7_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_7_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_7_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_7_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_7_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_7_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_7_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_7_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_7_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_7_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_7_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_7_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_7_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_7_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_7_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_7_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_7_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_7_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_7_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_7_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_7_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_7_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_7_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_7_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_7_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_7_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_7_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_7_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_7_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_7_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_7_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_7_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_7_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_7_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_7_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_7_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_7_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_7_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_7_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_7_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_7_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_7_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_7_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_7_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_7_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_7_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_7_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_7_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_7_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_7_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_7_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_7_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_7_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_7_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_7_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_7_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_7_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_7_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_7_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_7_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_7_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_7_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_7_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_7_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_7_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_7_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_7_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_7_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_7_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_7_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_7_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_7_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_7_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_7_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_7_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_7_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_7_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_7_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_7_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_7_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_7_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_7_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_7_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_7_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_7_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_7_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_7_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_7_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_7_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_7_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_7_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_7_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_7_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_7_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_7_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_7_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_7_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_7_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_7_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_7_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_7_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_7_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_7_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_7_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_7_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_7_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_7_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_7_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_7_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_7_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_7_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_7_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_7_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_7_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_7_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_7_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_7_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_7_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_7_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_7_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_7_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_7_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_7_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_7_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_7_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_7_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_7_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_7_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_7_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_7_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_7_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_7_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_7_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_7_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_7_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_7_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_7_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_7_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_7_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_7_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_7_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_7_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_7_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_7_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_7_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_7_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_7_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_7_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_7_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_7_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_7_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_7_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_7_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_7_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_7_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_7_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_7_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_7_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_7_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_7_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_7_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_7_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_7_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_7_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_7_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_7_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_7_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_7_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_7_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_7_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_7_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_7_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_7_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_7_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_7_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_7_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_7_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_7_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_7_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_7_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_7_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_7_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_7_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_7_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_7_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_8_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_8_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_8_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_8_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_8_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_8_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_8_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_8_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_8_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_8_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_8_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_8_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_8_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_8_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_8_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_8_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_8_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_8_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_8_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_8_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_8_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_8_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_8_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_8_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_8_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_8_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_8_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_8_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_8_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_8_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_8_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_8_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_8_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_8_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_8_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_8_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_8_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_8_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_8_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_8_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_8_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_8_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_8_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_8_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_8_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_8_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_8_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_8_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_8_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_8_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_8_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_8_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_8_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_8_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_8_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_8_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_8_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_8_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_8_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_8_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_8_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_8_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_8_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_8_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_8_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_8_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_8_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_8_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_8_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_8_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_8_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_8_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_8_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_8_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_8_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_8_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_8_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_8_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_8_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_8_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_8_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_8_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_8_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_8_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_8_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_8_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_8_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_8_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_8_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_8_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_8_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_8_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_8_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_8_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_8_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_8_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_8_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_8_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_8_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_8_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_8_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_8_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_8_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_8_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_8_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_8_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_8_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_8_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_8_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_8_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_8_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_8_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_8_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_8_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_8_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_8_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_8_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_8_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_8_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_8_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_8_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_8_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_8_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_8_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_8_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_8_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_8_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_8_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_8_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_8_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_8_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_8_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_8_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_8_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_8_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_8_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_8_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_8_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_8_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_8_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_8_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_8_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_8_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_8_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_8_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_8_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_8_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_8_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_8_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_8_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_8_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_8_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_8_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_8_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_8_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_8_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_8_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_8_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_8_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_8_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_8_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_8_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_8_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_8_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_8_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_8_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_8_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_8_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_8_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_8_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_8_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_8_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_8_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_8_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_8_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_8_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_8_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_8_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_8_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_8_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_8_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_8_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_8_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_8_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_8_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_8_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_8_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_8_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_8_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_8_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_8_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_8_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_8_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_8_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_8_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_8_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_8_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_8_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_8_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_8_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_8_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_8_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_8_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_8_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_8_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_8_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_8_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_8_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_8_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_8_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_8_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_8_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_8_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_8_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_8_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_8_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_8_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_8_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_8_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_8_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_8_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_8_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_8_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_8_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_8_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_8_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_8_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_8_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_8_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_8_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_8_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_8_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_8_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_8_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_8_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_8_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_8_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_8_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_8_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_8_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_8_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_8_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_8_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_8_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_8_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_8_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_8_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_8_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_8_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_8_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_8_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_8_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_8_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_8_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_8_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_8_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_8_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_8_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_8_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_8_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_8_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_8_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_8_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_8_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_8_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_8_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_8_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_8_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_8_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_8_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_8_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_8_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_8_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_8_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_8_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_8_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_8_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_8_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_8_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_8_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_8_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_8_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_8_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_8_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_8_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_8_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_8_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_8_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_8_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_8_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_8_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_8_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_8_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_8_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_8_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_8_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_8_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_8_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_8_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_8_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_8_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_8_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_8_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_8_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_8_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_8_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_8_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_8_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_8_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_8_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_8_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_8_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_8_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_8_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_8_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_8_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_8_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_8_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_8_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_8_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_8_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_8_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_8_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_8_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_8_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_8_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_8_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_8_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_8_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_8_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_8_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_8_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_8_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_8_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_8_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_8_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_8_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_8_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_8_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_8_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_8_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_8_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_9_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_9_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_9_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_9_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_9_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_9_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_9_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_9_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_9_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_9_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_9_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_9_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_9_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_9_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_9_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_9_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_9_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_9_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_9_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_9_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_9_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_9_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_9_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_9_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_9_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_9_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_9_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_9_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_9_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_9_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_9_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_9_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_9_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_9_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_9_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_9_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_9_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_9_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_9_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_9_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_9_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_9_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_9_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_9_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_9_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_9_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_9_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_9_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_9_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_9_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_9_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_9_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_9_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_9_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_9_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_9_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_9_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_9_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_9_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_9_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_9_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_9_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_9_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_9_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_9_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_9_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_9_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_9_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_9_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_9_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_9_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_9_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_9_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_9_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_9_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_9_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_9_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_9_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_9_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_9_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_9_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_9_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_9_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_9_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_9_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_9_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_9_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_9_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_9_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_9_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_9_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_9_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_9_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_9_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_9_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_9_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_9_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_9_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_9_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_9_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_9_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_9_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_9_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_9_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_9_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_9_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_9_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_9_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_9_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_9_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_9_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_9_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_9_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_9_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_9_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_9_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_9_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_9_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_9_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_9_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_9_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_9_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_9_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_9_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_9_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_9_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_9_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_9_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_9_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_9_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_9_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_9_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_9_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_9_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_9_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_9_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_9_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_9_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_9_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_9_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_9_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_9_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_9_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_9_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_9_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_9_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_9_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_9_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_9_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_9_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_9_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_9_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_9_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_9_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_9_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_9_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_9_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_9_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_9_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_9_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_9_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_9_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_9_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_9_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_9_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_9_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_9_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_9_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_9_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_9_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_9_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_9_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_9_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_9_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_9_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_9_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_9_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_9_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_9_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_9_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_9_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_9_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_9_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_9_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_9_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_9_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_9_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_9_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_9_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_9_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_9_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_9_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_9_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_9_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_9_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_9_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_9_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_9_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_9_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_9_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_9_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_9_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_9_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_9_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_9_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_9_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_9_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_9_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_9_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_9_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_9_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_9_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_9_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_9_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_9_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_9_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_9_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_9_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_9_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_9_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_9_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_9_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_9_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_9_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_9_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_9_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_9_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_9_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_9_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_9_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_9_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_9_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_9_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_9_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_9_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_9_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_9_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_9_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_9_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_9_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_9_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_9_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_9_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_9_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_9_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_9_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_9_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_9_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_9_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_9_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_9_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_9_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_9_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_9_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_9_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_9_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_9_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_9_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_9_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_9_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_9_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_9_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_9_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_9_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_9_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_9_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_9_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_9_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_9_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_9_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_9_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_9_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_9_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_9_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_9_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_9_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_9_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_9_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_9_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_9_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_9_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_9_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_9_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_9_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_9_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_9_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_9_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_9_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_9_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_9_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_9_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_9_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_9_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_9_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_9_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_9_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_9_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_9_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_9_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_9_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_9_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_9_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_9_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_9_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_9_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_9_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_9_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_9_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_9_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_9_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_9_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_9_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_9_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_9_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_9_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_9_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_9_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_9_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_9_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_9_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_9_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_9_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_9_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_9_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_9_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_9_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_9_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_9_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_9_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_9_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_9_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_9_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_9_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_9_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_9_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_9_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_9_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_9_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_9_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_9_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_9_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_9_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_10_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_10_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_10_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_10_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_10_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_10_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_10_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_10_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_10_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_10_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_10_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_10_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_10_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_10_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_10_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_10_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_10_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_10_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_10_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_10_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_10_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_10_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_10_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_10_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_10_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_10_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_10_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_10_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_10_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_10_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_10_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_10_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_10_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_10_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_10_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_10_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_10_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_10_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_10_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_10_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_10_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_10_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_10_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_10_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_10_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_10_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_10_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_10_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_10_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_10_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_10_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_10_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_10_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_10_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_10_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_10_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_10_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_10_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_10_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_10_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_10_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_10_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_10_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_10_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_10_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_10_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_10_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_10_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_10_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_10_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_10_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_10_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_10_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_10_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_10_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_10_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_10_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_10_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_10_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_10_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_10_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_10_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_10_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_10_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_10_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_10_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_10_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_10_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_10_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_10_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_10_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_10_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_10_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_10_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_10_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_10_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_10_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_10_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_10_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_10_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_10_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_10_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_10_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_10_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_10_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_10_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_10_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_10_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_10_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_10_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_10_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_10_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_10_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_10_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_10_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_10_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_10_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_10_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_10_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_10_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_10_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_10_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_10_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_10_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_10_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_10_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_10_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_10_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_10_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_10_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_10_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_10_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_10_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_10_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_10_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_10_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_10_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_10_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_10_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_10_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_10_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_10_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_10_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_10_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_10_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_10_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_10_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_10_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_10_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_10_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_10_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_10_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_10_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_10_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_10_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_10_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_10_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_10_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_10_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_10_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_10_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_10_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_10_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_10_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_10_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_10_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_10_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_10_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_10_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_10_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_10_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_10_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_10_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_10_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_10_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_10_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_10_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_10_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_10_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_10_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_10_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_10_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_10_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_10_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_10_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_10_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_10_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_10_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_10_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_10_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_10_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_10_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_10_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_10_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_10_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_10_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_10_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_10_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_10_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_10_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_10_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_10_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_10_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_10_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_10_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_10_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_10_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_10_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_10_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_10_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_10_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_10_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_10_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_10_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_10_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_10_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_10_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_10_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_10_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_10_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_10_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_10_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_10_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_10_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_10_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_10_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_10_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_10_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_10_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_10_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_10_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_10_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_10_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_10_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_10_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_10_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_10_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_10_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_10_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_10_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_10_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_10_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_10_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_10_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_10_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_10_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_10_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_10_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_10_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_10_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_10_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_10_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_10_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_10_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_10_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_10_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_10_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_10_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_10_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_10_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_10_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_10_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_10_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_10_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_10_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_10_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_10_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_10_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_10_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_10_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_10_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_10_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_10_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_10_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_10_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_10_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_10_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_10_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_10_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_10_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_10_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_10_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_10_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_10_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_10_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_10_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_10_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_10_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_10_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_10_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_10_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_10_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_10_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_10_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_10_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_10_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_10_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_10_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_10_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_10_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_10_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_10_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_10_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_10_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_10_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_10_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_10_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_10_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_10_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_10_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_10_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_10_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_10_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_10_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_10_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_10_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_10_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_10_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_10_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_10_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_10_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_10_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_10_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_10_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_10_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_10_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_10_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_10_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_10_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_10_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_10_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_10_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_10_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_10_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_10_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_10_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_10_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_10_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_10_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_10_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_10_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_10_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_11_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_11_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_11_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_11_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_11_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_11_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_11_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_11_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_11_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_11_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_11_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_11_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_11_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_11_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_11_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_11_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_11_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_11_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_11_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_11_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_11_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_11_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_11_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_11_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_11_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_11_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_11_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_11_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_11_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_11_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_11_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_11_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_11_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_11_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_11_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_11_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_11_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_11_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_11_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_11_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_11_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_11_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_11_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_11_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_11_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_11_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_11_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_11_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_11_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_11_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_11_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_11_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_11_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_11_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_11_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_11_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_11_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_11_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_11_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_11_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_11_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_11_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_11_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_11_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_11_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_11_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_11_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_11_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_11_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_11_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_11_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_11_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_11_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_11_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_11_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_11_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_11_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_11_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_11_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_11_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_11_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_11_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_11_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_11_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_11_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_11_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_11_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_11_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_11_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_11_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_11_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_11_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_11_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_11_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_11_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_11_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_11_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_11_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_11_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_11_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_11_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_11_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_11_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_11_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_11_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_11_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_11_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_11_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_11_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_11_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_11_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_11_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_11_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_11_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_11_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_11_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_11_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_11_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_11_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_11_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_11_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_11_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_11_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_11_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_11_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_11_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_11_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_11_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_11_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_11_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_11_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_11_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_11_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_11_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_11_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_11_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_11_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_11_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_11_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_11_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_11_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_11_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_11_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_11_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_11_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_11_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_11_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_11_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_11_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_11_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_11_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_11_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_11_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_11_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_11_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_11_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_11_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_11_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_11_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_11_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_11_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_11_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_11_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_11_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_11_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_11_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_11_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_11_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_11_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_11_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_11_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_11_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_11_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_11_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_11_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_11_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_11_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_11_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_11_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_11_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_11_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_11_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_11_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_11_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_11_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_11_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_11_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_11_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_11_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_11_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_11_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_11_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_11_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_11_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_11_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_11_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_11_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_11_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_11_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_11_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_11_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_11_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_11_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_11_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_11_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_11_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_11_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_11_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_11_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_11_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_11_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_11_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_11_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_11_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_11_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_11_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_11_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_11_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_11_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_11_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_11_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_11_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_11_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_11_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_11_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_11_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_11_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_11_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_11_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_11_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_11_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_11_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_11_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_11_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_11_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_11_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_11_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_11_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_11_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_11_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_11_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_11_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_11_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_11_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_11_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_11_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_11_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_11_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_11_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_11_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_11_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_11_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_11_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_11_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_11_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_11_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_11_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_11_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_11_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_11_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_11_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_11_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_11_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_11_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_11_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_11_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_11_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_11_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_11_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_11_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_11_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_11_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_11_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_11_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_11_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_11_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_11_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_11_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_11_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_11_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_11_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_11_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_11_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_11_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_11_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_11_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_11_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_11_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_11_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_11_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_11_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_11_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_11_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_11_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_11_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_11_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_11_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_11_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_11_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_11_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_11_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_11_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_11_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_11_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_11_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_11_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_11_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_11_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_11_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_11_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_11_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_11_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_11_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_11_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_11_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_11_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_11_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_11_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_11_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_11_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_11_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_11_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_11_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_11_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_11_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_11_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_11_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_11_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_11_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_11_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_11_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_11_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_11_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_11_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_11_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_11_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_11_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_11_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_11_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_11_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_11_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_11_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_12_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_12_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_12_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_12_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_12_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_12_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_12_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_12_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_12_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_12_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_12_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_12_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_12_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_12_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_12_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_12_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_12_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_12_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_12_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_12_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_12_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_12_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_12_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_12_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_12_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_12_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_12_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_12_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_12_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_12_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_12_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_12_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_12_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_12_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_12_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_12_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_12_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_12_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_12_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_12_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_12_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_12_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_12_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_12_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_12_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_12_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_12_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_12_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_12_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_12_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_12_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_12_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_12_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_12_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_12_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_12_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_12_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_12_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_12_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_12_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_12_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_12_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_12_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_12_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_12_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_12_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_12_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_12_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_12_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_12_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_12_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_12_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_12_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_12_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_12_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_12_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_12_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_12_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_12_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_12_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_12_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_12_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_12_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_12_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_12_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_12_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_12_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_12_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_12_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_12_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_12_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_12_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_12_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_12_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_12_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_12_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_12_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_12_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_12_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_12_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_12_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_12_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_12_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_12_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_12_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_12_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_12_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_12_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_12_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_12_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_12_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_12_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_12_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_12_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_12_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_12_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_12_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_12_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_12_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_12_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_12_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_12_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_12_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_12_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_12_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_12_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_12_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_12_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_12_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_12_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_12_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_12_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_12_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_12_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_12_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_12_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_12_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_12_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_12_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_12_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_12_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_12_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_12_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_12_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_12_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_12_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_12_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_12_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_12_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_12_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_12_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_12_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_12_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_12_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_12_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_12_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_12_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_12_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_12_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_12_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_12_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_12_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_12_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_12_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_12_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_12_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_12_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_12_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_12_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_12_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_12_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_12_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_12_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_12_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_12_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_12_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_12_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_12_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_12_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_12_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_12_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_12_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_12_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_12_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_12_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_12_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_12_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_12_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_12_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_12_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_12_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_12_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_12_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_12_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_12_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_12_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_12_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_12_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_12_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_12_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_12_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_12_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_12_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_12_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_12_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_12_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_12_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_12_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_12_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_12_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_12_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_12_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_12_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_12_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_12_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_12_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_12_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_12_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_12_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_12_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_12_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_12_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_12_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_12_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_12_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_12_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_12_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_12_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_12_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_12_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_12_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_12_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_12_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_12_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_12_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_12_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_12_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_12_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_12_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_12_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_12_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_12_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_12_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_12_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_12_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_12_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_12_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_12_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_12_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_12_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_12_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_12_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_12_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_12_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_12_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_12_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_12_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_12_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_12_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_12_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_12_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_12_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_12_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_12_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_12_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_12_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_12_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_12_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_12_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_12_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_12_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_12_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_12_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_12_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_12_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_12_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_12_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_12_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_12_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_12_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_12_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_12_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_12_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_12_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_12_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_12_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_12_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_12_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_12_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_12_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_12_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_12_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_12_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_12_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_12_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_12_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_12_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_12_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_12_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_12_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_12_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_12_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_12_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_12_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_12_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_12_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_12_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_12_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_12_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_12_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_12_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_12_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_12_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_12_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_12_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_12_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_12_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_12_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_12_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_12_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_12_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_12_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_12_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_12_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_12_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_12_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_12_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_12_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_12_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_12_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_12_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_12_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_12_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_12_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_12_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_12_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_12_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_12_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_12_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_12_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_12_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_12_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_13_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_13_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_13_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_13_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_13_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_13_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_13_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_13_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_13_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_13_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_13_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_13_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_13_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_13_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_13_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_13_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_13_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_13_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_13_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_13_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_13_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_13_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_13_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_13_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_13_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_13_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_13_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_13_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_13_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_13_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_13_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_13_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_13_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_13_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_13_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_13_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_13_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_13_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_13_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_13_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_13_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_13_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_13_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_13_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_13_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_13_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_13_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_13_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_13_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_13_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_13_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_13_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_13_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_13_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_13_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_13_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_13_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_13_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_13_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_13_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_13_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_13_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_13_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_13_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_13_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_13_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_13_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_13_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_13_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_13_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_13_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_13_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_13_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_13_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_13_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_13_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_13_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_13_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_13_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_13_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_13_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_13_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_13_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_13_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_13_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_13_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_13_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_13_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_13_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_13_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_13_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_13_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_13_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_13_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_13_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_13_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_13_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_13_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_13_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_13_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_13_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_13_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_13_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_13_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_13_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_13_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_13_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_13_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_13_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_13_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_13_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_13_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_13_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_13_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_13_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_13_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_13_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_13_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_13_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_13_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_13_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_13_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_13_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_13_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_13_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_13_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_13_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_13_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_13_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_13_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_13_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_13_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_13_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_13_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_13_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_13_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_13_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_13_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_13_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_13_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_13_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_13_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_13_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_13_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_13_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_13_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_13_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_13_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_13_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_13_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_13_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_13_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_13_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_13_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_13_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_13_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_13_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_13_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_13_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_13_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_13_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_13_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_13_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_13_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_13_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_13_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_13_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_13_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_13_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_13_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_13_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_13_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_13_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_13_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_13_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_13_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_13_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_13_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_13_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_13_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_13_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_13_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_13_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_13_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_13_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_13_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_13_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_13_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_13_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_13_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_13_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_13_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_13_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_13_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_13_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_13_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_13_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_13_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_13_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_13_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_13_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_13_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_13_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_13_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_13_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_13_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_13_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_13_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_13_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_13_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_13_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_13_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_13_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_13_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_13_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_13_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_13_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_13_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_13_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_13_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_13_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_13_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_13_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_13_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_13_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_13_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_13_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_13_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_13_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_13_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_13_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_13_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_13_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_13_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_13_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_13_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_13_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_13_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_13_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_13_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_13_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_13_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_13_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_13_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_13_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_13_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_13_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_13_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_13_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_13_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_13_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_13_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_13_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_13_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_13_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_13_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_13_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_13_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_13_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_13_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_13_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_13_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_13_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_13_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_13_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_13_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_13_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_13_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_13_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_13_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_13_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_13_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_13_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_13_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_13_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_13_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_13_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_13_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_13_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_13_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_13_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_13_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_13_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_13_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_13_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_13_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_13_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_13_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_13_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_13_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_13_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_13_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_13_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_13_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_13_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_13_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_13_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_13_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_13_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_13_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_13_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_13_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_13_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_13_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_13_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_13_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_13_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_13_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_13_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_13_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_13_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_13_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_13_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_13_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_13_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_13_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_13_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_13_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_13_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_13_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_13_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_13_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_13_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_13_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_13_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_13_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_13_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_13_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_13_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_13_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_13_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_13_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_13_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_13_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_13_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_13_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_13_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_13_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_13_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_13_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_13_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_13_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_14_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_14_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_14_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_14_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_14_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_14_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_14_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_14_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_14_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_14_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_14_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_14_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_14_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_14_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_14_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_14_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_14_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_14_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_14_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_14_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_14_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_14_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_14_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_14_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_14_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_14_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_14_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_14_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_14_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_14_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_14_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_14_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_14_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_14_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_14_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_14_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_14_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_14_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_14_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_14_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_14_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_14_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_14_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_14_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_14_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_14_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_14_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_14_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_14_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_14_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_14_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_14_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_14_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_14_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_14_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_14_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_14_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_14_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_14_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_14_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_14_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_14_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_14_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_14_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_14_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_14_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_14_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_14_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_14_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_14_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_14_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_14_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_14_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_14_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_14_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_14_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_14_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_14_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_14_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_14_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_14_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_14_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_14_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_14_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_14_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_14_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_14_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_14_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_14_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_14_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_14_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_14_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_14_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_14_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_14_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_14_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_14_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_14_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_14_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_14_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_14_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_14_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_14_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_14_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_14_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_14_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_14_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_14_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_14_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_14_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_14_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_14_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_14_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_14_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_14_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_14_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_14_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_14_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_14_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_14_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_14_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_14_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_14_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_14_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_14_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_14_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_14_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_14_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_14_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_14_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_14_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_14_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_14_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_14_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_14_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_14_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_14_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_14_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_14_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_14_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_14_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_14_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_14_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_14_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_14_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_14_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_14_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_14_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_14_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_14_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_14_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_14_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_14_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_14_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_14_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_14_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_14_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_14_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_14_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_14_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_14_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_14_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_14_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_14_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_14_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_14_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_14_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_14_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_14_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_14_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_14_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_14_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_14_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_14_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_14_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_14_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_14_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_14_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_14_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_14_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_14_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_14_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_14_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_14_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_14_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_14_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_14_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_14_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_14_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_14_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_14_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_14_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_14_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_14_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_14_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_14_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_14_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_14_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_14_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_14_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_14_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_14_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_14_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_14_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_14_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_14_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_14_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_14_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_14_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_14_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_14_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_14_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_14_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_14_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_14_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_14_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_14_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_14_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_14_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_14_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_14_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_14_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_14_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_14_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_14_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_14_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_14_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_14_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_14_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_14_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_14_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_14_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_14_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_14_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_14_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_14_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_14_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_14_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_14_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_14_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_14_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_14_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_14_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_14_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_14_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_14_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_14_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_14_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_14_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_14_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_14_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_14_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_14_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_14_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_14_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_14_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_14_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_14_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_14_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_14_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_14_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_14_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_14_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_14_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_14_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_14_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_14_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_14_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_14_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_14_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_14_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_14_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_14_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_14_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_14_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_14_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_14_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_14_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_14_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_14_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_14_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_14_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_14_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_14_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_14_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_14_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_14_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_14_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_14_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_14_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_14_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_14_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_14_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_14_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_14_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_14_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_14_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_14_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_14_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_14_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_14_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_14_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_14_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_14_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_14_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_14_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_14_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_14_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_14_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_14_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_14_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_14_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_14_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_14_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_14_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_14_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_14_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_14_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_14_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_14_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_14_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_14_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_14_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_14_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_14_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_14_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_14_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_14_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_14_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_14_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_14_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_14_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_14_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_14_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_14_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_14_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_14_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_14_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_14_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_14_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_14_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_14_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_15_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_15_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_15_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_15_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_15_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_15_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_15_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_15_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_15_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_15_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_15_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_15_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_15_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_15_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_15_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_15_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_15_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_15_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_15_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_15_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_15_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_15_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_15_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_15_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_15_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_15_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_15_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_15_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_15_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_15_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_15_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_15_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_15_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_15_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_15_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_15_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_15_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_15_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_15_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_15_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_15_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_15_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_15_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_15_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_15_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_15_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_15_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_15_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_15_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_15_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_15_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_15_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_15_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_15_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_15_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_15_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_15_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_15_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_15_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_15_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_15_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_15_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_15_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_15_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_15_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_15_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_15_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_15_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_15_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_15_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_15_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_15_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_15_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_15_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_15_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_15_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_15_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_15_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_15_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_15_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_15_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_15_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_15_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_15_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_15_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_15_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_15_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_15_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_15_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_15_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_15_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_15_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_15_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_15_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_15_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_15_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_15_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_15_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_15_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_15_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_15_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_15_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_15_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_15_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_15_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_15_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_15_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_15_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_15_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_15_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_15_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_15_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_15_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_15_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_15_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_15_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_15_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_15_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_15_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_15_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_15_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_15_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_15_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_15_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_15_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_15_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_15_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_15_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_15_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_15_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_15_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_15_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_15_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_15_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_15_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_15_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_15_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_15_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_15_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_15_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_15_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_15_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_15_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_15_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_15_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_15_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_15_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_15_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_15_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_15_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_15_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_15_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_15_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_15_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_15_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_15_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_15_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_15_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_15_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_15_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_15_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_15_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_15_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_15_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_15_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_15_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_15_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_15_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_15_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_15_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_15_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_15_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_15_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_15_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_15_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_15_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_15_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_15_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_15_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_15_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_15_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_15_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_15_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_15_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_15_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_15_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_15_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_15_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_15_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_15_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_15_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_15_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_15_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_15_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_15_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_15_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_15_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_15_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_15_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_15_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_15_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_15_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_15_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_15_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_15_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_15_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_15_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_15_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_15_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_15_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_15_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_15_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_15_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_15_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_15_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_15_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_15_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_15_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_15_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_15_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_15_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_15_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_15_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_15_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_15_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_15_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_15_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_15_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_15_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_15_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_15_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_15_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_15_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_15_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_15_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_15_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_15_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_15_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_15_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_15_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_15_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_15_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_15_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_15_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_15_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_15_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_15_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_15_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_15_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_15_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_15_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_15_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_15_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_15_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_15_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_15_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_15_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_15_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_15_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_15_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_15_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_15_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_15_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_15_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_15_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_15_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_15_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_15_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_15_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_15_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_15_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_15_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_15_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_15_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_15_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_15_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_15_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_15_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_15_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_15_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_15_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_15_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_15_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_15_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_15_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_15_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_15_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_15_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_15_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_15_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_15_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_15_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_15_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_15_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_15_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_15_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_15_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_15_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_15_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_15_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_15_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_15_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_15_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_15_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_15_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_15_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_15_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_15_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_15_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_15_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_15_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_15_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_15_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_15_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_15_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_15_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_15_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_15_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_15_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_15_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_15_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_15_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_15_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_15_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_15_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_15_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_15_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_15_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_15_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_15_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_15_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_15_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_15_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_15_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_15_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_15_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_15_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_15_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_15_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_15_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_15_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_15_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_16_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_16_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_16_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_16_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_16_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_16_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_16_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_16_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_16_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_16_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_16_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_16_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_16_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_16_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_16_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_16_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_16_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_16_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_16_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_16_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_16_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_16_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_16_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_16_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_16_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_16_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_16_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_16_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_16_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_16_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_16_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_16_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_16_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_16_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_16_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_16_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_16_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_16_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_16_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_16_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_16_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_16_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_16_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_16_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_16_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_16_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_16_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_16_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_16_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_16_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_16_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_16_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_16_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_16_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_16_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_16_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_16_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_16_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_16_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_16_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_16_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_16_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_16_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_16_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_16_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_16_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_16_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_16_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_16_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_16_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_16_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_16_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_16_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_16_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_16_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_16_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_16_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_16_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_16_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_16_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_16_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_16_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_16_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_16_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_16_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_16_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_16_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_16_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_16_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_16_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_16_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_16_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_16_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_16_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_16_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_16_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_16_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_16_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_16_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_16_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_16_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_16_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_16_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_16_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_16_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_16_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_16_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_16_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_16_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_16_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_16_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_16_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_16_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_16_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_16_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_16_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_16_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_16_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_16_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_16_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_16_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_16_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_16_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_16_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_16_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_16_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_16_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_16_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_16_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_16_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_16_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_16_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_16_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_16_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_16_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_16_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_16_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_16_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_16_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_16_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_16_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_16_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_16_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_16_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_16_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_16_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_16_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_16_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_16_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_16_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_16_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_16_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_16_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_16_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_16_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_16_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_16_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_16_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_16_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_16_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_16_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_16_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_16_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_16_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_16_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_16_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_16_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_16_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_16_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_16_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_16_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_16_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_16_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_16_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_16_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_16_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_16_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_16_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_16_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_16_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_16_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_16_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_16_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_16_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_16_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_16_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_16_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_16_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_16_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_16_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_16_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_16_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_16_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_16_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_16_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_16_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_16_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_16_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_16_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_16_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_16_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_16_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_16_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_16_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_16_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_16_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_16_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_16_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_16_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_16_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_16_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_16_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_16_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_16_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_16_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_16_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_16_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_16_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_16_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_16_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_16_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_16_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_16_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_16_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_16_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_16_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_16_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_16_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_16_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_16_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_16_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_16_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_16_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_16_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_16_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_16_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_16_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_16_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_16_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_16_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_16_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_16_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_16_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_16_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_16_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_16_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_16_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_16_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_16_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_16_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_16_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_16_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_16_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_16_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_16_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_16_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_16_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_16_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_16_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_16_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_16_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_16_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_16_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_16_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_16_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_16_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_16_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_16_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_16_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_16_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_16_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_16_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_16_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_16_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_16_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_16_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_16_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_16_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_16_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_16_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_16_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_16_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_16_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_16_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_16_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_16_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_16_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_16_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_16_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_16_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_16_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_16_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_16_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_16_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_16_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_16_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_16_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_16_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_16_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_16_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_16_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_16_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_16_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_16_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_16_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_16_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_16_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_16_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_16_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_16_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_16_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_16_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_16_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_16_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_16_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_16_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_16_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_16_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_16_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_16_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_16_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_16_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_16_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_16_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_16_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_16_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_16_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_16_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_16_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_16_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_16_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_16_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_16_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_16_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_16_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_16_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_16_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_16_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_16_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_16_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_16_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_16_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_17_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_17_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_17_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_17_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_17_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_17_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_17_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_17_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_17_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_17_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_17_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_17_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_17_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_17_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_17_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_17_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_17_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_17_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_17_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_17_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_17_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_17_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_17_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_17_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_17_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_17_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_17_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_17_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_17_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_17_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_17_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_17_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_17_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_17_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_17_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_17_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_17_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_17_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_17_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_17_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_17_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_17_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_17_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_17_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_17_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_17_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_17_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_17_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_17_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_17_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_17_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_17_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_17_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_17_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_17_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_17_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_17_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_17_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_17_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_17_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_17_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_17_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_17_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_17_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_17_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_17_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_17_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_17_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_17_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_17_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_17_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_17_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_17_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_17_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_17_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_17_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_17_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_17_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_17_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_17_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_17_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_17_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_17_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_17_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_17_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_17_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_17_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_17_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_17_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_17_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_17_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_17_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_17_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_17_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_17_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_17_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_17_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_17_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_17_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_17_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_17_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_17_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_17_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_17_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_17_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_17_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_17_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_17_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_17_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_17_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_17_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_17_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_17_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_17_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_17_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_17_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_17_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_17_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_17_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_17_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_17_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_17_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_17_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_17_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_17_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_17_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_17_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_17_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_17_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_17_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_17_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_17_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_17_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_17_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_17_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_17_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_17_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_17_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_17_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_17_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_17_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_17_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_17_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_17_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_17_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_17_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_17_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_17_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_17_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_17_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_17_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_17_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_17_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_17_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_17_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_17_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_17_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_17_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_17_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_17_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_17_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_17_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_17_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_17_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_17_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_17_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_17_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_17_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_17_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_17_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_17_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_17_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_17_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_17_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_17_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_17_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_17_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_17_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_17_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_17_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_17_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_17_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_17_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_17_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_17_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_17_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_17_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_17_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_17_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_17_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_17_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_17_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_17_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_17_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_17_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_17_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_17_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_17_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_17_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_17_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_17_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_17_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_17_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_17_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_17_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_17_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_17_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_17_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_17_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_17_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_17_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_17_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_17_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_17_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_17_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_17_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_17_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_17_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_17_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_17_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_17_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_17_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_17_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_17_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_17_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_17_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_17_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_17_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_17_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_17_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_17_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_17_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_17_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_17_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_17_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_17_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_17_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_17_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_17_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_17_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_17_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_17_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_17_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_17_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_17_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_17_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_17_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_17_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_17_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_17_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_17_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_17_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_17_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_17_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_17_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_17_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_17_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_17_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_17_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_17_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_17_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_17_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_17_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_17_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_17_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_17_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_17_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_17_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_17_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_17_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_17_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_17_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_17_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_17_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_17_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_17_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_17_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_17_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_17_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_17_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_17_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_17_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_17_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_17_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_17_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_17_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_17_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_17_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_17_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_17_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_17_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_17_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_17_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_17_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_17_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_17_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_17_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_17_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_17_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_17_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_17_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_17_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_17_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_17_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_17_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_17_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_17_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_17_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_17_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_17_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_17_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_17_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_17_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_17_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_17_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_17_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_17_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_17_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_17_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_17_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_17_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_17_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_17_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_17_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_17_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_17_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_17_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_17_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_17_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_17_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_17_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_17_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_17_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_17_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_17_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_17_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_17_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_17_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_17_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_17_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_17_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_17_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_18_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_18_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_18_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_18_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_18_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_18_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_18_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_18_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_18_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_18_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_18_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_18_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_18_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_18_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_18_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_18_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_18_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_18_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_18_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_18_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_18_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_18_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_18_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_18_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_18_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_18_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_18_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_18_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_18_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_18_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_18_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_18_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_18_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_18_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_18_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_18_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_18_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_18_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_18_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_18_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_18_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_18_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_18_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_18_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_18_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_18_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_18_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_18_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_18_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_18_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_18_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_18_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_18_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_18_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_18_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_18_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_18_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_18_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_18_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_18_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_18_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_18_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_18_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_18_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_18_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_18_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_18_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_18_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_18_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_18_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_18_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_18_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_18_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_18_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_18_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_18_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_18_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_18_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_18_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_18_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_18_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_18_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_18_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_18_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_18_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_18_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_18_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_18_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_18_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_18_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_18_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_18_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_18_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_18_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_18_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_18_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_18_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_18_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_18_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_18_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_18_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_18_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_18_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_18_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_18_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_18_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_18_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_18_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_18_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_18_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_18_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_18_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_18_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_18_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_18_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_18_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_18_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_18_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_18_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_18_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_18_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_18_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_18_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_18_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_18_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_18_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_18_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_18_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_18_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_18_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_18_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_18_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_18_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_18_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_18_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_18_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_18_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_18_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_18_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_18_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_18_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_18_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_18_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_18_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_18_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_18_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_18_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_18_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_18_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_18_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_18_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_18_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_18_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_18_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_18_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_18_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_18_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_18_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_18_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_18_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_18_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_18_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_18_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_18_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_18_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_18_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_18_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_18_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_18_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_18_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_18_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_18_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_18_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_18_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_18_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_18_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_18_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_18_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_18_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_18_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_18_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_18_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_18_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_18_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_18_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_18_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_18_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_18_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_18_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_18_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_18_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_18_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_18_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_18_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_18_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_18_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_18_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_18_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_18_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_18_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_18_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_18_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_18_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_18_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_18_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_18_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_18_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_18_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_18_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_18_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_18_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_18_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_18_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_18_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_18_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_18_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_18_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_18_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_18_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_18_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_18_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_18_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_18_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_18_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_18_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_18_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_18_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_18_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_18_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_18_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_18_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_18_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_18_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_18_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_18_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_18_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_18_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_18_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_18_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_18_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_18_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_18_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_18_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_18_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_18_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_18_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_18_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_18_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_18_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_18_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_18_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_18_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_18_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_18_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_18_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_18_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_18_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_18_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_18_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_18_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_18_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_18_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_18_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_18_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_18_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_18_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_18_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_18_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_18_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_18_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_18_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_18_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_18_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_18_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_18_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_18_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_18_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_18_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_18_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_18_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_18_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_18_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_18_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_18_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_18_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_18_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_18_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_18_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_18_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_18_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_18_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_18_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_18_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_18_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_18_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_18_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_18_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_18_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_18_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_18_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_18_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_18_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_18_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_18_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_18_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_18_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_18_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_18_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_18_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_18_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_18_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_18_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_18_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_18_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_18_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_18_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_18_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_18_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_18_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_18_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_18_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_18_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_18_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_18_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_18_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_18_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_18_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_18_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_18_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_18_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_18_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_18_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_18_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_18_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_18_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_18_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_18_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_18_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_18_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_18_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_18_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_18_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_19_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_19_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_19_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_19_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_19_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_19_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_19_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_19_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_19_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_19_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_19_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_19_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_19_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_19_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_19_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_19_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_19_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_19_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_19_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_19_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_19_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_19_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_19_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_19_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_19_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_19_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_19_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_19_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_19_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_19_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_19_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_19_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_19_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_19_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_19_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_19_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_19_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_19_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_19_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_19_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_19_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_19_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_19_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_19_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_19_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_19_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_19_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_19_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_19_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_19_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_19_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_19_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_19_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_19_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_19_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_19_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_19_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_19_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_19_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_19_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_19_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_19_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_19_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_19_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_19_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_19_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_19_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_19_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_19_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_19_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_19_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_19_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_19_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_19_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_19_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_19_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_19_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_19_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_19_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_19_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_19_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_19_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_19_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_19_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_19_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_19_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_19_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_19_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_19_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_19_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_19_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_19_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_19_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_19_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_19_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_19_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_19_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_19_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_19_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_19_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_19_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_19_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_19_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_19_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_19_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_19_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_19_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_19_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_19_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_19_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_19_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_19_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_19_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_19_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_19_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_19_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_19_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_19_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_19_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_19_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_19_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_19_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_19_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_19_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_19_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_19_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_19_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_19_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_19_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_19_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_19_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_19_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_19_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_19_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_19_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_19_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_19_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_19_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_19_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_19_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_19_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_19_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_19_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_19_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_19_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_19_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_19_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_19_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_19_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_19_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_19_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_19_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_19_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_19_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_19_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_19_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_19_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_19_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_19_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_19_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_19_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_19_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_19_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_19_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_19_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_19_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_19_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_19_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_19_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_19_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_19_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_19_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_19_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_19_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_19_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_19_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_19_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_19_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_19_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_19_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_19_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_19_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_19_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_19_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_19_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_19_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_19_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_19_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_19_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_19_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_19_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_19_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_19_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_19_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_19_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_19_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_19_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_19_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_19_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_19_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_19_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_19_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_19_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_19_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_19_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_19_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_19_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_19_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_19_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_19_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_19_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_19_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_19_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_19_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_19_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_19_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_19_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_19_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_19_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_19_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_19_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_19_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_19_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_19_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_19_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_19_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_19_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_19_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_19_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_19_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_19_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_19_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_19_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_19_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_19_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_19_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_19_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_19_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_19_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_19_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_19_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_19_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_19_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_19_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_19_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_19_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_19_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_19_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_19_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_19_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_19_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_19_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_19_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_19_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_19_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_19_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_19_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_19_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_19_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_19_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_19_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_19_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_19_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_19_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_19_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_19_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_19_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_19_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_19_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_19_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_19_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_19_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_19_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_19_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_19_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_19_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_19_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_19_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_19_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_19_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_19_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_19_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_19_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_19_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_19_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_19_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_19_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_19_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_19_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_19_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_19_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_19_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_19_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_19_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_19_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_19_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_19_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_19_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_19_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_19_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_19_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_19_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_19_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_19_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_19_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_19_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_19_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_19_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_19_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_19_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_19_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_19_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_19_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_19_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_19_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_19_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_19_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_19_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_19_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_19_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_19_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_19_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_19_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_19_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_19_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_19_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_19_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_19_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_19_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_19_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_19_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_19_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_19_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_19_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_19_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_19_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_19_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_19_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_19_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_19_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_19_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_19_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_20_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_20_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_20_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_20_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_20_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_20_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_20_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_20_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_20_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_20_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_20_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_20_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_20_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_20_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_20_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_20_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_20_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_20_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_20_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_20_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_20_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_20_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_20_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_20_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_20_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_20_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_20_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_20_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_20_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_20_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_20_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_20_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_20_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_20_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_20_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_20_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_20_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_20_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_20_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_20_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_20_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_20_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_20_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_20_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_20_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_20_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_20_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_20_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_20_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_20_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_20_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_20_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_20_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_20_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_20_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_20_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_20_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_20_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_20_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_20_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_20_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_20_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_20_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_20_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_20_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_20_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_20_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_20_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_20_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_20_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_20_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_20_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_20_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_20_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_20_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_20_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_20_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_20_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_20_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_20_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_20_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_20_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_20_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_20_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_20_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_20_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_20_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_20_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_20_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_20_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_20_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_20_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_20_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_20_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_20_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_20_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_20_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_20_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_20_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_20_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_20_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_20_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_20_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_20_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_20_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_20_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_20_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_20_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_20_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_20_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_20_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_20_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_20_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_20_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_20_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_20_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_20_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_20_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_20_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_20_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_20_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_20_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_20_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_20_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_20_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_20_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_20_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_20_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_20_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_20_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_20_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_20_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_20_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_20_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_20_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_20_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_20_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_20_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_20_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_20_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_20_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_20_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_20_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_20_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_20_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_20_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_20_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_20_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_20_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_20_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_20_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_20_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_20_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_20_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_20_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_20_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_20_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_20_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_20_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_20_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_20_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_20_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_20_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_20_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_20_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_20_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_20_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_20_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_20_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_20_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_20_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_20_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_20_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_20_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_20_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_20_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_20_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_20_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_20_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_20_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_20_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_20_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_20_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_20_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_20_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_20_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_20_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_20_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_20_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_20_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_20_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_20_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_20_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_20_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_20_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_20_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_20_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_20_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_20_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_20_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_20_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_20_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_20_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_20_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_20_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_20_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_20_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_20_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_20_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_20_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_20_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_20_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_20_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_20_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_20_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_20_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_20_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_20_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_20_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_20_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_20_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_20_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_20_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_20_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_20_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_20_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_20_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_20_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_20_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_20_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_20_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_20_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_20_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_20_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_20_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_20_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_20_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_20_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_20_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_20_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_20_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_20_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_20_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_20_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_20_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_20_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_20_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_20_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_20_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_20_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_20_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_20_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_20_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_20_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_20_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_20_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_20_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_20_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_20_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_20_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_20_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_20_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_20_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_20_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_20_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_20_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_20_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_20_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_20_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_20_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_20_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_20_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_20_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_20_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_20_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_20_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_20_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_20_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_20_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_20_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_20_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_20_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_20_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_20_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_20_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_20_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_20_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_20_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_20_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_20_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_20_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_20_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_20_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_20_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_20_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_20_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_20_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_20_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_20_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_20_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_20_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_20_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_20_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_20_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_20_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_20_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_20_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_20_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_20_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_20_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_20_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_20_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_20_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_20_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_20_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_20_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_20_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_20_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_20_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_20_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_20_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_20_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_20_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_20_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_20_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_20_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_20_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_20_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_20_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_20_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_20_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_20_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_20_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_20_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_20_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_20_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_20_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_20_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_20_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_20_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_20_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_20_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_21_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_21_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_21_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_21_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_21_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_21_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_21_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_21_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_21_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_21_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_21_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_21_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_21_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_21_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_21_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_21_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_21_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_21_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_21_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_21_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_21_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_21_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_21_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_21_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_21_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_21_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_21_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_21_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_21_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_21_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_21_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_21_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_21_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_21_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_21_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_21_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_21_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_21_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_21_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_21_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_21_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_21_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_21_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_21_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_21_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_21_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_21_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_21_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_21_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_21_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_21_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_21_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_21_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_21_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_21_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_21_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_21_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_21_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_21_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_21_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_21_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_21_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_21_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_21_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_21_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_21_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_21_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_21_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_21_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_21_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_21_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_21_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_21_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_21_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_21_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_21_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_21_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_21_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_21_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_21_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_21_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_21_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_21_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_21_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_21_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_21_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_21_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_21_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_21_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_21_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_21_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_21_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_21_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_21_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_21_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_21_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_21_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_21_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_21_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_21_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_21_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_21_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_21_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_21_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_21_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_21_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_21_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_21_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_21_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_21_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_21_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_21_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_21_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_21_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_21_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_21_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_21_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_21_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_21_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_21_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_21_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_21_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_21_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_21_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_21_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_21_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_21_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_21_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_21_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_21_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_21_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_21_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_21_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_21_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_21_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_21_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_21_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_21_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_21_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_21_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_21_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_21_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_21_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_21_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_21_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_21_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_21_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_21_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_21_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_21_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_21_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_21_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_21_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_21_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_21_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_21_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_21_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_21_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_21_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_21_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_21_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_21_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_21_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_21_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_21_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_21_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_21_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_21_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_21_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_21_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_21_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_21_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_21_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_21_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_21_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_21_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_21_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_21_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_21_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_21_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_21_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_21_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_21_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_21_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_21_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_21_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_21_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_21_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_21_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_21_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_21_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_21_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_21_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_21_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_21_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_21_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_21_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_21_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_21_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_21_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_21_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_21_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_21_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_21_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_21_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_21_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_21_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_21_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_21_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_21_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_21_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_21_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_21_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_21_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_21_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_21_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_21_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_21_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_21_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_21_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_21_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_21_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_21_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_21_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_21_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_21_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_21_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_21_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_21_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_21_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_21_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_21_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_21_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_21_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_21_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_21_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_21_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_21_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_21_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_21_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_21_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_21_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_21_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_21_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_21_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_21_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_21_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_21_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_21_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_21_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_21_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_21_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_21_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_21_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_21_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_21_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_21_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_21_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_21_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_21_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_21_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_21_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_21_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_21_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_21_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_21_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_21_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_21_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_21_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_21_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_21_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_21_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_21_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_21_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_21_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_21_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_21_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_21_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_21_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_21_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_21_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_21_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_21_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_21_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_21_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_21_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_21_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_21_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_21_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_21_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_21_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_21_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_21_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_21_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_21_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_21_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_21_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_21_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_21_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_21_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_21_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_21_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_21_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_21_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_21_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_21_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_21_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_21_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_21_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_21_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_21_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_21_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_21_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_21_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_21_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_21_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_21_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_21_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_21_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_21_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_21_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_21_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_21_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_21_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_21_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_21_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_21_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_21_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_21_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_21_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_21_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_21_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_21_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_21_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_21_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_21_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_21_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_21_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_21_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_21_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_21_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_21_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_22_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_22_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_22_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_22_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_22_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_22_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_22_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_22_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_22_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_22_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_22_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_22_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_22_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_22_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_22_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_22_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_22_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_22_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_22_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_22_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_22_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_22_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_22_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_22_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_22_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_22_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_22_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_22_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_22_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_22_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_22_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_22_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_22_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_22_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_22_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_22_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_22_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_22_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_22_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_22_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_22_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_22_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_22_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_22_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_22_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_22_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_22_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_22_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_22_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_22_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_22_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_22_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_22_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_22_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_22_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_22_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_22_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_22_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_22_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_22_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_22_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_22_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_22_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_22_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_22_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_22_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_22_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_22_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_22_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_22_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_22_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_22_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_22_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_22_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_22_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_22_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_22_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_22_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_22_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_22_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_22_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_22_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_22_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_22_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_22_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_22_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_22_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_22_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_22_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_22_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_22_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_22_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_22_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_22_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_22_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_22_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_22_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_22_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_22_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_22_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_22_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_22_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_22_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_22_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_22_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_22_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_22_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_22_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_22_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_22_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_22_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_22_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_22_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_22_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_22_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_22_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_22_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_22_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_22_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_22_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_22_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_22_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_22_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_22_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_22_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_22_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_22_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_22_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_22_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_22_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_22_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_22_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_22_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_22_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_22_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_22_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_22_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_22_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_22_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_22_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_22_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_22_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_22_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_22_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_22_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_22_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_22_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_22_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_22_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_22_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_22_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_22_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_22_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_22_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_22_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_22_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_22_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_22_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_22_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_22_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_22_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_22_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_22_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_22_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_22_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_22_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_22_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_22_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_22_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_22_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_22_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_22_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_22_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_22_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_22_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_22_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_22_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_22_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_22_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_22_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_22_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_22_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_22_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_22_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_22_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_22_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_22_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_22_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_22_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_22_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_22_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_22_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_22_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_22_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_22_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_22_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_22_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_22_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_22_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_22_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_22_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_22_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_22_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_22_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_22_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_22_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_22_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_22_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_22_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_22_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_22_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_22_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_22_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_22_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_22_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_22_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_22_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_22_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_22_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_22_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_22_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_22_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_22_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_22_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_22_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_22_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_22_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_22_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_22_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_22_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_22_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_22_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_22_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_22_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_22_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_22_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_22_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_22_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_22_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_22_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_22_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_22_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_22_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_22_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_22_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_22_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_22_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_22_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_22_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_22_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_22_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_22_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_22_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_22_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_22_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_22_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_22_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_22_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_22_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_22_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_22_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_22_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_22_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_22_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_22_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_22_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_22_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_22_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_22_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_22_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_22_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_22_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_22_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_22_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_22_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_22_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_22_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_22_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_22_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_22_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_22_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_22_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_22_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_22_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_22_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_22_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_22_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_22_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_22_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_22_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_22_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_22_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_22_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_22_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_22_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_22_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_22_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_22_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_22_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_22_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_22_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_22_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_22_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_22_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_22_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_22_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_22_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_22_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_22_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_22_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_22_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_22_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_22_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_22_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_22_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_22_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_22_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_22_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_22_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_22_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_22_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_22_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_22_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_22_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_22_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_22_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_22_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_22_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_22_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_22_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_22_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_22_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_22_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_22_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_22_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_22_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_22_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_22_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_22_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_22_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_22_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_22_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_23_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_23_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_23_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_23_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_23_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_23_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_23_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_23_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_23_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_23_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_23_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_23_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_23_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_23_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_23_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_23_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_23_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_23_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_23_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_23_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_23_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_23_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_23_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_23_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_23_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_23_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_23_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_23_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_23_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_23_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_23_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_23_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_23_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_23_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_23_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_23_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_23_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_23_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_23_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_23_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_23_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_23_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_23_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_23_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_23_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_23_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_23_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_23_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_23_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_23_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_23_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_23_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_23_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_23_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_23_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_23_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_23_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_23_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_23_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_23_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_23_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_23_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_23_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_23_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_23_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_23_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_23_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_23_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_23_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_23_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_23_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_23_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_23_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_23_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_23_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_23_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_23_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_23_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_23_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_23_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_23_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_23_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_23_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_23_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_23_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_23_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_23_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_23_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_23_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_23_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_23_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_23_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_23_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_23_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_23_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_23_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_23_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_23_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_23_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_23_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_23_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_23_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_23_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_23_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_23_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_23_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_23_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_23_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_23_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_23_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_23_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_23_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_23_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_23_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_23_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_23_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_23_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_23_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_23_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_23_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_23_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_23_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_23_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_23_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_23_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_23_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_23_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_23_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_23_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_23_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_23_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_23_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_23_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_23_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_23_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_23_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_23_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_23_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_23_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_23_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_23_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_23_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_23_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_23_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_23_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_23_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_23_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_23_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_23_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_23_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_23_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_23_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_23_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_23_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_23_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_23_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_23_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_23_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_23_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_23_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_23_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_23_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_23_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_23_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_23_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_23_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_23_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_23_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_23_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_23_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_23_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_23_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_23_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_23_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_23_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_23_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_23_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_23_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_23_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_23_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_23_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_23_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_23_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_23_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_23_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_23_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_23_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_23_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_23_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_23_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_23_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_23_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_23_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_23_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_23_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_23_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_23_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_23_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_23_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_23_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_23_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_23_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_23_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_23_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_23_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_23_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_23_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_23_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_23_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_23_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_23_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_23_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_23_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_23_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_23_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_23_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_23_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_23_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_23_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_23_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_23_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_23_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_23_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_23_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_23_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_23_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_23_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_23_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_23_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_23_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_23_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_23_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_23_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_23_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_23_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_23_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_23_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_23_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_23_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_23_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_23_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_23_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_23_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_23_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_23_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_23_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_23_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_23_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_23_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_23_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_23_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_23_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_23_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_23_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_23_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_23_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_23_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_23_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_23_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_23_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_23_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_23_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_23_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_23_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_23_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_23_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_23_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_23_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_23_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_23_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_23_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_23_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_23_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_23_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_23_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_23_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_23_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_23_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_23_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_23_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_23_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_23_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_23_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_23_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_23_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_23_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_23_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_23_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_23_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_23_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_23_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_23_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_23_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_23_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_23_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_23_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_23_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_23_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_23_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_23_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_23_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_23_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_23_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_23_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_23_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_23_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_23_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_23_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_23_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_23_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_23_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_23_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_23_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_23_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_23_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_23_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_23_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_23_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_23_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_23_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_23_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_23_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_23_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_23_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_23_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_23_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_23_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_23_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_23_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_23_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_23_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_23_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_23_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_23_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_23_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_23_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_23_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_23_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_23_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_23_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_23_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_23_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_24_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_24_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_24_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_24_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_24_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_24_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_24_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_24_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_24_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_24_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_24_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_24_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_24_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_24_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_24_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_24_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_24_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_24_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_24_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_24_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_24_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_24_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_24_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_24_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_24_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_24_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_24_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_24_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_24_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_24_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_24_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_24_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_24_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_24_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_24_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_24_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_24_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_24_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_24_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_24_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_24_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_24_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_24_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_24_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_24_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_24_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_24_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_24_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_24_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_24_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_24_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_24_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_24_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_24_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_24_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_24_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_24_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_24_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_24_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_24_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_24_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_24_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_24_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_24_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_24_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_24_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_24_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_24_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_24_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_24_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_24_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_24_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_24_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_24_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_24_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_24_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_24_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_24_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_24_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_24_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_24_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_24_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_24_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_24_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_24_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_24_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_24_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_24_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_24_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_24_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_24_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_24_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_24_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_24_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_24_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_24_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_24_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_24_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_24_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_24_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_24_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_24_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_24_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_24_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_24_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_24_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_24_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_24_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_24_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_24_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_24_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_24_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_24_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_24_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_24_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_24_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_24_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_24_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_24_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_24_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_24_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_24_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_24_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_24_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_24_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_24_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_24_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_24_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_24_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_24_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_24_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_24_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_24_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_24_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_24_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_24_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_24_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_24_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_24_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_24_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_24_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_24_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_24_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_24_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_24_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_24_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_24_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_24_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_24_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_24_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_24_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_24_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_24_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_24_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_24_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_24_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_24_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_24_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_24_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_24_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_24_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_24_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_24_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_24_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_24_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_24_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_24_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_24_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_24_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_24_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_24_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_24_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_24_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_24_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_24_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_24_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_24_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_24_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_24_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_24_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_24_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_24_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_24_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_24_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_24_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_24_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_24_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_24_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_24_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_24_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_24_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_24_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_24_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_24_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_24_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_24_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_24_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_24_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_24_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_24_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_24_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_24_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_24_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_24_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_24_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_24_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_24_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_24_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_24_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_24_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_24_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_24_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_24_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_24_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_24_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_24_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_24_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_24_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_24_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_24_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_24_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_24_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_24_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_24_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_24_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_24_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_24_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_24_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_24_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_24_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_24_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_24_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_24_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_24_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_24_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_24_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_24_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_24_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_24_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_24_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_24_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_24_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_24_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_24_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_24_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_24_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_24_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_24_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_24_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_24_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_24_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_24_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_24_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_24_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_24_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_24_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_24_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_24_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_24_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_24_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_24_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_24_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_24_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_24_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_24_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_24_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_24_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_24_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_24_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_24_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_24_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_24_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_24_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_24_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_24_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_24_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_24_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_24_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_24_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_24_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_24_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_24_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_24_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_24_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_24_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_24_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_24_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_24_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_24_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_24_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_24_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_24_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_24_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_24_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_24_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_24_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_24_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_24_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_24_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_24_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_24_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_24_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_24_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_24_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_24_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_24_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_24_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_24_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_24_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_24_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_24_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_24_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_24_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_24_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_24_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_24_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_24_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_24_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_24_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_24_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_24_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_24_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_24_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_24_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_24_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_24_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_24_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_24_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_24_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_24_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_24_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_24_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_24_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_24_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_24_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_24_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_24_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_24_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_24_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_24_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_24_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_24_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_25_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_25_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_25_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_25_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_25_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_25_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_25_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_25_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_25_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_25_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_25_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_25_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_25_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_25_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_25_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_25_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_25_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_25_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_25_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_25_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_25_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_25_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_25_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_25_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_25_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_25_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_25_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_25_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_25_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_25_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_25_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_25_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_25_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_25_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_25_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_25_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_25_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_25_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_25_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_25_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_25_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_25_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_25_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_25_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_25_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_25_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_25_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_25_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_25_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_25_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_25_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_25_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_25_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_25_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_25_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_25_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_25_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_25_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_25_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_25_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_25_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_25_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_25_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_25_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_25_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_25_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_25_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_25_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_25_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_25_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_25_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_25_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_25_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_25_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_25_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_25_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_25_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_25_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_25_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_25_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_25_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_25_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_25_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_25_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_25_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_25_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_25_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_25_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_25_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_25_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_25_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_25_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_25_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_25_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_25_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_25_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_25_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_25_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_25_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_25_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_25_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_25_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_25_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_25_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_25_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_25_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_25_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_25_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_25_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_25_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_25_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_25_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_25_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_25_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_25_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_25_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_25_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_25_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_25_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_25_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_25_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_25_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_25_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_25_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_25_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_25_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_25_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_25_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_25_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_25_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_25_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_25_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_25_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_25_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_25_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_25_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_25_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_25_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_25_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_25_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_25_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_25_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_25_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_25_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_25_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_25_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_25_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_25_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_25_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_25_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_25_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_25_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_25_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_25_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_25_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_25_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_25_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_25_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_25_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_25_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_25_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_25_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_25_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_25_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_25_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_25_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_25_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_25_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_25_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_25_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_25_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_25_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_25_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_25_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_25_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_25_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_25_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_25_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_25_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_25_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_25_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_25_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_25_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_25_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_25_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_25_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_25_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_25_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_25_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_25_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_25_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_25_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_25_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_25_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_25_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_25_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_25_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_25_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_25_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_25_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_25_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_25_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_25_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_25_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_25_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_25_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_25_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_25_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_25_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_25_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_25_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_25_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_25_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_25_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_25_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_25_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_25_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_25_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_25_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_25_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_25_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_25_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_25_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_25_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_25_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_25_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_25_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_25_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_25_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_25_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_25_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_25_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_25_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_25_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_25_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_25_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_25_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_25_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_25_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_25_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_25_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_25_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_25_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_25_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_25_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_25_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_25_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_25_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_25_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_25_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_25_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_25_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_25_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_25_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_25_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_25_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_25_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_25_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_25_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_25_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_25_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_25_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_25_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_25_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_25_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_25_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_25_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_25_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_25_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_25_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_25_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_25_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_25_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_25_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_25_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_25_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_25_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_25_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_25_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_25_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_25_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_25_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_25_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_25_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_25_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_25_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_25_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_25_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_25_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_25_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_25_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_25_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_25_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_25_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_25_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_25_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_25_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_25_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_25_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_25_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_25_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_25_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_25_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_25_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_25_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_25_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_25_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_25_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_25_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_25_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_25_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_25_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_25_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_25_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_25_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_25_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_25_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_25_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_25_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_25_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_25_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_25_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_25_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_25_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_25_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_25_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_25_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_25_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_25_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_25_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_25_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_25_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_25_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_25_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_25_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_25_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_25_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_25_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_25_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_25_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_25_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_25_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_26_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_26_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_26_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_26_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_26_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_26_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_26_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_26_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_26_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_26_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_26_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_26_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_26_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_26_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_26_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_26_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_26_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_26_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_26_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_26_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_26_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_26_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_26_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_26_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_26_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_26_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_26_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_26_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_26_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_26_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_26_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_26_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_26_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_26_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_26_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_26_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_26_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_26_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_26_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_26_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_26_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_26_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_26_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_26_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_26_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_26_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_26_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_26_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_26_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_26_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_26_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_26_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_26_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_26_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_26_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_26_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_26_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_26_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_26_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_26_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_26_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_26_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_26_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_26_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_26_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_26_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_26_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_26_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_26_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_26_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_26_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_26_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_26_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_26_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_26_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_26_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_26_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_26_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_26_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_26_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_26_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_26_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_26_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_26_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_26_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_26_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_26_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_26_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_26_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_26_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_26_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_26_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_26_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_26_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_26_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_26_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_26_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_26_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_26_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_26_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_26_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_26_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_26_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_26_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_26_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_26_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_26_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_26_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_26_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_26_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_26_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_26_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_26_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_26_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_26_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_26_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_26_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_26_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_26_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_26_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_26_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_26_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_26_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_26_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_26_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_26_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_26_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_26_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_26_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_26_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_26_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_26_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_26_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_26_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_26_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_26_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_26_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_26_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_26_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_26_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_26_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_26_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_26_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_26_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_26_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_26_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_26_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_26_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_26_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_26_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_26_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_26_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_26_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_26_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_26_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_26_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_26_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_26_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_26_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_26_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_26_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_26_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_26_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_26_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_26_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_26_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_26_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_26_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_26_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_26_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_26_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_26_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_26_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_26_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_26_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_26_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_26_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_26_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_26_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_26_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_26_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_26_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_26_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_26_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_26_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_26_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_26_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_26_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_26_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_26_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_26_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_26_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_26_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_26_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_26_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_26_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_26_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_26_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_26_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_26_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_26_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_26_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_26_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_26_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_26_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_26_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_26_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_26_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_26_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_26_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_26_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_26_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_26_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_26_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_26_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_26_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_26_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_26_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_26_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_26_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_26_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_26_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_26_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_26_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_26_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_26_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_26_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_26_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_26_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_26_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_26_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_26_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_26_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_26_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_26_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_26_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_26_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_26_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_26_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_26_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_26_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_26_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_26_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_26_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_26_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_26_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_26_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_26_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_26_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_26_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_26_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_26_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_26_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_26_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_26_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_26_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_26_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_26_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_26_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_26_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_26_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_26_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_26_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_26_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_26_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_26_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_26_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_26_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_26_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_26_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_26_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_26_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_26_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_26_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_26_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_26_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_26_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_26_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_26_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_26_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_26_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_26_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_26_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_26_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_26_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_26_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_26_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_26_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_26_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_26_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_26_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_26_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_26_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_26_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_26_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_26_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_26_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_26_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_26_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_26_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_26_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_26_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_26_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_26_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_26_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_26_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_26_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_26_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_26_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_26_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_26_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_26_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_26_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_26_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_26_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_26_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_26_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_26_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_26_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_26_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_26_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_26_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_26_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_26_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_26_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_26_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_26_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_26_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_26_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_26_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_26_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_26_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_26_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_26_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_26_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_26_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_26_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_26_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_26_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_26_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_26_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_26_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_27_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_27_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_27_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_27_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_27_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_27_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_27_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_27_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_27_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_27_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_27_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_27_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_27_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_27_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_27_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_27_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_27_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_27_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_27_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_27_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_27_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_27_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_27_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_27_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_27_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_27_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_27_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_27_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_27_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_27_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_27_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_27_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_27_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_27_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_27_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_27_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_27_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_27_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_27_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_27_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_27_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_27_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_27_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_27_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_27_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_27_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_27_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_27_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_27_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_27_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_27_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_27_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_27_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_27_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_27_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_27_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_27_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_27_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_27_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_27_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_27_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_27_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_27_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_27_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_27_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_27_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_27_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_27_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_27_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_27_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_27_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_27_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_27_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_27_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_27_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_27_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_27_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_27_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_27_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_27_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_27_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_27_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_27_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_27_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_27_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_27_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_27_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_27_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_27_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_27_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_27_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_27_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_27_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_27_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_27_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_27_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_27_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_27_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_27_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_27_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_27_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_27_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_27_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_27_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_27_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_27_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_27_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_27_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_27_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_27_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_27_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_27_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_27_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_27_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_27_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_27_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_27_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_27_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_27_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_27_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_27_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_27_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_27_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_27_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_27_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_27_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_27_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_27_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_27_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_27_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_27_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_27_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_27_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_27_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_27_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_27_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_27_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_27_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_27_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_27_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_27_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_27_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_27_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_27_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_27_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_27_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_27_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_27_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_27_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_27_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_27_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_27_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_27_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_27_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_27_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_27_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_27_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_27_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_27_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_27_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_27_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_27_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_27_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_27_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_27_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_27_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_27_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_27_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_27_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_27_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_27_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_27_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_27_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_27_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_27_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_27_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_27_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_27_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_27_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_27_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_27_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_27_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_27_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_27_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_27_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_27_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_27_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_27_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_27_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_27_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_27_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_27_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_27_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_27_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_27_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_27_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_27_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_27_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_27_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_27_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_27_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_27_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_27_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_27_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_27_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_27_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_27_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_27_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_27_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_27_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_27_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_27_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_27_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_27_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_27_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_27_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_27_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_27_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_27_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_27_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_27_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_27_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_27_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_27_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_27_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_27_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_27_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_27_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_27_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_27_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_27_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_27_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_27_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_27_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_27_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_27_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_27_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_27_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_27_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_27_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_27_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_27_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_27_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_27_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_27_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_27_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_27_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_27_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_27_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_27_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_27_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_27_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_27_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_27_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_27_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_27_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_27_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_27_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_27_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_27_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_27_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_27_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_27_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_27_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_27_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_27_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_27_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_27_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_27_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_27_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_27_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_27_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_27_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_27_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_27_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_27_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_27_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_27_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_27_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_27_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_27_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_27_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_27_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_27_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_27_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_27_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_27_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_27_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_27_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_27_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_27_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_27_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_27_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_27_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_27_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_27_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_27_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_27_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_27_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_27_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_27_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_27_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_27_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_27_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_27_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_27_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_27_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_27_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_27_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_27_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_27_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_27_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_27_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_27_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_27_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_27_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_27_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_27_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_27_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_27_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_27_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_27_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_27_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_27_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_27_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_27_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_27_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_27_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_27_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_27_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_27_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_27_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_27_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_27_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_27_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_27_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_27_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_27_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_27_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_27_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_27_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_27_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_28_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_28_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_28_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_28_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_28_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_28_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_28_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_28_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_28_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_28_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_28_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_28_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_28_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_28_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_28_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_28_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_28_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_28_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_28_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_28_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_28_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_28_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_28_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_28_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_28_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_28_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_28_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_28_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_28_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_28_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_28_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_28_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_28_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_28_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_28_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_28_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_28_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_28_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_28_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_28_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_28_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_28_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_28_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_28_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_28_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_28_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_28_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_28_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_28_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_28_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_28_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_28_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_28_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_28_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_28_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_28_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_28_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_28_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_28_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_28_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_28_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_28_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_28_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_28_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_28_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_28_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_28_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_28_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_28_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_28_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_28_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_28_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_28_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_28_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_28_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_28_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_28_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_28_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_28_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_28_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_28_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_28_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_28_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_28_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_28_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_28_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_28_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_28_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_28_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_28_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_28_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_28_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_28_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_28_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_28_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_28_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_28_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_28_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_28_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_28_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_28_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_28_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_28_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_28_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_28_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_28_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_28_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_28_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_28_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_28_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_28_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_28_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_28_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_28_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_28_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_28_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_28_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_28_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_28_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_28_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_28_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_28_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_28_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_28_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_28_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_28_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_28_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_28_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_28_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_28_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_28_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_28_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_28_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_28_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_28_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_28_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_28_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_28_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_28_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_28_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_28_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_28_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_28_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_28_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_28_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_28_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_28_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_28_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_28_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_28_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_28_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_28_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_28_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_28_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_28_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_28_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_28_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_28_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_28_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_28_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_28_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_28_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_28_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_28_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_28_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_28_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_28_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_28_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_28_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_28_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_28_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_28_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_28_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_28_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_28_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_28_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_28_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_28_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_28_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_28_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_28_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_28_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_28_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_28_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_28_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_28_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_28_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_28_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_28_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_28_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_28_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_28_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_28_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_28_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_28_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_28_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_28_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_28_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_28_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_28_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_28_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_28_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_28_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_28_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_28_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_28_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_28_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_28_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_28_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_28_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_28_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_28_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_28_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_28_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_28_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_28_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_28_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_28_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_28_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_28_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_28_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_28_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_28_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_28_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_28_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_28_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_28_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_28_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_28_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_28_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_28_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_28_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_28_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_28_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_28_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_28_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_28_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_28_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_28_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_28_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_28_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_28_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_28_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_28_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_28_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_28_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_28_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_28_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_28_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_28_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_28_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_28_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_28_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_28_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_28_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_28_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_28_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_28_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_28_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_28_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_28_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_28_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_28_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_28_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_28_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_28_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_28_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_28_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_28_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_28_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_28_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_28_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_28_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_28_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_28_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_28_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_28_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_28_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_28_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_28_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_28_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_28_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_28_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_28_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_28_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_28_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_28_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_28_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_28_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_28_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_28_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_28_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_28_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_28_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_28_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_28_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_28_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_28_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_28_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_28_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_28_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_28_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_28_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_28_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_28_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_28_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_28_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_28_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_28_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_28_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_28_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_28_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_28_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_28_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_28_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_28_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_28_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_28_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_28_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_28_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_28_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_28_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_28_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_28_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_28_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_28_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_28_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_28_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_28_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_28_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_28_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_28_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_28_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_28_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_28_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_28_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_28_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_28_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_28_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_28_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_28_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_28_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_29_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_29_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_29_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_29_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_29_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_29_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_29_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_29_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_29_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_29_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_29_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_29_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_29_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_29_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_29_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_29_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_29_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_29_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_29_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_29_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_29_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_29_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_29_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_29_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_29_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_29_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_29_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_29_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_29_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_29_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_29_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_29_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_29_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_29_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_29_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_29_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_29_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_29_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_29_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_29_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_29_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_29_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_29_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_29_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_29_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_29_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_29_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_29_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_29_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_29_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_29_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_29_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_29_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_29_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_29_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_29_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_29_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_29_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_29_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_29_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_29_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_29_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_29_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_29_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_29_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_29_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_29_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_29_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_29_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_29_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_29_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_29_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_29_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_29_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_29_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_29_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_29_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_29_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_29_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_29_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_29_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_29_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_29_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_29_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_29_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_29_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_29_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_29_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_29_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_29_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_29_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_29_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_29_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_29_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_29_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_29_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_29_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_29_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_29_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_29_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_29_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_29_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_29_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_29_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_29_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_29_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_29_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_29_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_29_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_29_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_29_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_29_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_29_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_29_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_29_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_29_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_29_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_29_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_29_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_29_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_29_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_29_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_29_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_29_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_29_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_29_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_29_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_29_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_29_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_29_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_29_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_29_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_29_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_29_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_29_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_29_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_29_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_29_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_29_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_29_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_29_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_29_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_29_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_29_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_29_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_29_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_29_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_29_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_29_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_29_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_29_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_29_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_29_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_29_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_29_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_29_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_29_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_29_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_29_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_29_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_29_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_29_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_29_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_29_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_29_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_29_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_29_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_29_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_29_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_29_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_29_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_29_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_29_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_29_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_29_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_29_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_29_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_29_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_29_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_29_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_29_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_29_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_29_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_29_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_29_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_29_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_29_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_29_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_29_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_29_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_29_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_29_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_29_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_29_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_29_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_29_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_29_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_29_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_29_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_29_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_29_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_29_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_29_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_29_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_29_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_29_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_29_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_29_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_29_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_29_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_29_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_29_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_29_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_29_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_29_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_29_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_29_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_29_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_29_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_29_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_29_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_29_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_29_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_29_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_29_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_29_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_29_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_29_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_29_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_29_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_29_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_29_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_29_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_29_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_29_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_29_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_29_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_29_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_29_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_29_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_29_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_29_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_29_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_29_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_29_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_29_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_29_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_29_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_29_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_29_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_29_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_29_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_29_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_29_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_29_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_29_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_29_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_29_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_29_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_29_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_29_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_29_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_29_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_29_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_29_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_29_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_29_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_29_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_29_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_29_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_29_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_29_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_29_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_29_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_29_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_29_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_29_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_29_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_29_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_29_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_29_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_29_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_29_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_29_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_29_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_29_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_29_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_29_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_29_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_29_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_29_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_29_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_29_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_29_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_29_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_29_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_29_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_29_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_29_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_29_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_29_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_29_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_29_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_29_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_29_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_29_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_29_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_29_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_29_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_29_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_29_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_29_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_29_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_29_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_29_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_29_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_29_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_29_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_29_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_29_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_29_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_29_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_29_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_29_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_29_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_29_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_29_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_29_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_29_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_29_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_29_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_29_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_29_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_29_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_29_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_29_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_29_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_29_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_29_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_29_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_29_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_29_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_30_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_30_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_30_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_30_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_30_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_30_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_30_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_30_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_30_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_30_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_30_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_30_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_30_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_30_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_30_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_30_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_30_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_30_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_30_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_30_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_30_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_30_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_30_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_30_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_30_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_30_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_30_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_30_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_30_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_30_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_30_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_30_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_30_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_30_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_30_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_30_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_30_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_30_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_30_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_30_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_30_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_30_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_30_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_30_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_30_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_30_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_30_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_30_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_30_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_30_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_30_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_30_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_30_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_30_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_30_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_30_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_30_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_30_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_30_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_30_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_30_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_30_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_30_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_30_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_30_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_30_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_30_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_30_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_30_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_30_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_30_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_30_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_30_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_30_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_30_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_30_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_30_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_30_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_30_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_30_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_30_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_30_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_30_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_30_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_30_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_30_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_30_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_30_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_30_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_30_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_30_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_30_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_30_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_30_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_30_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_30_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_30_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_30_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_30_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_30_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_30_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_30_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_30_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_30_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_30_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_30_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_30_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_30_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_30_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_30_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_30_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_30_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_30_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_30_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_30_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_30_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_30_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_30_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_30_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_30_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_30_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_30_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_30_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_30_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_30_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_30_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_30_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_30_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_30_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_30_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_30_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_30_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_30_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_30_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_30_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_30_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_30_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_30_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_30_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_30_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_30_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_30_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_30_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_30_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_30_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_30_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_30_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_30_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_30_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_30_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_30_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_30_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_30_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_30_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_30_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_30_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_30_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_30_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_30_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_30_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_30_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_30_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_30_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_30_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_30_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_30_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_30_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_30_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_30_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_30_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_30_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_30_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_30_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_30_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_30_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_30_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_30_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_30_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_30_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_30_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_30_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_30_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_30_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_30_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_30_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_30_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_30_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_30_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_30_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_30_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_30_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_30_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_30_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_30_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_30_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_30_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_30_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_30_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_30_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_30_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_30_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_30_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_30_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_30_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_30_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_30_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_30_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_30_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_30_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_30_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_30_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_30_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_30_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_30_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_30_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_30_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_30_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_30_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_30_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_30_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_30_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_30_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_30_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_30_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_30_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_30_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_30_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_30_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_30_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_30_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_30_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_30_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_30_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_30_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_30_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_30_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_30_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_30_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_30_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_30_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_30_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_30_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_30_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_30_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_30_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_30_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_30_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_30_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_30_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_30_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_30_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_30_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_30_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_30_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_30_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_30_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_30_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_30_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_30_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_30_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_30_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_30_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_30_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_30_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_30_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_30_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_30_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_30_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_30_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_30_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_30_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_30_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_30_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_30_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_30_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_30_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_30_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_30_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_30_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_30_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_30_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_30_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_30_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_30_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_30_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_30_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_30_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_30_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_30_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_30_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_30_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_30_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_30_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_30_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_30_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_30_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_30_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_30_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_30_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_30_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_30_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_30_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_30_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_30_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_30_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_30_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_30_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_30_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_30_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_30_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_30_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_30_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_30_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_30_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_30_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_30_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_30_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_30_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_30_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_30_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_30_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_30_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_30_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_30_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_30_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_30_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_30_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_30_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_30_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_30_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_30_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_30_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_30_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_30_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_30_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_30_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_30_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_30_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_30_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_30_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_30_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_30_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_31_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_31_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_31_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_31_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_31_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_31_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_31_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_31_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_31_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_31_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_31_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_31_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_31_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_31_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_31_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_31_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_31_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_31_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_31_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_31_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_31_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_31_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_31_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_31_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_31_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_31_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_31_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_31_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_31_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_31_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_31_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_31_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_31_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_31_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_31_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_31_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_31_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_31_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_31_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_31_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_31_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_31_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_31_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_31_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_31_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_31_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_31_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_31_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_31_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_31_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_31_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_31_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_31_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_31_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_31_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_31_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_31_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_31_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_31_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_31_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_31_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_31_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_31_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_31_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_31_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_31_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_31_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_31_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_31_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_31_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_31_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_31_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_31_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_31_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_31_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_31_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_31_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_31_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_31_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_31_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_31_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_31_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_31_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_31_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_31_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_31_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_31_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_31_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_31_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_31_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_31_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_31_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_31_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_31_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_31_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_31_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_31_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_31_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_31_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_31_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_31_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_31_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_31_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_31_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_31_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_31_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_31_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_31_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_31_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_31_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_31_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_31_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_31_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_31_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_31_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_31_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_31_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_31_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_31_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_31_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_31_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_31_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_31_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_31_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_31_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_31_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_31_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_31_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_31_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_31_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_31_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_31_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_31_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_31_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_31_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_31_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_31_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_31_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_31_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_31_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_31_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_31_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_31_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_31_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_31_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_31_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_31_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_31_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_31_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_31_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_31_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_31_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_31_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_31_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_31_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_31_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_31_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_31_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_31_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_31_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_31_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_31_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_31_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_31_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_31_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_31_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_31_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_31_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_31_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_31_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_31_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_31_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_31_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_31_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_31_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_31_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_31_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_31_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_31_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_31_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_31_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_31_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_31_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_31_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_31_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_31_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_31_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_31_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_31_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_31_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_31_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_31_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_31_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_31_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_31_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_31_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_31_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_31_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_31_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_31_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_31_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_31_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_31_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_31_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_31_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_31_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_31_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_31_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_31_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_31_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_31_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_31_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_31_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_31_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_31_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_31_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_31_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_31_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_31_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_31_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_31_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_31_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_31_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_31_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_31_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_31_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_31_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_31_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_31_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_31_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_31_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_31_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_31_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_31_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_31_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_31_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_31_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_31_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_31_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_31_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_31_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_31_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_31_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_31_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_31_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_31_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_31_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_31_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_31_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_31_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_31_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_31_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_31_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_31_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_31_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_31_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_31_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_31_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_31_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_31_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_31_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_31_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_31_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_31_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_31_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_31_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_31_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_31_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_31_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_31_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_31_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_31_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_31_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_31_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_31_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_31_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_31_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_31_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_31_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_31_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_31_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_31_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_31_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_31_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_31_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_31_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_31_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_31_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_31_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_31_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_31_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_31_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_31_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_31_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_31_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_31_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_31_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_31_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_31_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_31_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_31_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_31_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_31_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_31_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_31_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_31_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_31_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_31_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_31_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_31_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_31_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_31_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_31_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_31_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_31_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_31_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_31_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_31_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_31_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_31_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_31_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_31_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_31_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_31_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_31_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_31_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_31_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_31_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_31_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_31_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_31_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_31_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_31_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_31_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_31_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_31_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_31_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_31_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_31_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_31_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_31_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_31_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_32_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_32_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_32_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_32_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_32_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_32_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_32_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_32_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_32_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_32_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_32_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_32_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_32_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_32_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_32_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_32_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_32_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_32_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_32_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_32_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_32_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_32_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_32_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_32_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_32_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_32_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_32_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_32_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_32_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_32_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_32_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_32_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_32_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_32_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_32_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_32_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_32_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_32_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_32_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_32_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_32_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_32_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_32_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_32_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_32_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_32_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_32_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_32_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_32_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_32_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_32_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_32_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_32_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_32_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_32_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_32_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_32_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_32_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_32_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_32_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_32_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_32_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_32_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_32_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_32_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_32_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_32_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_32_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_32_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_32_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_32_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_32_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_32_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_32_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_32_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_32_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_32_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_32_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_32_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_32_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_32_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_32_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_32_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_32_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_32_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_32_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_32_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_32_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_32_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_32_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_32_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_32_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_32_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_32_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_32_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_32_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_32_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_32_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_32_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_32_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_32_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_32_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_32_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_32_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_32_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_32_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_32_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_32_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_32_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_32_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_32_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_32_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_32_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_32_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_32_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_32_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_32_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_32_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_32_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_32_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_32_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_32_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_32_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_32_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_32_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_32_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_32_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_32_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_32_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_32_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_32_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_32_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_32_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_32_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_32_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_32_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_32_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_32_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_32_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_32_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_32_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_32_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_32_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_32_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_32_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_32_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_32_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_32_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_32_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_32_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_32_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_32_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_32_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_32_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_32_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_32_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_32_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_32_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_32_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_32_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_32_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_32_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_32_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_32_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_32_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_32_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_32_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_32_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_32_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_32_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_32_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_32_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_32_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_32_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_32_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_32_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_32_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_32_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_32_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_32_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_32_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_32_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_32_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_32_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_32_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_32_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_32_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_32_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_32_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_32_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_32_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_32_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_32_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_32_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_32_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_32_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_32_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_32_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_32_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_32_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_32_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_32_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_32_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_32_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_32_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_32_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_32_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_32_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_32_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_32_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_32_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_32_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_32_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_32_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_32_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_32_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_32_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_32_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_32_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_32_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_32_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_32_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_32_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_32_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_32_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_32_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_32_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_32_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_32_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_32_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_32_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_32_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_32_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_32_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_32_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_32_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_32_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_32_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_32_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_32_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_32_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_32_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_32_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_32_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_32_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_32_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_32_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_32_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_32_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_32_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_32_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_32_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_32_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_32_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_32_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_32_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_32_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_32_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_32_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_32_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_32_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_32_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_32_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_32_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_32_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_32_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_32_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_32_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_32_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_32_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_32_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_32_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_32_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_32_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_32_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_32_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_32_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_32_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_32_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_32_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_32_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_32_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_32_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_32_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_32_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_32_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_32_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_32_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_32_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_32_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_32_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_32_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_32_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_32_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_32_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_32_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_32_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_32_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_32_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_32_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_32_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_32_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_32_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_32_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_32_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_32_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_32_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_32_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_32_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_32_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_32_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_32_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_32_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_32_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_32_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_32_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_32_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_32_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_32_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_32_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_32_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_32_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_32_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_32_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_32_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_32_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_32_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_32_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_32_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_32_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_32_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_32_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_32_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_32_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_32_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_32_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_32_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_32_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_32_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_32_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_32_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_32_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_33_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_33_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_33_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_33_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_33_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_33_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_33_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_33_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_33_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_33_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_33_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_33_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_33_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_33_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_33_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_33_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_33_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_33_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_33_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_33_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_33_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_33_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_33_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_33_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_33_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_33_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_33_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_33_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_33_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_33_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_33_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_33_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_33_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_33_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_33_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_33_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_33_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_33_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_33_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_33_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_33_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_33_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_33_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_33_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_33_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_33_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_33_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_33_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_33_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_33_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_33_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_33_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_33_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_33_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_33_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_33_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_33_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_33_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_33_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_33_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_33_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_33_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_33_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_33_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_33_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_33_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_33_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_33_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_33_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_33_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_33_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_33_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_33_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_33_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_33_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_33_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_33_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_33_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_33_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_33_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_33_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_33_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_33_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_33_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_33_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_33_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_33_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_33_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_33_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_33_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_33_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_33_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_33_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_33_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_33_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_33_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_33_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_33_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_33_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_33_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_33_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_33_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_33_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_33_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_33_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_33_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_33_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_33_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_33_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_33_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_33_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_33_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_33_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_33_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_33_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_33_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_33_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_33_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_33_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_33_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_33_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_33_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_33_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_33_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_33_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_33_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_33_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_33_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_33_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_33_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_33_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_33_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_33_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_33_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_33_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_33_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_33_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_33_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_33_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_33_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_33_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_33_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_33_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_33_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_33_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_33_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_33_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_33_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_33_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_33_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_33_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_33_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_33_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_33_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_33_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_33_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_33_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_33_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_33_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_33_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_33_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_33_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_33_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_33_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_33_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_33_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_33_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_33_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_33_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_33_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_33_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_33_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_33_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_33_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_33_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_33_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_33_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_33_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_33_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_33_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_33_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_33_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_33_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_33_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_33_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_33_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_33_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_33_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_33_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_33_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_33_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_33_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_33_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_33_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_33_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_33_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_33_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_33_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_33_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_33_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_33_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_33_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_33_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_33_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_33_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_33_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_33_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_33_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_33_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_33_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_33_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_33_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_33_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_33_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_33_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_33_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_33_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_33_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_33_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_33_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_33_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_33_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_33_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_33_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_33_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_33_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_33_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_33_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_33_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_33_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_33_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_33_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_33_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_33_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_33_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_33_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_33_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_33_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_33_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_33_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_33_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_33_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_33_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_33_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_33_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_33_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_33_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_33_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_33_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_33_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_33_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_33_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_33_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_33_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_33_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_33_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_33_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_33_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_33_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_33_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_33_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_33_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_33_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_33_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_33_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_33_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_33_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_33_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_33_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_33_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_33_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_33_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_33_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_33_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_33_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_33_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_33_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_33_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_33_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_33_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_33_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_33_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_33_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_33_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_33_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_33_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_33_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_33_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_33_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_33_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_33_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_33_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_33_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_33_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_33_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_33_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_33_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_33_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_33_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_33_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_33_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_33_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_33_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_33_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_33_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_33_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_33_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_33_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_33_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_33_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_33_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_33_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_33_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_33_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_33_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_33_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_33_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_33_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_33_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_33_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_33_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_33_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_33_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_33_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_33_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_33_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_33_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_33_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_33_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_33_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_33_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_33_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_33_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_33_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_33_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_33_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_33_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_33_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_33_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_33_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_33_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_33_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_34_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_34_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_34_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_34_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_34_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_34_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_34_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_34_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_34_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_34_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_34_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_34_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_34_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_34_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_34_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_34_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_34_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_34_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_34_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_34_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_34_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_34_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_34_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_34_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_34_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_34_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_34_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_34_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_34_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_34_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_34_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_34_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_34_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_34_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_34_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_34_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_34_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_34_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_34_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_34_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_34_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_34_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_34_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_34_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_34_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_34_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_34_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_34_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_34_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_34_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_34_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_34_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_34_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_34_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_34_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_34_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_34_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_34_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_34_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_34_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_34_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_34_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_34_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_34_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_34_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_34_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_34_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_34_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_34_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_34_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_34_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_34_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_34_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_34_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_34_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_34_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_34_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_34_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_34_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_34_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_34_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_34_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_34_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_34_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_34_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_34_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_34_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_34_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_34_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_34_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_34_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_34_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_34_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_34_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_34_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_34_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_34_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_34_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_34_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_34_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_34_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_34_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_34_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_34_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_34_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_34_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_34_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_34_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_34_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_34_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_34_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_34_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_34_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_34_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_34_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_34_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_34_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_34_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_34_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_34_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_34_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_34_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_34_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_34_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_34_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_34_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_34_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_34_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_34_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_34_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_34_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_34_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_34_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_34_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_34_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_34_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_34_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_34_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_34_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_34_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_34_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_34_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_34_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_34_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_34_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_34_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_34_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_34_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_34_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_34_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_34_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_34_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_34_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_34_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_34_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_34_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_34_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_34_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_34_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_34_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_34_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_34_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_34_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_34_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_34_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_34_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_34_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_34_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_34_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_34_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_34_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_34_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_34_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_34_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_34_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_34_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_34_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_34_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_34_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_34_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_34_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_34_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_34_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_34_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_34_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_34_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_34_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_34_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_34_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_34_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_34_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_34_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_34_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_34_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_34_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_34_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_34_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_34_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_34_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_34_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_34_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_34_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_34_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_34_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_34_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_34_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_34_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_34_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_34_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_34_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_34_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_34_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_34_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_34_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_34_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_34_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_34_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_34_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_34_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_34_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_34_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_34_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_34_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_34_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_34_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_34_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_34_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_34_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_34_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_34_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_34_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_34_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_34_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_34_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_34_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_34_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_34_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_34_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_34_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_34_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_34_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_34_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_34_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_34_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_34_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_34_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_34_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_34_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_34_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_34_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_34_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_34_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_34_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_34_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_34_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_34_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_34_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_34_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_34_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_34_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_34_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_34_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_34_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_34_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_34_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_34_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_34_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_34_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_34_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_34_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_34_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_34_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_34_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_34_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_34_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_34_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_34_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_34_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_34_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_34_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_34_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_34_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_34_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_34_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_34_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_34_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_34_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_34_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_34_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_34_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_34_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_34_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_34_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_34_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_34_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_34_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_34_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_34_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_34_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_34_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_34_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_34_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_34_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_34_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_34_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_34_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_34_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_34_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_34_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_34_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_34_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_34_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_34_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_34_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_34_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_34_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_34_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_34_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_34_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_34_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_34_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_34_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_34_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_34_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_34_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_34_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_34_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_34_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_34_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_34_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_34_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_34_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_34_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_34_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_34_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_34_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_34_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_34_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_34_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_34_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_34_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_34_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_35_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_35_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_35_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_35_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_35_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_35_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_35_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_35_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_35_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_35_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_35_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_35_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_35_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_35_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_35_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_35_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_35_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_35_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_35_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_35_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_35_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_35_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_35_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_35_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_35_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_35_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_35_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_35_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_35_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_35_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_35_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_35_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_35_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_35_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_35_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_35_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_35_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_35_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_35_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_35_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_35_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_35_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_35_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_35_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_35_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_35_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_35_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_35_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_35_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_35_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_35_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_35_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_35_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_35_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_35_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_35_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_35_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_35_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_35_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_35_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_35_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_35_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_35_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_35_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_35_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_35_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_35_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_35_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_35_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_35_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_35_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_35_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_35_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_35_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_35_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_35_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_35_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_35_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_35_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_35_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_35_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_35_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_35_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_35_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_35_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_35_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_35_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_35_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_35_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_35_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_35_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_35_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_35_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_35_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_35_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_35_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_35_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_35_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_35_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_35_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_35_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_35_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_35_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_35_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_35_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_35_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_35_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_35_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_35_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_35_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_35_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_35_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_35_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_35_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_35_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_35_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_35_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_35_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_35_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_35_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_35_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_35_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_35_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_35_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_35_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_35_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_35_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_35_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_35_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_35_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_35_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_35_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_35_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_35_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_35_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_35_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_35_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_35_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_35_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_35_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_35_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_35_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_35_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_35_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_35_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_35_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_35_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_35_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_35_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_35_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_35_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_35_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_35_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_35_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_35_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_35_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_35_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_35_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_35_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_35_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_35_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_35_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_35_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_35_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_35_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_35_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_35_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_35_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_35_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_35_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_35_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_35_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_35_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_35_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_35_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_35_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_35_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_35_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_35_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_35_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_35_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_35_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_35_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_35_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_35_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_35_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_35_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_35_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_35_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_35_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_35_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_35_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_35_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_35_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_35_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_35_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_35_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_35_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_35_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_35_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_35_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_35_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_35_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_35_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_35_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_35_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_35_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_35_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_35_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_35_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_35_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_35_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_35_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_35_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_35_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_35_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_35_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_35_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_35_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_35_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_35_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_35_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_35_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_35_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_35_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_35_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_35_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_35_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_35_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_35_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_35_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_35_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_35_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_35_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_35_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_35_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_35_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_35_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_35_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_35_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_35_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_35_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_35_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_35_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_35_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_35_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_35_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_35_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_35_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_35_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_35_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_35_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_35_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_35_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_35_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_35_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_35_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_35_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_35_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_35_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_35_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_35_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_35_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_35_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_35_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_35_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_35_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_35_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_35_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_35_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_35_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_35_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_35_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_35_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_35_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_35_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_35_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_35_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_35_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_35_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_35_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_35_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_35_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_35_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_35_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_35_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_35_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_35_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_35_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_35_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_35_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_35_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_35_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_35_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_35_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_35_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_35_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_35_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_35_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_35_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_35_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_35_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_35_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_35_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_35_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_35_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_35_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_35_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_35_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_35_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_35_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_35_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_35_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_35_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_35_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_35_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_35_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_35_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_35_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_35_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_35_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_35_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_35_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_35_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_35_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_35_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_35_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_35_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_35_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_35_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_35_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_35_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_35_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_35_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_35_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_35_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_35_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_35_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_35_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_35_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_35_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_35_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_36_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_36_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_36_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_36_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_36_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_36_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_36_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_36_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_36_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_36_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_36_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_36_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_36_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_36_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_36_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_36_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_36_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_36_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_36_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_36_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_36_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_36_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_36_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_36_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_36_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_36_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_36_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_36_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_36_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_36_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_36_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_36_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_36_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_36_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_36_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_36_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_36_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_36_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_36_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_36_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_36_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_36_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_36_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_36_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_36_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_36_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_36_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_36_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_36_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_36_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_36_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_36_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_36_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_36_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_36_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_36_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_36_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_36_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_36_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_36_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_36_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_36_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_36_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_36_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_36_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_36_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_36_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_36_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_36_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_36_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_36_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_36_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_36_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_36_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_36_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_36_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_36_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_36_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_36_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_36_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_36_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_36_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_36_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_36_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_36_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_36_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_36_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_36_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_36_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_36_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_36_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_36_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_36_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_36_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_36_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_36_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_36_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_36_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_36_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_36_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_36_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_36_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_36_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_36_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_36_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_36_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_36_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_36_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_36_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_36_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_36_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_36_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_36_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_36_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_36_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_36_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_36_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_36_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_36_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_36_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_36_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_36_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_36_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_36_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_36_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_36_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_36_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_36_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_36_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_36_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_36_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_36_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_36_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_36_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_36_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_36_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_36_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_36_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_36_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_36_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_36_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_36_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_36_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_36_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_36_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_36_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_36_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_36_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_36_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_36_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_36_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_36_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_36_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_36_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_36_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_36_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_36_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_36_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_36_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_36_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_36_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_36_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_36_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_36_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_36_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_36_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_36_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_36_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_36_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_36_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_36_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_36_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_36_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_36_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_36_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_36_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_36_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_36_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_36_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_36_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_36_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_36_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_36_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_36_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_36_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_36_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_36_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_36_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_36_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_36_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_36_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_36_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_36_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_36_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_36_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_36_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_36_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_36_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_36_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_36_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_36_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_36_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_36_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_36_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_36_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_36_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_36_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_36_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_36_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_36_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_36_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_36_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_36_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_36_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_36_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_36_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_36_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_36_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_36_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_36_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_36_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_36_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_36_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_36_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_36_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_36_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_36_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_36_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_36_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_36_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_36_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_36_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_36_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_36_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_36_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_36_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_36_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_36_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_36_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_36_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_36_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_36_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_36_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_36_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_36_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_36_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_36_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_36_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_36_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_36_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_36_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_36_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_36_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_36_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_36_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_36_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_36_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_36_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_36_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_36_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_36_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_36_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_36_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_36_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_36_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_36_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_36_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_36_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_36_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_36_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_36_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_36_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_36_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_36_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_36_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_36_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_36_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_36_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_36_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_36_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_36_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_36_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_36_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_36_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_36_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_36_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_36_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_36_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_36_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_36_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_36_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_36_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_36_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_36_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_36_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_36_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_36_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_36_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_36_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_36_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_36_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_36_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_36_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_36_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_36_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_36_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_36_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_36_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_36_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_36_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_36_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_36_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_36_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_36_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_36_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_36_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_36_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_36_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_36_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_36_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_36_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_36_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_36_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_36_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_36_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_36_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_36_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_36_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_36_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_36_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_36_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_36_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_36_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_36_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_36_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_36_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_36_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_36_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_36_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_36_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_36_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_36_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_37_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_37_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_37_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_37_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_37_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_37_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_37_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_37_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_37_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_37_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_37_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_37_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_37_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_37_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_37_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_37_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_37_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_37_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_37_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_37_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_37_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_37_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_37_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_37_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_37_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_37_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_37_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_37_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_37_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_37_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_37_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_37_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_37_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_37_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_37_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_37_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_37_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_37_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_37_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_37_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_37_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_37_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_37_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_37_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_37_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_37_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_37_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_37_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_37_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_37_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_37_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_37_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_37_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_37_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_37_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_37_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_37_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_37_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_37_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_37_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_37_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_37_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_37_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_37_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_37_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_37_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_37_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_37_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_37_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_37_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_37_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_37_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_37_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_37_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_37_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_37_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_37_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_37_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_37_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_37_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_37_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_37_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_37_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_37_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_37_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_37_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_37_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_37_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_37_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_37_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_37_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_37_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_37_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_37_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_37_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_37_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_37_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_37_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_37_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_37_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_37_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_37_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_37_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_37_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_37_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_37_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_37_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_37_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_37_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_37_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_37_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_37_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_37_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_37_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_37_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_37_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_37_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_37_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_37_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_37_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_37_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_37_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_37_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_37_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_37_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_37_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_37_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_37_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_37_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_37_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_37_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_37_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_37_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_37_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_37_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_37_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_37_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_37_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_37_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_37_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_37_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_37_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_37_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_37_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_37_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_37_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_37_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_37_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_37_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_37_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_37_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_37_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_37_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_37_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_37_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_37_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_37_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_37_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_37_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_37_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_37_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_37_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_37_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_37_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_37_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_37_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_37_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_37_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_37_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_37_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_37_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_37_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_37_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_37_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_37_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_37_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_37_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_37_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_37_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_37_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_37_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_37_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_37_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_37_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_37_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_37_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_37_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_37_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_37_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_37_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_37_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_37_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_37_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_37_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_37_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_37_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_37_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_37_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_37_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_37_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_37_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_37_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_37_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_37_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_37_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_37_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_37_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_37_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_37_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_37_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_37_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_37_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_37_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_37_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_37_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_37_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_37_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_37_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_37_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_37_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_37_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_37_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_37_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_37_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_37_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_37_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_37_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_37_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_37_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_37_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_37_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_37_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_37_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_37_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_37_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_37_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_37_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_37_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_37_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_37_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_37_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_37_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_37_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_37_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_37_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_37_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_37_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_37_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_37_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_37_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_37_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_37_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_37_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_37_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_37_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_37_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_37_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_37_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_37_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_37_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_37_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_37_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_37_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_37_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_37_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_37_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_37_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_37_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_37_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_37_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_37_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_37_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_37_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_37_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_37_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_37_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_37_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_37_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_37_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_37_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_37_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_37_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_37_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_37_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_37_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_37_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_37_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_37_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_37_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_37_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_37_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_37_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_37_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_37_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_37_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_37_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_37_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_37_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_37_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_37_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_37_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_37_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_37_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_37_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_37_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_37_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_37_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_37_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_37_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_37_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_37_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_37_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_37_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_37_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_37_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_37_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_37_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_37_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_37_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_37_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_37_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_37_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_37_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_37_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_37_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_37_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_37_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_37_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_37_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_37_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_37_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_37_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_37_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_37_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_37_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_37_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_37_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_37_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_37_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_37_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_37_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_37_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_38_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_38_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_38_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_38_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_38_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_38_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_38_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_38_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_38_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_38_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_38_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_38_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_38_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_38_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_38_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_38_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_38_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_38_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_38_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_38_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_38_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_38_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_38_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_38_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_38_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_38_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_38_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_38_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_38_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_38_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_38_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_38_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_38_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_38_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_38_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_38_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_38_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_38_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_38_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_38_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_38_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_38_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_38_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_38_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_38_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_38_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_38_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_38_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_38_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_38_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_38_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_38_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_38_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_38_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_38_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_38_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_38_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_38_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_38_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_38_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_38_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_38_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_38_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_38_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_38_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_38_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_38_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_38_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_38_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_38_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_38_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_38_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_38_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_38_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_38_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_38_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_38_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_38_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_38_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_38_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_38_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_38_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_38_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_38_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_38_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_38_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_38_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_38_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_38_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_38_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_38_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_38_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_38_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_38_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_38_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_38_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_38_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_38_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_38_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_38_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_38_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_38_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_38_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_38_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_38_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_38_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_38_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_38_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_38_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_38_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_38_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_38_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_38_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_38_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_38_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_38_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_38_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_38_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_38_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_38_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_38_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_38_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_38_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_38_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_38_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_38_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_38_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_38_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_38_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_38_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_38_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_38_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_38_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_38_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_38_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_38_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_38_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_38_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_38_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_38_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_38_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_38_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_38_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_38_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_38_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_38_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_38_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_38_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_38_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_38_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_38_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_38_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_38_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_38_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_38_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_38_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_38_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_38_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_38_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_38_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_38_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_38_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_38_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_38_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_38_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_38_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_38_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_38_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_38_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_38_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_38_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_38_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_38_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_38_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_38_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_38_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_38_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_38_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_38_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_38_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_38_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_38_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_38_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_38_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_38_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_38_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_38_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_38_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_38_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_38_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_38_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_38_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_38_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_38_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_38_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_38_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_38_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_38_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_38_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_38_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_38_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_38_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_38_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_38_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_38_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_38_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_38_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_38_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_38_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_38_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_38_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_38_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_38_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_38_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_38_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_38_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_38_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_38_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_38_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_38_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_38_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_38_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_38_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_38_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_38_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_38_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_38_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_38_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_38_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_38_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_38_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_38_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_38_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_38_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_38_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_38_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_38_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_38_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_38_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_38_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_38_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_38_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_38_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_38_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_38_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_38_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_38_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_38_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_38_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_38_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_38_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_38_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_38_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_38_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_38_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_38_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_38_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_38_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_38_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_38_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_38_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_38_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_38_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_38_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_38_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_38_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_38_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_38_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_38_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_38_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_38_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_38_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_38_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_38_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_38_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_38_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_38_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_38_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_38_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_38_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_38_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_38_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_38_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_38_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_38_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_38_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_38_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_38_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_38_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_38_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_38_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_38_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_38_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_38_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_38_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_38_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_38_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_38_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_38_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_38_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_38_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_38_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_38_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_38_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_38_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_38_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_38_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_38_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_38_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_38_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_38_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_38_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_38_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_38_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_38_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_38_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_38_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_38_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_38_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_38_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_38_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_38_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_38_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_38_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_38_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_38_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_38_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_38_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_38_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_38_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_38_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_38_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_38_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_38_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_38_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_38_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_38_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_38_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_38_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_38_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_38_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_38_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_39_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_39_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_39_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_39_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_39_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_39_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_39_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_39_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_39_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_39_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_39_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_39_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_39_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_39_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_39_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_39_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_39_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_39_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_39_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_39_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_39_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_39_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_39_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_39_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_39_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_39_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_39_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_39_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_39_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_39_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_39_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_39_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_39_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_39_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_39_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_39_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_39_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_39_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_39_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_39_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_39_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_39_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_39_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_39_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_39_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_39_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_39_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_39_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_39_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_39_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_39_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_39_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_39_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_39_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_39_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_39_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_39_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_39_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_39_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_39_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_39_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_39_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_39_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_39_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_39_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_39_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_39_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_39_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_39_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_39_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_39_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_39_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_39_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_39_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_39_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_39_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_39_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_39_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_39_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_39_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_39_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_39_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_39_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_39_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_39_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_39_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_39_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_39_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_39_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_39_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_39_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_39_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_39_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_39_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_39_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_39_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_39_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_39_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_39_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_39_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_39_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_39_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_39_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_39_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_39_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_39_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_39_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_39_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_39_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_39_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_39_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_39_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_39_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_39_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_39_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_39_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_39_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_39_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_39_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_39_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_39_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_39_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_39_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_39_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_39_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_39_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_39_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_39_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_39_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_39_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_39_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_39_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_39_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_39_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_39_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_39_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_39_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_39_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_39_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_39_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_39_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_39_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_39_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_39_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_39_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_39_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_39_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_39_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_39_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_39_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_39_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_39_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_39_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_39_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_39_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_39_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_39_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_39_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_39_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_39_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_39_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_39_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_39_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_39_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_39_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_39_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_39_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_39_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_39_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_39_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_39_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_39_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_39_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_39_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_39_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_39_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_39_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_39_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_39_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_39_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_39_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_39_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_39_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_39_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_39_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_39_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_39_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_39_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_39_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_39_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_39_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_39_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_39_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_39_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_39_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_39_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_39_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_39_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_39_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_39_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_39_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_39_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_39_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_39_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_39_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_39_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_39_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_39_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_39_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_39_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_39_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_39_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_39_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_39_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_39_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_39_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_39_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_39_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_39_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_39_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_39_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_39_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_39_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_39_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_39_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_39_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_39_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_39_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_39_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_39_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_39_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_39_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_39_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_39_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_39_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_39_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_39_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_39_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_39_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_39_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_39_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_39_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_39_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_39_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_39_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_39_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_39_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_39_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_39_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_39_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_39_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_39_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_39_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_39_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_39_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_39_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_39_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_39_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_39_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_39_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_39_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_39_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_39_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_39_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_39_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_39_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_39_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_39_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_39_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_39_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_39_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_39_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_39_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_39_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_39_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_39_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_39_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_39_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_39_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_39_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_39_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_39_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_39_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_39_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_39_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_39_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_39_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_39_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_39_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_39_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_39_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_39_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_39_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_39_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_39_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_39_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_39_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_39_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_39_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_39_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_39_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_39_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_39_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_39_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_39_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_39_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_39_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_39_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_39_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_39_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_39_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_39_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_39_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_39_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_39_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_39_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_39_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_39_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_39_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_39_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_39_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_39_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_39_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_39_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_39_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_39_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_39_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_39_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_39_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_39_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_39_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_39_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_39_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_39_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_39_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_39_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_39_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_39_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_39_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_39_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_39_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_39_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_40_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_40_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_40_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_40_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_40_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_40_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_40_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_40_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_40_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_40_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_40_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_40_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_40_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_40_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_40_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_40_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_40_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_40_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_40_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_40_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_40_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_40_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_40_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_40_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_40_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_40_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_40_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_40_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_40_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_40_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_40_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_40_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_40_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_40_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_40_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_40_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_40_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_40_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_40_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_40_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_40_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_40_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_40_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_40_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_40_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_40_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_40_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_40_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_40_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_40_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_40_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_40_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_40_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_40_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_40_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_40_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_40_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_40_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_40_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_40_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_40_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_40_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_40_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_40_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_40_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_40_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_40_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_40_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_40_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_40_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_40_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_40_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_40_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_40_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_40_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_40_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_40_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_40_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_40_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_40_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_40_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_40_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_40_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_40_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_40_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_40_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_40_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_40_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_40_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_40_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_40_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_40_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_40_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_40_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_40_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_40_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_40_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_40_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_40_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_40_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_40_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_40_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_40_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_40_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_40_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_40_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_40_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_40_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_40_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_40_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_40_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_40_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_40_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_40_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_40_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_40_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_40_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_40_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_40_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_40_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_40_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_40_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_40_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_40_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_40_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_40_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_40_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_40_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_40_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_40_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_40_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_40_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_40_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_40_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_40_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_40_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_40_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_40_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_40_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_40_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_40_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_40_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_40_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_40_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_40_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_40_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_40_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_40_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_40_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_40_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_40_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_40_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_40_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_40_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_40_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_40_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_40_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_40_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_40_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_40_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_40_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_40_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_40_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_40_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_40_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_40_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_40_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_40_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_40_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_40_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_40_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_40_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_40_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_40_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_40_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_40_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_40_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_40_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_40_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_40_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_40_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_40_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_40_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_40_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_40_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_40_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_40_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_40_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_40_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_40_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_40_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_40_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_40_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_40_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_40_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_40_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_40_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_40_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_40_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_40_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_40_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_40_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_40_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_40_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_40_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_40_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_40_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_40_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_40_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_40_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_40_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_40_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_40_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_40_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_40_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_40_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_40_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_40_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_40_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_40_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_40_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_40_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_40_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_40_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_40_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_40_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_40_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_40_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_40_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_40_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_40_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_40_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_40_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_40_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_40_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_40_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_40_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_40_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_40_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_40_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_40_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_40_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_40_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_40_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_40_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_40_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_40_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_40_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_40_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_40_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_40_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_40_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_40_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_40_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_40_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_40_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_40_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_40_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_40_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_40_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_40_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_40_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_40_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_40_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_40_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_40_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_40_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_40_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_40_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_40_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_40_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_40_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_40_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_40_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_40_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_40_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_40_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_40_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_40_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_40_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_40_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_40_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_40_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_40_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_40_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_40_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_40_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_40_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_40_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_40_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_40_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_40_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_40_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_40_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_40_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_40_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_40_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_40_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_40_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_40_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_40_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_40_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_40_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_40_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_40_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_40_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_40_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_40_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_40_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_40_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_40_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_40_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_40_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_40_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_40_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_40_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_40_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_40_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_40_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_40_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_40_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_40_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_40_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_40_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_40_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_40_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_40_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_40_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_40_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_40_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_40_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_40_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_40_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_40_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_40_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_40_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_40_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_40_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_40_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_40_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_40_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_40_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_41_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_41_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_41_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_41_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_41_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_41_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_41_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_41_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_41_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_41_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_41_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_41_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_41_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_41_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_41_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_41_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_41_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_41_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_41_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_41_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_41_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_41_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_41_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_41_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_41_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_41_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_41_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_41_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_41_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_41_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_41_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_41_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_41_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_41_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_41_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_41_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_41_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_41_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_41_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_41_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_41_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_41_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_41_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_41_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_41_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_41_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_41_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_41_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_41_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_41_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_41_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_41_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_41_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_41_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_41_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_41_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_41_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_41_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_41_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_41_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_41_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_41_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_41_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_41_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_41_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_41_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_41_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_41_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_41_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_41_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_41_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_41_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_41_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_41_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_41_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_41_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_41_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_41_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_41_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_41_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_41_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_41_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_41_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_41_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_41_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_41_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_41_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_41_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_41_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_41_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_41_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_41_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_41_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_41_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_41_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_41_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_41_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_41_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_41_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_41_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_41_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_41_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_41_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_41_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_41_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_41_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_41_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_41_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_41_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_41_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_41_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_41_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_41_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_41_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_41_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_41_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_41_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_41_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_41_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_41_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_41_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_41_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_41_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_41_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_41_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_41_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_41_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_41_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_41_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_41_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_41_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_41_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_41_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_41_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_41_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_41_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_41_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_41_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_41_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_41_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_41_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_41_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_41_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_41_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_41_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_41_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_41_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_41_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_41_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_41_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_41_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_41_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_41_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_41_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_41_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_41_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_41_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_41_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_41_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_41_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_41_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_41_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_41_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_41_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_41_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_41_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_41_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_41_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_41_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_41_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_41_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_41_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_41_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_41_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_41_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_41_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_41_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_41_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_41_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_41_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_41_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_41_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_41_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_41_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_41_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_41_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_41_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_41_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_41_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_41_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_41_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_41_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_41_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_41_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_41_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_41_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_41_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_41_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_41_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_41_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_41_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_41_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_41_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_41_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_41_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_41_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_41_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_41_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_41_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_41_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_41_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_41_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_41_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_41_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_41_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_41_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_41_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_41_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_41_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_41_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_41_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_41_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_41_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_41_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_41_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_41_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_41_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_41_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_41_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_41_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_41_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_41_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_41_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_41_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_41_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_41_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_41_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_41_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_41_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_41_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_41_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_41_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_41_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_41_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_41_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_41_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_41_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_41_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_41_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_41_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_41_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_41_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_41_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_41_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_41_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_41_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_41_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_41_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_41_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_41_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_41_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_41_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_41_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_41_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_41_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_41_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_41_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_41_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_41_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_41_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_41_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_41_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_41_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_41_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_41_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_41_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_41_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_41_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_41_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_41_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_41_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_41_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_41_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_41_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_41_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_41_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_41_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_41_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_41_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_41_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_41_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_41_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_41_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_41_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_41_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_41_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_41_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_41_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_41_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_41_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_41_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_41_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_41_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_41_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_41_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_41_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_41_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_41_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_41_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_41_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_41_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_41_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_41_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_41_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_41_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_41_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_41_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_41_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_41_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_41_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_41_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_41_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_41_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_41_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_41_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_41_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_41_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_41_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_41_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_41_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_41_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_41_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_41_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_41_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_41_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_41_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_41_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_41_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_41_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_41_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_41_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_41_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_42_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_42_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_42_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_42_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_42_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_42_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_42_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_42_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_42_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_42_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_42_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_42_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_42_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_42_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_42_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_42_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_42_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_42_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_42_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_42_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_42_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_42_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_42_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_42_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_42_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_42_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_42_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_42_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_42_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_42_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_42_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_42_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_42_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_42_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_42_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_42_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_42_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_42_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_42_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_42_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_42_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_42_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_42_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_42_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_42_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_42_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_42_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_42_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_42_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_42_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_42_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_42_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_42_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_42_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_42_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_42_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_42_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_42_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_42_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_42_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_42_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_42_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_42_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_42_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_42_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_42_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_42_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_42_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_42_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_42_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_42_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_42_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_42_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_42_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_42_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_42_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_42_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_42_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_42_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_42_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_42_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_42_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_42_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_42_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_42_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_42_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_42_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_42_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_42_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_42_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_42_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_42_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_42_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_42_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_42_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_42_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_42_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_42_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_42_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_42_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_42_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_42_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_42_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_42_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_42_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_42_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_42_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_42_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_42_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_42_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_42_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_42_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_42_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_42_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_42_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_42_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_42_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_42_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_42_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_42_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_42_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_42_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_42_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_42_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_42_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_42_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_42_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_42_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_42_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_42_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_42_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_42_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_42_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_42_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_42_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_42_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_42_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_42_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_42_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_42_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_42_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_42_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_42_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_42_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_42_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_42_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_42_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_42_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_42_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_42_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_42_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_42_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_42_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_42_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_42_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_42_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_42_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_42_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_42_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_42_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_42_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_42_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_42_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_42_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_42_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_42_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_42_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_42_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_42_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_42_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_42_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_42_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_42_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_42_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_42_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_42_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_42_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_42_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_42_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_42_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_42_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_42_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_42_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_42_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_42_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_42_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_42_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_42_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_42_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_42_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_42_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_42_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_42_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_42_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_42_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_42_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_42_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_42_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_42_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_42_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_42_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_42_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_42_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_42_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_42_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_42_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_42_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_42_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_42_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_42_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_42_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_42_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_42_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_42_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_42_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_42_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_42_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_42_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_42_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_42_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_42_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_42_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_42_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_42_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_42_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_42_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_42_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_42_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_42_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_42_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_42_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_42_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_42_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_42_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_42_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_42_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_42_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_42_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_42_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_42_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_42_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_42_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_42_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_42_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_42_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_42_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_42_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_42_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_42_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_42_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_42_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_42_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_42_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_42_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_42_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_42_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_42_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_42_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_42_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_42_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_42_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_42_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_42_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_42_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_42_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_42_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_42_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_42_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_42_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_42_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_42_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_42_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_42_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_42_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_42_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_42_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_42_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_42_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_42_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_42_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_42_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_42_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_42_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_42_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_42_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_42_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_42_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_42_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_42_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_42_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_42_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_42_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_42_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_42_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_42_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_42_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_42_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_42_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_42_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_42_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_42_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_42_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_42_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_42_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_42_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_42_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_42_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_42_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_42_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_42_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_42_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_42_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_42_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_42_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_42_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_42_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_42_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_42_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_42_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_42_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_42_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_42_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_42_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_42_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_42_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_42_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_42_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_42_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_42_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_42_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_42_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_42_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_42_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_42_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_42_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_42_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_42_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_42_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_42_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_42_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_42_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_42_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_43_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_43_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_43_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_43_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_43_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_43_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_43_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_43_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_43_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_43_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_43_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_43_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_43_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_43_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_43_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_43_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_43_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_43_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_43_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_43_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_43_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_43_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_43_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_43_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_43_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_43_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_43_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_43_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_43_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_43_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_43_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_43_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_43_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_43_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_43_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_43_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_43_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_43_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_43_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_43_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_43_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_43_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_43_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_43_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_43_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_43_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_43_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_43_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_43_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_43_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_43_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_43_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_43_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_43_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_43_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_43_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_43_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_43_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_43_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_43_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_43_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_43_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_43_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_43_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_43_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_43_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_43_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_43_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_43_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_43_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_43_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_43_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_43_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_43_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_43_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_43_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_43_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_43_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_43_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_43_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_43_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_43_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_43_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_43_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_43_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_43_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_43_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_43_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_43_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_43_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_43_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_43_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_43_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_43_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_43_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_43_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_43_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_43_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_43_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_43_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_43_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_43_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_43_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_43_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_43_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_43_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_43_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_43_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_43_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_43_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_43_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_43_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_43_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_43_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_43_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_43_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_43_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_43_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_43_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_43_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_43_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_43_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_43_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_43_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_43_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_43_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_43_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_43_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_43_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_43_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_43_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_43_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_43_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_43_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_43_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_43_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_43_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_43_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_43_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_43_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_43_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_43_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_43_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_43_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_43_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_43_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_43_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_43_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_43_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_43_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_43_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_43_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_43_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_43_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_43_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_43_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_43_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_43_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_43_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_43_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_43_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_43_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_43_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_43_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_43_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_43_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_43_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_43_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_43_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_43_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_43_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_43_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_43_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_43_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_43_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_43_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_43_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_43_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_43_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_43_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_43_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_43_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_43_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_43_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_43_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_43_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_43_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_43_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_43_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_43_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_43_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_43_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_43_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_43_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_43_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_43_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_43_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_43_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_43_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_43_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_43_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_43_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_43_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_43_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_43_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_43_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_43_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_43_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_43_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_43_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_43_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_43_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_43_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_43_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_43_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_43_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_43_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_43_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_43_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_43_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_43_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_43_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_43_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_43_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_43_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_43_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_43_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_43_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_43_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_43_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_43_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_43_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_43_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_43_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_43_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_43_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_43_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_43_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_43_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_43_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_43_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_43_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_43_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_43_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_43_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_43_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_43_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_43_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_43_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_43_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_43_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_43_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_43_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_43_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_43_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_43_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_43_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_43_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_43_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_43_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_43_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_43_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_43_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_43_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_43_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_43_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_43_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_43_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_43_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_43_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_43_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_43_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_43_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_43_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_43_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_43_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_43_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_43_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_43_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_43_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_43_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_43_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_43_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_43_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_43_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_43_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_43_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_43_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_43_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_43_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_43_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_43_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_43_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_43_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_43_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_43_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_43_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_43_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_43_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_43_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_43_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_43_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_43_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_43_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_43_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_43_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_43_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_43_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_43_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_43_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_43_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_43_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_43_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_43_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_43_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_43_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_43_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_43_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_43_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_43_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_43_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_43_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_43_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_43_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_43_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_43_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_43_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_43_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_43_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_43_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_43_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_43_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_43_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_43_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_43_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_43_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_43_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_43_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_43_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_43_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_43_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_43_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_44_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_44_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_44_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_44_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_44_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_44_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_44_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_44_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_44_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_44_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_44_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_44_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_44_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_44_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_44_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_44_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_44_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_44_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_44_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_44_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_44_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_44_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_44_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_44_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_44_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_44_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_44_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_44_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_44_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_44_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_44_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_44_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_44_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_44_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_44_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_44_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_44_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_44_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_44_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_44_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_44_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_44_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_44_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_44_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_44_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_44_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_44_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_44_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_44_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_44_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_44_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_44_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_44_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_44_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_44_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_44_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_44_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_44_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_44_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_44_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_44_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_44_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_44_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_44_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_44_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_44_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_44_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_44_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_44_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_44_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_44_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_44_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_44_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_44_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_44_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_44_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_44_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_44_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_44_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_44_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_44_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_44_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_44_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_44_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_44_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_44_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_44_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_44_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_44_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_44_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_44_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_44_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_44_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_44_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_44_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_44_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_44_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_44_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_44_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_44_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_44_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_44_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_44_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_44_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_44_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_44_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_44_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_44_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_44_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_44_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_44_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_44_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_44_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_44_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_44_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_44_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_44_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_44_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_44_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_44_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_44_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_44_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_44_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_44_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_44_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_44_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_44_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_44_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_44_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_44_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_44_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_44_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_44_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_44_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_44_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_44_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_44_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_44_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_44_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_44_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_44_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_44_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_44_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_44_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_44_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_44_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_44_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_44_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_44_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_44_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_44_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_44_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_44_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_44_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_44_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_44_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_44_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_44_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_44_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_44_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_44_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_44_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_44_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_44_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_44_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_44_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_44_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_44_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_44_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_44_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_44_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_44_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_44_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_44_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_44_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_44_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_44_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_44_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_44_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_44_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_44_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_44_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_44_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_44_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_44_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_44_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_44_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_44_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_44_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_44_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_44_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_44_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_44_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_44_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_44_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_44_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_44_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_44_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_44_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_44_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_44_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_44_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_44_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_44_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_44_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_44_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_44_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_44_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_44_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_44_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_44_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_44_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_44_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_44_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_44_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_44_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_44_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_44_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_44_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_44_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_44_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_44_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_44_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_44_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_44_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_44_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_44_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_44_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_44_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_44_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_44_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_44_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_44_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_44_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_44_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_44_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_44_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_44_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_44_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_44_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_44_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_44_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_44_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_44_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_44_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_44_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_44_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_44_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_44_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_44_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_44_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_44_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_44_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_44_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_44_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_44_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_44_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_44_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_44_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_44_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_44_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_44_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_44_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_44_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_44_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_44_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_44_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_44_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_44_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_44_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_44_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_44_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_44_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_44_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_44_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_44_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_44_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_44_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_44_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_44_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_44_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_44_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_44_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_44_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_44_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_44_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_44_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_44_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_44_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_44_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_44_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_44_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_44_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_44_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_44_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_44_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_44_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_44_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_44_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_44_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_44_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_44_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_44_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_44_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_44_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_44_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_44_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_44_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_44_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_44_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_44_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_44_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_44_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_44_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_44_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_44_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_44_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_44_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_44_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_44_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_44_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_44_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_44_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_44_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_44_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_44_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_44_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_44_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_44_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_44_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_44_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_44_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_44_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_44_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_44_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_44_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_44_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_44_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_44_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_44_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_44_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_44_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_45_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_45_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_45_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_45_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_45_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_45_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_45_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_45_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_45_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_45_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_45_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_45_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_45_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_45_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_45_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_45_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_45_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_45_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_45_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_45_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_45_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_45_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_45_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_45_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_45_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_45_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_45_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_45_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_45_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_45_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_45_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_45_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_45_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_45_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_45_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_45_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_45_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_45_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_45_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_45_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_45_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_45_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_45_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_45_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_45_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_45_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_45_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_45_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_45_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_45_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_45_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_45_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_45_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_45_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_45_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_45_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_45_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_45_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_45_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_45_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_45_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_45_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_45_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_45_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_45_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_45_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_45_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_45_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_45_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_45_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_45_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_45_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_45_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_45_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_45_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_45_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_45_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_45_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_45_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_45_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_45_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_45_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_45_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_45_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_45_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_45_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_45_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_45_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_45_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_45_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_45_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_45_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_45_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_45_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_45_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_45_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_45_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_45_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_45_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_45_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_45_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_45_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_45_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_45_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_45_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_45_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_45_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_45_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_45_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_45_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_45_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_45_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_45_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_45_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_45_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_45_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_45_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_45_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_45_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_45_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_45_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_45_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_45_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_45_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_45_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_45_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_45_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_45_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_45_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_45_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_45_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_45_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_45_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_45_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_45_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_45_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_45_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_45_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_45_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_45_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_45_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_45_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_45_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_45_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_45_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_45_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_45_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_45_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_45_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_45_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_45_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_45_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_45_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_45_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_45_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_45_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_45_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_45_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_45_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_45_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_45_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_45_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_45_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_45_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_45_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_45_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_45_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_45_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_45_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_45_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_45_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_45_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_45_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_45_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_45_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_45_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_45_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_45_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_45_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_45_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_45_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_45_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_45_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_45_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_45_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_45_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_45_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_45_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_45_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_45_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_45_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_45_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_45_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_45_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_45_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_45_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_45_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_45_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_45_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_45_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_45_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_45_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_45_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_45_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_45_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_45_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_45_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_45_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_45_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_45_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_45_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_45_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_45_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_45_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_45_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_45_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_45_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_45_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_45_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_45_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_45_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_45_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_45_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_45_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_45_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_45_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_45_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_45_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_45_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_45_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_45_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_45_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_45_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_45_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_45_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_45_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_45_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_45_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_45_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_45_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_45_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_45_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_45_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_45_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_45_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_45_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_45_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_45_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_45_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_45_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_45_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_45_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_45_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_45_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_45_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_45_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_45_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_45_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_45_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_45_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_45_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_45_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_45_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_45_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_45_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_45_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_45_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_45_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_45_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_45_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_45_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_45_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_45_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_45_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_45_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_45_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_45_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_45_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_45_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_45_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_45_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_45_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_45_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_45_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_45_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_45_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_45_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_45_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_45_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_45_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_45_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_45_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_45_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_45_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_45_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_45_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_45_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_45_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_45_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_45_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_45_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_45_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_45_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_45_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_45_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_45_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_45_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_45_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_45_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_45_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_45_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_45_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_45_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_45_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_45_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_45_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_45_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_45_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_45_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_45_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_45_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_45_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_45_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_45_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_45_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_45_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_45_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_45_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_45_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_45_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_45_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_45_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_45_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_45_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_45_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_45_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_45_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_45_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_45_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_45_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_45_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_45_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_46_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_46_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_46_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_46_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_46_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_46_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_46_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_46_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_46_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_46_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_46_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_46_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_46_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_46_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_46_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_46_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_46_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_46_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_46_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_46_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_46_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_46_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_46_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_46_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_46_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_46_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_46_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_46_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_46_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_46_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_46_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_46_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_46_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_46_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_46_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_46_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_46_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_46_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_46_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_46_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_46_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_46_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_46_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_46_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_46_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_46_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_46_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_46_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_46_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_46_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_46_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_46_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_46_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_46_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_46_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_46_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_46_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_46_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_46_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_46_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_46_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_46_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_46_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_46_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_46_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_46_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_46_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_46_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_46_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_46_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_46_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_46_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_46_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_46_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_46_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_46_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_46_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_46_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_46_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_46_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_46_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_46_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_46_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_46_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_46_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_46_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_46_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_46_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_46_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_46_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_46_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_46_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_46_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_46_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_46_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_46_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_46_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_46_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_46_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_46_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_46_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_46_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_46_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_46_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_46_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_46_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_46_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_46_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_46_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_46_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_46_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_46_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_46_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_46_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_46_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_46_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_46_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_46_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_46_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_46_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_46_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_46_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_46_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_46_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_46_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_46_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_46_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_46_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_46_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_46_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_46_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_46_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_46_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_46_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_46_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_46_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_46_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_46_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_46_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_46_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_46_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_46_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_46_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_46_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_46_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_46_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_46_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_46_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_46_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_46_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_46_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_46_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_46_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_46_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_46_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_46_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_46_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_46_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_46_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_46_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_46_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_46_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_46_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_46_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_46_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_46_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_46_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_46_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_46_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_46_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_46_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_46_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_46_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_46_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_46_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_46_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_46_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_46_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_46_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_46_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_46_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_46_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_46_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_46_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_46_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_46_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_46_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_46_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_46_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_46_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_46_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_46_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_46_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_46_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_46_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_46_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_46_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_46_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_46_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_46_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_46_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_46_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_46_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_46_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_46_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_46_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_46_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_46_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_46_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_46_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_46_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_46_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_46_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_46_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_46_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_46_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_46_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_46_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_46_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_46_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_46_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_46_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_46_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_46_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_46_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_46_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_46_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_46_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_46_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_46_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_46_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_46_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_46_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_46_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_46_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_46_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_46_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_46_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_46_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_46_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_46_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_46_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_46_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_46_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_46_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_46_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_46_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_46_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_46_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_46_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_46_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_46_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_46_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_46_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_46_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_46_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_46_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_46_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_46_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_46_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_46_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_46_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_46_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_46_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_46_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_46_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_46_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_46_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_46_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_46_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_46_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_46_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_46_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_46_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_46_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_46_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_46_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_46_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_46_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_46_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_46_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_46_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_46_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_46_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_46_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_46_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_46_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_46_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_46_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_46_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_46_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_46_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_46_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_46_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_46_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_46_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_46_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_46_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_46_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_46_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_46_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_46_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_46_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_46_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_46_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_46_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_46_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_46_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_46_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_46_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_46_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_46_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_46_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_46_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_46_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_46_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_46_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_46_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_46_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_46_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_46_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_46_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_46_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_46_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_46_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_46_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_46_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_46_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_46_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_46_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_46_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_46_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_46_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_46_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_46_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_46_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_46_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_46_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_46_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_46_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_46_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_46_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_47_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_47_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_47_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_47_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_47_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_47_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_47_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_47_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_47_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_47_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_47_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_47_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_47_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_47_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_47_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_47_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_47_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_47_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_47_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_47_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_47_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_47_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_47_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_47_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_47_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_47_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_47_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_47_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_47_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_47_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_47_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_47_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_47_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_47_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_47_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_47_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_47_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_47_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_47_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_47_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_47_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_47_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_47_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_47_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_47_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_47_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_47_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_47_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_47_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_47_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_47_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_47_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_47_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_47_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_47_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_47_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_47_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_47_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_47_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_47_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_47_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_47_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_47_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_47_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_47_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_47_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_47_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_47_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_47_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_47_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_47_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_47_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_47_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_47_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_47_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_47_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_47_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_47_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_47_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_47_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_47_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_47_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_47_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_47_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_47_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_47_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_47_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_47_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_47_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_47_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_47_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_47_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_47_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_47_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_47_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_47_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_47_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_47_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_47_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_47_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_47_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_47_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_47_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_47_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_47_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_47_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_47_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_47_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_47_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_47_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_47_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_47_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_47_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_47_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_47_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_47_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_47_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_47_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_47_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_47_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_47_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_47_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_47_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_47_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_47_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_47_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_47_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_47_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_47_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_47_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_47_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_47_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_47_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_47_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_47_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_47_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_47_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_47_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_47_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_47_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_47_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_47_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_47_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_47_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_47_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_47_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_47_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_47_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_47_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_47_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_47_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_47_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_47_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_47_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_47_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_47_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_47_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_47_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_47_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_47_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_47_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_47_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_47_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_47_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_47_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_47_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_47_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_47_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_47_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_47_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_47_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_47_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_47_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_47_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_47_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_47_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_47_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_47_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_47_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_47_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_47_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_47_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_47_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_47_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_47_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_47_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_47_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_47_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_47_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_47_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_47_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_47_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_47_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_47_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_47_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_47_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_47_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_47_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_47_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_47_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_47_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_47_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_47_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_47_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_47_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_47_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_47_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_47_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_47_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_47_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_47_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_47_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_47_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_47_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_47_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_47_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_47_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_47_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_47_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_47_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_47_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_47_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_47_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_47_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_47_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_47_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_47_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_47_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_47_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_47_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_47_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_47_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_47_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_47_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_47_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_47_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_47_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_47_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_47_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_47_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_47_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_47_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_47_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_47_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_47_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_47_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_47_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_47_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_47_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_47_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_47_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_47_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_47_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_47_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_47_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_47_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_47_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_47_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_47_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_47_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_47_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_47_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_47_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_47_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_47_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_47_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_47_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_47_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_47_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_47_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_47_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_47_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_47_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_47_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_47_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_47_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_47_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_47_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_47_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_47_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_47_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_47_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_47_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_47_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_47_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_47_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_47_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_47_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_47_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_47_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_47_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_47_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_47_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_47_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_47_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_47_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_47_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_47_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_47_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_47_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_47_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_47_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_47_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_47_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_47_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_47_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_47_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_47_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_47_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_47_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_47_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_47_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_47_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_47_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_47_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_47_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_47_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_47_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_47_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_47_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_47_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_47_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_47_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_47_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_47_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_47_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_47_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_47_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_47_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_47_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_47_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_47_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_47_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_47_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_47_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_47_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_47_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_47_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_47_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_47_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_47_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_47_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_48_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_48_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_48_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_48_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_48_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_48_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_48_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_48_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_48_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_48_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_48_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_48_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_48_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_48_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_48_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_48_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_48_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_48_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_48_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_48_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_48_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_48_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_48_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_48_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_48_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_48_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_48_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_48_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_48_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_48_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_48_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_48_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_48_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_48_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_48_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_48_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_48_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_48_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_48_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_48_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_48_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_48_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_48_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_48_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_48_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_48_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_48_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_48_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_48_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_48_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_48_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_48_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_48_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_48_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_48_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_48_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_48_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_48_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_48_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_48_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_48_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_48_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_48_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_48_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_48_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_48_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_48_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_48_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_48_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_48_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_48_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_48_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_48_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_48_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_48_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_48_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_48_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_48_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_48_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_48_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_48_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_48_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_48_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_48_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_48_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_48_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_48_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_48_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_48_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_48_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_48_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_48_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_48_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_48_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_48_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_48_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_48_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_48_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_48_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_48_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_48_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_48_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_48_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_48_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_48_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_48_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_48_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_48_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_48_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_48_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_48_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_48_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_48_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_48_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_48_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_48_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_48_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_48_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_48_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_48_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_48_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_48_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_48_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_48_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_48_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_48_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_48_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_48_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_48_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_48_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_48_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_48_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_48_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_48_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_48_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_48_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_48_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_48_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_48_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_48_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_48_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_48_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_48_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_48_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_48_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_48_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_48_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_48_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_48_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_48_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_48_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_48_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_48_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_48_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_48_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_48_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_48_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_48_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_48_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_48_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_48_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_48_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_48_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_48_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_48_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_48_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_48_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_48_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_48_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_48_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_48_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_48_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_48_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_48_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_48_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_48_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_48_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_48_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_48_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_48_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_48_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_48_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_48_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_48_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_48_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_48_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_48_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_48_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_48_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_48_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_48_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_48_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_48_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_48_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_48_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_48_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_48_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_48_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_48_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_48_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_48_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_48_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_48_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_48_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_48_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_48_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_48_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_48_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_48_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_48_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_48_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_48_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_48_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_48_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_48_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_48_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_48_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_48_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_48_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_48_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_48_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_48_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_48_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_48_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_48_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_48_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_48_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_48_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_48_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_48_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_48_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_48_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_48_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_48_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_48_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_48_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_48_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_48_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_48_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_48_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_48_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_48_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_48_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_48_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_48_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_48_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_48_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_48_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_48_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_48_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_48_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_48_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_48_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_48_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_48_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_48_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_48_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_48_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_48_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_48_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_48_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_48_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_48_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_48_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_48_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_48_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_48_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_48_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_48_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_48_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_48_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_48_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_48_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_48_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_48_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_48_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_48_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_48_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_48_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_48_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_48_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_48_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_48_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_48_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_48_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_48_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_48_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_48_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_48_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_48_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_48_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_48_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_48_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_48_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_48_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_48_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_48_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_48_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_48_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_48_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_48_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_48_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_48_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_48_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_48_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_48_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_48_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_48_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_48_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_48_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_48_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_48_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_48_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_48_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_48_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_48_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_48_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_48_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_48_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_48_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_48_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_48_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_48_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_48_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_48_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_48_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_48_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_48_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_48_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_48_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_48_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_48_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_48_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_48_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_48_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_48_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_48_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_48_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_48_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_48_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_48_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_48_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_49_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_49_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_49_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_49_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_49_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_49_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_49_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_49_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_49_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_49_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_49_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_49_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_49_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_49_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_49_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_49_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_49_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_49_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_49_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_49_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_49_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_49_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_49_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_49_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_49_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_49_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_49_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_49_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_49_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_49_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_49_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_49_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_49_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_49_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_49_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_49_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_49_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_49_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_49_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_49_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_49_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_49_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_49_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_49_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_49_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_49_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_49_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_49_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_49_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_49_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_49_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_49_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_49_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_49_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_49_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_49_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_49_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_49_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_49_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_49_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_49_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_49_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_49_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_49_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_49_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_49_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_49_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_49_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_49_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_49_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_49_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_49_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_49_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_49_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_49_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_49_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_49_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_49_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_49_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_49_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_49_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_49_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_49_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_49_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_49_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_49_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_49_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_49_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_49_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_49_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_49_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_49_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_49_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_49_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_49_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_49_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_49_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_49_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_49_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_49_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_49_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_49_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_49_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_49_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_49_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_49_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_49_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_49_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_49_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_49_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_49_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_49_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_49_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_49_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_49_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_49_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_49_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_49_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_49_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_49_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_49_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_49_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_49_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_49_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_49_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_49_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_49_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_49_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_49_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_49_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_49_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_49_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_49_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_49_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_49_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_49_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_49_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_49_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_49_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_49_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_49_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_49_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_49_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_49_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_49_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_49_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_49_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_49_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_49_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_49_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_49_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_49_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_49_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_49_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_49_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_49_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_49_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_49_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_49_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_49_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_49_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_49_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_49_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_49_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_49_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_49_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_49_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_49_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_49_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_49_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_49_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_49_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_49_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_49_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_49_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_49_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_49_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_49_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_49_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_49_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_49_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_49_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_49_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_49_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_49_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_49_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_49_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_49_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_49_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_49_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_49_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_49_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_49_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_49_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_49_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_49_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_49_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_49_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_49_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_49_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_49_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_49_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_49_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_49_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_49_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_49_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_49_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_49_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_49_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_49_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_49_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_49_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_49_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_49_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_49_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_49_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_49_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_49_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_49_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_49_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_49_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_49_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_49_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_49_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_49_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_49_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_49_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_49_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_49_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_49_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_49_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_49_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_49_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_49_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_49_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_49_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_49_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_49_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_49_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_49_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_49_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_49_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_49_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_49_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_49_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_49_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_49_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_49_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_49_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_49_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_49_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_49_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_49_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_49_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_49_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_49_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_49_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_49_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_49_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_49_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_49_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_49_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_49_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_49_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_49_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_49_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_49_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_49_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_49_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_49_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_49_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_49_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_49_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_49_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_49_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_49_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_49_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_49_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_49_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_49_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_49_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_49_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_49_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_49_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_49_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_49_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_49_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_49_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_49_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_49_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_49_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_49_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_49_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_49_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_49_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_49_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_49_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_49_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_49_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_49_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_49_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_49_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_49_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_49_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_49_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_49_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_49_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_49_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_49_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_49_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_49_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_49_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_49_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_49_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_49_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_49_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_49_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_49_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_49_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_49_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_49_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_49_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_49_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_49_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_49_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_49_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_49_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_49_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_49_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_49_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_49_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_49_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_49_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_49_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_49_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_49_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_49_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_49_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_49_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_49_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_49_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_49_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_50_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_50_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_50_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_50_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_50_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_50_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_50_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_50_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_50_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_50_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_50_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_50_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_50_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_50_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_50_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_50_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_50_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_50_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_50_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_50_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_50_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_50_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_50_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_50_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_50_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_50_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_50_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_50_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_50_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_50_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_50_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_50_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_50_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_50_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_50_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_50_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_50_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_50_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_50_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_50_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_50_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_50_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_50_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_50_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_50_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_50_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_50_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_50_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_50_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_50_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_50_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_50_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_50_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_50_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_50_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_50_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_50_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_50_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_50_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_50_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_50_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_50_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_50_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_50_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_50_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_50_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_50_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_50_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_50_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_50_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_50_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_50_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_50_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_50_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_50_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_50_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_50_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_50_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_50_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_50_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_50_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_50_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_50_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_50_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_50_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_50_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_50_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_50_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_50_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_50_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_50_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_50_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_50_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_50_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_50_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_50_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_50_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_50_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_50_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_50_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_50_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_50_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_50_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_50_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_50_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_50_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_50_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_50_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_50_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_50_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_50_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_50_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_50_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_50_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_50_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_50_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_50_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_50_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_50_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_50_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_50_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_50_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_50_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_50_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_50_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_50_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_50_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_50_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_50_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_50_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_50_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_50_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_50_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_50_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_50_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_50_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_50_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_50_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_50_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_50_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_50_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_50_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_50_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_50_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_50_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_50_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_50_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_50_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_50_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_50_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_50_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_50_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_50_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_50_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_50_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_50_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_50_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_50_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_50_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_50_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_50_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_50_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_50_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_50_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_50_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_50_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_50_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_50_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_50_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_50_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_50_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_50_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_50_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_50_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_50_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_50_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_50_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_50_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_50_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_50_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_50_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_50_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_50_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_50_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_50_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_50_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_50_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_50_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_50_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_50_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_50_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_50_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_50_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_50_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_50_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_50_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_50_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_50_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_50_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_50_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_50_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_50_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_50_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_50_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_50_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_50_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_50_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_50_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_50_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_50_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_50_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_50_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_50_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_50_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_50_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_50_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_50_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_50_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_50_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_50_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_50_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_50_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_50_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_50_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_50_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_50_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_50_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_50_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_50_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_50_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_50_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_50_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_50_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_50_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_50_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_50_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_50_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_50_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_50_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_50_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_50_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_50_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_50_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_50_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_50_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_50_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_50_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_50_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_50_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_50_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_50_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_50_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_50_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_50_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_50_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_50_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_50_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_50_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_50_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_50_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_50_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_50_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_50_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_50_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_50_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_50_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_50_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_50_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_50_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_50_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_50_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_50_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_50_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_50_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_50_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_50_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_50_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_50_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_50_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_50_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_50_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_50_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_50_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_50_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_50_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_50_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_50_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_50_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_50_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_50_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_50_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_50_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_50_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_50_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_50_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_50_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_50_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_50_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_50_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_50_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_50_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_50_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_50_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_50_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_50_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_50_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_50_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_50_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_50_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_50_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_50_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_50_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_50_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_50_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_50_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_50_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_50_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_50_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_50_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_50_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_50_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_50_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_50_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_50_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_50_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_50_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_50_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_50_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_50_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_50_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_50_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_50_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_50_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_50_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_50_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_50_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_50_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_50_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_50_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_50_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_50_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_50_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_51_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_51_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_51_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_51_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_51_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_51_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_51_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_51_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_51_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_51_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_51_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_51_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_51_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_51_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_51_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_51_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_51_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_51_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_51_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_51_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_51_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_51_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_51_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_51_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_51_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_51_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_51_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_51_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_51_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_51_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_51_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_51_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_51_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_51_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_51_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_51_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_51_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_51_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_51_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_51_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_51_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_51_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_51_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_51_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_51_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_51_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_51_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_51_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_51_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_51_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_51_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_51_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_51_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_51_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_51_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_51_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_51_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_51_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_51_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_51_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_51_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_51_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_51_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_51_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_51_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_51_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_51_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_51_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_51_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_51_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_51_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_51_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_51_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_51_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_51_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_51_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_51_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_51_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_51_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_51_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_51_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_51_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_51_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_51_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_51_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_51_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_51_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_51_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_51_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_51_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_51_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_51_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_51_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_51_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_51_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_51_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_51_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_51_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_51_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_51_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_51_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_51_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_51_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_51_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_51_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_51_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_51_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_51_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_51_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_51_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_51_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_51_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_51_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_51_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_51_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_51_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_51_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_51_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_51_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_51_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_51_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_51_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_51_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_51_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_51_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_51_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_51_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_51_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_51_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_51_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_51_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_51_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_51_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_51_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_51_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_51_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_51_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_51_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_51_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_51_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_51_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_51_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_51_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_51_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_51_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_51_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_51_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_51_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_51_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_51_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_51_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_51_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_51_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_51_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_51_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_51_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_51_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_51_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_51_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_51_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_51_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_51_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_51_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_51_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_51_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_51_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_51_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_51_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_51_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_51_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_51_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_51_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_51_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_51_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_51_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_51_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_51_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_51_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_51_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_51_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_51_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_51_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_51_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_51_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_51_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_51_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_51_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_51_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_51_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_51_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_51_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_51_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_51_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_51_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_51_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_51_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_51_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_51_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_51_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_51_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_51_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_51_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_51_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_51_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_51_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_51_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_51_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_51_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_51_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_51_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_51_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_51_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_51_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_51_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_51_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_51_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_51_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_51_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_51_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_51_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_51_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_51_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_51_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_51_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_51_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_51_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_51_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_51_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_51_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_51_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_51_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_51_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_51_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_51_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_51_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_51_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_51_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_51_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_51_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_51_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_51_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_51_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_51_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_51_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_51_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_51_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_51_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_51_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_51_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_51_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_51_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_51_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_51_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_51_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_51_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_51_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_51_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_51_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_51_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_51_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_51_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_51_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_51_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_51_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_51_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_51_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_51_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_51_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_51_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_51_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_51_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_51_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_51_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_51_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_51_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_51_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_51_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_51_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_51_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_51_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_51_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_51_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_51_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_51_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_51_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_51_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_51_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_51_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_51_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_51_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_51_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_51_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_51_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_51_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_51_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_51_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_51_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_51_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_51_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_51_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_51_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_51_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_51_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_51_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_51_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_51_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_51_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_51_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_51_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_51_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_51_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_51_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_51_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_51_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_51_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_51_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_51_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_51_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_51_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_51_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_51_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_51_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_51_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_51_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_51_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_51_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_51_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_51_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_51_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_51_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_51_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_51_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_51_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_51_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_51_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_51_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_51_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_51_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_51_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_51_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_51_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_51_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_52_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_52_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_52_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_52_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_52_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_52_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_52_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_52_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_52_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_52_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_52_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_52_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_52_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_52_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_52_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_52_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_52_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_52_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_52_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_52_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_52_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_52_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_52_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_52_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_52_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_52_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_52_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_52_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_52_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_52_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_52_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_52_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_52_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_52_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_52_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_52_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_52_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_52_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_52_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_52_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_52_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_52_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_52_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_52_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_52_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_52_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_52_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_52_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_52_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_52_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_52_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_52_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_52_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_52_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_52_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_52_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_52_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_52_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_52_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_52_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_52_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_52_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_52_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_52_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_52_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_52_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_52_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_52_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_52_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_52_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_52_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_52_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_52_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_52_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_52_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_52_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_52_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_52_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_52_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_52_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_52_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_52_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_52_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_52_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_52_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_52_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_52_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_52_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_52_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_52_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_52_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_52_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_52_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_52_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_52_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_52_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_52_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_52_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_52_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_52_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_52_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_52_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_52_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_52_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_52_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_52_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_52_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_52_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_52_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_52_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_52_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_52_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_52_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_52_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_52_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_52_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_52_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_52_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_52_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_52_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_52_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_52_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_52_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_52_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_52_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_52_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_52_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_52_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_52_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_52_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_52_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_52_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_52_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_52_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_52_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_52_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_52_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_52_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_52_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_52_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_52_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_52_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_52_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_52_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_52_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_52_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_52_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_52_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_52_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_52_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_52_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_52_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_52_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_52_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_52_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_52_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_52_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_52_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_52_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_52_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_52_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_52_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_52_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_52_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_52_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_52_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_52_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_52_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_52_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_52_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_52_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_52_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_52_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_52_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_52_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_52_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_52_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_52_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_52_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_52_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_52_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_52_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_52_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_52_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_52_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_52_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_52_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_52_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_52_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_52_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_52_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_52_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_52_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_52_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_52_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_52_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_52_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_52_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_52_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_52_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_52_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_52_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_52_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_52_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_52_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_52_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_52_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_52_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_52_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_52_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_52_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_52_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_52_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_52_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_52_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_52_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_52_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_52_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_52_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_52_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_52_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_52_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_52_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_52_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_52_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_52_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_52_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_52_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_52_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_52_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_52_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_52_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_52_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_52_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_52_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_52_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_52_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_52_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_52_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_52_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_52_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_52_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_52_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_52_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_52_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_52_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_52_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_52_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_52_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_52_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_52_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_52_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_52_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_52_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_52_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_52_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_52_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_52_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_52_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_52_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_52_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_52_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_52_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_52_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_52_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_52_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_52_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_52_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_52_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_52_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_52_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_52_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_52_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_52_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_52_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_52_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_52_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_52_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_52_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_52_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_52_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_52_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_52_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_52_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_52_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_52_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_52_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_52_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_52_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_52_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_52_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_52_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_52_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_52_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_52_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_52_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_52_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_52_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_52_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_52_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_52_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_52_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_52_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_52_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_52_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_52_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_52_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_52_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_52_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_52_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_52_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_52_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_52_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_52_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_52_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_52_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_52_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_52_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_52_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_52_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_52_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_52_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_52_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_52_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_52_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_52_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_52_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_52_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_52_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_52_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_52_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_52_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_52_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_52_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_52_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_52_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_52_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_52_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_52_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_52_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_52_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_52_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_53_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_53_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_53_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_53_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_53_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_53_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_53_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_53_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_53_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_53_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_53_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_53_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_53_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_53_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_53_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_53_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_53_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_53_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_53_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_53_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_53_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_53_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_53_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_53_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_53_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_53_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_53_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_53_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_53_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_53_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_53_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_53_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_53_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_53_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_53_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_53_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_53_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_53_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_53_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_53_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_53_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_53_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_53_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_53_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_53_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_53_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_53_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_53_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_53_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_53_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_53_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_53_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_53_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_53_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_53_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_53_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_53_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_53_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_53_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_53_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_53_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_53_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_53_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_53_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_53_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_53_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_53_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_53_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_53_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_53_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_53_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_53_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_53_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_53_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_53_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_53_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_53_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_53_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_53_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_53_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_53_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_53_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_53_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_53_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_53_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_53_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_53_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_53_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_53_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_53_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_53_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_53_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_53_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_53_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_53_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_53_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_53_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_53_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_53_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_53_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_53_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_53_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_53_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_53_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_53_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_53_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_53_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_53_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_53_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_53_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_53_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_53_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_53_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_53_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_53_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_53_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_53_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_53_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_53_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_53_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_53_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_53_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_53_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_53_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_53_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_53_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_53_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_53_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_53_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_53_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_53_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_53_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_53_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_53_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_53_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_53_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_53_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_53_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_53_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_53_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_53_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_53_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_53_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_53_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_53_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_53_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_53_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_53_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_53_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_53_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_53_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_53_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_53_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_53_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_53_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_53_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_53_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_53_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_53_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_53_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_53_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_53_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_53_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_53_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_53_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_53_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_53_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_53_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_53_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_53_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_53_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_53_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_53_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_53_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_53_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_53_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_53_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_53_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_53_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_53_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_53_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_53_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_53_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_53_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_53_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_53_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_53_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_53_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_53_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_53_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_53_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_53_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_53_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_53_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_53_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_53_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_53_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_53_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_53_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_53_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_53_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_53_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_53_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_53_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_53_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_53_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_53_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_53_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_53_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_53_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_53_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_53_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_53_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_53_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_53_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_53_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_53_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_53_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_53_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_53_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_53_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_53_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_53_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_53_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_53_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_53_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_53_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_53_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_53_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_53_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_53_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_53_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_53_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_53_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_53_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_53_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_53_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_53_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_53_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_53_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_53_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_53_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_53_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_53_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_53_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_53_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_53_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_53_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_53_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_53_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_53_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_53_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_53_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_53_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_53_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_53_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_53_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_53_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_53_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_53_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_53_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_53_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_53_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_53_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_53_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_53_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_53_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_53_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_53_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_53_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_53_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_53_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_53_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_53_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_53_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_53_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_53_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_53_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_53_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_53_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_53_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_53_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_53_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_53_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_53_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_53_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_53_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_53_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_53_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_53_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_53_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_53_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_53_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_53_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_53_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_53_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_53_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_53_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_53_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_53_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_53_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_53_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_53_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_53_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_53_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_53_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_53_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_53_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_53_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_53_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_53_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_53_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_53_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_53_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_53_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_53_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_53_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_53_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_53_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_53_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_53_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_53_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_53_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_53_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_53_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_53_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_53_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_53_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_53_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_53_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_53_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_53_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_53_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_53_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_53_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_53_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_53_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_53_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_53_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_53_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_53_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_53_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_54_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_54_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_54_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_54_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_54_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_54_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_54_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_54_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_54_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_54_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_54_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_54_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_54_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_54_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_54_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_54_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_54_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_54_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_54_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_54_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_54_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_54_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_54_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_54_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_54_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_54_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_54_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_54_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_54_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_54_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_54_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_54_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_54_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_54_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_54_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_54_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_54_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_54_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_54_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_54_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_54_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_54_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_54_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_54_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_54_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_54_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_54_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_54_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_54_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_54_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_54_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_54_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_54_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_54_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_54_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_54_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_54_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_54_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_54_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_54_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_54_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_54_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_54_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_54_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_54_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_54_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_54_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_54_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_54_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_54_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_54_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_54_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_54_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_54_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_54_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_54_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_54_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_54_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_54_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_54_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_54_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_54_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_54_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_54_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_54_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_54_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_54_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_54_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_54_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_54_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_54_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_54_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_54_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_54_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_54_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_54_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_54_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_54_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_54_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_54_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_54_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_54_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_54_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_54_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_54_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_54_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_54_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_54_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_54_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_54_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_54_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_54_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_54_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_54_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_54_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_54_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_54_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_54_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_54_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_54_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_54_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_54_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_54_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_54_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_54_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_54_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_54_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_54_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_54_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_54_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_54_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_54_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_54_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_54_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_54_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_54_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_54_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_54_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_54_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_54_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_54_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_54_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_54_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_54_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_54_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_54_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_54_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_54_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_54_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_54_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_54_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_54_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_54_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_54_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_54_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_54_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_54_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_54_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_54_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_54_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_54_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_54_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_54_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_54_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_54_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_54_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_54_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_54_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_54_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_54_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_54_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_54_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_54_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_54_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_54_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_54_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_54_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_54_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_54_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_54_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_54_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_54_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_54_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_54_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_54_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_54_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_54_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_54_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_54_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_54_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_54_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_54_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_54_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_54_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_54_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_54_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_54_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_54_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_54_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_54_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_54_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_54_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_54_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_54_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_54_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_54_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_54_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_54_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_54_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_54_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_54_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_54_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_54_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_54_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_54_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_54_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_54_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_54_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_54_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_54_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_54_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_54_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_54_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_54_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_54_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_54_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_54_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_54_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_54_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_54_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_54_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_54_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_54_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_54_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_54_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_54_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_54_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_54_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_54_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_54_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_54_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_54_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_54_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_54_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_54_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_54_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_54_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_54_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_54_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_54_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_54_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_54_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_54_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_54_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_54_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_54_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_54_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_54_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_54_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_54_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_54_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_54_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_54_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_54_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_54_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_54_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_54_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_54_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_54_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_54_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_54_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_54_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_54_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_54_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_54_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_54_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_54_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_54_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_54_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_54_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_54_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_54_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_54_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_54_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_54_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_54_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_54_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_54_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_54_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_54_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_54_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_54_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_54_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_54_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_54_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_54_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_54_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_54_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_54_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_54_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_54_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_54_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_54_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_54_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_54_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_54_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_54_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_54_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_54_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_54_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_54_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_54_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_54_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_54_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_54_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_54_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_54_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_54_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_54_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_54_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_54_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_54_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_54_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_54_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_54_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_54_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_54_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_54_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_54_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_54_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_54_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_54_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_54_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_54_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_54_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_54_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_54_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_54_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_54_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_54_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_54_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_54_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_55_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_55_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_55_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_55_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_55_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_55_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_55_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_55_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_55_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_55_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_55_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_55_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_55_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_55_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_55_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_55_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_55_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_55_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_55_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_55_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_55_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_55_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_55_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_55_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_55_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_55_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_55_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_55_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_55_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_55_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_55_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_55_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_55_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_55_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_55_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_55_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_55_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_55_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_55_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_55_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_55_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_55_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_55_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_55_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_55_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_55_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_55_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_55_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_55_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_55_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_55_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_55_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_55_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_55_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_55_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_55_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_55_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_55_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_55_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_55_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_55_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_55_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_55_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_55_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_55_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_55_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_55_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_55_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_55_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_55_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_55_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_55_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_55_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_55_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_55_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_55_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_55_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_55_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_55_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_55_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_55_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_55_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_55_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_55_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_55_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_55_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_55_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_55_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_55_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_55_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_55_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_55_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_55_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_55_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_55_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_55_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_55_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_55_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_55_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_55_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_55_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_55_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_55_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_55_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_55_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_55_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_55_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_55_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_55_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_55_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_55_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_55_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_55_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_55_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_55_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_55_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_55_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_55_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_55_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_55_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_55_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_55_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_55_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_55_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_55_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_55_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_55_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_55_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_55_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_55_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_55_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_55_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_55_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_55_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_55_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_55_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_55_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_55_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_55_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_55_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_55_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_55_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_55_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_55_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_55_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_55_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_55_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_55_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_55_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_55_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_55_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_55_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_55_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_55_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_55_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_55_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_55_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_55_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_55_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_55_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_55_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_55_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_55_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_55_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_55_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_55_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_55_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_55_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_55_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_55_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_55_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_55_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_55_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_55_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_55_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_55_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_55_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_55_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_55_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_55_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_55_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_55_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_55_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_55_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_55_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_55_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_55_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_55_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_55_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_55_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_55_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_55_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_55_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_55_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_55_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_55_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_55_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_55_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_55_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_55_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_55_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_55_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_55_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_55_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_55_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_55_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_55_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_55_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_55_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_55_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_55_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_55_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_55_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_55_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_55_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_55_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_55_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_55_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_55_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_55_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_55_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_55_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_55_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_55_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_55_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_55_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_55_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_55_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_55_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_55_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_55_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_55_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_55_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_55_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_55_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_55_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_55_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_55_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_55_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_55_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_55_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_55_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_55_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_55_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_55_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_55_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_55_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_55_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_55_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_55_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_55_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_55_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_55_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_55_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_55_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_55_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_55_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_55_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_55_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_55_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_55_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_55_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_55_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_55_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_55_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_55_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_55_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_55_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_55_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_55_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_55_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_55_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_55_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_55_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_55_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_55_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_55_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_55_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_55_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_55_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_55_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_55_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_55_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_55_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_55_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_55_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_55_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_55_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_55_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_55_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_55_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_55_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_55_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_55_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_55_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_55_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_55_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_55_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_55_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_55_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_55_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_55_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_55_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_55_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_55_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_55_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_55_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_55_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_55_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_55_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_55_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_55_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_55_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_55_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_55_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_55_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_55_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_55_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_55_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_55_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_55_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_55_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_55_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_55_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_55_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_55_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_55_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_55_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_55_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_55_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_55_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_55_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_55_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_55_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_55_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_55_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_55_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_55_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_55_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_55_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_55_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_55_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_56_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_56_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_56_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_56_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_56_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_56_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_56_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_56_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_56_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_56_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_56_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_56_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_56_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_56_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_56_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_56_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_56_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_56_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_56_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_56_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_56_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_56_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_56_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_56_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_56_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_56_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_56_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_56_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_56_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_56_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_56_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_56_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_56_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_56_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_56_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_56_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_56_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_56_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_56_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_56_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_56_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_56_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_56_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_56_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_56_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_56_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_56_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_56_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_56_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_56_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_56_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_56_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_56_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_56_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_56_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_56_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_56_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_56_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_56_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_56_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_56_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_56_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_56_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_56_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_56_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_56_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_56_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_56_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_56_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_56_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_56_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_56_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_56_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_56_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_56_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_56_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_56_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_56_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_56_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_56_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_56_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_56_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_56_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_56_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_56_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_56_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_56_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_56_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_56_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_56_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_56_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_56_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_56_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_56_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_56_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_56_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_56_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_56_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_56_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_56_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_56_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_56_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_56_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_56_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_56_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_56_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_56_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_56_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_56_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_56_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_56_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_56_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_56_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_56_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_56_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_56_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_56_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_56_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_56_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_56_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_56_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_56_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_56_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_56_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_56_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_56_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_56_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_56_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_56_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_56_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_56_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_56_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_56_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_56_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_56_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_56_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_56_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_56_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_56_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_56_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_56_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_56_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_56_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_56_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_56_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_56_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_56_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_56_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_56_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_56_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_56_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_56_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_56_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_56_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_56_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_56_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_56_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_56_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_56_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_56_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_56_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_56_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_56_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_56_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_56_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_56_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_56_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_56_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_56_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_56_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_56_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_56_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_56_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_56_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_56_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_56_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_56_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_56_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_56_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_56_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_56_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_56_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_56_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_56_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_56_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_56_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_56_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_56_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_56_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_56_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_56_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_56_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_56_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_56_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_56_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_56_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_56_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_56_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_56_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_56_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_56_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_56_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_56_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_56_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_56_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_56_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_56_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_56_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_56_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_56_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_56_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_56_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_56_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_56_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_56_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_56_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_56_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_56_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_56_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_56_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_56_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_56_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_56_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_56_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_56_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_56_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_56_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_56_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_56_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_56_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_56_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_56_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_56_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_56_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_56_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_56_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_56_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_56_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_56_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_56_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_56_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_56_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_56_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_56_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_56_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_56_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_56_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_56_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_56_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_56_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_56_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_56_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_56_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_56_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_56_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_56_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_56_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_56_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_56_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_56_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_56_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_56_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_56_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_56_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_56_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_56_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_56_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_56_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_56_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_56_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_56_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_56_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_56_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_56_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_56_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_56_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_56_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_56_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_56_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_56_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_56_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_56_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_56_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_56_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_56_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_56_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_56_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_56_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_56_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_56_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_56_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_56_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_56_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_56_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_56_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_56_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_56_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_56_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_56_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_56_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_56_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_56_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_56_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_56_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_56_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_56_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_56_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_56_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_56_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_56_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_56_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_56_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_56_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_56_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_56_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_56_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_56_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_56_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_56_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_56_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_56_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_56_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_56_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_56_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_56_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_56_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_56_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_56_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_56_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_56_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_56_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_56_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_56_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_56_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_56_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_56_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_56_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_56_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_56_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_56_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_56_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_56_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_57_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_57_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_57_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_57_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_57_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_57_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_57_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_57_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_57_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_57_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_57_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_57_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_57_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_57_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_57_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_57_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_57_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_57_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_57_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_57_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_57_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_57_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_57_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_57_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_57_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_57_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_57_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_57_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_57_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_57_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_57_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_57_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_57_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_57_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_57_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_57_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_57_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_57_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_57_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_57_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_57_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_57_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_57_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_57_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_57_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_57_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_57_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_57_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_57_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_57_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_57_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_57_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_57_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_57_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_57_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_57_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_57_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_57_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_57_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_57_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_57_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_57_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_57_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_57_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_57_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_57_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_57_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_57_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_57_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_57_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_57_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_57_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_57_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_57_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_57_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_57_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_57_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_57_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_57_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_57_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_57_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_57_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_57_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_57_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_57_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_57_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_57_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_57_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_57_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_57_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_57_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_57_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_57_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_57_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_57_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_57_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_57_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_57_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_57_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_57_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_57_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_57_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_57_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_57_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_57_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_57_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_57_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_57_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_57_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_57_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_57_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_57_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_57_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_57_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_57_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_57_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_57_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_57_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_57_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_57_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_57_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_57_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_57_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_57_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_57_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_57_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_57_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_57_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_57_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_57_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_57_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_57_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_57_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_57_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_57_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_57_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_57_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_57_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_57_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_57_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_57_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_57_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_57_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_57_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_57_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_57_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_57_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_57_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_57_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_57_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_57_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_57_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_57_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_57_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_57_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_57_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_57_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_57_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_57_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_57_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_57_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_57_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_57_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_57_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_57_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_57_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_57_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_57_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_57_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_57_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_57_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_57_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_57_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_57_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_57_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_57_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_57_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_57_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_57_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_57_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_57_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_57_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_57_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_57_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_57_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_57_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_57_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_57_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_57_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_57_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_57_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_57_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_57_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_57_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_57_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_57_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_57_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_57_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_57_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_57_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_57_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_57_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_57_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_57_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_57_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_57_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_57_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_57_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_57_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_57_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_57_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_57_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_57_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_57_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_57_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_57_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_57_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_57_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_57_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_57_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_57_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_57_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_57_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_57_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_57_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_57_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_57_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_57_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_57_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_57_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_57_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_57_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_57_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_57_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_57_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_57_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_57_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_57_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_57_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_57_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_57_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_57_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_57_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_57_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_57_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_57_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_57_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_57_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_57_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_57_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_57_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_57_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_57_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_57_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_57_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_57_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_57_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_57_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_57_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_57_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_57_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_57_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_57_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_57_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_57_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_57_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_57_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_57_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_57_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_57_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_57_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_57_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_57_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_57_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_57_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_57_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_57_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_57_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_57_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_57_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_57_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_57_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_57_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_57_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_57_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_57_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_57_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_57_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_57_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_57_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_57_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_57_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_57_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_57_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_57_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_57_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_57_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_57_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_57_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_57_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_57_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_57_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_57_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_57_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_57_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_57_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_57_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_57_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_57_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_57_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_57_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_57_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_57_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_57_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_57_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_57_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_57_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_57_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_57_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_57_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_57_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_57_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_57_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_57_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_57_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_57_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_57_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_57_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_57_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_57_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_57_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_57_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_57_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_57_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_57_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_57_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_57_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_57_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_57_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_57_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_57_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_57_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_58_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_58_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_58_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_58_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_58_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_58_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_58_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_58_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_58_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_58_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_58_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_58_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_58_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_58_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_58_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_58_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_58_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_58_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_58_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_58_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_58_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_58_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_58_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_58_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_58_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_58_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_58_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_58_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_58_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_58_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_58_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_58_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_58_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_58_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_58_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_58_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_58_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_58_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_58_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_58_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_58_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_58_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_58_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_58_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_58_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_58_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_58_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_58_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_58_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_58_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_58_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_58_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_58_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_58_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_58_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_58_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_58_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_58_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_58_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_58_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_58_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_58_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_58_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_58_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_58_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_58_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_58_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_58_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_58_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_58_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_58_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_58_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_58_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_58_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_58_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_58_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_58_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_58_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_58_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_58_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_58_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_58_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_58_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_58_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_58_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_58_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_58_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_58_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_58_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_58_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_58_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_58_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_58_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_58_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_58_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_58_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_58_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_58_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_58_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_58_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_58_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_58_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_58_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_58_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_58_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_58_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_58_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_58_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_58_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_58_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_58_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_58_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_58_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_58_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_58_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_58_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_58_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_58_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_58_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_58_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_58_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_58_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_58_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_58_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_58_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_58_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_58_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_58_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_58_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_58_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_58_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_58_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_58_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_58_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_58_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_58_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_58_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_58_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_58_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_58_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_58_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_58_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_58_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_58_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_58_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_58_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_58_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_58_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_58_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_58_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_58_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_58_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_58_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_58_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_58_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_58_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_58_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_58_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_58_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_58_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_58_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_58_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_58_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_58_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_58_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_58_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_58_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_58_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_58_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_58_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_58_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_58_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_58_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_58_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_58_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_58_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_58_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_58_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_58_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_58_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_58_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_58_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_58_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_58_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_58_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_58_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_58_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_58_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_58_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_58_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_58_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_58_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_58_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_58_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_58_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_58_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_58_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_58_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_58_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_58_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_58_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_58_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_58_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_58_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_58_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_58_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_58_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_58_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_58_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_58_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_58_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_58_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_58_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_58_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_58_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_58_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_58_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_58_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_58_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_58_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_58_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_58_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_58_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_58_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_58_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_58_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_58_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_58_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_58_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_58_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_58_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_58_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_58_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_58_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_58_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_58_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_58_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_58_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_58_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_58_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_58_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_58_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_58_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_58_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_58_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_58_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_58_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_58_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_58_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_58_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_58_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_58_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_58_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_58_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_58_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_58_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_58_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_58_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_58_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_58_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_58_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_58_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_58_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_58_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_58_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_58_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_58_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_58_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_58_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_58_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_58_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_58_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_58_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_58_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_58_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_58_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_58_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_58_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_58_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_58_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_58_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_58_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_58_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_58_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_58_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_58_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_58_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_58_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_58_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_58_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_58_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_58_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_58_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_58_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_58_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_58_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_58_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_58_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_58_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_58_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_58_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_58_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_58_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_58_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_58_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_58_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_58_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_58_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_58_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_58_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_58_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_58_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_58_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_58_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_58_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_58_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_58_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_58_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_58_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_58_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_58_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_58_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_58_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_58_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_58_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_58_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_58_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_58_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_58_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_58_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_58_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_58_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_58_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_58_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_58_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_58_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_58_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_58_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_58_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_58_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_58_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_58_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_59_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_59_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_59_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_59_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_59_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_59_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_59_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_59_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_59_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_59_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_59_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_59_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_59_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_59_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_59_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_59_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_59_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_59_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_59_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_59_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_59_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_59_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_59_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_59_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_59_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_59_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_59_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_59_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_59_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_59_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_59_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_59_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_59_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_59_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_59_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_59_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_59_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_59_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_59_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_59_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_59_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_59_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_59_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_59_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_59_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_59_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_59_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_59_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_59_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_59_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_59_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_59_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_59_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_59_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_59_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_59_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_59_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_59_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_59_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_59_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_59_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_59_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_59_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_59_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_59_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_59_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_59_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_59_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_59_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_59_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_59_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_59_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_59_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_59_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_59_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_59_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_59_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_59_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_59_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_59_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_59_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_59_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_59_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_59_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_59_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_59_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_59_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_59_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_59_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_59_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_59_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_59_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_59_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_59_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_59_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_59_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_59_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_59_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_59_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_59_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_59_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_59_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_59_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_59_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_59_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_59_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_59_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_59_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_59_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_59_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_59_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_59_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_59_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_59_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_59_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_59_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_59_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_59_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_59_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_59_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_59_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_59_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_59_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_59_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_59_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_59_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_59_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_59_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_59_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_59_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_59_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_59_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_59_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_59_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_59_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_59_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_59_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_59_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_59_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_59_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_59_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_59_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_59_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_59_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_59_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_59_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_59_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_59_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_59_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_59_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_59_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_59_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_59_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_59_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_59_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_59_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_59_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_59_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_59_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_59_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_59_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_59_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_59_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_59_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_59_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_59_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_59_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_59_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_59_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_59_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_59_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_59_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_59_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_59_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_59_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_59_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_59_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_59_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_59_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_59_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_59_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_59_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_59_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_59_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_59_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_59_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_59_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_59_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_59_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_59_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_59_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_59_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_59_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_59_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_59_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_59_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_59_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_59_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_59_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_59_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_59_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_59_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_59_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_59_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_59_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_59_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_59_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_59_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_59_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_59_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_59_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_59_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_59_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_59_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_59_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_59_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_59_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_59_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_59_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_59_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_59_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_59_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_59_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_59_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_59_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_59_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_59_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_59_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_59_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_59_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_59_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_59_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_59_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_59_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_59_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_59_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_59_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_59_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_59_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_59_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_59_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_59_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_59_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_59_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_59_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_59_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_59_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_59_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_59_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_59_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_59_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_59_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_59_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_59_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_59_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_59_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_59_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_59_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_59_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_59_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_59_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_59_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_59_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_59_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_59_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_59_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_59_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_59_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_59_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_59_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_59_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_59_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_59_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_59_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_59_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_59_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_59_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_59_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_59_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_59_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_59_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_59_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_59_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_59_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_59_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_59_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_59_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_59_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_59_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_59_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_59_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_59_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_59_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_59_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_59_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_59_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_59_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_59_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_59_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_59_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_59_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_59_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_59_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_59_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_59_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_59_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_59_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_59_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_59_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_59_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_59_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_59_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_59_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_59_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_59_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_59_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_59_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_59_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_59_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_59_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_59_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_59_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_59_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_59_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_59_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_59_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_59_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_59_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_59_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_59_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_59_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_59_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_59_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_59_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_59_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_59_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_59_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_59_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_59_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_59_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_59_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_59_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_60_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_60_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_60_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_60_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_60_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_60_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_60_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_60_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_60_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_60_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_60_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_60_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_60_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_60_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_60_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_60_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_60_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_60_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_60_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_60_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_60_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_60_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_60_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_60_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_60_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_60_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_60_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_60_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_60_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_60_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_60_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_60_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_60_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_60_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_60_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_60_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_60_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_60_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_60_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_60_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_60_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_60_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_60_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_60_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_60_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_60_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_60_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_60_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_60_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_60_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_60_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_60_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_60_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_60_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_60_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_60_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_60_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_60_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_60_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_60_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_60_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_60_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_60_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_60_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_60_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_60_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_60_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_60_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_60_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_60_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_60_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_60_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_60_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_60_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_60_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_60_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_60_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_60_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_60_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_60_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_60_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_60_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_60_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_60_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_60_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_60_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_60_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_60_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_60_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_60_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_60_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_60_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_60_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_60_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_60_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_60_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_60_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_60_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_60_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_60_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_60_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_60_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_60_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_60_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_60_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_60_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_60_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_60_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_60_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_60_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_60_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_60_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_60_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_60_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_60_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_60_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_60_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_60_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_60_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_60_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_60_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_60_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_60_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_60_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_60_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_60_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_60_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_60_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_60_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_60_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_60_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_60_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_60_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_60_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_60_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_60_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_60_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_60_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_60_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_60_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_60_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_60_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_60_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_60_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_60_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_60_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_60_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_60_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_60_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_60_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_60_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_60_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_60_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_60_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_60_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_60_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_60_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_60_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_60_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_60_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_60_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_60_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_60_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_60_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_60_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_60_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_60_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_60_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_60_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_60_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_60_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_60_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_60_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_60_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_60_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_60_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_60_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_60_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_60_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_60_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_60_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_60_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_60_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_60_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_60_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_60_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_60_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_60_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_60_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_60_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_60_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_60_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_60_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_60_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_60_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_60_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_60_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_60_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_60_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_60_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_60_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_60_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_60_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_60_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_60_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_60_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_60_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_60_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_60_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_60_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_60_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_60_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_60_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_60_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_60_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_60_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_60_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_60_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_60_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_60_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_60_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_60_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_60_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_60_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_60_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_60_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_60_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_60_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_60_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_60_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_60_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_60_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_60_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_60_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_60_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_60_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_60_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_60_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_60_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_60_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_60_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_60_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_60_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_60_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_60_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_60_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_60_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_60_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_60_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_60_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_60_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_60_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_60_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_60_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_60_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_60_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_60_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_60_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_60_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_60_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_60_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_60_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_60_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_60_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_60_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_60_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_60_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_60_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_60_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_60_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_60_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_60_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_60_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_60_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_60_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_60_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_60_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_60_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_60_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_60_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_60_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_60_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_60_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_60_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_60_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_60_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_60_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_60_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_60_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_60_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_60_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_60_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_60_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_60_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_60_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_60_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_60_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_60_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_60_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_60_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_60_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_60_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_60_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_60_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_60_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_60_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_60_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_60_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_60_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_60_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_60_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_60_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_60_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_60_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_60_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_60_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_60_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_60_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_60_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_60_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_60_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_60_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_60_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_60_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_60_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_60_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_60_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_60_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_60_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_60_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_60_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_60_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_60_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_60_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_60_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_60_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_60_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_60_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_60_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_60_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_60_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_60_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_61_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_61_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_61_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_61_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_61_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_61_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_61_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_61_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_61_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_61_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_61_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_61_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_61_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_61_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_61_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_61_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_61_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_61_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_61_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_61_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_61_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_61_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_61_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_61_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_61_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_61_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_61_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_61_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_61_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_61_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_61_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_61_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_61_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_61_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_61_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_61_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_61_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_61_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_61_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_61_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_61_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_61_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_61_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_61_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_61_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_61_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_61_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_61_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_61_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_61_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_61_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_61_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_61_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_61_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_61_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_61_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_61_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_61_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_61_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_61_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_61_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_61_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_61_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_61_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_61_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_61_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_61_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_61_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_61_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_61_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_61_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_61_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_61_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_61_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_61_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_61_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_61_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_61_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_61_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_61_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_61_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_61_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_61_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_61_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_61_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_61_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_61_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_61_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_61_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_61_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_61_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_61_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_61_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_61_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_61_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_61_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_61_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_61_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_61_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_61_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_61_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_61_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_61_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_61_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_61_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_61_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_61_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_61_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_61_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_61_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_61_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_61_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_61_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_61_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_61_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_61_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_61_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_61_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_61_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_61_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_61_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_61_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_61_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_61_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_61_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_61_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_61_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_61_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_61_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_61_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_61_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_61_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_61_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_61_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_61_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_61_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_61_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_61_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_61_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_61_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_61_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_61_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_61_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_61_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_61_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_61_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_61_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_61_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_61_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_61_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_61_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_61_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_61_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_61_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_61_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_61_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_61_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_61_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_61_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_61_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_61_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_61_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_61_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_61_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_61_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_61_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_61_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_61_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_61_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_61_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_61_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_61_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_61_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_61_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_61_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_61_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_61_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_61_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_61_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_61_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_61_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_61_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_61_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_61_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_61_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_61_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_61_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_61_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_61_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_61_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_61_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_61_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_61_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_61_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_61_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_61_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_61_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_61_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_61_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_61_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_61_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_61_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_61_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_61_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_61_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_61_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_61_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_61_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_61_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_61_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_61_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_61_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_61_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_61_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_61_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_61_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_61_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_61_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_61_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_61_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_61_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_61_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_61_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_61_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_61_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_61_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_61_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_61_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_61_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_61_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_61_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_61_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_61_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_61_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_61_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_61_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_61_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_61_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_61_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_61_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_61_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_61_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_61_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_61_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_61_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_61_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_61_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_61_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_61_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_61_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_61_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_61_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_61_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_61_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_61_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_61_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_61_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_61_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_61_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_61_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_61_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_61_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_61_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_61_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_61_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_61_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_61_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_61_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_61_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_61_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_61_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_61_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_61_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_61_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_61_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_61_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_61_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_61_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_61_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_61_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_61_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_61_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_61_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_61_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_61_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_61_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_61_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_61_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_61_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_61_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_61_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_61_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_61_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_61_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_61_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_61_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_61_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_61_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_61_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_61_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_61_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_61_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_61_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_61_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_61_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_61_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_61_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_61_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_61_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_61_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_61_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_61_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_61_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_61_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_61_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_61_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_61_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_61_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_61_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_61_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_61_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_61_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_61_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_61_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_61_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_61_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_61_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_61_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_61_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_61_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_61_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_61_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_61_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_61_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_61_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_61_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_61_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_61_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_61_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_61_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_61_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_61_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_62_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_62_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_62_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_62_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_62_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_62_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_62_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_62_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_62_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_62_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_62_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_62_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_62_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_62_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_62_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_62_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_62_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_62_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_62_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_62_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_62_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_62_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_62_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_62_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_62_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_62_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_62_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_62_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_62_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_62_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_62_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_62_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_62_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_62_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_62_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_62_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_62_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_62_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_62_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_62_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_62_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_62_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_62_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_62_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_62_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_62_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_62_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_62_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_62_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_62_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_62_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_62_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_62_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_62_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_62_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_62_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_62_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_62_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_62_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_62_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_62_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_62_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_62_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_62_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_62_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_62_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_62_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_62_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_62_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_62_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_62_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_62_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_62_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_62_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_62_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_62_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_62_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_62_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_62_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_62_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_62_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_62_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_62_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_62_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_62_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_62_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_62_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_62_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_62_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_62_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_62_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_62_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_62_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_62_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_62_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_62_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_62_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_62_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_62_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_62_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_62_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_62_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_62_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_62_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_62_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_62_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_62_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_62_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_62_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_62_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_62_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_62_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_62_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_62_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_62_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_62_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_62_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_62_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_62_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_62_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_62_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_62_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_62_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_62_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_62_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_62_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_62_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_62_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_62_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_62_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_62_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_62_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_62_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_62_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_62_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_62_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_62_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_62_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_62_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_62_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_62_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_62_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_62_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_62_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_62_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_62_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_62_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_62_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_62_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_62_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_62_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_62_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_62_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_62_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_62_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_62_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_62_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_62_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_62_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_62_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_62_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_62_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_62_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_62_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_62_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_62_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_62_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_62_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_62_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_62_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_62_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_62_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_62_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_62_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_62_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_62_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_62_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_62_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_62_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_62_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_62_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_62_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_62_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_62_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_62_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_62_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_62_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_62_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_62_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_62_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_62_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_62_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_62_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_62_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_62_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_62_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_62_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_62_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_62_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_62_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_62_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_62_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_62_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_62_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_62_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_62_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_62_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_62_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_62_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_62_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_62_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_62_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_62_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_62_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_62_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_62_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_62_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_62_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_62_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_62_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_62_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_62_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_62_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_62_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_62_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_62_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_62_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_62_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_62_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_62_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_62_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_62_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_62_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_62_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_62_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_62_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_62_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_62_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_62_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_62_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_62_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_62_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_62_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_62_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_62_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_62_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_62_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_62_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_62_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_62_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_62_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_62_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_62_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_62_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_62_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_62_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_62_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_62_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_62_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_62_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_62_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_62_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_62_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_62_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_62_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_62_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_62_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_62_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_62_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_62_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_62_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_62_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_62_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_62_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_62_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_62_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_62_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_62_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_62_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_62_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_62_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_62_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_62_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_62_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_62_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_62_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_62_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_62_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_62_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_62_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_62_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_62_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_62_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_62_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_62_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_62_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_62_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_62_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_62_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_62_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_62_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_62_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_62_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_62_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_62_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_62_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_62_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_62_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_62_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_62_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_62_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_62_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_62_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_62_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_62_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_62_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_62_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_62_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_62_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_62_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_62_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_62_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_62_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_62_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_62_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_62_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_62_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_62_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_62_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_62_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_62_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_62_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_62_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_62_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_62_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_62_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_62_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_62_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_62_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_62_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_62_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_62_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_63_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_63_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_63_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_63_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_63_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_63_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_63_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_63_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_63_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_63_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_63_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_63_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_63_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_63_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_63_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_63_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_63_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_63_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_63_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_63_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_63_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_63_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_63_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_63_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_63_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_63_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_63_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_63_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_63_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_63_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_63_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_63_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_63_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_63_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_63_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_63_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_63_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_63_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_63_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_63_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_63_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_63_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_63_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_63_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_63_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_63_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_63_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_63_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_63_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_63_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_63_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_63_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_63_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_63_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_63_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_63_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_63_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_63_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_63_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_63_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_63_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_63_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_63_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_63_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_63_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_63_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_63_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_63_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_63_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_63_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_63_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_63_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_63_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_63_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_63_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_63_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_63_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_63_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_63_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_63_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_63_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_63_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_63_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_63_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_63_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_63_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_63_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_63_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_63_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_63_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_63_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_63_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_63_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_63_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_63_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_63_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_63_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_63_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_63_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_63_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_63_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_63_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_63_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_63_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_63_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_63_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_63_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_63_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_63_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_63_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_63_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_63_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_63_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_63_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_63_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_63_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_63_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_63_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_63_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_63_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_63_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_63_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_63_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_63_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_63_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_63_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_63_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_63_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_63_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_63_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_63_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_63_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_63_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_63_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_63_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_63_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_63_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_63_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_63_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_63_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_63_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_63_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_63_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_63_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_63_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_63_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_63_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_63_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_63_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_63_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_63_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_63_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_63_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_63_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_63_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_63_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_63_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_63_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_63_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_63_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_63_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_63_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_63_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_63_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_63_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_63_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_63_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_63_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_63_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_63_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_63_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_63_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_63_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_63_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_63_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_63_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_63_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_63_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_63_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_63_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_63_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_63_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_63_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_63_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_63_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_63_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_63_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_63_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_63_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_63_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_63_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_63_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_63_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_63_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_63_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_63_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_63_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_63_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_63_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_63_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_63_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_63_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_63_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_63_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_63_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_63_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_63_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_63_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_63_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_63_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_63_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_63_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_63_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_63_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_63_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_63_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_63_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_63_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_63_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_63_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_63_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_63_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_63_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_63_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_63_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_63_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_63_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_63_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_63_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_63_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_63_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_63_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_63_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_63_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_63_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_63_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_63_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_63_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_63_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_63_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_63_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_63_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_63_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_63_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_63_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_63_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_63_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_63_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_63_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_63_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_63_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_63_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_63_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_63_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_63_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_63_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_63_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_63_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_63_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_63_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_63_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_63_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_63_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_63_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_63_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_63_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_63_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_63_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_63_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_63_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_63_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_63_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_63_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_63_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_63_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_63_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_63_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_63_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_63_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_63_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_63_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_63_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_63_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_63_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_63_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_63_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_63_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_63_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_63_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_63_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_63_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_63_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_63_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_63_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_63_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_63_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_63_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_63_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_63_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_63_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_63_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_63_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_63_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_63_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_63_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_63_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_63_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_63_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_63_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_63_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_63_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_63_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_63_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_63_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_63_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_63_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_63_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_63_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_63_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_63_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_63_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_63_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_63_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_63_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_63_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_63_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_63_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_63_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_63_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_63_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_63_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_63_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_63_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_63_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_63_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_63_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_63_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_63_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_63_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_63_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_63_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_63_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_64_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_64_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_64_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_64_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_64_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_64_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_64_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_64_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_64_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_64_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_64_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_64_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_64_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_64_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_64_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_64_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_64_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_64_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_64_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_64_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_64_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_64_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_64_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_64_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_64_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_64_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_64_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_64_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_64_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_64_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_64_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_64_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_64_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_64_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_64_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_64_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_64_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_64_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_64_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_64_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_64_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_64_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_64_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_64_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_64_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_64_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_64_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_64_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_64_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_64_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_64_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_64_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_64_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_64_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_64_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_64_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_64_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_64_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_64_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_64_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_64_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_64_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_64_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_64_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_64_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_64_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_64_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_64_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_64_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_64_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_64_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_64_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_64_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_64_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_64_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_64_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_64_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_64_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_64_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_64_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_64_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_64_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_64_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_64_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_64_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_64_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_64_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_64_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_64_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_64_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_64_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_64_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_64_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_64_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_64_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_64_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_64_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_64_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_64_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_64_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_64_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_64_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_64_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_64_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_64_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_64_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_64_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_64_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_64_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_64_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_64_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_64_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_64_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_64_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_64_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_64_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_64_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_64_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_64_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_64_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_64_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_64_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_64_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_64_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_64_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_64_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_64_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_64_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_64_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_64_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_64_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_64_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_64_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_64_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_64_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_64_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_64_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_64_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_64_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_64_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_64_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_64_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_64_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_64_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_64_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_64_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_64_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_64_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_64_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_64_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_64_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_64_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_64_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_64_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_64_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_64_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_64_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_64_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_64_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_64_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_64_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_64_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_64_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_64_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_64_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_64_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_64_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_64_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_64_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_64_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_64_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_64_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_64_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_64_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_64_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_64_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_64_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_64_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_64_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_64_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_64_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_64_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_64_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_64_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_64_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_64_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_64_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_64_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_64_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_64_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_64_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_64_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_64_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_64_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_64_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_64_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_64_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_64_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_64_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_64_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_64_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_64_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_64_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_64_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_64_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_64_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_64_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_64_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_64_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_64_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_64_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_64_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_64_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_64_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_64_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_64_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_64_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_64_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_64_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_64_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_64_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_64_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_64_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_64_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_64_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_64_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_64_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_64_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_64_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_64_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_64_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_64_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_64_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_64_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_64_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_64_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_64_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_64_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_64_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_64_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_64_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_64_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_64_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_64_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_64_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_64_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_64_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_64_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_64_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_64_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_64_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_64_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_64_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_64_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_64_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_64_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_64_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_64_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_64_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_64_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_64_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_64_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_64_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_64_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_64_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_64_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_64_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_64_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_64_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_64_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_64_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_64_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_64_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_64_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_64_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_64_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_64_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_64_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_64_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_64_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_64_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_64_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_64_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_64_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_64_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_64_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_64_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_64_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_64_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_64_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_64_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_64_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_64_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_64_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_64_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_64_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_64_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_64_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_64_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_64_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_64_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_64_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_64_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_64_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_64_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_64_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_64_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_64_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_64_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_64_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_64_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_64_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_64_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_64_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_64_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_64_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_64_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_64_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_64_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_64_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_64_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_64_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_64_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_64_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_64_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_64_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_64_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_64_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_64_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_64_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_64_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_64_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_64_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_64_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_64_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_64_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_64_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_64_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_64_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_64_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_64_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_64_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_65_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_65_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_65_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_65_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_65_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_65_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_65_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_65_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_65_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_65_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_65_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_65_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_65_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_65_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_65_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_65_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_65_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_65_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_65_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_65_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_65_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_65_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_65_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_65_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_65_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_65_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_65_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_65_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_65_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_65_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_65_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_65_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_65_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_65_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_65_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_65_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_65_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_65_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_65_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_65_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_65_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_65_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_65_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_65_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_65_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_65_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_65_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_65_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_65_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_65_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_65_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_65_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_65_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_65_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_65_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_65_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_65_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_65_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_65_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_65_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_65_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_65_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_65_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_65_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_65_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_65_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_65_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_65_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_65_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_65_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_65_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_65_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_65_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_65_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_65_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_65_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_65_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_65_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_65_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_65_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_65_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_65_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_65_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_65_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_65_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_65_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_65_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_65_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_65_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_65_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_65_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_65_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_65_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_65_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_65_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_65_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_65_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_65_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_65_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_65_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_65_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_65_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_65_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_65_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_65_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_65_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_65_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_65_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_65_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_65_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_65_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_65_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_65_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_65_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_65_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_65_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_65_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_65_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_65_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_65_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_65_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_65_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_65_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_65_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_65_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_65_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_65_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_65_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_65_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_65_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_65_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_65_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_65_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_65_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_65_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_65_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_65_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_65_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_65_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_65_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_65_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_65_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_65_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_65_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_65_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_65_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_65_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_65_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_65_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_65_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_65_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_65_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_65_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_65_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_65_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_65_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_65_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_65_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_65_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_65_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_65_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_65_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_65_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_65_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_65_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_65_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_65_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_65_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_65_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_65_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_65_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_65_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_65_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_65_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_65_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_65_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_65_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_65_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_65_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_65_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_65_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_65_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_65_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_65_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_65_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_65_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_65_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_65_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_65_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_65_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_65_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_65_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_65_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_65_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_65_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_65_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_65_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_65_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_65_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_65_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_65_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_65_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_65_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_65_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_65_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_65_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_65_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_65_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_65_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_65_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_65_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_65_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_65_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_65_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_65_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_65_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_65_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_65_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_65_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_65_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_65_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_65_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_65_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_65_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_65_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_65_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_65_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_65_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_65_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_65_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_65_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_65_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_65_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_65_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_65_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_65_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_65_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_65_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_65_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_65_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_65_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_65_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_65_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_65_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_65_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_65_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_65_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_65_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_65_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_65_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_65_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_65_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_65_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_65_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_65_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_65_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_65_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_65_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_65_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_65_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_65_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_65_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_65_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_65_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_65_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_65_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_65_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_65_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_65_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_65_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_65_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_65_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_65_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_65_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_65_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_65_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_65_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_65_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_65_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_65_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_65_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_65_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_65_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_65_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_65_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_65_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_65_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_65_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_65_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_65_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_65_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_65_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_65_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_65_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_65_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_65_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_65_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_65_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_65_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_65_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_65_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_65_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_65_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_65_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_65_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_65_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_65_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_65_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_65_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_65_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_65_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_65_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_65_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_65_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_65_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_65_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_65_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_65_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_65_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_65_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_65_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_65_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_65_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_65_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_65_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_65_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_65_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_65_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_65_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_65_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_65_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_65_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_65_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_65_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_65_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_65_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_65_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_65_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_65_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_65_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_65_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_65_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_66_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_66_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_66_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_66_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_66_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_66_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_66_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_66_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_66_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_66_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_66_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_66_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_66_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_66_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_66_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_66_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_66_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_66_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_66_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_66_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_66_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_66_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_66_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_66_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_66_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_66_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_66_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_66_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_66_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_66_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_66_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_66_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_66_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_66_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_66_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_66_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_66_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_66_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_66_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_66_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_66_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_66_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_66_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_66_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_66_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_66_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_66_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_66_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_66_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_66_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_66_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_66_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_66_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_66_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_66_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_66_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_66_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_66_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_66_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_66_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_66_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_66_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_66_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_66_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_66_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_66_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_66_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_66_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_66_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_66_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_66_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_66_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_66_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_66_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_66_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_66_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_66_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_66_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_66_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_66_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_66_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_66_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_66_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_66_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_66_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_66_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_66_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_66_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_66_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_66_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_66_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_66_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_66_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_66_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_66_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_66_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_66_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_66_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_66_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_66_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_66_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_66_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_66_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_66_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_66_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_66_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_66_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_66_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_66_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_66_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_66_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_66_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_66_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_66_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_66_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_66_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_66_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_66_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_66_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_66_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_66_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_66_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_66_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_66_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_66_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_66_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_66_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_66_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_66_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_66_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_66_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_66_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_66_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_66_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_66_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_66_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_66_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_66_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_66_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_66_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_66_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_66_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_66_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_66_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_66_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_66_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_66_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_66_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_66_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_66_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_66_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_66_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_66_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_66_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_66_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_66_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_66_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_66_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_66_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_66_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_66_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_66_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_66_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_66_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_66_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_66_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_66_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_66_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_66_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_66_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_66_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_66_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_66_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_66_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_66_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_66_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_66_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_66_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_66_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_66_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_66_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_66_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_66_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_66_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_66_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_66_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_66_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_66_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_66_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_66_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_66_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_66_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_66_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_66_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_66_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_66_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_66_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_66_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_66_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_66_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_66_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_66_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_66_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_66_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_66_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_66_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_66_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_66_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_66_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_66_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_66_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_66_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_66_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_66_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_66_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_66_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_66_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_66_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_66_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_66_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_66_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_66_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_66_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_66_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_66_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_66_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_66_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_66_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_66_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_66_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_66_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_66_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_66_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_66_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_66_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_66_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_66_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_66_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_66_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_66_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_66_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_66_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_66_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_66_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_66_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_66_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_66_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_66_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_66_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_66_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_66_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_66_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_66_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_66_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_66_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_66_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_66_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_66_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_66_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_66_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_66_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_66_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_66_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_66_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_66_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_66_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_66_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_66_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_66_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_66_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_66_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_66_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_66_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_66_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_66_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_66_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_66_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_66_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_66_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_66_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_66_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_66_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_66_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_66_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_66_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_66_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_66_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_66_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_66_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_66_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_66_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_66_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_66_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_66_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_66_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_66_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_66_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_66_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_66_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_66_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_66_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_66_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_66_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_66_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_66_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_66_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_66_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_66_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_66_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_66_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_66_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_66_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_66_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_66_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_66_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_66_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_66_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_66_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_66_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_66_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_66_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_66_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_66_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_66_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_66_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_66_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_66_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_66_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_66_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_66_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_66_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_66_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_66_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_66_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_66_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_66_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_66_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_66_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_66_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_66_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_66_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_66_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_67_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_67_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_67_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_67_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_67_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_67_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_67_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_67_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_67_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_67_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_67_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_67_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_67_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_67_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_67_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_67_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_67_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_67_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_67_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_67_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_67_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_67_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_67_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_67_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_67_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_67_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_67_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_67_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_67_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_67_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_67_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_67_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_67_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_67_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_67_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_67_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_67_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_67_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_67_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_67_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_67_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_67_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_67_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_67_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_67_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_67_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_67_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_67_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_67_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_67_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_67_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_67_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_67_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_67_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_67_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_67_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_67_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_67_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_67_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_67_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_67_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_67_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_67_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_67_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_67_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_67_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_67_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_67_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_67_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_67_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_67_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_67_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_67_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_67_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_67_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_67_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_67_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_67_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_67_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_67_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_67_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_67_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_67_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_67_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_67_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_67_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_67_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_67_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_67_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_67_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_67_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_67_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_67_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_67_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_67_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_67_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_67_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_67_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_67_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_67_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_67_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_67_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_67_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_67_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_67_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_67_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_67_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_67_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_67_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_67_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_67_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_67_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_67_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_67_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_67_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_67_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_67_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_67_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_67_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_67_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_67_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_67_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_67_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_67_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_67_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_67_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_67_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_67_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_67_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_67_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_67_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_67_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_67_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_67_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_67_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_67_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_67_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_67_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_67_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_67_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_67_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_67_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_67_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_67_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_67_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_67_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_67_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_67_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_67_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_67_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_67_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_67_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_67_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_67_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_67_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_67_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_67_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_67_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_67_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_67_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_67_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_67_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_67_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_67_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_67_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_67_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_67_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_67_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_67_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_67_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_67_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_67_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_67_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_67_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_67_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_67_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_67_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_67_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_67_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_67_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_67_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_67_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_67_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_67_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_67_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_67_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_67_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_67_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_67_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_67_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_67_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_67_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_67_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_67_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_67_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_67_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_67_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_67_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_67_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_67_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_67_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_67_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_67_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_67_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_67_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_67_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_67_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_67_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_67_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_67_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_67_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_67_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_67_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_67_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_67_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_67_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_67_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_67_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_67_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_67_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_67_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_67_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_67_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_67_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_67_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_67_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_67_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_67_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_67_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_67_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_67_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_67_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_67_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_67_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_67_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_67_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_67_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_67_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_67_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_67_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_67_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_67_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_67_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_67_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_67_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_67_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_67_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_67_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_67_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_67_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_67_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_67_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_67_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_67_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_67_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_67_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_67_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_67_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_67_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_67_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_67_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_67_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_67_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_67_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_67_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_67_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_67_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_67_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_67_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_67_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_67_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_67_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_67_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_67_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_67_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_67_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_67_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_67_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_67_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_67_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_67_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_67_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_67_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_67_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_67_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_67_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_67_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_67_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_67_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_67_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_67_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_67_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_67_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_67_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_67_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_67_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_67_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_67_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_67_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_67_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_67_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_67_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_67_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_67_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_67_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_67_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_67_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_67_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_67_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_67_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_67_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_67_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_67_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_67_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_67_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_67_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_67_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_67_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_67_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_67_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_67_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_67_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_67_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_67_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_67_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_67_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_67_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_67_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_67_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_67_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_67_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_67_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_67_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_67_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_67_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_67_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_67_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_67_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_67_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_67_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_67_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_67_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_68_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_68_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_68_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_68_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_68_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_68_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_68_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_68_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_68_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_68_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_68_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_68_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_68_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_68_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_68_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_68_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_68_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_68_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_68_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_68_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_68_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_68_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_68_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_68_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_68_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_68_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_68_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_68_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_68_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_68_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_68_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_68_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_68_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_68_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_68_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_68_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_68_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_68_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_68_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_68_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_68_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_68_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_68_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_68_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_68_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_68_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_68_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_68_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_68_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_68_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_68_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_68_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_68_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_68_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_68_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_68_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_68_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_68_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_68_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_68_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_68_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_68_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_68_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_68_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_68_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_68_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_68_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_68_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_68_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_68_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_68_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_68_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_68_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_68_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_68_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_68_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_68_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_68_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_68_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_68_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_68_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_68_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_68_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_68_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_68_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_68_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_68_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_68_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_68_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_68_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_68_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_68_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_68_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_68_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_68_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_68_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_68_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_68_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_68_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_68_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_68_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_68_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_68_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_68_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_68_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_68_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_68_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_68_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_68_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_68_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_68_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_68_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_68_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_68_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_68_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_68_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_68_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_68_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_68_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_68_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_68_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_68_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_68_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_68_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_68_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_68_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_68_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_68_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_68_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_68_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_68_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_68_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_68_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_68_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_68_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_68_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_68_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_68_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_68_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_68_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_68_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_68_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_68_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_68_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_68_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_68_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_68_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_68_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_68_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_68_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_68_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_68_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_68_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_68_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_68_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_68_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_68_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_68_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_68_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_68_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_68_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_68_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_68_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_68_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_68_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_68_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_68_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_68_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_68_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_68_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_68_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_68_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_68_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_68_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_68_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_68_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_68_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_68_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_68_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_68_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_68_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_68_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_68_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_68_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_68_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_68_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_68_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_68_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_68_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_68_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_68_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_68_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_68_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_68_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_68_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_68_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_68_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_68_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_68_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_68_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_68_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_68_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_68_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_68_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_68_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_68_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_68_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_68_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_68_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_68_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_68_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_68_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_68_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_68_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_68_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_68_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_68_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_68_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_68_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_68_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_68_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_68_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_68_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_68_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_68_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_68_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_68_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_68_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_68_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_68_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_68_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_68_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_68_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_68_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_68_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_68_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_68_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_68_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_68_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_68_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_68_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_68_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_68_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_68_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_68_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_68_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_68_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_68_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_68_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_68_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_68_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_68_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_68_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_68_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_68_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_68_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_68_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_68_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_68_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_68_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_68_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_68_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_68_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_68_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_68_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_68_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_68_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_68_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_68_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_68_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_68_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_68_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_68_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_68_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_68_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_68_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_68_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_68_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_68_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_68_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_68_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_68_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_68_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_68_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_68_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_68_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_68_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_68_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_68_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_68_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_68_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_68_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_68_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_68_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_68_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_68_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_68_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_68_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_68_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_68_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_68_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_68_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_68_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_68_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_68_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_68_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_68_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_68_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_68_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_68_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_68_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_68_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_68_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_68_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_68_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_68_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_68_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_68_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_68_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_68_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_68_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_68_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_68_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_68_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_68_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_68_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_68_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_68_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_68_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_68_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_68_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_68_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_68_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_68_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_68_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_68_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_68_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_68_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_68_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_68_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_68_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_68_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_69_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_69_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_69_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_69_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_69_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_69_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_69_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_69_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_69_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_69_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_69_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_69_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_69_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_69_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_69_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_69_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_69_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_69_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_69_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_69_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_69_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_69_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_69_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_69_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_69_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_69_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_69_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_69_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_69_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_69_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_69_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_69_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_69_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_69_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_69_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_69_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_69_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_69_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_69_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_69_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_69_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_69_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_69_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_69_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_69_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_69_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_69_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_69_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_69_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_69_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_69_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_69_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_69_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_69_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_69_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_69_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_69_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_69_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_69_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_69_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_69_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_69_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_69_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_69_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_69_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_69_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_69_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_69_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_69_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_69_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_69_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_69_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_69_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_69_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_69_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_69_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_69_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_69_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_69_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_69_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_69_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_69_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_69_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_69_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_69_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_69_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_69_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_69_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_69_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_69_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_69_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_69_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_69_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_69_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_69_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_69_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_69_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_69_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_69_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_69_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_69_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_69_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_69_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_69_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_69_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_69_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_69_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_69_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_69_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_69_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_69_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_69_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_69_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_69_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_69_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_69_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_69_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_69_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_69_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_69_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_69_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_69_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_69_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_69_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_69_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_69_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_69_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_69_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_69_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_69_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_69_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_69_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_69_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_69_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_69_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_69_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_69_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_69_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_69_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_69_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_69_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_69_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_69_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_69_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_69_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_69_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_69_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_69_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_69_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_69_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_69_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_69_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_69_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_69_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_69_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_69_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_69_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_69_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_69_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_69_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_69_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_69_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_69_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_69_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_69_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_69_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_69_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_69_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_69_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_69_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_69_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_69_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_69_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_69_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_69_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_69_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_69_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_69_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_69_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_69_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_69_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_69_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_69_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_69_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_69_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_69_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_69_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_69_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_69_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_69_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_69_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_69_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_69_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_69_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_69_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_69_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_69_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_69_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_69_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_69_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_69_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_69_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_69_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_69_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_69_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_69_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_69_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_69_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_69_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_69_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_69_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_69_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_69_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_69_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_69_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_69_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_69_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_69_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_69_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_69_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_69_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_69_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_69_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_69_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_69_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_69_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_69_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_69_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_69_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_69_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_69_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_69_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_69_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_69_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_69_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_69_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_69_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_69_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_69_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_69_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_69_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_69_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_69_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_69_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_69_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_69_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_69_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_69_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_69_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_69_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_69_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_69_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_69_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_69_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_69_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_69_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_69_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_69_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_69_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_69_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_69_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_69_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_69_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_69_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_69_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_69_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_69_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_69_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_69_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_69_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_69_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_69_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_69_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_69_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_69_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_69_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_69_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_69_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_69_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_69_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_69_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_69_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_69_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_69_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_69_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_69_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_69_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_69_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_69_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_69_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_69_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_69_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_69_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_69_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_69_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_69_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_69_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_69_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_69_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_69_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_69_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_69_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_69_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_69_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_69_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_69_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_69_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_69_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_69_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_69_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_69_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_69_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_69_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_69_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_69_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_69_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_69_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_69_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_69_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_69_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_69_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_69_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_69_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_69_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_69_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_69_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_69_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_69_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_69_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_69_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_69_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_69_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_69_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_69_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_69_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_69_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_69_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_69_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_69_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_69_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_69_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_69_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_70_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_70_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_70_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_70_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_70_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_70_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_70_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_70_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_70_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_70_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_70_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_70_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_70_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_70_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_70_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_70_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_70_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_70_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_70_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_70_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_70_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_70_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_70_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_70_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_70_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_70_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_70_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_70_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_70_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_70_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_70_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_70_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_70_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_70_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_70_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_70_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_70_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_70_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_70_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_70_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_70_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_70_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_70_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_70_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_70_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_70_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_70_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_70_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_70_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_70_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_70_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_70_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_70_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_70_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_70_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_70_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_70_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_70_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_70_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_70_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_70_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_70_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_70_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_70_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_70_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_70_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_70_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_70_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_70_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_70_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_70_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_70_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_70_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_70_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_70_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_70_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_70_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_70_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_70_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_70_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_70_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_70_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_70_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_70_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_70_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_70_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_70_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_70_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_70_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_70_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_70_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_70_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_70_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_70_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_70_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_70_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_70_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_70_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_70_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_70_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_70_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_70_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_70_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_70_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_70_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_70_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_70_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_70_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_70_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_70_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_70_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_70_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_70_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_70_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_70_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_70_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_70_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_70_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_70_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_70_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_70_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_70_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_70_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_70_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_70_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_70_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_70_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_70_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_70_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_70_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_70_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_70_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_70_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_70_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_70_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_70_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_70_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_70_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_70_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_70_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_70_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_70_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_70_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_70_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_70_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_70_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_70_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_70_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_70_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_70_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_70_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_70_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_70_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_70_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_70_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_70_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_70_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_70_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_70_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_70_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_70_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_70_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_70_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_70_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_70_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_70_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_70_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_70_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_70_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_70_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_70_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_70_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_70_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_70_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_70_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_70_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_70_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_70_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_70_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_70_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_70_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_70_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_70_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_70_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_70_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_70_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_70_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_70_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_70_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_70_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_70_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_70_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_70_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_70_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_70_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_70_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_70_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_70_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_70_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_70_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_70_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_70_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_70_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_70_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_70_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_70_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_70_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_70_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_70_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_70_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_70_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_70_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_70_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_70_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_70_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_70_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_70_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_70_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_70_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_70_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_70_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_70_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_70_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_70_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_70_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_70_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_70_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_70_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_70_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_70_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_70_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_70_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_70_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_70_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_70_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_70_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_70_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_70_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_70_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_70_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_70_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_70_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_70_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_70_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_70_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_70_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_70_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_70_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_70_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_70_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_70_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_70_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_70_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_70_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_70_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_70_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_70_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_70_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_70_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_70_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_70_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_70_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_70_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_70_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_70_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_70_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_70_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_70_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_70_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_70_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_70_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_70_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_70_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_70_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_70_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_70_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_70_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_70_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_70_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_70_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_70_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_70_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_70_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_70_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_70_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_70_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_70_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_70_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_70_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_70_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_70_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_70_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_70_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_70_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_70_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_70_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_70_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_70_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_70_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_70_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_70_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_70_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_70_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_70_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_70_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_70_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_70_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_70_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_70_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_70_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_70_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_70_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_70_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_70_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_70_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_70_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_70_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_70_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_70_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_70_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_70_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_70_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_70_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_70_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_70_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_70_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_70_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_70_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_70_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_70_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_70_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_70_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_70_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_70_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_70_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_70_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_70_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_70_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_70_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_70_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_70_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_70_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_71_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_71_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_71_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_71_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_71_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_71_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_71_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_71_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_71_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_71_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_71_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_71_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_71_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_71_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_71_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_71_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_71_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_71_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_71_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_71_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_71_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_71_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_71_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_71_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_71_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_71_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_71_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_71_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_71_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_71_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_71_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_71_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_71_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_71_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_71_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_71_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_71_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_71_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_71_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_71_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_71_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_71_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_71_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_71_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_71_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_71_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_71_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_71_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_71_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_71_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_71_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_71_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_71_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_71_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_71_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_71_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_71_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_71_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_71_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_71_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_71_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_71_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_71_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_71_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_71_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_71_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_71_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_71_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_71_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_71_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_71_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_71_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_71_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_71_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_71_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_71_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_71_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_71_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_71_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_71_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_71_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_71_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_71_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_71_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_71_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_71_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_71_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_71_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_71_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_71_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_71_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_71_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_71_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_71_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_71_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_71_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_71_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_71_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_71_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_71_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_71_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_71_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_71_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_71_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_71_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_71_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_71_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_71_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_71_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_71_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_71_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_71_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_71_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_71_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_71_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_71_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_71_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_71_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_71_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_71_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_71_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_71_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_71_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_71_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_71_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_71_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_71_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_71_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_71_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_71_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_71_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_71_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_71_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_71_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_71_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_71_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_71_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_71_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_71_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_71_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_71_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_71_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_71_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_71_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_71_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_71_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_71_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_71_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_71_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_71_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_71_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_71_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_71_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_71_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_71_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_71_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_71_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_71_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_71_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_71_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_71_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_71_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_71_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_71_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_71_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_71_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_71_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_71_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_71_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_71_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_71_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_71_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_71_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_71_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_71_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_71_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_71_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_71_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_71_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_71_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_71_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_71_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_71_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_71_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_71_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_71_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_71_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_71_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_71_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_71_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_71_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_71_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_71_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_71_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_71_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_71_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_71_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_71_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_71_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_71_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_71_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_71_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_71_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_71_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_71_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_71_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_71_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_71_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_71_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_71_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_71_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_71_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_71_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_71_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_71_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_71_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_71_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_71_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_71_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_71_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_71_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_71_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_71_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_71_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_71_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_71_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_71_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_71_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_71_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_71_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_71_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_71_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_71_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_71_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_71_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_71_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_71_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_71_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_71_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_71_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_71_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_71_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_71_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_71_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_71_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_71_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_71_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_71_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_71_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_71_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_71_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_71_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_71_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_71_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_71_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_71_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_71_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_71_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_71_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_71_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_71_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_71_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_71_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_71_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_71_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_71_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_71_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_71_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_71_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_71_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_71_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_71_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_71_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_71_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_71_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_71_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_71_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_71_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_71_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_71_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_71_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_71_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_71_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_71_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_71_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_71_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_71_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_71_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_71_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_71_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_71_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_71_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_71_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_71_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_71_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_71_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_71_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_71_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_71_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_71_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_71_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_71_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_71_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_71_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_71_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_71_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_71_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_71_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_71_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_71_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_71_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_71_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_71_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_71_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_71_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_71_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_71_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_71_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_71_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_71_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_71_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_71_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_71_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_71_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_71_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_71_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_71_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_71_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_71_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_71_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_71_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_71_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_71_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_71_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_71_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_71_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_71_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_71_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_71_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_71_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_71_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_71_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_72_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_72_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_72_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_72_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_72_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_72_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_72_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_72_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_72_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_72_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_72_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_72_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_72_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_72_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_72_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_72_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_72_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_72_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_72_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_72_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_72_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_72_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_72_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_72_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_72_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_72_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_72_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_72_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_72_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_72_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_72_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_72_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_72_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_72_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_72_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_72_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_72_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_72_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_72_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_72_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_72_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_72_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_72_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_72_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_72_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_72_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_72_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_72_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_72_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_72_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_72_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_72_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_72_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_72_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_72_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_72_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_72_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_72_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_72_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_72_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_72_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_72_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_72_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_72_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_72_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_72_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_72_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_72_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_72_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_72_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_72_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_72_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_72_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_72_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_72_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_72_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_72_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_72_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_72_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_72_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_72_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_72_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_72_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_72_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_72_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_72_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_72_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_72_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_72_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_72_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_72_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_72_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_72_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_72_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_72_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_72_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_72_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_72_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_72_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_72_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_72_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_72_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_72_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_72_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_72_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_72_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_72_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_72_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_72_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_72_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_72_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_72_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_72_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_72_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_72_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_72_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_72_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_72_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_72_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_72_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_72_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_72_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_72_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_72_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_72_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_72_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_72_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_72_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_72_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_72_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_72_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_72_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_72_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_72_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_72_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_72_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_72_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_72_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_72_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_72_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_72_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_72_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_72_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_72_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_72_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_72_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_72_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_72_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_72_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_72_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_72_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_72_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_72_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_72_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_72_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_72_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_72_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_72_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_72_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_72_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_72_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_72_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_72_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_72_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_72_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_72_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_72_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_72_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_72_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_72_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_72_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_72_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_72_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_72_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_72_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_72_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_72_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_72_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_72_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_72_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_72_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_72_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_72_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_72_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_72_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_72_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_72_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_72_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_72_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_72_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_72_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_72_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_72_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_72_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_72_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_72_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_72_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_72_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_72_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_72_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_72_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_72_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_72_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_72_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_72_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_72_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_72_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_72_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_72_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_72_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_72_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_72_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_72_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_72_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_72_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_72_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_72_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_72_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_72_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_72_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_72_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_72_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_72_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_72_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_72_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_72_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_72_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_72_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_72_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_72_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_72_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_72_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_72_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_72_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_72_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_72_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_72_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_72_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_72_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_72_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_72_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_72_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_72_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_72_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_72_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_72_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_72_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_72_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_72_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_72_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_72_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_72_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_72_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_72_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_72_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_72_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_72_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_72_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_72_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_72_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_72_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_72_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_72_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_72_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_72_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_72_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_72_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_72_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_72_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_72_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_72_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_72_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_72_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_72_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_72_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_72_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_72_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_72_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_72_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_72_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_72_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_72_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_72_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_72_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_72_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_72_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_72_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_72_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_72_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_72_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_72_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_72_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_72_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_72_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_72_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_72_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_72_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_72_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_72_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_72_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_72_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_72_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_72_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_72_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_72_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_72_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_72_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_72_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_72_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_72_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_72_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_72_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_72_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_72_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_72_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_72_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_72_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_72_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_72_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_72_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_72_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_72_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_72_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_72_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_72_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_72_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_72_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_72_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_72_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_72_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_72_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_72_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_72_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_72_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_72_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_72_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_72_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_72_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_72_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_72_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_72_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_72_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_73_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_73_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_73_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_73_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_73_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_73_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_73_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_73_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_73_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_73_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_73_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_73_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_73_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_73_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_73_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_73_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_73_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_73_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_73_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_73_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_73_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_73_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_73_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_73_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_73_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_73_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_73_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_73_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_73_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_73_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_73_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_73_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_73_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_73_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_73_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_73_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_73_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_73_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_73_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_73_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_73_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_73_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_73_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_73_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_73_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_73_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_73_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_73_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_73_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_73_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_73_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_73_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_73_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_73_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_73_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_73_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_73_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_73_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_73_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_73_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_73_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_73_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_73_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_73_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_73_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_73_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_73_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_73_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_73_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_73_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_73_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_73_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_73_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_73_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_73_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_73_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_73_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_73_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_73_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_73_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_73_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_73_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_73_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_73_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_73_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_73_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_73_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_73_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_73_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_73_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_73_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_73_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_73_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_73_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_73_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_73_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_73_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_73_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_73_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_73_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_73_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_73_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_73_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_73_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_73_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_73_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_73_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_73_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_73_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_73_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_73_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_73_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_73_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_73_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_73_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_73_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_73_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_73_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_73_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_73_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_73_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_73_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_73_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_73_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_73_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_73_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_73_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_73_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_73_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_73_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_73_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_73_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_73_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_73_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_73_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_73_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_73_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_73_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_73_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_73_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_73_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_73_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_73_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_73_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_73_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_73_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_73_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_73_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_73_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_73_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_73_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_73_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_73_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_73_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_73_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_73_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_73_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_73_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_73_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_73_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_73_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_73_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_73_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_73_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_73_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_73_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_73_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_73_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_73_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_73_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_73_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_73_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_73_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_73_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_73_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_73_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_73_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_73_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_73_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_73_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_73_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_73_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_73_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_73_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_73_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_73_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_73_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_73_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_73_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_73_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_73_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_73_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_73_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_73_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_73_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_73_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_73_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_73_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_73_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_73_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_73_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_73_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_73_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_73_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_73_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_73_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_73_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_73_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_73_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_73_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_73_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_73_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_73_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_73_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_73_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_73_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_73_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_73_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_73_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_73_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_73_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_73_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_73_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_73_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_73_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_73_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_73_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_73_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_73_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_73_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_73_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_73_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_73_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_73_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_73_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_73_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_73_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_73_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_73_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_73_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_73_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_73_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_73_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_73_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_73_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_73_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_73_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_73_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_73_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_73_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_73_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_73_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_73_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_73_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_73_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_73_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_73_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_73_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_73_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_73_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_73_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_73_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_73_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_73_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_73_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_73_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_73_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_73_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_73_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_73_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_73_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_73_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_73_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_73_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_73_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_73_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_73_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_73_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_73_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_73_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_73_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_73_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_73_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_73_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_73_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_73_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_73_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_73_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_73_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_73_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_73_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_73_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_73_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_73_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_73_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_73_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_73_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_73_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_73_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_73_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_73_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_73_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_73_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_73_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_73_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_73_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_73_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_73_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_73_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_73_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_73_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_73_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_73_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_73_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_73_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_73_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_73_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_73_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_73_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_73_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_73_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_73_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_73_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_73_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_73_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_73_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_73_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_73_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_73_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_73_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_73_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_73_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_73_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_73_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_73_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_73_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_73_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_73_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_73_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_73_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_73_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_73_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_74_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_74_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_74_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_74_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_74_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_74_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_74_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_74_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_74_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_74_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_74_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_74_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_74_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_74_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_74_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_74_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_74_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_74_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_74_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_74_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_74_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_74_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_74_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_74_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_74_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_74_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_74_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_74_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_74_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_74_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_74_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_74_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_74_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_74_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_74_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_74_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_74_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_74_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_74_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_74_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_74_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_74_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_74_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_74_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_74_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_74_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_74_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_74_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_74_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_74_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_74_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_74_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_74_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_74_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_74_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_74_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_74_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_74_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_74_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_74_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_74_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_74_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_74_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_74_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_74_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_74_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_74_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_74_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_74_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_74_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_74_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_74_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_74_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_74_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_74_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_74_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_74_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_74_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_74_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_74_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_74_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_74_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_74_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_74_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_74_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_74_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_74_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_74_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_74_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_74_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_74_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_74_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_74_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_74_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_74_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_74_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_74_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_74_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_74_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_74_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_74_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_74_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_74_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_74_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_74_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_74_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_74_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_74_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_74_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_74_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_74_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_74_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_74_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_74_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_74_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_74_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_74_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_74_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_74_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_74_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_74_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_74_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_74_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_74_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_74_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_74_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_74_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_74_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_74_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_74_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_74_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_74_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_74_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_74_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_74_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_74_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_74_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_74_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_74_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_74_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_74_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_74_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_74_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_74_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_74_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_74_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_74_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_74_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_74_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_74_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_74_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_74_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_74_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_74_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_74_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_74_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_74_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_74_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_74_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_74_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_74_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_74_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_74_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_74_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_74_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_74_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_74_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_74_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_74_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_74_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_74_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_74_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_74_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_74_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_74_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_74_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_74_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_74_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_74_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_74_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_74_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_74_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_74_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_74_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_74_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_74_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_74_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_74_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_74_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_74_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_74_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_74_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_74_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_74_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_74_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_74_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_74_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_74_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_74_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_74_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_74_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_74_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_74_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_74_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_74_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_74_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_74_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_74_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_74_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_74_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_74_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_74_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_74_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_74_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_74_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_74_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_74_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_74_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_74_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_74_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_74_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_74_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_74_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_74_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_74_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_74_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_74_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_74_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_74_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_74_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_74_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_74_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_74_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_74_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_74_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_74_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_74_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_74_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_74_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_74_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_74_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_74_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_74_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_74_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_74_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_74_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_74_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_74_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_74_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_74_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_74_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_74_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_74_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_74_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_74_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_74_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_74_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_74_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_74_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_74_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_74_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_74_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_74_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_74_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_74_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_74_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_74_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_74_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_74_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_74_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_74_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_74_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_74_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_74_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_74_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_74_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_74_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_74_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_74_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_74_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_74_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_74_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_74_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_74_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_74_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_74_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_74_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_74_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_74_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_74_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_74_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_74_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_74_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_74_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_74_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_74_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_74_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_74_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_74_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_74_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_74_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_74_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_74_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_74_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_74_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_74_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_74_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_74_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_74_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_74_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_74_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_74_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_74_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_74_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_74_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_74_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_74_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_74_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_74_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_74_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_74_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_74_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_74_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_74_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_74_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_74_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_74_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_74_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_74_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_74_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_74_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_74_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_74_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_74_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_74_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_74_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_74_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_74_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_74_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_74_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_74_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_74_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_75_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_75_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_75_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_75_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_75_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_75_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_75_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_75_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_75_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_75_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_75_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_75_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_75_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_75_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_75_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_75_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_75_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_75_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_75_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_75_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_75_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_75_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_75_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_75_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_75_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_75_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_75_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_75_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_75_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_75_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_75_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_75_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_75_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_75_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_75_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_75_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_75_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_75_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_75_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_75_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_75_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_75_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_75_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_75_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_75_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_75_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_75_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_75_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_75_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_75_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_75_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_75_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_75_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_75_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_75_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_75_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_75_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_75_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_75_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_75_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_75_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_75_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_75_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_75_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_75_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_75_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_75_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_75_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_75_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_75_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_75_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_75_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_75_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_75_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_75_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_75_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_75_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_75_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_75_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_75_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_75_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_75_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_75_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_75_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_75_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_75_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_75_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_75_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_75_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_75_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_75_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_75_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_75_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_75_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_75_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_75_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_75_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_75_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_75_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_75_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_75_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_75_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_75_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_75_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_75_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_75_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_75_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_75_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_75_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_75_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_75_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_75_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_75_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_75_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_75_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_75_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_75_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_75_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_75_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_75_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_75_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_75_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_75_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_75_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_75_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_75_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_75_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_75_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_75_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_75_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_75_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_75_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_75_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_75_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_75_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_75_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_75_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_75_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_75_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_75_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_75_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_75_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_75_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_75_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_75_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_75_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_75_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_75_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_75_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_75_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_75_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_75_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_75_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_75_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_75_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_75_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_75_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_75_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_75_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_75_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_75_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_75_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_75_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_75_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_75_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_75_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_75_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_75_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_75_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_75_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_75_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_75_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_75_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_75_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_75_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_75_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_75_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_75_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_75_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_75_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_75_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_75_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_75_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_75_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_75_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_75_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_75_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_75_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_75_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_75_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_75_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_75_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_75_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_75_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_75_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_75_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_75_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_75_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_75_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_75_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_75_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_75_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_75_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_75_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_75_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_75_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_75_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_75_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_75_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_75_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_75_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_75_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_75_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_75_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_75_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_75_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_75_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_75_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_75_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_75_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_75_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_75_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_75_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_75_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_75_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_75_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_75_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_75_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_75_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_75_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_75_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_75_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_75_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_75_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_75_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_75_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_75_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_75_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_75_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_75_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_75_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_75_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_75_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_75_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_75_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_75_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_75_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_75_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_75_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_75_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_75_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_75_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_75_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_75_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_75_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_75_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_75_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_75_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_75_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_75_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_75_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_75_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_75_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_75_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_75_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_75_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_75_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_75_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_75_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_75_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_75_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_75_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_75_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_75_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_75_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_75_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_75_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_75_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_75_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_75_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_75_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_75_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_75_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_75_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_75_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_75_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_75_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_75_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_75_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_75_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_75_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_75_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_75_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_75_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_75_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_75_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_75_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_75_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_75_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_75_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_75_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_75_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_75_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_75_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_75_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_75_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_75_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_75_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_75_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_75_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_75_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_75_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_75_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_75_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_75_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_75_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_75_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_75_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_75_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_75_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_75_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_75_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_75_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_75_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_75_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_75_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_75_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_75_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_75_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_75_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_75_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_75_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_75_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_75_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_75_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_75_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_75_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_75_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_75_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_75_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_75_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_75_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_76_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_76_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_76_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_76_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_76_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_76_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_76_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_76_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_76_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_76_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_76_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_76_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_76_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_76_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_76_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_76_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_76_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_76_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_76_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_76_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_76_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_76_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_76_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_76_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_76_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_76_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_76_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_76_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_76_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_76_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_76_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_76_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_76_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_76_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_76_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_76_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_76_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_76_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_76_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_76_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_76_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_76_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_76_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_76_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_76_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_76_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_76_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_76_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_76_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_76_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_76_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_76_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_76_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_76_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_76_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_76_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_76_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_76_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_76_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_76_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_76_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_76_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_76_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_76_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_76_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_76_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_76_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_76_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_76_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_76_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_76_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_76_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_76_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_76_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_76_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_76_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_76_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_76_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_76_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_76_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_76_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_76_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_76_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_76_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_76_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_76_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_76_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_76_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_76_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_76_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_76_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_76_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_76_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_76_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_76_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_76_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_76_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_76_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_76_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_76_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_76_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_76_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_76_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_76_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_76_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_76_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_76_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_76_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_76_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_76_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_76_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_76_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_76_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_76_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_76_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_76_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_76_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_76_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_76_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_76_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_76_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_76_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_76_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_76_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_76_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_76_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_76_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_76_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_76_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_76_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_76_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_76_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_76_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_76_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_76_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_76_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_76_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_76_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_76_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_76_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_76_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_76_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_76_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_76_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_76_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_76_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_76_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_76_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_76_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_76_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_76_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_76_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_76_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_76_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_76_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_76_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_76_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_76_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_76_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_76_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_76_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_76_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_76_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_76_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_76_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_76_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_76_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_76_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_76_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_76_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_76_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_76_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_76_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_76_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_76_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_76_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_76_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_76_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_76_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_76_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_76_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_76_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_76_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_76_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_76_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_76_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_76_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_76_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_76_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_76_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_76_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_76_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_76_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_76_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_76_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_76_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_76_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_76_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_76_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_76_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_76_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_76_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_76_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_76_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_76_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_76_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_76_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_76_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_76_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_76_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_76_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_76_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_76_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_76_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_76_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_76_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_76_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_76_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_76_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_76_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_76_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_76_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_76_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_76_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_76_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_76_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_76_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_76_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_76_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_76_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_76_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_76_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_76_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_76_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_76_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_76_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_76_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_76_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_76_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_76_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_76_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_76_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_76_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_76_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_76_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_76_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_76_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_76_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_76_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_76_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_76_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_76_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_76_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_76_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_76_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_76_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_76_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_76_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_76_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_76_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_76_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_76_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_76_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_76_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_76_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_76_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_76_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_76_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_76_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_76_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_76_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_76_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_76_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_76_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_76_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_76_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_76_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_76_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_76_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_76_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_76_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_76_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_76_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_76_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_76_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_76_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_76_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_76_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_76_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_76_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_76_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_76_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_76_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_76_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_76_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_76_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_76_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_76_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_76_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_76_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_76_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_76_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_76_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_76_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_76_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_76_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_76_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_76_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_76_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_76_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_76_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_76_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_76_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_76_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_76_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_76_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_76_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_76_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_76_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_76_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_76_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_76_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_76_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_76_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_76_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_76_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_76_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_76_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_76_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_76_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_76_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_76_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_76_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_76_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_76_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_76_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_76_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_76_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_76_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_76_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_76_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_76_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_77_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_77_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_77_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_77_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_77_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_77_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_77_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_77_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_77_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_77_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_77_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_77_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_77_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_77_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_77_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_77_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_77_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_77_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_77_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_77_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_77_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_77_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_77_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_77_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_77_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_77_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_77_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_77_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_77_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_77_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_77_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_77_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_77_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_77_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_77_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_77_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_77_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_77_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_77_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_77_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_77_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_77_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_77_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_77_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_77_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_77_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_77_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_77_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_77_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_77_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_77_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_77_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_77_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_77_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_77_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_77_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_77_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_77_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_77_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_77_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_77_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_77_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_77_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_77_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_77_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_77_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_77_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_77_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_77_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_77_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_77_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_77_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_77_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_77_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_77_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_77_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_77_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_77_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_77_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_77_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_77_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_77_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_77_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_77_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_77_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_77_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_77_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_77_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_77_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_77_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_77_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_77_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_77_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_77_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_77_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_77_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_77_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_77_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_77_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_77_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_77_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_77_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_77_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_77_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_77_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_77_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_77_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_77_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_77_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_77_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_77_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_77_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_77_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_77_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_77_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_77_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_77_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_77_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_77_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_77_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_77_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_77_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_77_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_77_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_77_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_77_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_77_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_77_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_77_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_77_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_77_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_77_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_77_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_77_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_77_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_77_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_77_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_77_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_77_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_77_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_77_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_77_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_77_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_77_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_77_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_77_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_77_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_77_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_77_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_77_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_77_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_77_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_77_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_77_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_77_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_77_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_77_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_77_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_77_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_77_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_77_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_77_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_77_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_77_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_77_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_77_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_77_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_77_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_77_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_77_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_77_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_77_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_77_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_77_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_77_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_77_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_77_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_77_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_77_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_77_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_77_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_77_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_77_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_77_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_77_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_77_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_77_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_77_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_77_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_77_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_77_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_77_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_77_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_77_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_77_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_77_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_77_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_77_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_77_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_77_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_77_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_77_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_77_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_77_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_77_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_77_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_77_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_77_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_77_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_77_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_77_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_77_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_77_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_77_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_77_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_77_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_77_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_77_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_77_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_77_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_77_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_77_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_77_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_77_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_77_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_77_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_77_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_77_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_77_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_77_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_77_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_77_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_77_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_77_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_77_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_77_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_77_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_77_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_77_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_77_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_77_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_77_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_77_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_77_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_77_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_77_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_77_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_77_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_77_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_77_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_77_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_77_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_77_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_77_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_77_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_77_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_77_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_77_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_77_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_77_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_77_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_77_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_77_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_77_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_77_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_77_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_77_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_77_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_77_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_77_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_77_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_77_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_77_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_77_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_77_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_77_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_77_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_77_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_77_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_77_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_77_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_77_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_77_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_77_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_77_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_77_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_77_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_77_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_77_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_77_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_77_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_77_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_77_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_77_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_77_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_77_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_77_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_77_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_77_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_77_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_77_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_77_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_77_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_77_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_77_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_77_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_77_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_77_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_77_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_77_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_77_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_77_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_77_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_77_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_77_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_77_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_77_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_77_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_77_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_77_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_77_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_77_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_77_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_77_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_77_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_77_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_77_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_77_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_77_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_77_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_77_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_77_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_77_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_77_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_77_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_77_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_77_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_77_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_77_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_77_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_77_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_77_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_78_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_78_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_78_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_78_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_78_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_78_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_78_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_78_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_78_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_78_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_78_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_78_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_78_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_78_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_78_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_78_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_78_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_78_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_78_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_78_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_78_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_78_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_78_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_78_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_78_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_78_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_78_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_78_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_78_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_78_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_78_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_78_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_78_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_78_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_78_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_78_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_78_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_78_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_78_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_78_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_78_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_78_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_78_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_78_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_78_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_78_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_78_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_78_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_78_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_78_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_78_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_78_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_78_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_78_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_78_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_78_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_78_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_78_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_78_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_78_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_78_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_78_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_78_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_78_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_78_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_78_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_78_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_78_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_78_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_78_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_78_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_78_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_78_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_78_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_78_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_78_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_78_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_78_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_78_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_78_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_78_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_78_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_78_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_78_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_78_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_78_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_78_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_78_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_78_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_78_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_78_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_78_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_78_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_78_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_78_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_78_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_78_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_78_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_78_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_78_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_78_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_78_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_78_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_78_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_78_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_78_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_78_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_78_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_78_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_78_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_78_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_78_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_78_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_78_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_78_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_78_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_78_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_78_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_78_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_78_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_78_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_78_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_78_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_78_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_78_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_78_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_78_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_78_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_78_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_78_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_78_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_78_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_78_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_78_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_78_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_78_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_78_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_78_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_78_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_78_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_78_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_78_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_78_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_78_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_78_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_78_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_78_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_78_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_78_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_78_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_78_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_78_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_78_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_78_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_78_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_78_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_78_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_78_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_78_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_78_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_78_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_78_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_78_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_78_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_78_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_78_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_78_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_78_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_78_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_78_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_78_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_78_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_78_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_78_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_78_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_78_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_78_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_78_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_78_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_78_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_78_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_78_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_78_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_78_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_78_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_78_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_78_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_78_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_78_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_78_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_78_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_78_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_78_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_78_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_78_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_78_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_78_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_78_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_78_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_78_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_78_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_78_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_78_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_78_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_78_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_78_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_78_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_78_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_78_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_78_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_78_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_78_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_78_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_78_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_78_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_78_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_78_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_78_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_78_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_78_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_78_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_78_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_78_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_78_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_78_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_78_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_78_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_78_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_78_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_78_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_78_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_78_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_78_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_78_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_78_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_78_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_78_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_78_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_78_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_78_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_78_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_78_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_78_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_78_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_78_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_78_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_78_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_78_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_78_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_78_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_78_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_78_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_78_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_78_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_78_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_78_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_78_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_78_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_78_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_78_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_78_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_78_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_78_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_78_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_78_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_78_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_78_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_78_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_78_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_78_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_78_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_78_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_78_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_78_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_78_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_78_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_78_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_78_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_78_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_78_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_78_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_78_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_78_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_78_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_78_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_78_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_78_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_78_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_78_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_78_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_78_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_78_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_78_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_78_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_78_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_78_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_78_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_78_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_78_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_78_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_78_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_78_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_78_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_78_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_78_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_78_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_78_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_78_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_78_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_78_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_78_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_78_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_78_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_78_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_78_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_78_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_78_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_78_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_78_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_78_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_78_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_78_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_78_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_78_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_78_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_78_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_78_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_78_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_78_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_78_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_78_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_78_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_78_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_78_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_78_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_78_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_78_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_78_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_78_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_78_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_78_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_78_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_79_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_79_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_79_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_79_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_79_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_79_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_79_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_79_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_79_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_79_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_79_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_79_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_79_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_79_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_79_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_79_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_79_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_79_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_79_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_79_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_79_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_79_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_79_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_79_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_79_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_79_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_79_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_79_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_79_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_79_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_79_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_79_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_79_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_79_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_79_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_79_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_79_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_79_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_79_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_79_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_79_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_79_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_79_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_79_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_79_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_79_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_79_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_79_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_79_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_79_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_79_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_79_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_79_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_79_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_79_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_79_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_79_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_79_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_79_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_79_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_79_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_79_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_79_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_79_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_79_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_79_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_79_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_79_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_79_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_79_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_79_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_79_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_79_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_79_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_79_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_79_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_79_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_79_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_79_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_79_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_79_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_79_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_79_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_79_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_79_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_79_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_79_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_79_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_79_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_79_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_79_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_79_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_79_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_79_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_79_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_79_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_79_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_79_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_79_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_79_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_79_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_79_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_79_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_79_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_79_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_79_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_79_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_79_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_79_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_79_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_79_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_79_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_79_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_79_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_79_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_79_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_79_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_79_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_79_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_79_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_79_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_79_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_79_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_79_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_79_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_79_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_79_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_79_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_79_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_79_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_79_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_79_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_79_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_79_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_79_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_79_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_79_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_79_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_79_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_79_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_79_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_79_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_79_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_79_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_79_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_79_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_79_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_79_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_79_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_79_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_79_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_79_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_79_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_79_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_79_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_79_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_79_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_79_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_79_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_79_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_79_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_79_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_79_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_79_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_79_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_79_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_79_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_79_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_79_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_79_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_79_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_79_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_79_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_79_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_79_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_79_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_79_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_79_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_79_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_79_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_79_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_79_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_79_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_79_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_79_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_79_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_79_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_79_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_79_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_79_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_79_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_79_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_79_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_79_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_79_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_79_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_79_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_79_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_79_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_79_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_79_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_79_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_79_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_79_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_79_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_79_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_79_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_79_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_79_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_79_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_79_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_79_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_79_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_79_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_79_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_79_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_79_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_79_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_79_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_79_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_79_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_79_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_79_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_79_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_79_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_79_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_79_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_79_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_79_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_79_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_79_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_79_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_79_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_79_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_79_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_79_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_79_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_79_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_79_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_79_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_79_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_79_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_79_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_79_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_79_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_79_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_79_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_79_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_79_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_79_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_79_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_79_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_79_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_79_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_79_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_79_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_79_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_79_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_79_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_79_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_79_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_79_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_79_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_79_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_79_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_79_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_79_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_79_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_79_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_79_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_79_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_79_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_79_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_79_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_79_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_79_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_79_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_79_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_79_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_79_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_79_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_79_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_79_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_79_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_79_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_79_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_79_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_79_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_79_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_79_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_79_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_79_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_79_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_79_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_79_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_79_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_79_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_79_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_79_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_79_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_79_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_79_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_79_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_79_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_79_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_79_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_79_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_79_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_79_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_79_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_79_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_79_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_79_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_79_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_79_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_79_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_79_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_79_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_79_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_79_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_79_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_79_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_79_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_79_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_79_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_79_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_79_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_79_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_79_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_79_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_79_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_79_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_79_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_79_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_79_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_79_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_79_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_79_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_79_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_79_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_79_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_79_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_80_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_80_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_80_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_80_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_80_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_80_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_80_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_80_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_80_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_80_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_80_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_80_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_80_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_80_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_80_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_80_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_80_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_80_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_80_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_80_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_80_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_80_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_80_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_80_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_80_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_80_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_80_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_80_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_80_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_80_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_80_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_80_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_80_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_80_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_80_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_80_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_80_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_80_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_80_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_80_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_80_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_80_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_80_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_80_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_80_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_80_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_80_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_80_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_80_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_80_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_80_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_80_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_80_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_80_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_80_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_80_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_80_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_80_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_80_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_80_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_80_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_80_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_80_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_80_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_80_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_80_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_80_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_80_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_80_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_80_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_80_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_80_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_80_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_80_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_80_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_80_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_80_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_80_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_80_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_80_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_80_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_80_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_80_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_80_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_80_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_80_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_80_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_80_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_80_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_80_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_80_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_80_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_80_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_80_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_80_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_80_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_80_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_80_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_80_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_80_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_80_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_80_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_80_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_80_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_80_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_80_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_80_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_80_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_80_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_80_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_80_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_80_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_80_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_80_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_80_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_80_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_80_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_80_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_80_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_80_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_80_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_80_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_80_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_80_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_80_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_80_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_80_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_80_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_80_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_80_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_80_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_80_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_80_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_80_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_80_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_80_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_80_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_80_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_80_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_80_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_80_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_80_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_80_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_80_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_80_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_80_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_80_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_80_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_80_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_80_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_80_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_80_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_80_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_80_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_80_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_80_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_80_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_80_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_80_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_80_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_80_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_80_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_80_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_80_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_80_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_80_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_80_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_80_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_80_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_80_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_80_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_80_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_80_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_80_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_80_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_80_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_80_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_80_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_80_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_80_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_80_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_80_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_80_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_80_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_80_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_80_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_80_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_80_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_80_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_80_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_80_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_80_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_80_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_80_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_80_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_80_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_80_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_80_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_80_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_80_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_80_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_80_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_80_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_80_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_80_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_80_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_80_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_80_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_80_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_80_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_80_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_80_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_80_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_80_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_80_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_80_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_80_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_80_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_80_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_80_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_80_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_80_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_80_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_80_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_80_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_80_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_80_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_80_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_80_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_80_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_80_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_80_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_80_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_80_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_80_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_80_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_80_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_80_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_80_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_80_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_80_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_80_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_80_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_80_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_80_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_80_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_80_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_80_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_80_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_80_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_80_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_80_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_80_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_80_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_80_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_80_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_80_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_80_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_80_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_80_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_80_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_80_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_80_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_80_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_80_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_80_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_80_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_80_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_80_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_80_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_80_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_80_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_80_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_80_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_80_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_80_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_80_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_80_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_80_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_80_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_80_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_80_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_80_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_80_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_80_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_80_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_80_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_80_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_80_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_80_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_80_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_80_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_80_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_80_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_80_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_80_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_80_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_80_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_80_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_80_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_80_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_80_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_80_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_80_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_80_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_80_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_80_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_80_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_80_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_80_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_80_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_80_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_80_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_80_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_80_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_80_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_80_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_80_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_80_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_80_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_80_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_80_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_80_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_80_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_80_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_80_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_80_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_80_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_80_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_80_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_80_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_80_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_80_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_80_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_80_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_80_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_80_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_80_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_80_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_80_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_80_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_80_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_81_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_81_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_81_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_81_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_81_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_81_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_81_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_81_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_81_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_81_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_81_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_81_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_81_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_81_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_81_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_81_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_81_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_81_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_81_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_81_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_81_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_81_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_81_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_81_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_81_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_81_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_81_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_81_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_81_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_81_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_81_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_81_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_81_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_81_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_81_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_81_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_81_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_81_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_81_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_81_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_81_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_81_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_81_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_81_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_81_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_81_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_81_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_81_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_81_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_81_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_81_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_81_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_81_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_81_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_81_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_81_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_81_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_81_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_81_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_81_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_81_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_81_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_81_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_81_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_81_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_81_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_81_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_81_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_81_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_81_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_81_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_81_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_81_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_81_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_81_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_81_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_81_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_81_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_81_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_81_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_81_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_81_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_81_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_81_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_81_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_81_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_81_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_81_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_81_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_81_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_81_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_81_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_81_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_81_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_81_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_81_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_81_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_81_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_81_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_81_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_81_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_81_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_81_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_81_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_81_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_81_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_81_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_81_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_81_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_81_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_81_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_81_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_81_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_81_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_81_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_81_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_81_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_81_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_81_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_81_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_81_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_81_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_81_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_81_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_81_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_81_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_81_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_81_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_81_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_81_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_81_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_81_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_81_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_81_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_81_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_81_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_81_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_81_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_81_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_81_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_81_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_81_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_81_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_81_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_81_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_81_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_81_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_81_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_81_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_81_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_81_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_81_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_81_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_81_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_81_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_81_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_81_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_81_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_81_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_81_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_81_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_81_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_81_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_81_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_81_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_81_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_81_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_81_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_81_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_81_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_81_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_81_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_81_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_81_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_81_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_81_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_81_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_81_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_81_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_81_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_81_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_81_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_81_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_81_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_81_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_81_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_81_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_81_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_81_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_81_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_81_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_81_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_81_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_81_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_81_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_81_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_81_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_81_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_81_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_81_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_81_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_81_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_81_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_81_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_81_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_81_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_81_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_81_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_81_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_81_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_81_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_81_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_81_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_81_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_81_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_81_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_81_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_81_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_81_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_81_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_81_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_81_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_81_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_81_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_81_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_81_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_81_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_81_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_81_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_81_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_81_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_81_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_81_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_81_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_81_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_81_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_81_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_81_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_81_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_81_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_81_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_81_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_81_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_81_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_81_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_81_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_81_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_81_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_81_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_81_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_81_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_81_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_81_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_81_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_81_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_81_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_81_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_81_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_81_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_81_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_81_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_81_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_81_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_81_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_81_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_81_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_81_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_81_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_81_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_81_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_81_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_81_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_81_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_81_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_81_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_81_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_81_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_81_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_81_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_81_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_81_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_81_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_81_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_81_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_81_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_81_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_81_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_81_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_81_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_81_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_81_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_81_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_81_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_81_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_81_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_81_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_81_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_81_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_81_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_81_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_81_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_81_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_81_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_81_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_81_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_81_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_81_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_81_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_81_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_81_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_81_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_81_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_81_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_81_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_81_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_81_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_81_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_81_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_81_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_81_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_81_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_81_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_81_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_81_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_81_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_81_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_81_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_81_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_81_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_81_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_81_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_81_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_81_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_81_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_81_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_81_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_81_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_81_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_81_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_81_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_81_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_81_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_82_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_82_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_82_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_82_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_82_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_82_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_82_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_82_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_82_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_82_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_82_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_82_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_82_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_82_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_82_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_82_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_82_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_82_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_82_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_82_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_82_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_82_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_82_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_82_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_82_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_82_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_82_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_82_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_82_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_82_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_82_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_82_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_82_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_82_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_82_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_82_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_82_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_82_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_82_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_82_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_82_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_82_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_82_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_82_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_82_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_82_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_82_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_82_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_82_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_82_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_82_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_82_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_82_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_82_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_82_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_82_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_82_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_82_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_82_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_82_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_82_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_82_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_82_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_82_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_82_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_82_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_82_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_82_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_82_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_82_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_82_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_82_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_82_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_82_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_82_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_82_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_82_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_82_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_82_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_82_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_82_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_82_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_82_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_82_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_82_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_82_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_82_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_82_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_82_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_82_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_82_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_82_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_82_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_82_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_82_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_82_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_82_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_82_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_82_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_82_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_82_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_82_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_82_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_82_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_82_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_82_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_82_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_82_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_82_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_82_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_82_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_82_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_82_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_82_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_82_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_82_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_82_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_82_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_82_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_82_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_82_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_82_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_82_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_82_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_82_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_82_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_82_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_82_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_82_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_82_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_82_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_82_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_82_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_82_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_82_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_82_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_82_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_82_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_82_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_82_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_82_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_82_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_82_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_82_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_82_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_82_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_82_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_82_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_82_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_82_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_82_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_82_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_82_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_82_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_82_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_82_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_82_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_82_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_82_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_82_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_82_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_82_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_82_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_82_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_82_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_82_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_82_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_82_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_82_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_82_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_82_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_82_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_82_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_82_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_82_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_82_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_82_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_82_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_82_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_82_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_82_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_82_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_82_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_82_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_82_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_82_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_82_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_82_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_82_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_82_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_82_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_82_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_82_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_82_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_82_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_82_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_82_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_82_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_82_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_82_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_82_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_82_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_82_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_82_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_82_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_82_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_82_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_82_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_82_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_82_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_82_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_82_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_82_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_82_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_82_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_82_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_82_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_82_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_82_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_82_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_82_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_82_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_82_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_82_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_82_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_82_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_82_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_82_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_82_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_82_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_82_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_82_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_82_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_82_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_82_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_82_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_82_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_82_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_82_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_82_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_82_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_82_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_82_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_82_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_82_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_82_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_82_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_82_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_82_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_82_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_82_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_82_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_82_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_82_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_82_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_82_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_82_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_82_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_82_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_82_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_82_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_82_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_82_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_82_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_82_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_82_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_82_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_82_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_82_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_82_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_82_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_82_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_82_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_82_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_82_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_82_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_82_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_82_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_82_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_82_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_82_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_82_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_82_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_82_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_82_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_82_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_82_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_82_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_82_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_82_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_82_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_82_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_82_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_82_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_82_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_82_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_82_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_82_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_82_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_82_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_82_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_82_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_82_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_82_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_82_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_82_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_82_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_82_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_82_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_82_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_82_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_82_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_82_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_82_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_82_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_82_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_82_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_82_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_82_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_82_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_82_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_82_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_82_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_82_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_82_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_82_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_82_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_82_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_82_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_82_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_82_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_82_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_82_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_82_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_82_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_82_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_82_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_82_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_82_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_82_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_82_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_82_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_83_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_83_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_83_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_83_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_83_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_83_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_83_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_83_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_83_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_83_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_83_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_83_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_83_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_83_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_83_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_83_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_83_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_83_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_83_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_83_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_83_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_83_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_83_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_83_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_83_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_83_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_83_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_83_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_83_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_83_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_83_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_83_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_83_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_83_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_83_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_83_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_83_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_83_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_83_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_83_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_83_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_83_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_83_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_83_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_83_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_83_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_83_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_83_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_83_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_83_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_83_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_83_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_83_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_83_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_83_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_83_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_83_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_83_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_83_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_83_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_83_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_83_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_83_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_83_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_83_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_83_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_83_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_83_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_83_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_83_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_83_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_83_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_83_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_83_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_83_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_83_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_83_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_83_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_83_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_83_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_83_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_83_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_83_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_83_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_83_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_83_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_83_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_83_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_83_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_83_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_83_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_83_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_83_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_83_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_83_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_83_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_83_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_83_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_83_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_83_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_83_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_83_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_83_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_83_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_83_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_83_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_83_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_83_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_83_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_83_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_83_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_83_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_83_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_83_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_83_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_83_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_83_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_83_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_83_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_83_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_83_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_83_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_83_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_83_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_83_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_83_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_83_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_83_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_83_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_83_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_83_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_83_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_83_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_83_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_83_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_83_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_83_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_83_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_83_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_83_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_83_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_83_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_83_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_83_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_83_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_83_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_83_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_83_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_83_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_83_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_83_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_83_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_83_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_83_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_83_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_83_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_83_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_83_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_83_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_83_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_83_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_83_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_83_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_83_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_83_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_83_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_83_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_83_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_83_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_83_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_83_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_83_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_83_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_83_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_83_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_83_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_83_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_83_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_83_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_83_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_83_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_83_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_83_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_83_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_83_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_83_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_83_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_83_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_83_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_83_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_83_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_83_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_83_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_83_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_83_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_83_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_83_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_83_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_83_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_83_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_83_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_83_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_83_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_83_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_83_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_83_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_83_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_83_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_83_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_83_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_83_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_83_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_83_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_83_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_83_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_83_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_83_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_83_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_83_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_83_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_83_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_83_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_83_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_83_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_83_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_83_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_83_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_83_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_83_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_83_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_83_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_83_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_83_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_83_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_83_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_83_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_83_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_83_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_83_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_83_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_83_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_83_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_83_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_83_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_83_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_83_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_83_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_83_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_83_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_83_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_83_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_83_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_83_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_83_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_83_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_83_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_83_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_83_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_83_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_83_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_83_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_83_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_83_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_83_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_83_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_83_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_83_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_83_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_83_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_83_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_83_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_83_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_83_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_83_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_83_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_83_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_83_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_83_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_83_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_83_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_83_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_83_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_83_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_83_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_83_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_83_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_83_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_83_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_83_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_83_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_83_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_83_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_83_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_83_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_83_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_83_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_83_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_83_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_83_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_83_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_83_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_83_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_83_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_83_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_83_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_83_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_83_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_83_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_83_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_83_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_83_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_83_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_83_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_83_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_83_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_83_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_83_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_83_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_83_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_83_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_83_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_83_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_83_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_83_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_83_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_83_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_83_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_83_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_83_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_83_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_83_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_83_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_83_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_83_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_83_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_83_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_83_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_83_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_83_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_83_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_83_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_83_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_84_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_84_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_84_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_84_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_84_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_84_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_84_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_84_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_84_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_84_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_84_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_84_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_84_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_84_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_84_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_84_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_84_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_84_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_84_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_84_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_84_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_84_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_84_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_84_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_84_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_84_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_84_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_84_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_84_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_84_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_84_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_84_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_84_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_84_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_84_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_84_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_84_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_84_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_84_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_84_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_84_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_84_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_84_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_84_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_84_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_84_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_84_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_84_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_84_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_84_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_84_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_84_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_84_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_84_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_84_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_84_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_84_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_84_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_84_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_84_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_84_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_84_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_84_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_84_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_84_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_84_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_84_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_84_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_84_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_84_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_84_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_84_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_84_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_84_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_84_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_84_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_84_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_84_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_84_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_84_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_84_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_84_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_84_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_84_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_84_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_84_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_84_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_84_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_84_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_84_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_84_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_84_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_84_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_84_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_84_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_84_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_84_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_84_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_84_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_84_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_84_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_84_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_84_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_84_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_84_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_84_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_84_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_84_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_84_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_84_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_84_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_84_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_84_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_84_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_84_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_84_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_84_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_84_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_84_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_84_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_84_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_84_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_84_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_84_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_84_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_84_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_84_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_84_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_84_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_84_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_84_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_84_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_84_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_84_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_84_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_84_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_84_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_84_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_84_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_84_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_84_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_84_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_84_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_84_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_84_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_84_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_84_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_84_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_84_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_84_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_84_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_84_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_84_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_84_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_84_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_84_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_84_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_84_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_84_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_84_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_84_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_84_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_84_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_84_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_84_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_84_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_84_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_84_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_84_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_84_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_84_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_84_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_84_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_84_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_84_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_84_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_84_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_84_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_84_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_84_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_84_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_84_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_84_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_84_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_84_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_84_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_84_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_84_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_84_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_84_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_84_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_84_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_84_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_84_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_84_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_84_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_84_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_84_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_84_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_84_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_84_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_84_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_84_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_84_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_84_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_84_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_84_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_84_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_84_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_84_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_84_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_84_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_84_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_84_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_84_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_84_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_84_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_84_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_84_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_84_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_84_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_84_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_84_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_84_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_84_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_84_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_84_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_84_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_84_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_84_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_84_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_84_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_84_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_84_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_84_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_84_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_84_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_84_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_84_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_84_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_84_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_84_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_84_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_84_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_84_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_84_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_84_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_84_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_84_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_84_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_84_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_84_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_84_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_84_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_84_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_84_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_84_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_84_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_84_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_84_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_84_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_84_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_84_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_84_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_84_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_84_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_84_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_84_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_84_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_84_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_84_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_84_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_84_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_84_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_84_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_84_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_84_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_84_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_84_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_84_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_84_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_84_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_84_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_84_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_84_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_84_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_84_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_84_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_84_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_84_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_84_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_84_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_84_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_84_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_84_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_84_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_84_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_84_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_84_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_84_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_84_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_84_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_84_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_84_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_84_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_84_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_84_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_84_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_84_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_84_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_84_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_84_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_84_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_84_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_84_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_84_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_84_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_84_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_84_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_84_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_84_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_84_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_84_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_84_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_84_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_84_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_84_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_84_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_84_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_84_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_84_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_84_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_84_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_84_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_84_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_84_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_84_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_84_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_84_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_84_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_84_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_84_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_85_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_85_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_85_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_85_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_85_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_85_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_85_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_85_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_85_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_85_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_85_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_85_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_85_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_85_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_85_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_85_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_85_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_85_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_85_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_85_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_85_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_85_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_85_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_85_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_85_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_85_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_85_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_85_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_85_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_85_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_85_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_85_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_85_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_85_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_85_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_85_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_85_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_85_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_85_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_85_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_85_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_85_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_85_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_85_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_85_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_85_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_85_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_85_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_85_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_85_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_85_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_85_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_85_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_85_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_85_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_85_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_85_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_85_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_85_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_85_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_85_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_85_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_85_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_85_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_85_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_85_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_85_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_85_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_85_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_85_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_85_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_85_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_85_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_85_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_85_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_85_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_85_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_85_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_85_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_85_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_85_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_85_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_85_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_85_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_85_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_85_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_85_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_85_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_85_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_85_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_85_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_85_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_85_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_85_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_85_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_85_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_85_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_85_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_85_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_85_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_85_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_85_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_85_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_85_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_85_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_85_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_85_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_85_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_85_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_85_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_85_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_85_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_85_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_85_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_85_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_85_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_85_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_85_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_85_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_85_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_85_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_85_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_85_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_85_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_85_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_85_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_85_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_85_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_85_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_85_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_85_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_85_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_85_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_85_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_85_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_85_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_85_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_85_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_85_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_85_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_85_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_85_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_85_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_85_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_85_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_85_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_85_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_85_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_85_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_85_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_85_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_85_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_85_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_85_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_85_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_85_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_85_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_85_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_85_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_85_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_85_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_85_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_85_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_85_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_85_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_85_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_85_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_85_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_85_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_85_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_85_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_85_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_85_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_85_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_85_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_85_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_85_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_85_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_85_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_85_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_85_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_85_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_85_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_85_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_85_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_85_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_85_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_85_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_85_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_85_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_85_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_85_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_85_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_85_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_85_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_85_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_85_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_85_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_85_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_85_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_85_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_85_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_85_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_85_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_85_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_85_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_85_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_85_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_85_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_85_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_85_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_85_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_85_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_85_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_85_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_85_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_85_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_85_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_85_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_85_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_85_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_85_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_85_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_85_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_85_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_85_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_85_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_85_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_85_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_85_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_85_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_85_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_85_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_85_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_85_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_85_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_85_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_85_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_85_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_85_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_85_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_85_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_85_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_85_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_85_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_85_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_85_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_85_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_85_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_85_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_85_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_85_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_85_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_85_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_85_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_85_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_85_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_85_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_85_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_85_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_85_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_85_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_85_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_85_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_85_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_85_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_85_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_85_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_85_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_85_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_85_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_85_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_85_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_85_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_85_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_85_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_85_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_85_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_85_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_85_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_85_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_85_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_85_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_85_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_85_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_85_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_85_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_85_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_85_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_85_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_85_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_85_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_85_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_85_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_85_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_85_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_85_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_85_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_85_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_85_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_85_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_85_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_85_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_85_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_85_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_85_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_85_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_85_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_85_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_85_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_85_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_85_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_85_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_85_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_85_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_85_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_85_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_85_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_85_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_85_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_85_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_85_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_85_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_85_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_85_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_85_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_85_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_85_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_85_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_85_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_85_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_85_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_85_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_85_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_85_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_85_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_85_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_85_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_85_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_85_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_85_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_85_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_86_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_86_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_86_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_86_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_86_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_86_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_86_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_86_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_86_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_86_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_86_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_86_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_86_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_86_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_86_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_86_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_86_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_86_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_86_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_86_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_86_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_86_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_86_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_86_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_86_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_86_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_86_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_86_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_86_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_86_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_86_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_86_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_86_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_86_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_86_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_86_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_86_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_86_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_86_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_86_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_86_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_86_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_86_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_86_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_86_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_86_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_86_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_86_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_86_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_86_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_86_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_86_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_86_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_86_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_86_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_86_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_86_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_86_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_86_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_86_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_86_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_86_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_86_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_86_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_86_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_86_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_86_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_86_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_86_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_86_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_86_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_86_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_86_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_86_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_86_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_86_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_86_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_86_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_86_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_86_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_86_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_86_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_86_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_86_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_86_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_86_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_86_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_86_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_86_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_86_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_86_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_86_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_86_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_86_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_86_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_86_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_86_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_86_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_86_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_86_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_86_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_86_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_86_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_86_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_86_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_86_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_86_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_86_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_86_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_86_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_86_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_86_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_86_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_86_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_86_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_86_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_86_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_86_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_86_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_86_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_86_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_86_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_86_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_86_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_86_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_86_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_86_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_86_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_86_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_86_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_86_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_86_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_86_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_86_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_86_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_86_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_86_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_86_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_86_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_86_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_86_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_86_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_86_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_86_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_86_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_86_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_86_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_86_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_86_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_86_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_86_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_86_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_86_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_86_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_86_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_86_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_86_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_86_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_86_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_86_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_86_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_86_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_86_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_86_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_86_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_86_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_86_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_86_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_86_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_86_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_86_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_86_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_86_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_86_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_86_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_86_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_86_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_86_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_86_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_86_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_86_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_86_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_86_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_86_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_86_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_86_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_86_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_86_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_86_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_86_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_86_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_86_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_86_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_86_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_86_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_86_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_86_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_86_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_86_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_86_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_86_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_86_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_86_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_86_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_86_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_86_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_86_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_86_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_86_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_86_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_86_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_86_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_86_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_86_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_86_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_86_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_86_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_86_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_86_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_86_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_86_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_86_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_86_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_86_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_86_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_86_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_86_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_86_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_86_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_86_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_86_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_86_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_86_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_86_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_86_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_86_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_86_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_86_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_86_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_86_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_86_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_86_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_86_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_86_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_86_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_86_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_86_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_86_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_86_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_86_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_86_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_86_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_86_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_86_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_86_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_86_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_86_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_86_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_86_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_86_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_86_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_86_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_86_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_86_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_86_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_86_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_86_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_86_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_86_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_86_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_86_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_86_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_86_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_86_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_86_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_86_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_86_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_86_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_86_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_86_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_86_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_86_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_86_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_86_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_86_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_86_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_86_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_86_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_86_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_86_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_86_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_86_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_86_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_86_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_86_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_86_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_86_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_86_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_86_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_86_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_86_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_86_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_86_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_86_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_86_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_86_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_86_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_86_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_86_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_86_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_86_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_86_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_86_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_86_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_86_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_86_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_86_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_86_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_86_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_86_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_86_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_86_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_86_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_86_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_86_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_86_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_86_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_86_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_86_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_86_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_86_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_86_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_86_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_86_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_86_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_86_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_86_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_86_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_86_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_86_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_86_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_86_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_87_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_87_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_87_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_87_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_87_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_87_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_87_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_87_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_87_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_87_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_87_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_87_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_87_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_87_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_87_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_87_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_87_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_87_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_87_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_87_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_87_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_87_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_87_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_87_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_87_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_87_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_87_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_87_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_87_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_87_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_87_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_87_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_87_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_87_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_87_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_87_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_87_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_87_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_87_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_87_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_87_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_87_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_87_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_87_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_87_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_87_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_87_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_87_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_87_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_87_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_87_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_87_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_87_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_87_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_87_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_87_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_87_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_87_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_87_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_87_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_87_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_87_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_87_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_87_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_87_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_87_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_87_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_87_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_87_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_87_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_87_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_87_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_87_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_87_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_87_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_87_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_87_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_87_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_87_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_87_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_87_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_87_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_87_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_87_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_87_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_87_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_87_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_87_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_87_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_87_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_87_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_87_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_87_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_87_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_87_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_87_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_87_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_87_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_87_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_87_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_87_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_87_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_87_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_87_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_87_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_87_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_87_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_87_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_87_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_87_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_87_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_87_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_87_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_87_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_87_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_87_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_87_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_87_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_87_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_87_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_87_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_87_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_87_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_87_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_87_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_87_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_87_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_87_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_87_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_87_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_87_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_87_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_87_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_87_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_87_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_87_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_87_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_87_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_87_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_87_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_87_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_87_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_87_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_87_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_87_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_87_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_87_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_87_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_87_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_87_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_87_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_87_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_87_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_87_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_87_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_87_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_87_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_87_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_87_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_87_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_87_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_87_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_87_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_87_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_87_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_87_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_87_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_87_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_87_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_87_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_87_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_87_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_87_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_87_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_87_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_87_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_87_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_87_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_87_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_87_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_87_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_87_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_87_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_87_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_87_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_87_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_87_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_87_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_87_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_87_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_87_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_87_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_87_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_87_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_87_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_87_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_87_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_87_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_87_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_87_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_87_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_87_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_87_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_87_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_87_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_87_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_87_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_87_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_87_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_87_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_87_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_87_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_87_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_87_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_87_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_87_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_87_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_87_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_87_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_87_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_87_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_87_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_87_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_87_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_87_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_87_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_87_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_87_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_87_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_87_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_87_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_87_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_87_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_87_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_87_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_87_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_87_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_87_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_87_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_87_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_87_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_87_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_87_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_87_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_87_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_87_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_87_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_87_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_87_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_87_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_87_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_87_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_87_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_87_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_87_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_87_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_87_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_87_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_87_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_87_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_87_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_87_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_87_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_87_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_87_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_87_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_87_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_87_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_87_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_87_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_87_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_87_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_87_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_87_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_87_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_87_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_87_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_87_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_87_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_87_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_87_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_87_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_87_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_87_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_87_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_87_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_87_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_87_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_87_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_87_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_87_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_87_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_87_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_87_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_87_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_87_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_87_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_87_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_87_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_87_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_87_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_87_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_87_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_87_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_87_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_87_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_87_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_87_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_87_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_87_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_87_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_87_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_87_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_87_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_87_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_87_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_87_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_87_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_87_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_87_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_87_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_87_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_87_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_87_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_87_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_87_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_87_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_87_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_87_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_87_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_87_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_87_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_87_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_87_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_87_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_87_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_87_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_87_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_87_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_87_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_87_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_87_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_88_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_88_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_88_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_88_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_88_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_88_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_88_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_88_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_88_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_88_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_88_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_88_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_88_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_88_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_88_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_88_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_88_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_88_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_88_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_88_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_88_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_88_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_88_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_88_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_88_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_88_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_88_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_88_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_88_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_88_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_88_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_88_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_88_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_88_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_88_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_88_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_88_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_88_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_88_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_88_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_88_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_88_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_88_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_88_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_88_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_88_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_88_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_88_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_88_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_88_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_88_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_88_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_88_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_88_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_88_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_88_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_88_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_88_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_88_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_88_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_88_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_88_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_88_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_88_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_88_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_88_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_88_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_88_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_88_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_88_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_88_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_88_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_88_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_88_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_88_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_88_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_88_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_88_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_88_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_88_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_88_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_88_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_88_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_88_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_88_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_88_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_88_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_88_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_88_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_88_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_88_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_88_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_88_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_88_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_88_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_88_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_88_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_88_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_88_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_88_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_88_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_88_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_88_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_88_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_88_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_88_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_88_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_88_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_88_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_88_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_88_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_88_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_88_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_88_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_88_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_88_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_88_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_88_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_88_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_88_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_88_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_88_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_88_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_88_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_88_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_88_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_88_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_88_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_88_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_88_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_88_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_88_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_88_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_88_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_88_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_88_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_88_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_88_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_88_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_88_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_88_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_88_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_88_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_88_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_88_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_88_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_88_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_88_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_88_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_88_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_88_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_88_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_88_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_88_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_88_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_88_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_88_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_88_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_88_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_88_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_88_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_88_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_88_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_88_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_88_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_88_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_88_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_88_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_88_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_88_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_88_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_88_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_88_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_88_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_88_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_88_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_88_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_88_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_88_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_88_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_88_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_88_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_88_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_88_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_88_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_88_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_88_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_88_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_88_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_88_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_88_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_88_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_88_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_88_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_88_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_88_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_88_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_88_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_88_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_88_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_88_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_88_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_88_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_88_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_88_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_88_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_88_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_88_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_88_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_88_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_88_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_88_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_88_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_88_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_88_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_88_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_88_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_88_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_88_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_88_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_88_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_88_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_88_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_88_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_88_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_88_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_88_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_88_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_88_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_88_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_88_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_88_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_88_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_88_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_88_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_88_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_88_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_88_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_88_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_88_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_88_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_88_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_88_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_88_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_88_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_88_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_88_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_88_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_88_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_88_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_88_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_88_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_88_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_88_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_88_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_88_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_88_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_88_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_88_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_88_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_88_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_88_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_88_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_88_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_88_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_88_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_88_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_88_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_88_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_88_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_88_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_88_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_88_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_88_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_88_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_88_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_88_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_88_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_88_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_88_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_88_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_88_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_88_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_88_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_88_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_88_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_88_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_88_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_88_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_88_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_88_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_88_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_88_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_88_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_88_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_88_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_88_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_88_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_88_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_88_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_88_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_88_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_88_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_88_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_88_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_88_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_88_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_88_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_88_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_88_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_88_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_88_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_88_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_88_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_88_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_88_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_88_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_88_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_88_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_88_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_88_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_88_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_88_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_88_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_88_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_88_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_88_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_88_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_88_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_88_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_88_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_88_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_88_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_88_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_88_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_88_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_88_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_88_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_88_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_88_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_88_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_88_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_89_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_89_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_89_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_89_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_89_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_89_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_89_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_89_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_89_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_89_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_89_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_89_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_89_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_89_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_89_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_89_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_89_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_89_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_89_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_89_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_89_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_89_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_89_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_89_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_89_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_89_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_89_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_89_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_89_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_89_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_89_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_89_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_89_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_89_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_89_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_89_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_89_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_89_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_89_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_89_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_89_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_89_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_89_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_89_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_89_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_89_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_89_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_89_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_89_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_89_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_89_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_89_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_89_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_89_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_89_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_89_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_89_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_89_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_89_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_89_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_89_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_89_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_89_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_89_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_89_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_89_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_89_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_89_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_89_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_89_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_89_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_89_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_89_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_89_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_89_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_89_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_89_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_89_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_89_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_89_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_89_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_89_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_89_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_89_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_89_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_89_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_89_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_89_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_89_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_89_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_89_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_89_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_89_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_89_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_89_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_89_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_89_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_89_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_89_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_89_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_89_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_89_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_89_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_89_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_89_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_89_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_89_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_89_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_89_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_89_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_89_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_89_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_89_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_89_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_89_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_89_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_89_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_89_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_89_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_89_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_89_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_89_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_89_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_89_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_89_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_89_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_89_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_89_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_89_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_89_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_89_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_89_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_89_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_89_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_89_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_89_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_89_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_89_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_89_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_89_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_89_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_89_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_89_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_89_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_89_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_89_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_89_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_89_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_89_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_89_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_89_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_89_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_89_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_89_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_89_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_89_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_89_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_89_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_89_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_89_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_89_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_89_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_89_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_89_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_89_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_89_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_89_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_89_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_89_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_89_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_89_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_89_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_89_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_89_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_89_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_89_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_89_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_89_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_89_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_89_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_89_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_89_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_89_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_89_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_89_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_89_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_89_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_89_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_89_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_89_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_89_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_89_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_89_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_89_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_89_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_89_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_89_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_89_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_89_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_89_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_89_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_89_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_89_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_89_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_89_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_89_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_89_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_89_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_89_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_89_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_89_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_89_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_89_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_89_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_89_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_89_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_89_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_89_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_89_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_89_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_89_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_89_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_89_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_89_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_89_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_89_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_89_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_89_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_89_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_89_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_89_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_89_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_89_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_89_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_89_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_89_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_89_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_89_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_89_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_89_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_89_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_89_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_89_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_89_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_89_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_89_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_89_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_89_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_89_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_89_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_89_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_89_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_89_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_89_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_89_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_89_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_89_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_89_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_89_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_89_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_89_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_89_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_89_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_89_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_89_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_89_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_89_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_89_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_89_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_89_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_89_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_89_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_89_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_89_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_89_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_89_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_89_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_89_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_89_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_89_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_89_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_89_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_89_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_89_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_89_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_89_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_89_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_89_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_89_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_89_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_89_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_89_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_89_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_89_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_89_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_89_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_89_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_89_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_89_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_89_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_89_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_89_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_89_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_89_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_89_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_89_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_89_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_89_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_89_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_89_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_89_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_89_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_89_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_89_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_89_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_89_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_89_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_89_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_89_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_89_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_89_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_89_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_89_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_89_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_89_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_89_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_89_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_89_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_89_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_89_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_89_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_89_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_89_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_89_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_89_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_89_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_89_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_89_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_89_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_89_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_89_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_89_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_90_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_90_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_90_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_90_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_90_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_90_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_90_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_90_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_90_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_90_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_90_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_90_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_90_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_90_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_90_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_90_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_90_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_90_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_90_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_90_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_90_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_90_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_90_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_90_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_90_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_90_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_90_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_90_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_90_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_90_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_90_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_90_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_90_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_90_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_90_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_90_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_90_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_90_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_90_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_90_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_90_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_90_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_90_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_90_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_90_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_90_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_90_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_90_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_90_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_90_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_90_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_90_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_90_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_90_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_90_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_90_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_90_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_90_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_90_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_90_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_90_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_90_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_90_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_90_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_90_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_90_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_90_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_90_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_90_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_90_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_90_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_90_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_90_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_90_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_90_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_90_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_90_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_90_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_90_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_90_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_90_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_90_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_90_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_90_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_90_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_90_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_90_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_90_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_90_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_90_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_90_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_90_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_90_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_90_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_90_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_90_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_90_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_90_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_90_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_90_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_90_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_90_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_90_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_90_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_90_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_90_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_90_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_90_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_90_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_90_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_90_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_90_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_90_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_90_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_90_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_90_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_90_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_90_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_90_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_90_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_90_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_90_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_90_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_90_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_90_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_90_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_90_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_90_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_90_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_90_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_90_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_90_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_90_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_90_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_90_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_90_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_90_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_90_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_90_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_90_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_90_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_90_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_90_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_90_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_90_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_90_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_90_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_90_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_90_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_90_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_90_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_90_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_90_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_90_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_90_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_90_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_90_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_90_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_90_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_90_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_90_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_90_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_90_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_90_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_90_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_90_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_90_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_90_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_90_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_90_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_90_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_90_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_90_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_90_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_90_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_90_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_90_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_90_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_90_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_90_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_90_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_90_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_90_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_90_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_90_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_90_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_90_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_90_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_90_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_90_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_90_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_90_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_90_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_90_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_90_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_90_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_90_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_90_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_90_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_90_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_90_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_90_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_90_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_90_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_90_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_90_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_90_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_90_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_90_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_90_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_90_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_90_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_90_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_90_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_90_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_90_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_90_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_90_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_90_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_90_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_90_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_90_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_90_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_90_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_90_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_90_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_90_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_90_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_90_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_90_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_90_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_90_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_90_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_90_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_90_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_90_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_90_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_90_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_90_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_90_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_90_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_90_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_90_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_90_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_90_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_90_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_90_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_90_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_90_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_90_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_90_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_90_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_90_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_90_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_90_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_90_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_90_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_90_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_90_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_90_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_90_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_90_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_90_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_90_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_90_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_90_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_90_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_90_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_90_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_90_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_90_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_90_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_90_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_90_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_90_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_90_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_90_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_90_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_90_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_90_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_90_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_90_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_90_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_90_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_90_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_90_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_90_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_90_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_90_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_90_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_90_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_90_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_90_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_90_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_90_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_90_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_90_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_90_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_90_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_90_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_90_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_90_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_90_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_90_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_90_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_90_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_90_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_90_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_90_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_90_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_90_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_90_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_90_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_90_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_90_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_90_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_90_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_90_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_90_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_90_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_90_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_90_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_90_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_90_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_90_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_90_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_90_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_90_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_90_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_90_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_90_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_90_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_90_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_90_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_90_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_90_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_90_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_90_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_90_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_90_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_90_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_90_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_91_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_91_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_91_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_91_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_91_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_91_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_91_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_91_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_91_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_91_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_91_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_91_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_91_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_91_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_91_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_91_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_91_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_91_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_91_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_91_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_91_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_91_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_91_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_91_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_91_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_91_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_91_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_91_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_91_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_91_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_91_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_91_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_91_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_91_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_91_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_91_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_91_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_91_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_91_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_91_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_91_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_91_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_91_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_91_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_91_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_91_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_91_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_91_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_91_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_91_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_91_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_91_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_91_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_91_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_91_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_91_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_91_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_91_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_91_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_91_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_91_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_91_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_91_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_91_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_91_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_91_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_91_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_91_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_91_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_91_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_91_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_91_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_91_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_91_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_91_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_91_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_91_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_91_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_91_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_91_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_91_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_91_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_91_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_91_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_91_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_91_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_91_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_91_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_91_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_91_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_91_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_91_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_91_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_91_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_91_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_91_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_91_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_91_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_91_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_91_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_91_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_91_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_91_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_91_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_91_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_91_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_91_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_91_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_91_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_91_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_91_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_91_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_91_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_91_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_91_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_91_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_91_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_91_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_91_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_91_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_91_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_91_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_91_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_91_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_91_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_91_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_91_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_91_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_91_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_91_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_91_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_91_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_91_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_91_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_91_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_91_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_91_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_91_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_91_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_91_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_91_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_91_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_91_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_91_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_91_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_91_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_91_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_91_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_91_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_91_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_91_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_91_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_91_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_91_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_91_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_91_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_91_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_91_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_91_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_91_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_91_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_91_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_91_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_91_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_91_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_91_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_91_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_91_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_91_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_91_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_91_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_91_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_91_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_91_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_91_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_91_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_91_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_91_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_91_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_91_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_91_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_91_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_91_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_91_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_91_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_91_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_91_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_91_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_91_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_91_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_91_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_91_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_91_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_91_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_91_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_91_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_91_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_91_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_91_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_91_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_91_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_91_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_91_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_91_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_91_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_91_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_91_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_91_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_91_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_91_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_91_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_91_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_91_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_91_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_91_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_91_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_91_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_91_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_91_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_91_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_91_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_91_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_91_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_91_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_91_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_91_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_91_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_91_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_91_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_91_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_91_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_91_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_91_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_91_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_91_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_91_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_91_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_91_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_91_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_91_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_91_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_91_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_91_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_91_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_91_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_91_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_91_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_91_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_91_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_91_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_91_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_91_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_91_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_91_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_91_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_91_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_91_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_91_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_91_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_91_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_91_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_91_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_91_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_91_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_91_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_91_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_91_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_91_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_91_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_91_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_91_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_91_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_91_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_91_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_91_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_91_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_91_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_91_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_91_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_91_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_91_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_91_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_91_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_91_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_91_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_91_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_91_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_91_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_91_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_91_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_91_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_91_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_91_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_91_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_91_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_91_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_91_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_91_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_91_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_91_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_91_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_91_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_91_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_91_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_91_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_91_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_91_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_91_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_91_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_91_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_91_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_91_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_91_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_91_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_91_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_91_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_91_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_91_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_91_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_91_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_91_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_91_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_91_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_91_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_91_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_91_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_91_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_91_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_91_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_91_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_91_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_91_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_91_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_91_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_91_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_91_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_91_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_91_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_91_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_91_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_91_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_91_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_92_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_92_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_92_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_92_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_92_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_92_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_92_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_92_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_92_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_92_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_92_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_92_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_92_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_92_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_92_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_92_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_92_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_92_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_92_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_92_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_92_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_92_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_92_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_92_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_92_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_92_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_92_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_92_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_92_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_92_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_92_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_92_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_92_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_92_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_92_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_92_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_92_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_92_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_92_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_92_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_92_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_92_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_92_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_92_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_92_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_92_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_92_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_92_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_92_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_92_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_92_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_92_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_92_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_92_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_92_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_92_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_92_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_92_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_92_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_92_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_92_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_92_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_92_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_92_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_92_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_92_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_92_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_92_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_92_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_92_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_92_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_92_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_92_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_92_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_92_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_92_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_92_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_92_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_92_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_92_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_92_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_92_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_92_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_92_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_92_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_92_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_92_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_92_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_92_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_92_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_92_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_92_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_92_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_92_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_92_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_92_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_92_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_92_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_92_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_92_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_92_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_92_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_92_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_92_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_92_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_92_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_92_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_92_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_92_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_92_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_92_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_92_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_92_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_92_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_92_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_92_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_92_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_92_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_92_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_92_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_92_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_92_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_92_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_92_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_92_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_92_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_92_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_92_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_92_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_92_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_92_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_92_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_92_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_92_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_92_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_92_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_92_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_92_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_92_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_92_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_92_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_92_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_92_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_92_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_92_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_92_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_92_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_92_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_92_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_92_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_92_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_92_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_92_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_92_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_92_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_92_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_92_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_92_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_92_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_92_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_92_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_92_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_92_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_92_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_92_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_92_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_92_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_92_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_92_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_92_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_92_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_92_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_92_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_92_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_92_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_92_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_92_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_92_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_92_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_92_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_92_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_92_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_92_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_92_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_92_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_92_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_92_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_92_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_92_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_92_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_92_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_92_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_92_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_92_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_92_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_92_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_92_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_92_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_92_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_92_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_92_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_92_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_92_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_92_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_92_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_92_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_92_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_92_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_92_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_92_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_92_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_92_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_92_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_92_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_92_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_92_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_92_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_92_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_92_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_92_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_92_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_92_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_92_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_92_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_92_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_92_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_92_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_92_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_92_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_92_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_92_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_92_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_92_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_92_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_92_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_92_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_92_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_92_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_92_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_92_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_92_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_92_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_92_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_92_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_92_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_92_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_92_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_92_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_92_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_92_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_92_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_92_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_92_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_92_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_92_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_92_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_92_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_92_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_92_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_92_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_92_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_92_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_92_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_92_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_92_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_92_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_92_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_92_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_92_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_92_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_92_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_92_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_92_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_92_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_92_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_92_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_92_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_92_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_92_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_92_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_92_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_92_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_92_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_92_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_92_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_92_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_92_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_92_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_92_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_92_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_92_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_92_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_92_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_92_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_92_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_92_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_92_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_92_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_92_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_92_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_92_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_92_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_92_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_92_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_92_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_92_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_92_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_92_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_92_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_92_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_92_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_92_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_92_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_92_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_92_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_92_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_92_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_92_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_92_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_92_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_92_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_92_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_92_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_92_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_92_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_92_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_92_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_92_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_92_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_92_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_92_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_92_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_92_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_92_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_92_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_92_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_92_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_92_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_92_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_92_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_92_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_92_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_93_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_93_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_93_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_93_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_93_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_93_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_93_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_93_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_93_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_93_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_93_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_93_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_93_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_93_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_93_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_93_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_93_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_93_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_93_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_93_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_93_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_93_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_93_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_93_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_93_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_93_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_93_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_93_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_93_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_93_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_93_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_93_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_93_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_93_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_93_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_93_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_93_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_93_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_93_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_93_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_93_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_93_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_93_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_93_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_93_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_93_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_93_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_93_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_93_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_93_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_93_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_93_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_93_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_93_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_93_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_93_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_93_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_93_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_93_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_93_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_93_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_93_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_93_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_93_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_93_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_93_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_93_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_93_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_93_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_93_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_93_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_93_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_93_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_93_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_93_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_93_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_93_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_93_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_93_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_93_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_93_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_93_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_93_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_93_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_93_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_93_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_93_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_93_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_93_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_93_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_93_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_93_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_93_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_93_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_93_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_93_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_93_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_93_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_93_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_93_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_93_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_93_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_93_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_93_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_93_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_93_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_93_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_93_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_93_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_93_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_93_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_93_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_93_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_93_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_93_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_93_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_93_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_93_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_93_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_93_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_93_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_93_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_93_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_93_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_93_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_93_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_93_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_93_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_93_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_93_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_93_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_93_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_93_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_93_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_93_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_93_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_93_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_93_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_93_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_93_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_93_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_93_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_93_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_93_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_93_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_93_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_93_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_93_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_93_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_93_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_93_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_93_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_93_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_93_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_93_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_93_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_93_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_93_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_93_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_93_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_93_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_93_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_93_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_93_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_93_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_93_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_93_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_93_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_93_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_93_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_93_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_93_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_93_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_93_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_93_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_93_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_93_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_93_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_93_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_93_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_93_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_93_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_93_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_93_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_93_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_93_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_93_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_93_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_93_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_93_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_93_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_93_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_93_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_93_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_93_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_93_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_93_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_93_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_93_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_93_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_93_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_93_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_93_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_93_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_93_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_93_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_93_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_93_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_93_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_93_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_93_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_93_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_93_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_93_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_93_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_93_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_93_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_93_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_93_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_93_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_93_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_93_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_93_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_93_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_93_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_93_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_93_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_93_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_93_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_93_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_93_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_93_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_93_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_93_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_93_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_93_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_93_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_93_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_93_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_93_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_93_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_93_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_93_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_93_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_93_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_93_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_93_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_93_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_93_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_93_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_93_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_93_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_93_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_93_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_93_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_93_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_93_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_93_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_93_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_93_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_93_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_93_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_93_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_93_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_93_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_93_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_93_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_93_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_93_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_93_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_93_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_93_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_93_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_93_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_93_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_93_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_93_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_93_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_93_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_93_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_93_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_93_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_93_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_93_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_93_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_93_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_93_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_93_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_93_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_93_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_93_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_93_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_93_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_93_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_93_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_93_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_93_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_93_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_93_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_93_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_93_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_93_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_93_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_93_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_93_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_93_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_93_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_93_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_93_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_93_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_93_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_93_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_93_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_93_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_93_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_93_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_93_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_93_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_93_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_93_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_93_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_93_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_93_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_93_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_93_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_93_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_93_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_93_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_93_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_93_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_93_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_93_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_93_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_93_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_93_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_93_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_93_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_93_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_93_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_93_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_93_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_93_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_94_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_94_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_94_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_94_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_94_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_94_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_94_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_94_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_94_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_94_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_94_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_94_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_94_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_94_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_94_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_94_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_94_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_94_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_94_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_94_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_94_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_94_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_94_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_94_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_94_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_94_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_94_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_94_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_94_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_94_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_94_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_94_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_94_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_94_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_94_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_94_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_94_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_94_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_94_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_94_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_94_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_94_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_94_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_94_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_94_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_94_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_94_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_94_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_94_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_94_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_94_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_94_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_94_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_94_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_94_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_94_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_94_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_94_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_94_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_94_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_94_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_94_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_94_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_94_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_94_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_94_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_94_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_94_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_94_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_94_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_94_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_94_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_94_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_94_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_94_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_94_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_94_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_94_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_94_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_94_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_94_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_94_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_94_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_94_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_94_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_94_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_94_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_94_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_94_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_94_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_94_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_94_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_94_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_94_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_94_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_94_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_94_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_94_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_94_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_94_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_94_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_94_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_94_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_94_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_94_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_94_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_94_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_94_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_94_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_94_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_94_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_94_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_94_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_94_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_94_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_94_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_94_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_94_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_94_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_94_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_94_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_94_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_94_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_94_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_94_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_94_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_94_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_94_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_94_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_94_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_94_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_94_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_94_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_94_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_94_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_94_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_94_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_94_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_94_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_94_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_94_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_94_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_94_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_94_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_94_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_94_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_94_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_94_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_94_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_94_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_94_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_94_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_94_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_94_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_94_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_94_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_94_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_94_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_94_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_94_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_94_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_94_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_94_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_94_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_94_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_94_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_94_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_94_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_94_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_94_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_94_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_94_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_94_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_94_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_94_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_94_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_94_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_94_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_94_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_94_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_94_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_94_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_94_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_94_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_94_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_94_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_94_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_94_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_94_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_94_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_94_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_94_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_94_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_94_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_94_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_94_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_94_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_94_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_94_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_94_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_94_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_94_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_94_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_94_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_94_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_94_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_94_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_94_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_94_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_94_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_94_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_94_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_94_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_94_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_94_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_94_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_94_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_94_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_94_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_94_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_94_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_94_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_94_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_94_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_94_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_94_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_94_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_94_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_94_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_94_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_94_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_94_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_94_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_94_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_94_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_94_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_94_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_94_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_94_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_94_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_94_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_94_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_94_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_94_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_94_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_94_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_94_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_94_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_94_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_94_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_94_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_94_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_94_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_94_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_94_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_94_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_94_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_94_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_94_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_94_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_94_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_94_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_94_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_94_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_94_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_94_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_94_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_94_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_94_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_94_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_94_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_94_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_94_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_94_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_94_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_94_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_94_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_94_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_94_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_94_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_94_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_94_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_94_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_94_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_94_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_94_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_94_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_94_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_94_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_94_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_94_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_94_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_94_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_94_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_94_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_94_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_94_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_94_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_94_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_94_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_94_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_94_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_94_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_94_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_94_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_94_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_94_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_94_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_94_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_94_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_94_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_94_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_94_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_94_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_94_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_94_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_94_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_94_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_94_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_94_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_94_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_94_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_94_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_94_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_94_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_94_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_94_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_94_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_94_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_94_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_94_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_94_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_94_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_94_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_94_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_94_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_94_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_94_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_94_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_94_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_94_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_94_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_95_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_95_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_95_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_95_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_95_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_95_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_95_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_95_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_95_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_95_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_95_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_95_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_95_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_95_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_95_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_95_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_95_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_95_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_95_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_95_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_95_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_95_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_95_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_95_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_95_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_95_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_95_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_95_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_95_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_95_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_95_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_95_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_95_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_95_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_95_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_95_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_95_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_95_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_95_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_95_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_95_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_95_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_95_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_95_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_95_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_95_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_95_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_95_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_95_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_95_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_95_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_95_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_95_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_95_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_95_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_95_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_95_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_95_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_95_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_95_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_95_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_95_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_95_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_95_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_95_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_95_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_95_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_95_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_95_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_95_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_95_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_95_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_95_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_95_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_95_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_95_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_95_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_95_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_95_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_95_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_95_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_95_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_95_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_95_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_95_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_95_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_95_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_95_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_95_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_95_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_95_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_95_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_95_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_95_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_95_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_95_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_95_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_95_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_95_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_95_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_95_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_95_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_95_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_95_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_95_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_95_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_95_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_95_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_95_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_95_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_95_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_95_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_95_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_95_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_95_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_95_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_95_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_95_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_95_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_95_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_95_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_95_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_95_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_95_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_95_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_95_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_95_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_95_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_95_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_95_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_95_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_95_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_95_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_95_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_95_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_95_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_95_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_95_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_95_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_95_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_95_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_95_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_95_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_95_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_95_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_95_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_95_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_95_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_95_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_95_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_95_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_95_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_95_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_95_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_95_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_95_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_95_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_95_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_95_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_95_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_95_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_95_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_95_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_95_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_95_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_95_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_95_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_95_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_95_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_95_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_95_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_95_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_95_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_95_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_95_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_95_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_95_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_95_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_95_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_95_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_95_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_95_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_95_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_95_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_95_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_95_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_95_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_95_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_95_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_95_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_95_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_95_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_95_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_95_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_95_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_95_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_95_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_95_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_95_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_95_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_95_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_95_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_95_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_95_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_95_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_95_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_95_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_95_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_95_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_95_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_95_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_95_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_95_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_95_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_95_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_95_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_95_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_95_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_95_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_95_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_95_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_95_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_95_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_95_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_95_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_95_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_95_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_95_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_95_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_95_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_95_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_95_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_95_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_95_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_95_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_95_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_95_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_95_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_95_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_95_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_95_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_95_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_95_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_95_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_95_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_95_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_95_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_95_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_95_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_95_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_95_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_95_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_95_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_95_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_95_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_95_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_95_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_95_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_95_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_95_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_95_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_95_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_95_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_95_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_95_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_95_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_95_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_95_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_95_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_95_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_95_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_95_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_95_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_95_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_95_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_95_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_95_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_95_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_95_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_95_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_95_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_95_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_95_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_95_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_95_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_95_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_95_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_95_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_95_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_95_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_95_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_95_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_95_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_95_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_95_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_95_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_95_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_95_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_95_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_95_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_95_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_95_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_95_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_95_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_95_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_95_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_95_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_95_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_95_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_95_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_95_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_95_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_95_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_95_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_95_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_95_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_95_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_95_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_95_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_95_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_95_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_95_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_95_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_95_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_95_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_95_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_95_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_95_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_95_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_95_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_95_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_95_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_95_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_95_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_95_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_95_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_95_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_95_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_95_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_95_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_95_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_95_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_96_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_96_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_96_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_96_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_96_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_96_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_96_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_96_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_96_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_96_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_96_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_96_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_96_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_96_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_96_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_96_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_96_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_96_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_96_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_96_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_96_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_96_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_96_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_96_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_96_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_96_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_96_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_96_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_96_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_96_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_96_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_96_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_96_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_96_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_96_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_96_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_96_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_96_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_96_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_96_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_96_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_96_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_96_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_96_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_96_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_96_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_96_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_96_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_96_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_96_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_96_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_96_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_96_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_96_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_96_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_96_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_96_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_96_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_96_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_96_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_96_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_96_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_96_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_96_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_96_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_96_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_96_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_96_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_96_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_96_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_96_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_96_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_96_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_96_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_96_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_96_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_96_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_96_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_96_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_96_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_96_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_96_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_96_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_96_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_96_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_96_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_96_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_96_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_96_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_96_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_96_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_96_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_96_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_96_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_96_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_96_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_96_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_96_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_96_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_96_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_96_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_96_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_96_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_96_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_96_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_96_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_96_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_96_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_96_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_96_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_96_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_96_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_96_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_96_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_96_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_96_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_96_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_96_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_96_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_96_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_96_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_96_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_96_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_96_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_96_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_96_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_96_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_96_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_96_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_96_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_96_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_96_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_96_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_96_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_96_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_96_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_96_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_96_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_96_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_96_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_96_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_96_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_96_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_96_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_96_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_96_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_96_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_96_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_96_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_96_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_96_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_96_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_96_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_96_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_96_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_96_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_96_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_96_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_96_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_96_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_96_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_96_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_96_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_96_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_96_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_96_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_96_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_96_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_96_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_96_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_96_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_96_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_96_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_96_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_96_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_96_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_96_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_96_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_96_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_96_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_96_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_96_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_96_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_96_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_96_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_96_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_96_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_96_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_96_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_96_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_96_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_96_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_96_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_96_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_96_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_96_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_96_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_96_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_96_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_96_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_96_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_96_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_96_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_96_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_96_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_96_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_96_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_96_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_96_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_96_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_96_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_96_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_96_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_96_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_96_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_96_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_96_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_96_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_96_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_96_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_96_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_96_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_96_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_96_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_96_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_96_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_96_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_96_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_96_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_96_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_96_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_96_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_96_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_96_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_96_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_96_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_96_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_96_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_96_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_96_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_96_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_96_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_96_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_96_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_96_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_96_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_96_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_96_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_96_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_96_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_96_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_96_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_96_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_96_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_96_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_96_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_96_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_96_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_96_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_96_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_96_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_96_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_96_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_96_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_96_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_96_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_96_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_96_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_96_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_96_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_96_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_96_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_96_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_96_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_96_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_96_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_96_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_96_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_96_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_96_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_96_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_96_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_96_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_96_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_96_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_96_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_96_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_96_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_96_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_96_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_96_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_96_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_96_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_96_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_96_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_96_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_96_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_96_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_96_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_96_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_96_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_96_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_96_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_96_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_96_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_96_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_96_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_96_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_96_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_96_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_96_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_96_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_96_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_96_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_96_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_96_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_96_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_96_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_96_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_96_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_96_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_96_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_96_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_96_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_96_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_96_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_96_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_96_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_96_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_96_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_96_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_96_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_96_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_96_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_96_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_96_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_96_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_96_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_96_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_96_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_96_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_96_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_97_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_97_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_97_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_97_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_97_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_97_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_97_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_97_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_97_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_97_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_97_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_97_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_97_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_97_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_97_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_97_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_97_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_97_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_97_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_97_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_97_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_97_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_97_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_97_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_97_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_97_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_97_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_97_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_97_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_97_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_97_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_97_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_97_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_97_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_97_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_97_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_97_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_97_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_97_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_97_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_97_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_97_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_97_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_97_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_97_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_97_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_97_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_97_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_97_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_97_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_97_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_97_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_97_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_97_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_97_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_97_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_97_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_97_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_97_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_97_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_97_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_97_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_97_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_97_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_97_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_97_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_97_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_97_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_97_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_97_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_97_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_97_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_97_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_97_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_97_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_97_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_97_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_97_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_97_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_97_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_97_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_97_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_97_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_97_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_97_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_97_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_97_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_97_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_97_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_97_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_97_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_97_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_97_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_97_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_97_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_97_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_97_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_97_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_97_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_97_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_97_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_97_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_97_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_97_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_97_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_97_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_97_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_97_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_97_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_97_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_97_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_97_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_97_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_97_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_97_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_97_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_97_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_97_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_97_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_97_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_97_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_97_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_97_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_97_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_97_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_97_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_97_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_97_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_97_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_97_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_97_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_97_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_97_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_97_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_97_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_97_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_97_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_97_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_97_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_97_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_97_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_97_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_97_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_97_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_97_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_97_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_97_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_97_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_97_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_97_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_97_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_97_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_97_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_97_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_97_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_97_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_97_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_97_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_97_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_97_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_97_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_97_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_97_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_97_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_97_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_97_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_97_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_97_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_97_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_97_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_97_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_97_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_97_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_97_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_97_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_97_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_97_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_97_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_97_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_97_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_97_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_97_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_97_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_97_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_97_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_97_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_97_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_97_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_97_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_97_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_97_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_97_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_97_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_97_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_97_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_97_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_97_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_97_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_97_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_97_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_97_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_97_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_97_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_97_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_97_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_97_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_97_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_97_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_97_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_97_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_97_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_97_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_97_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_97_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_97_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_97_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_97_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_97_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_97_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_97_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_97_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_97_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_97_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_97_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_97_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_97_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_97_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_97_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_97_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_97_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_97_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_97_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_97_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_97_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_97_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_97_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_97_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_97_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_97_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_97_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_97_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_97_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_97_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_97_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_97_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_97_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_97_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_97_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_97_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_97_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_97_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_97_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_97_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_97_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_97_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_97_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_97_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_97_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_97_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_97_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_97_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_97_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_97_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_97_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_97_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_97_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_97_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_97_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_97_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_97_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_97_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_97_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_97_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_97_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_97_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_97_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_97_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_97_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_97_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_97_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_97_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_97_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_97_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_97_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_97_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_97_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_97_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_97_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_97_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_97_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_97_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_97_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_97_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_97_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_97_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_97_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_97_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_97_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_97_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_97_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_97_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_97_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_97_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_97_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_97_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_97_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_97_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_97_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_97_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_97_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_97_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_97_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_97_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_97_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_97_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_97_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_97_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_97_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_97_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_97_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_97_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_97_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_97_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_97_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_97_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_97_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_97_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_97_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_97_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_97_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_97_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_97_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_97_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_97_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_97_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_97_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_97_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_97_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_97_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_97_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_97_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_97_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_98_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_98_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_98_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_98_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_98_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_98_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_98_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_98_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_98_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_98_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_98_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_98_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_98_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_98_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_98_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_98_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_98_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_98_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_98_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_98_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_98_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_98_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_98_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_98_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_98_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_98_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_98_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_98_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_98_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_98_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_98_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_98_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_98_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_98_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_98_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_98_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_98_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_98_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_98_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_98_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_98_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_98_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_98_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_98_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_98_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_98_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_98_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_98_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_98_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_98_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_98_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_98_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_98_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_98_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_98_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_98_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_98_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_98_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_98_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_98_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_98_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_98_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_98_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_98_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_98_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_98_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_98_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_98_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_98_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_98_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_98_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_98_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_98_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_98_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_98_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_98_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_98_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_98_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_98_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_98_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_98_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_98_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_98_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_98_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_98_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_98_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_98_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_98_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_98_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_98_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_98_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_98_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_98_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_98_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_98_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_98_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_98_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_98_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_98_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_98_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_98_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_98_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_98_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_98_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_98_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_98_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_98_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_98_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_98_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_98_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_98_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_98_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_98_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_98_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_98_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_98_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_98_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_98_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_98_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_98_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_98_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_98_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_98_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_98_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_98_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_98_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_98_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_98_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_98_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_98_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_98_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_98_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_98_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_98_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_98_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_98_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_98_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_98_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_98_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_98_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_98_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_98_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_98_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_98_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_98_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_98_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_98_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_98_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_98_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_98_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_98_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_98_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_98_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_98_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_98_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_98_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_98_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_98_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_98_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_98_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_98_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_98_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_98_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_98_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_98_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_98_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_98_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_98_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_98_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_98_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_98_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_98_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_98_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_98_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_98_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_98_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_98_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_98_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_98_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_98_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_98_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_98_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_98_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_98_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_98_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_98_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_98_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_98_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_98_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_98_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_98_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_98_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_98_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_98_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_98_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_98_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_98_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_98_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_98_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_98_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_98_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_98_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_98_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_98_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_98_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_98_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_98_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_98_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_98_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_98_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_98_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_98_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_98_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_98_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_98_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_98_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_98_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_98_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_98_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_98_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_98_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_98_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_98_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_98_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_98_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_98_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_98_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_98_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_98_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_98_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_98_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_98_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_98_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_98_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_98_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_98_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_98_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_98_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_98_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_98_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_98_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_98_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_98_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_98_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_98_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_98_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_98_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_98_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_98_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_98_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_98_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_98_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_98_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_98_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_98_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_98_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_98_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_98_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_98_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_98_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_98_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_98_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_98_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_98_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_98_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_98_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_98_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_98_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_98_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_98_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_98_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_98_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_98_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_98_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_98_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_98_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_98_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_98_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_98_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_98_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_98_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_98_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_98_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_98_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_98_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_98_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_98_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_98_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_98_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_98_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_98_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_98_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_98_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_98_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_98_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_98_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_98_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_98_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_98_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_98_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_98_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_98_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_98_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_98_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_98_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_98_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_98_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_98_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_98_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_98_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_98_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_98_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_98_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_98_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_98_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_98_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_98_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_98_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_98_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_98_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_98_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_98_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_98_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_98_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_98_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_98_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_98_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_98_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_98_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_98_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_98_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_98_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_98_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_98_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_98_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_98_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_98_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_98_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_98_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_98_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_98_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_98_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_99_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_99_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_99_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_99_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_99_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_99_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_99_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_99_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_99_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_99_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_99_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_99_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_99_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_99_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_99_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_99_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_99_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_99_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_99_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_99_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_99_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_99_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_99_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_99_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_99_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_99_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_99_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_99_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_99_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_99_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_99_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_99_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_99_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_99_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_99_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_99_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_99_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_99_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_99_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_99_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_99_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_99_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_99_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_99_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_99_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_99_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_99_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_99_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_99_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_99_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_99_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_99_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_99_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_99_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_99_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_99_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_99_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_99_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_99_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_99_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_99_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_99_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_99_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_99_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_99_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_99_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_99_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_99_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_99_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_99_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_99_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_99_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_99_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_99_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_99_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_99_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_99_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_99_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_99_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_99_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_99_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_99_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_99_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_99_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_99_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_99_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_99_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_99_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_99_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_99_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_99_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_99_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_99_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_99_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_99_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_99_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_99_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_99_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_99_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_99_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_99_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_99_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_99_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_99_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_99_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_99_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_99_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_99_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_99_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_99_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_99_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_99_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_99_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_99_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_99_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_99_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_99_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_99_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_99_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_99_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_99_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_99_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_99_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_99_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_99_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_99_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_99_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_99_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_99_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_99_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_99_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_99_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_99_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_99_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_99_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_99_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_99_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_99_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_99_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_99_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_99_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_99_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_99_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_99_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_99_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_99_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_99_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_99_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_99_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_99_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_99_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_99_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_99_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_99_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_99_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_99_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_99_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_99_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_99_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_99_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_99_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_99_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_99_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_99_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_99_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_99_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_99_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_99_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_99_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_99_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_99_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_99_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_99_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_99_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_99_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_99_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_99_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_99_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_99_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_99_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_99_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_99_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_99_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_99_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_99_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_99_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_99_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_99_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_99_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_99_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_99_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_99_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_99_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_99_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_99_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_99_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_99_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_99_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_99_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_99_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_99_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_99_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_99_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_99_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_99_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_99_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_99_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_99_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_99_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_99_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_99_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_99_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_99_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_99_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_99_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_99_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_99_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_99_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_99_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_99_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_99_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_99_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_99_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_99_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_99_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_99_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_99_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_99_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_99_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_99_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_99_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_99_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_99_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_99_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_99_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_99_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_99_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_99_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_99_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_99_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_99_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_99_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_99_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_99_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_99_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_99_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_99_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_99_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_99_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_99_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_99_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_99_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_99_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_99_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_99_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_99_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_99_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_99_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_99_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_99_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_99_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_99_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_99_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_99_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_99_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_99_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_99_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_99_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_99_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_99_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_99_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_99_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_99_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_99_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_99_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_99_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_99_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_99_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_99_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_99_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_99_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_99_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_99_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_99_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_99_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_99_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_99_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_99_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_99_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_99_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_99_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_99_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_99_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_99_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_99_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_99_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_99_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_99_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_99_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_99_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_99_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_99_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_99_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_99_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_99_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_99_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_99_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_99_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_99_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_99_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_99_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_99_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_99_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_99_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_99_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_99_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_99_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_99_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_99_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_99_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_99_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_99_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_99_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_99_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_99_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_99_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_99_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_99_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_99_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_99_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_99_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_99_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_99_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_99_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_99_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_99_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_99_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_99_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_99_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_99_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_99_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_99_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_100_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_100_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_100_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_100_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_100_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_100_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_100_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_100_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_100_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_100_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_100_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_100_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_100_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_100_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_100_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_100_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_100_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_100_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_100_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_100_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_100_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_100_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_100_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_100_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_100_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_100_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_100_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_100_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_100_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_100_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_100_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_100_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_100_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_100_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_100_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_100_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_100_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_100_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_100_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_100_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_100_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_100_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_100_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_100_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_100_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_100_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_100_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_100_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_100_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_100_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_100_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_100_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_100_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_100_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_100_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_100_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_100_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_100_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_100_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_100_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_100_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_100_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_100_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_100_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_100_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_100_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_100_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_100_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_100_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_100_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_100_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_100_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_100_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_100_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_100_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_100_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_100_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_100_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_100_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_100_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_100_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_100_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_100_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_100_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_100_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_100_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_100_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_100_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_100_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_100_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_100_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_100_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_100_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_100_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_100_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_100_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_100_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_100_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_100_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_100_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_100_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_100_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_100_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_100_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_100_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_100_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_100_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_100_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_100_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_100_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_100_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_100_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_100_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_100_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_100_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_100_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_100_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_100_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_100_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_100_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_100_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_100_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_100_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_100_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_100_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_100_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_100_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_100_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_100_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_100_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_100_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_100_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_100_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_100_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_100_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_100_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_100_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_100_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_100_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_100_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_100_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_100_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_100_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_100_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_100_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_100_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_100_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_100_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_100_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_100_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_100_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_100_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_100_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_100_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_100_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_100_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_100_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_100_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_100_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_100_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_100_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_100_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_100_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_100_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_100_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_100_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_100_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_100_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_100_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_100_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_100_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_100_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_100_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_100_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_100_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_100_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_100_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_100_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_100_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_100_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_100_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_100_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_100_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_100_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_100_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_100_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_100_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_100_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_100_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_100_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_100_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_100_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_100_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_100_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_100_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_100_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_100_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_100_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_100_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_100_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_100_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_100_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_100_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_100_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_100_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_100_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_100_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_100_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_100_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_100_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_100_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_100_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_100_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_100_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_100_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_100_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_100_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_100_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_100_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_100_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_100_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_100_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_100_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_100_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_100_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_100_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_100_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_100_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_100_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_100_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_100_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_100_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_100_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_100_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_100_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_100_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_100_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_100_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_100_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_100_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_100_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_100_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_100_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_100_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_100_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_100_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_100_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_100_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_100_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_100_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_100_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_100_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_100_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_100_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_100_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_100_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_100_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_100_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_100_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_100_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_100_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_100_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_100_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_100_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_100_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_100_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_100_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_100_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_100_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_100_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_100_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_100_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_100_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_100_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_100_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_100_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_100_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_100_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_100_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_100_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_100_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_100_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_100_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_100_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_100_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_100_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_100_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_100_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_100_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_100_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_100_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_100_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_100_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_100_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_100_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_100_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_100_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_100_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_100_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_100_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_100_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_100_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_100_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_100_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_100_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_100_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_100_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_100_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_100_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_100_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_100_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_100_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_100_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_100_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_100_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_100_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_100_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_100_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_100_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_100_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_100_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_100_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_100_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_100_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_100_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_100_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_100_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_100_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_100_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_100_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_100_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_100_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_100_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_100_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_100_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_100_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_100_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_100_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_100_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_100_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_100_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_100_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_101_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_101_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_101_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_101_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_101_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_101_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_101_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_101_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_101_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_101_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_101_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_101_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_101_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_101_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_101_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_101_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_101_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_101_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_101_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_101_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_101_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_101_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_101_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_101_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_101_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_101_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_101_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_101_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_101_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_101_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_101_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_101_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_101_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_101_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_101_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_101_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_101_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_101_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_101_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_101_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_101_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_101_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_101_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_101_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_101_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_101_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_101_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_101_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_101_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_101_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_101_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_101_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_101_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_101_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_101_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_101_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_101_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_101_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_101_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_101_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_101_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_101_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_101_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_101_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_101_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_101_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_101_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_101_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_101_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_101_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_101_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_101_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_101_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_101_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_101_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_101_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_101_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_101_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_101_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_101_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_101_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_101_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_101_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_101_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_101_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_101_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_101_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_101_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_101_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_101_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_101_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_101_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_101_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_101_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_101_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_101_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_101_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_101_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_101_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_101_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_101_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_101_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_101_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_101_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_101_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_101_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_101_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_101_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_101_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_101_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_101_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_101_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_101_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_101_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_101_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_101_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_101_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_101_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_101_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_101_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_101_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_101_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_101_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_101_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_101_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_101_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_101_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_101_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_101_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_101_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_101_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_101_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_101_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_101_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_101_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_101_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_101_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_101_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_101_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_101_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_101_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_101_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_101_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_101_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_101_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_101_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_101_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_101_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_101_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_101_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_101_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_101_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_101_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_101_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_101_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_101_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_101_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_101_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_101_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_101_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_101_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_101_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_101_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_101_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_101_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_101_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_101_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_101_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_101_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_101_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_101_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_101_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_101_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_101_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_101_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_101_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_101_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_101_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_101_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_101_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_101_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_101_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_101_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_101_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_101_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_101_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_101_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_101_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_101_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_101_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_101_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_101_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_101_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_101_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_101_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_101_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_101_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_101_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_101_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_101_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_101_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_101_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_101_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_101_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_101_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_101_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_101_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_101_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_101_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_101_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_101_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_101_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_101_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_101_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_101_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_101_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_101_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_101_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_101_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_101_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_101_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_101_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_101_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_101_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_101_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_101_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_101_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_101_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_101_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_101_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_101_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_101_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_101_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_101_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_101_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_101_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_101_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_101_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_101_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_101_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_101_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_101_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_101_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_101_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_101_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_101_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_101_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_101_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_101_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_101_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_101_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_101_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_101_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_101_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_101_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_101_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_101_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_101_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_101_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_101_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_101_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_101_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_101_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_101_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_101_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_101_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_101_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_101_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_101_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_101_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_101_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_101_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_101_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_101_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_101_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_101_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_101_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_101_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_101_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_101_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_101_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_101_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_101_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_101_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_101_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_101_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_101_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_101_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_101_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_101_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_101_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_101_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_101_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_101_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_101_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_101_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_101_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_101_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_101_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_101_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_101_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_101_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_101_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_101_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_101_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_101_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_101_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_101_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_101_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_101_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_101_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_101_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_101_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_101_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_101_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_101_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_101_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_101_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_101_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_101_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_101_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_101_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_101_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_101_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_101_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_101_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_101_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_101_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_101_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_101_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_101_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_101_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_101_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_101_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_101_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_101_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_101_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_101_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_101_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_101_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_101_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_101_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_102_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_102_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_102_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_102_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_102_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_102_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_102_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_102_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_102_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_102_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_102_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_102_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_102_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_102_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_102_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_102_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_102_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_102_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_102_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_102_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_102_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_102_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_102_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_102_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_102_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_102_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_102_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_102_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_102_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_102_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_102_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_102_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_102_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_102_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_102_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_102_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_102_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_102_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_102_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_102_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_102_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_102_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_102_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_102_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_102_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_102_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_102_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_102_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_102_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_102_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_102_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_102_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_102_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_102_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_102_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_102_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_102_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_102_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_102_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_102_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_102_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_102_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_102_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_102_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_102_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_102_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_102_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_102_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_102_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_102_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_102_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_102_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_102_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_102_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_102_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_102_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_102_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_102_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_102_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_102_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_102_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_102_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_102_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_102_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_102_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_102_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_102_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_102_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_102_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_102_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_102_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_102_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_102_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_102_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_102_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_102_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_102_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_102_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_102_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_102_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_102_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_102_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_102_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_102_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_102_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_102_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_102_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_102_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_102_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_102_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_102_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_102_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_102_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_102_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_102_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_102_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_102_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_102_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_102_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_102_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_102_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_102_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_102_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_102_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_102_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_102_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_102_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_102_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_102_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_102_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_102_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_102_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_102_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_102_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_102_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_102_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_102_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_102_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_102_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_102_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_102_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_102_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_102_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_102_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_102_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_102_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_102_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_102_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_102_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_102_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_102_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_102_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_102_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_102_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_102_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_102_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_102_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_102_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_102_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_102_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_102_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_102_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_102_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_102_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_102_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_102_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_102_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_102_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_102_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_102_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_102_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_102_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_102_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_102_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_102_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_102_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_102_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_102_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_102_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_102_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_102_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_102_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_102_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_102_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_102_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_102_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_102_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_102_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_102_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_102_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_102_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_102_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_102_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_102_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_102_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_102_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_102_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_102_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_102_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_102_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_102_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_102_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_102_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_102_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_102_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_102_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_102_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_102_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_102_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_102_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_102_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_102_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_102_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_102_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_102_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_102_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_102_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_102_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_102_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_102_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_102_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_102_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_102_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_102_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_102_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_102_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_102_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_102_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_102_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_102_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_102_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_102_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_102_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_102_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_102_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_102_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_102_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_102_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_102_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_102_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_102_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_102_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_102_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_102_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_102_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_102_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_102_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_102_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_102_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_102_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_102_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_102_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_102_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_102_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_102_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_102_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_102_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_102_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_102_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_102_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_102_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_102_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_102_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_102_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_102_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_102_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_102_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_102_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_102_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_102_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_102_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_102_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_102_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_102_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_102_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_102_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_102_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_102_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_102_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_102_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_102_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_102_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_102_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_102_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_102_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_102_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_102_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_102_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_102_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_102_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_102_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_102_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_102_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_102_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_102_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_102_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_102_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_102_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_102_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_102_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_102_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_102_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_102_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_102_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_102_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_102_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_102_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_102_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_102_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_102_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_102_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_102_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_102_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_102_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_102_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_102_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_102_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_102_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_102_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_102_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_102_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_102_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_102_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_102_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_102_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_102_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_102_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_102_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_102_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_102_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_102_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_102_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_102_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_102_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_102_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_102_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_102_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_102_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_102_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_102_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_102_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_102_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_103_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_103_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_103_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_103_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_103_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_103_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_103_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_103_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_103_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_103_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_103_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_103_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_103_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_103_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_103_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_103_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_103_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_103_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_103_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_103_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_103_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_103_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_103_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_103_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_103_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_103_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_103_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_103_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_103_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_103_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_103_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_103_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_103_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_103_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_103_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_103_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_103_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_103_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_103_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_103_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_103_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_103_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_103_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_103_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_103_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_103_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_103_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_103_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_103_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_103_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_103_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_103_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_103_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_103_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_103_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_103_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_103_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_103_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_103_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_103_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_103_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_103_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_103_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_103_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_103_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_103_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_103_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_103_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_103_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_103_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_103_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_103_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_103_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_103_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_103_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_103_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_103_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_103_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_103_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_103_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_103_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_103_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_103_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_103_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_103_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_103_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_103_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_103_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_103_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_103_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_103_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_103_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_103_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_103_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_103_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_103_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_103_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_103_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_103_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_103_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_103_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_103_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_103_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_103_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_103_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_103_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_103_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_103_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_103_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_103_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_103_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_103_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_103_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_103_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_103_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_103_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_103_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_103_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_103_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_103_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_103_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_103_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_103_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_103_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_103_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_103_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_103_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_103_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_103_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_103_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_103_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_103_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_103_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_103_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_103_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_103_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_103_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_103_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_103_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_103_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_103_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_103_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_103_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_103_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_103_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_103_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_103_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_103_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_103_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_103_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_103_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_103_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_103_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_103_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_103_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_103_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_103_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_103_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_103_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_103_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_103_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_103_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_103_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_103_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_103_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_103_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_103_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_103_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_103_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_103_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_103_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_103_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_103_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_103_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_103_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_103_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_103_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_103_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_103_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_103_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_103_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_103_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_103_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_103_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_103_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_103_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_103_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_103_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_103_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_103_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_103_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_103_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_103_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_103_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_103_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_103_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_103_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_103_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_103_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_103_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_103_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_103_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_103_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_103_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_103_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_103_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_103_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_103_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_103_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_103_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_103_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_103_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_103_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_103_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_103_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_103_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_103_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_103_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_103_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_103_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_103_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_103_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_103_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_103_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_103_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_103_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_103_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_103_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_103_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_103_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_103_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_103_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_103_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_103_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_103_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_103_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_103_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_103_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_103_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_103_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_103_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_103_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_103_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_103_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_103_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_103_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_103_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_103_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_103_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_103_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_103_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_103_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_103_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_103_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_103_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_103_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_103_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_103_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_103_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_103_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_103_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_103_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_103_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_103_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_103_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_103_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_103_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_103_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_103_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_103_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_103_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_103_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_103_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_103_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_103_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_103_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_103_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_103_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_103_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_103_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_103_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_103_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_103_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_103_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_103_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_103_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_103_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_103_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_103_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_103_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_103_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_103_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_103_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_103_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_103_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_103_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_103_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_103_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_103_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_103_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_103_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_103_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_103_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_103_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_103_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_103_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_103_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_103_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_103_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_103_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_103_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_103_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_103_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_103_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_103_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_103_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_103_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_103_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_103_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_103_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_103_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_103_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_103_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_103_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_103_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_103_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_103_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_103_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_103_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_103_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_103_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_103_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_103_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_103_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_103_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_103_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_103_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_103_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_103_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_103_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_103_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_103_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_104_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_104_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_104_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_104_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_104_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_104_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_104_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_104_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_104_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_104_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_104_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_104_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_104_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_104_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_104_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_104_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_104_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_104_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_104_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_104_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_104_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_104_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_104_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_104_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_104_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_104_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_104_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_104_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_104_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_104_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_104_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_104_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_104_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_104_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_104_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_104_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_104_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_104_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_104_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_104_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_104_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_104_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_104_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_104_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_104_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_104_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_104_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_104_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_104_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_104_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_104_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_104_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_104_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_104_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_104_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_104_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_104_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_104_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_104_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_104_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_104_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_104_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_104_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_104_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_104_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_104_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_104_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_104_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_104_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_104_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_104_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_104_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_104_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_104_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_104_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_104_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_104_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_104_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_104_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_104_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_104_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_104_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_104_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_104_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_104_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_104_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_104_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_104_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_104_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_104_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_104_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_104_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_104_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_104_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_104_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_104_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_104_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_104_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_104_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_104_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_104_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_104_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_104_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_104_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_104_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_104_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_104_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_104_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_104_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_104_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_104_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_104_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_104_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_104_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_104_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_104_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_104_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_104_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_104_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_104_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_104_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_104_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_104_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_104_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_104_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_104_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_104_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_104_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_104_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_104_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_104_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_104_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_104_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_104_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_104_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_104_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_104_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_104_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_104_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_104_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_104_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_104_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_104_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_104_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_104_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_104_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_104_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_104_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_104_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_104_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_104_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_104_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_104_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_104_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_104_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_104_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_104_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_104_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_104_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_104_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_104_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_104_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_104_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_104_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_104_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_104_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_104_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_104_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_104_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_104_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_104_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_104_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_104_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_104_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_104_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_104_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_104_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_104_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_104_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_104_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_104_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_104_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_104_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_104_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_104_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_104_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_104_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_104_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_104_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_104_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_104_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_104_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_104_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_104_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_104_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_104_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_104_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_104_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_104_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_104_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_104_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_104_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_104_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_104_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_104_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_104_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_104_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_104_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_104_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_104_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_104_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_104_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_104_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_104_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_104_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_104_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_104_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_104_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_104_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_104_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_104_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_104_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_104_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_104_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_104_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_104_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_104_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_104_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_104_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_104_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_104_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_104_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_104_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_104_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_104_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_104_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_104_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_104_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_104_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_104_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_104_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_104_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_104_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_104_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_104_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_104_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_104_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_104_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_104_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_104_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_104_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_104_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_104_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_104_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_104_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_104_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_104_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_104_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_104_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_104_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_104_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_104_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_104_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_104_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_104_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_104_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_104_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_104_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_104_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_104_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_104_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_104_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_104_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_104_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_104_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_104_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_104_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_104_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_104_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_104_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_104_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_104_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_104_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_104_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_104_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_104_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_104_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_104_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_104_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_104_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_104_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_104_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_104_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_104_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_104_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_104_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_104_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_104_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_104_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_104_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_104_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_104_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_104_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_104_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_104_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_104_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_104_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_104_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_104_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_104_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_104_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_104_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_104_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_104_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_104_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_104_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_104_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_104_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_104_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_104_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_104_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_104_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_104_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_104_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_104_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_104_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_104_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_104_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_104_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_104_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_104_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_104_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_104_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_104_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_104_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_104_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_104_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_104_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_104_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_104_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_104_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_104_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_105_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_105_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_105_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_105_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_105_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_105_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_105_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_105_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_105_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_105_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_105_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_105_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_105_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_105_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_105_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_105_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_105_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_105_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_105_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_105_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_105_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_105_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_105_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_105_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_105_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_105_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_105_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_105_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_105_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_105_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_105_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_105_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_105_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_105_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_105_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_105_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_105_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_105_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_105_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_105_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_105_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_105_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_105_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_105_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_105_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_105_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_105_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_105_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_105_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_105_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_105_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_105_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_105_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_105_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_105_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_105_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_105_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_105_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_105_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_105_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_105_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_105_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_105_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_105_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_105_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_105_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_105_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_105_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_105_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_105_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_105_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_105_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_105_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_105_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_105_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_105_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_105_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_105_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_105_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_105_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_105_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_105_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_105_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_105_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_105_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_105_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_105_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_105_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_105_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_105_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_105_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_105_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_105_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_105_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_105_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_105_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_105_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_105_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_105_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_105_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_105_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_105_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_105_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_105_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_105_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_105_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_105_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_105_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_105_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_105_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_105_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_105_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_105_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_105_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_105_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_105_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_105_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_105_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_105_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_105_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_105_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_105_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_105_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_105_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_105_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_105_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_105_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_105_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_105_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_105_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_105_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_105_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_105_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_105_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_105_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_105_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_105_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_105_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_105_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_105_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_105_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_105_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_105_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_105_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_105_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_105_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_105_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_105_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_105_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_105_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_105_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_105_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_105_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_105_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_105_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_105_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_105_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_105_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_105_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_105_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_105_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_105_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_105_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_105_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_105_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_105_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_105_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_105_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_105_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_105_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_105_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_105_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_105_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_105_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_105_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_105_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_105_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_105_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_105_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_105_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_105_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_105_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_105_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_105_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_105_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_105_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_105_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_105_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_105_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_105_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_105_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_105_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_105_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_105_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_105_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_105_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_105_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_105_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_105_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_105_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_105_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_105_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_105_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_105_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_105_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_105_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_105_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_105_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_105_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_105_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_105_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_105_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_105_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_105_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_105_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_105_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_105_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_105_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_105_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_105_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_105_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_105_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_105_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_105_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_105_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_105_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_105_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_105_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_105_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_105_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_105_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_105_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_105_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_105_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_105_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_105_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_105_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_105_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_105_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_105_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_105_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_105_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_105_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_105_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_105_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_105_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_105_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_105_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_105_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_105_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_105_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_105_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_105_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_105_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_105_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_105_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_105_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_105_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_105_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_105_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_105_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_105_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_105_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_105_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_105_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_105_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_105_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_105_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_105_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_105_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_105_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_105_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_105_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_105_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_105_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_105_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_105_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_105_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_105_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_105_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_105_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_105_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_105_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_105_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_105_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_105_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_105_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_105_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_105_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_105_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_105_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_105_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_105_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_105_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_105_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_105_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_105_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_105_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_105_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_105_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_105_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_105_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_105_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_105_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_105_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_105_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_105_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_105_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_105_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_105_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_105_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_105_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_105_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_105_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_105_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_105_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_105_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_105_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_105_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_105_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_105_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_105_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_105_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_105_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_105_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_105_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_105_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_105_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_105_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_105_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_105_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_105_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_105_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_105_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_105_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_105_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_105_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_105_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_105_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_105_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_105_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_105_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_106_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_106_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_106_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_106_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_106_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_106_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_106_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_106_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_106_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_106_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_106_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_106_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_106_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_106_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_106_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_106_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_106_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_106_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_106_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_106_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_106_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_106_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_106_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_106_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_106_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_106_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_106_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_106_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_106_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_106_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_106_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_106_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_106_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_106_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_106_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_106_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_106_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_106_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_106_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_106_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_106_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_106_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_106_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_106_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_106_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_106_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_106_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_106_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_106_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_106_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_106_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_106_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_106_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_106_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_106_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_106_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_106_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_106_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_106_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_106_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_106_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_106_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_106_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_106_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_106_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_106_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_106_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_106_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_106_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_106_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_106_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_106_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_106_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_106_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_106_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_106_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_106_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_106_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_106_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_106_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_106_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_106_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_106_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_106_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_106_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_106_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_106_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_106_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_106_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_106_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_106_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_106_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_106_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_106_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_106_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_106_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_106_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_106_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_106_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_106_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_106_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_106_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_106_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_106_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_106_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_106_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_106_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_106_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_106_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_106_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_106_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_106_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_106_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_106_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_106_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_106_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_106_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_106_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_106_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_106_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_106_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_106_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_106_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_106_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_106_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_106_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_106_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_106_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_106_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_106_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_106_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_106_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_106_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_106_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_106_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_106_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_106_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_106_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_106_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_106_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_106_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_106_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_106_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_106_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_106_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_106_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_106_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_106_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_106_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_106_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_106_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_106_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_106_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_106_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_106_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_106_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_106_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_106_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_106_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_106_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_106_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_106_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_106_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_106_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_106_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_106_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_106_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_106_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_106_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_106_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_106_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_106_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_106_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_106_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_106_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_106_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_106_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_106_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_106_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_106_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_106_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_106_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_106_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_106_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_106_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_106_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_106_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_106_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_106_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_106_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_106_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_106_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_106_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_106_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_106_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_106_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_106_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_106_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_106_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_106_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_106_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_106_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_106_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_106_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_106_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_106_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_106_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_106_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_106_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_106_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_106_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_106_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_106_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_106_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_106_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_106_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_106_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_106_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_106_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_106_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_106_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_106_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_106_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_106_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_106_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_106_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_106_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_106_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_106_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_106_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_106_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_106_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_106_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_106_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_106_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_106_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_106_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_106_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_106_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_106_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_106_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_106_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_106_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_106_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_106_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_106_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_106_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_106_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_106_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_106_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_106_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_106_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_106_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_106_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_106_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_106_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_106_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_106_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_106_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_106_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_106_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_106_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_106_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_106_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_106_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_106_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_106_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_106_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_106_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_106_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_106_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_106_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_106_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_106_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_106_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_106_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_106_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_106_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_106_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_106_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_106_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_106_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_106_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_106_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_106_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_106_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_106_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_106_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_106_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_106_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_106_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_106_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_106_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_106_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_106_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_106_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_106_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_106_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_106_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_106_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_106_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_106_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_106_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_106_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_106_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_106_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_106_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_106_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_106_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_106_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_106_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_106_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_106_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_106_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_106_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_106_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_106_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_106_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_106_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_106_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_106_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_106_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_106_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_106_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_106_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_106_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_106_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_106_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_106_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_106_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_106_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_106_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_106_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_106_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_106_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_106_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_106_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_106_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_106_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_106_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_106_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_106_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_107_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_107_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_107_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_107_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_107_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_107_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_107_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_107_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_107_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_107_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_107_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_107_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_107_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_107_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_107_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_107_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_107_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_107_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_107_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_107_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_107_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_107_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_107_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_107_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_107_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_107_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_107_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_107_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_107_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_107_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_107_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_107_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_107_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_107_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_107_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_107_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_107_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_107_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_107_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_107_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_107_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_107_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_107_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_107_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_107_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_107_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_107_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_107_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_107_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_107_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_107_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_107_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_107_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_107_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_107_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_107_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_107_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_107_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_107_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_107_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_107_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_107_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_107_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_107_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_107_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_107_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_107_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_107_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_107_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_107_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_107_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_107_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_107_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_107_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_107_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_107_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_107_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_107_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_107_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_107_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_107_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_107_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_107_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_107_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_107_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_107_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_107_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_107_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_107_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_107_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_107_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_107_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_107_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_107_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_107_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_107_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_107_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_107_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_107_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_107_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_107_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_107_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_107_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_107_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_107_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_107_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_107_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_107_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_107_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_107_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_107_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_107_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_107_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_107_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_107_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_107_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_107_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_107_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_107_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_107_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_107_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_107_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_107_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_107_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_107_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_107_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_107_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_107_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_107_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_107_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_107_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_107_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_107_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_107_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_107_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_107_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_107_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_107_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_107_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_107_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_107_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_107_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_107_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_107_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_107_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_107_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_107_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_107_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_107_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_107_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_107_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_107_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_107_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_107_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_107_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_107_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_107_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_107_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_107_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_107_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_107_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_107_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_107_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_107_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_107_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_107_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_107_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_107_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_107_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_107_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_107_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_107_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_107_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_107_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_107_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_107_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_107_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_107_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_107_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_107_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_107_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_107_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_107_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_107_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_107_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_107_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_107_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_107_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_107_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_107_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_107_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_107_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_107_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_107_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_107_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_107_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_107_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_107_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_107_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_107_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_107_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_107_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_107_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_107_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_107_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_107_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_107_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_107_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_107_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_107_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_107_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_107_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_107_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_107_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_107_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_107_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_107_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_107_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_107_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_107_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_107_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_107_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_107_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_107_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_107_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_107_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_107_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_107_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_107_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_107_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_107_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_107_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_107_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_107_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_107_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_107_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_107_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_107_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_107_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_107_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_107_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_107_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_107_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_107_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_107_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_107_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_107_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_107_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_107_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_107_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_107_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_107_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_107_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_107_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_107_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_107_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_107_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_107_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_107_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_107_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_107_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_107_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_107_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_107_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_107_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_107_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_107_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_107_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_107_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_107_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_107_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_107_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_107_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_107_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_107_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_107_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_107_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_107_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_107_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_107_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_107_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_107_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_107_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_107_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_107_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_107_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_107_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_107_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_107_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_107_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_107_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_107_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_107_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_107_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_107_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_107_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_107_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_107_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_107_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_107_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_107_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_107_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_107_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_107_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_107_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_107_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_107_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_107_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_107_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_107_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_107_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_107_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_107_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_107_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_107_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_107_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_107_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_107_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_107_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_107_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_107_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_107_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_107_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_107_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_107_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_107_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_107_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_107_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_107_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_107_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_107_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_107_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_107_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_107_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_107_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_107_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_107_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_107_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_107_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_107_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_107_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_107_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_108_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_108_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_108_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_108_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_108_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_108_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_108_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_108_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_108_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_108_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_108_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_108_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_108_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_108_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_108_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_108_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_108_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_108_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_108_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_108_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_108_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_108_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_108_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_108_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_108_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_108_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_108_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_108_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_108_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_108_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_108_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_108_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_108_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_108_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_108_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_108_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_108_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_108_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_108_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_108_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_108_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_108_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_108_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_108_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_108_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_108_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_108_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_108_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_108_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_108_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_108_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_108_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_108_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_108_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_108_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_108_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_108_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_108_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_108_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_108_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_108_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_108_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_108_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_108_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_108_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_108_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_108_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_108_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_108_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_108_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_108_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_108_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_108_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_108_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_108_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_108_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_108_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_108_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_108_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_108_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_108_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_108_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_108_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_108_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_108_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_108_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_108_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_108_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_108_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_108_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_108_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_108_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_108_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_108_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_108_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_108_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_108_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_108_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_108_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_108_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_108_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_108_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_108_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_108_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_108_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_108_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_108_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_108_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_108_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_108_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_108_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_108_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_108_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_108_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_108_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_108_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_108_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_108_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_108_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_108_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_108_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_108_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_108_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_108_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_108_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_108_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_108_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_108_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_108_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_108_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_108_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_108_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_108_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_108_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_108_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_108_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_108_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_108_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_108_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_108_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_108_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_108_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_108_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_108_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_108_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_108_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_108_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_108_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_108_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_108_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_108_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_108_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_108_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_108_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_108_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_108_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_108_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_108_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_108_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_108_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_108_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_108_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_108_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_108_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_108_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_108_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_108_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_108_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_108_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_108_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_108_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_108_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_108_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_108_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_108_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_108_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_108_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_108_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_108_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_108_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_108_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_108_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_108_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_108_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_108_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_108_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_108_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_108_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_108_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_108_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_108_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_108_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_108_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_108_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_108_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_108_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_108_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_108_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_108_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_108_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_108_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_108_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_108_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_108_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_108_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_108_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_108_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_108_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_108_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_108_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_108_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_108_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_108_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_108_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_108_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_108_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_108_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_108_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_108_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_108_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_108_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_108_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_108_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_108_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_108_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_108_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_108_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_108_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_108_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_108_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_108_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_108_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_108_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_108_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_108_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_108_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_108_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_108_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_108_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_108_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_108_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_108_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_108_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_108_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_108_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_108_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_108_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_108_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_108_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_108_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_108_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_108_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_108_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_108_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_108_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_108_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_108_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_108_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_108_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_108_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_108_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_108_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_108_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_108_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_108_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_108_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_108_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_108_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_108_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_108_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_108_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_108_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_108_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_108_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_108_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_108_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_108_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_108_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_108_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_108_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_108_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_108_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_108_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_108_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_108_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_108_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_108_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_108_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_108_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_108_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_108_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_108_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_108_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_108_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_108_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_108_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_108_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_108_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_108_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_108_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_108_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_108_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_108_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_108_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_108_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_108_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_108_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_108_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_108_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_108_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_108_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_108_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_108_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_108_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_108_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_108_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_108_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_108_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_108_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_108_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_108_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_108_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_108_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_108_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_108_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_108_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_108_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_108_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_108_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_108_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_108_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_108_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_108_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_108_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_108_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_108_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_108_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_108_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_108_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_108_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_108_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_108_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_109_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_109_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_109_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_109_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_109_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_109_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_109_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_109_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_109_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_109_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_109_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_109_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_109_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_109_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_109_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_109_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_109_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_109_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_109_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_109_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_109_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_109_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_109_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_109_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_109_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_109_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_109_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_109_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_109_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_109_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_109_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_109_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_109_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_109_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_109_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_109_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_109_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_109_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_109_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_109_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_109_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_109_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_109_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_109_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_109_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_109_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_109_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_109_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_109_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_109_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_109_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_109_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_109_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_109_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_109_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_109_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_109_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_109_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_109_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_109_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_109_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_109_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_109_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_109_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_109_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_109_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_109_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_109_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_109_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_109_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_109_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_109_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_109_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_109_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_109_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_109_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_109_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_109_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_109_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_109_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_109_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_109_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_109_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_109_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_109_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_109_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_109_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_109_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_109_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_109_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_109_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_109_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_109_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_109_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_109_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_109_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_109_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_109_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_109_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_109_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_109_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_109_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_109_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_109_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_109_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_109_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_109_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_109_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_109_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_109_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_109_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_109_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_109_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_109_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_109_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_109_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_109_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_109_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_109_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_109_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_109_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_109_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_109_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_109_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_109_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_109_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_109_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_109_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_109_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_109_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_109_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_109_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_109_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_109_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_109_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_109_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_109_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_109_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_109_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_109_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_109_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_109_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_109_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_109_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_109_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_109_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_109_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_109_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_109_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_109_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_109_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_109_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_109_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_109_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_109_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_109_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_109_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_109_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_109_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_109_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_109_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_109_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_109_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_109_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_109_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_109_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_109_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_109_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_109_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_109_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_109_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_109_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_109_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_109_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_109_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_109_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_109_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_109_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_109_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_109_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_109_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_109_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_109_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_109_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_109_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_109_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_109_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_109_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_109_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_109_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_109_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_109_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_109_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_109_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_109_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_109_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_109_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_109_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_109_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_109_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_109_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_109_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_109_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_109_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_109_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_109_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_109_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_109_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_109_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_109_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_109_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_109_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_109_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_109_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_109_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_109_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_109_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_109_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_109_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_109_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_109_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_109_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_109_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_109_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_109_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_109_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_109_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_109_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_109_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_109_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_109_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_109_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_109_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_109_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_109_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_109_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_109_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_109_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_109_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_109_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_109_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_109_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_109_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_109_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_109_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_109_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_109_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_109_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_109_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_109_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_109_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_109_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_109_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_109_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_109_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_109_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_109_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_109_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_109_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_109_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_109_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_109_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_109_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_109_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_109_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_109_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_109_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_109_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_109_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_109_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_109_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_109_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_109_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_109_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_109_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_109_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_109_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_109_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_109_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_109_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_109_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_109_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_109_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_109_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_109_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_109_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_109_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_109_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_109_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_109_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_109_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_109_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_109_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_109_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_109_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_109_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_109_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_109_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_109_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_109_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_109_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_109_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_109_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_109_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_109_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_109_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_109_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_109_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_109_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_109_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_109_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_109_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_109_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_109_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_109_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_109_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_109_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_109_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_109_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_109_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_109_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_109_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_109_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_109_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_109_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_109_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_109_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_109_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_109_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_109_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_109_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_109_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_109_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_109_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_109_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_109_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_109_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_109_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_109_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_109_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_109_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_109_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_110_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_110_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_110_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_110_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_110_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_110_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_110_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_110_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_110_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_110_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_110_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_110_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_110_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_110_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_110_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_110_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_110_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_110_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_110_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_110_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_110_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_110_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_110_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_110_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_110_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_110_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_110_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_110_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_110_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_110_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_110_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_110_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_110_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_110_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_110_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_110_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_110_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_110_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_110_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_110_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_110_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_110_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_110_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_110_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_110_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_110_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_110_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_110_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_110_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_110_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_110_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_110_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_110_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_110_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_110_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_110_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_110_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_110_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_110_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_110_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_110_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_110_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_110_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_110_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_110_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_110_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_110_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_110_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_110_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_110_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_110_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_110_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_110_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_110_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_110_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_110_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_110_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_110_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_110_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_110_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_110_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_110_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_110_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_110_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_110_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_110_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_110_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_110_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_110_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_110_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_110_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_110_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_110_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_110_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_110_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_110_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_110_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_110_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_110_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_110_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_110_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_110_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_110_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_110_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_110_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_110_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_110_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_110_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_110_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_110_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_110_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_110_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_110_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_110_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_110_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_110_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_110_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_110_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_110_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_110_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_110_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_110_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_110_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_110_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_110_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_110_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_110_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_110_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_110_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_110_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_110_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_110_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_110_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_110_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_110_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_110_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_110_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_110_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_110_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_110_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_110_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_110_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_110_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_110_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_110_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_110_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_110_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_110_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_110_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_110_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_110_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_110_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_110_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_110_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_110_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_110_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_110_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_110_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_110_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_110_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_110_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_110_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_110_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_110_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_110_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_110_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_110_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_110_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_110_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_110_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_110_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_110_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_110_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_110_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_110_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_110_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_110_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_110_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_110_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_110_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_110_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_110_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_110_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_110_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_110_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_110_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_110_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_110_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_110_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_110_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_110_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_110_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_110_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_110_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_110_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_110_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_110_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_110_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_110_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_110_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_110_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_110_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_110_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_110_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_110_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_110_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_110_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_110_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_110_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_110_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_110_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_110_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_110_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_110_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_110_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_110_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_110_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_110_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_110_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_110_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_110_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_110_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_110_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_110_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_110_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_110_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_110_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_110_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_110_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_110_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_110_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_110_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_110_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_110_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_110_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_110_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_110_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_110_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_110_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_110_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_110_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_110_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_110_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_110_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_110_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_110_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_110_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_110_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_110_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_110_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_110_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_110_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_110_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_110_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_110_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_110_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_110_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_110_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_110_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_110_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_110_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_110_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_110_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_110_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_110_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_110_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_110_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_110_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_110_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_110_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_110_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_110_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_110_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_110_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_110_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_110_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_110_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_110_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_110_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_110_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_110_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_110_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_110_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_110_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_110_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_110_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_110_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_110_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_110_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_110_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_110_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_110_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_110_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_110_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_110_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_110_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_110_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_110_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_110_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_110_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_110_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_110_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_110_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_110_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_110_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_110_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_110_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_110_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_110_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_110_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_110_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_110_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_110_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_110_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_110_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_110_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_110_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_110_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_110_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_110_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_110_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_110_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_110_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_110_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_110_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_110_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_110_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_110_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_110_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_110_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_110_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_110_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_110_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_110_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_110_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_110_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_110_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_110_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_110_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_110_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_110_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_110_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_111_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_111_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_111_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_111_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_111_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_111_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_111_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_111_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_111_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_111_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_111_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_111_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_111_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_111_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_111_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_111_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_111_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_111_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_111_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_111_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_111_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_111_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_111_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_111_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_111_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_111_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_111_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_111_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_111_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_111_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_111_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_111_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_111_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_111_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_111_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_111_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_111_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_111_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_111_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_111_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_111_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_111_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_111_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_111_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_111_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_111_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_111_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_111_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_111_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_111_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_111_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_111_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_111_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_111_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_111_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_111_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_111_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_111_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_111_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_111_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_111_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_111_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_111_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_111_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_111_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_111_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_111_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_111_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_111_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_111_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_111_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_111_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_111_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_111_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_111_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_111_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_111_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_111_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_111_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_111_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_111_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_111_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_111_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_111_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_111_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_111_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_111_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_111_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_111_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_111_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_111_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_111_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_111_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_111_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_111_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_111_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_111_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_111_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_111_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_111_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_111_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_111_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_111_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_111_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_111_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_111_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_111_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_111_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_111_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_111_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_111_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_111_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_111_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_111_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_111_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_111_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_111_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_111_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_111_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_111_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_111_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_111_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_111_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_111_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_111_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_111_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_111_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_111_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_111_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_111_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_111_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_111_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_111_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_111_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_111_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_111_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_111_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_111_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_111_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_111_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_111_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_111_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_111_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_111_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_111_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_111_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_111_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_111_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_111_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_111_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_111_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_111_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_111_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_111_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_111_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_111_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_111_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_111_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_111_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_111_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_111_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_111_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_111_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_111_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_111_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_111_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_111_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_111_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_111_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_111_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_111_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_111_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_111_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_111_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_111_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_111_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_111_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_111_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_111_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_111_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_111_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_111_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_111_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_111_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_111_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_111_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_111_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_111_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_111_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_111_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_111_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_111_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_111_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_111_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_111_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_111_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_111_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_111_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_111_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_111_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_111_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_111_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_111_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_111_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_111_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_111_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_111_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_111_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_111_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_111_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_111_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_111_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_111_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_111_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_111_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_111_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_111_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_111_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_111_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_111_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_111_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_111_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_111_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_111_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_111_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_111_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_111_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_111_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_111_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_111_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_111_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_111_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_111_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_111_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_111_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_111_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_111_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_111_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_111_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_111_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_111_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_111_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_111_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_111_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_111_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_111_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_111_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_111_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_111_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_111_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_111_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_111_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_111_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_111_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_111_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_111_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_111_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_111_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_111_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_111_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_111_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_111_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_111_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_111_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_111_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_111_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_111_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_111_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_111_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_111_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_111_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_111_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_111_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_111_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_111_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_111_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_111_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_111_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_111_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_111_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_111_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_111_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_111_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_111_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_111_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_111_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_111_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_111_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_111_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_111_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_111_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_111_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_111_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_111_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_111_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_111_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_111_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_111_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_111_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_111_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_111_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_111_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_111_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_111_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_111_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_111_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_111_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_111_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_111_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_111_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_111_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_111_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_111_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_111_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_111_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_111_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_111_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_111_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_111_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_111_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_111_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_111_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_111_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_111_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_111_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_111_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_111_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_111_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_111_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_111_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_111_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_111_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_111_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_111_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_111_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_111_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_111_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_111_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_111_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_111_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_111_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_111_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_112_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_112_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_112_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_112_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_112_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_112_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_112_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_112_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_112_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_112_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_112_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_112_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_112_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_112_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_112_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_112_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_112_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_112_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_112_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_112_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_112_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_112_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_112_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_112_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_112_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_112_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_112_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_112_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_112_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_112_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_112_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_112_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_112_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_112_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_112_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_112_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_112_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_112_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_112_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_112_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_112_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_112_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_112_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_112_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_112_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_112_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_112_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_112_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_112_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_112_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_112_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_112_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_112_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_112_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_112_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_112_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_112_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_112_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_112_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_112_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_112_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_112_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_112_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_112_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_112_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_112_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_112_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_112_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_112_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_112_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_112_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_112_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_112_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_112_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_112_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_112_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_112_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_112_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_112_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_112_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_112_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_112_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_112_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_112_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_112_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_112_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_112_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_112_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_112_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_112_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_112_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_112_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_112_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_112_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_112_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_112_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_112_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_112_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_112_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_112_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_112_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_112_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_112_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_112_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_112_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_112_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_112_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_112_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_112_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_112_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_112_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_112_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_112_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_112_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_112_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_112_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_112_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_112_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_112_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_112_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_112_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_112_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_112_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_112_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_112_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_112_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_112_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_112_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_112_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_112_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_112_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_112_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_112_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_112_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_112_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_112_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_112_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_112_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_112_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_112_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_112_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_112_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_112_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_112_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_112_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_112_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_112_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_112_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_112_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_112_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_112_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_112_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_112_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_112_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_112_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_112_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_112_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_112_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_112_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_112_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_112_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_112_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_112_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_112_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_112_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_112_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_112_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_112_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_112_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_112_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_112_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_112_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_112_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_112_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_112_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_112_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_112_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_112_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_112_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_112_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_112_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_112_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_112_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_112_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_112_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_112_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_112_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_112_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_112_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_112_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_112_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_112_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_112_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_112_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_112_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_112_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_112_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_112_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_112_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_112_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_112_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_112_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_112_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_112_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_112_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_112_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_112_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_112_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_112_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_112_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_112_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_112_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_112_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_112_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_112_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_112_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_112_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_112_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_112_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_112_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_112_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_112_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_112_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_112_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_112_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_112_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_112_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_112_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_112_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_112_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_112_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_112_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_112_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_112_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_112_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_112_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_112_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_112_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_112_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_112_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_112_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_112_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_112_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_112_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_112_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_112_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_112_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_112_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_112_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_112_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_112_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_112_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_112_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_112_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_112_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_112_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_112_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_112_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_112_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_112_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_112_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_112_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_112_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_112_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_112_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_112_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_112_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_112_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_112_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_112_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_112_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_112_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_112_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_112_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_112_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_112_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_112_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_112_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_112_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_112_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_112_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_112_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_112_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_112_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_112_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_112_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_112_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_112_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_112_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_112_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_112_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_112_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_112_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_112_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_112_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_112_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_112_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_112_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_112_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_112_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_112_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_112_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_112_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_112_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_112_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_112_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_112_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_112_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_112_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_112_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_112_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_112_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_112_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_112_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_112_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_112_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_112_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_112_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_112_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_112_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_112_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_112_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_112_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_112_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_112_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_112_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_112_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_112_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_112_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_112_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_112_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_112_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_112_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_112_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_112_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_112_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_112_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_112_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_112_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_112_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_112_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_112_combo_113_loc2 ': (i==2 and int(my_lab)==113),
        'Hero_113_combo_0_loc0 ': (i==0 and int(my_lab)==0),
        'Hero_113_combo_1_loc0 ': (i==0 and int(my_lab)==1),
        'Hero_113_combo_2_loc0 ': (i==0 and int(my_lab)==2),
        'Hero_113_combo_3_loc0 ': (i==0 and int(my_lab)==3),
        'Hero_113_combo_4_loc0 ': (i==0 and int(my_lab)==4),
        'Hero_113_combo_5_loc0 ': (i==0 and int(my_lab)==5),
        'Hero_113_combo_6_loc0 ': (i==0 and int(my_lab)==6),
        'Hero_113_combo_7_loc0 ': (i==0 and int(my_lab)==7),
        'Hero_113_combo_8_loc0 ': (i==0 and int(my_lab)==8),
        'Hero_113_combo_9_loc0 ': (i==0 and int(my_lab)==9),
        'Hero_113_combo_10_loc0 ': (i==0 and int(my_lab)==10),
        'Hero_113_combo_11_loc0 ': (i==0 and int(my_lab)==11),
        'Hero_113_combo_12_loc0 ': (i==0 and int(my_lab)==12),
        'Hero_113_combo_13_loc0 ': (i==0 and int(my_lab)==13),
        'Hero_113_combo_14_loc0 ': (i==0 and int(my_lab)==14),
        'Hero_113_combo_15_loc0 ': (i==0 and int(my_lab)==15),
        'Hero_113_combo_16_loc0 ': (i==0 and int(my_lab)==16),
        'Hero_113_combo_17_loc0 ': (i==0 and int(my_lab)==17),
        'Hero_113_combo_18_loc0 ': (i==0 and int(my_lab)==18),
        'Hero_113_combo_19_loc0 ': (i==0 and int(my_lab)==19),
        'Hero_113_combo_20_loc0 ': (i==0 and int(my_lab)==20),
        'Hero_113_combo_21_loc0 ': (i==0 and int(my_lab)==21),
        'Hero_113_combo_22_loc0 ': (i==0 and int(my_lab)==22),
        'Hero_113_combo_23_loc0 ': (i==0 and int(my_lab)==23),
        'Hero_113_combo_24_loc0 ': (i==0 and int(my_lab)==24),
        'Hero_113_combo_25_loc0 ': (i==0 and int(my_lab)==25),
        'Hero_113_combo_26_loc0 ': (i==0 and int(my_lab)==26),
        'Hero_113_combo_27_loc0 ': (i==0 and int(my_lab)==27),
        'Hero_113_combo_28_loc0 ': (i==0 and int(my_lab)==28),
        'Hero_113_combo_29_loc0 ': (i==0 and int(my_lab)==29),
        'Hero_113_combo_30_loc0 ': (i==0 and int(my_lab)==30),
        'Hero_113_combo_31_loc0 ': (i==0 and int(my_lab)==31),
        'Hero_113_combo_32_loc0 ': (i==0 and int(my_lab)==32),
        'Hero_113_combo_33_loc0 ': (i==0 and int(my_lab)==33),
        'Hero_113_combo_34_loc0 ': (i==0 and int(my_lab)==34),
        'Hero_113_combo_35_loc0 ': (i==0 and int(my_lab)==35),
        'Hero_113_combo_36_loc0 ': (i==0 and int(my_lab)==36),
        'Hero_113_combo_37_loc0 ': (i==0 and int(my_lab)==37),
        'Hero_113_combo_38_loc0 ': (i==0 and int(my_lab)==38),
        'Hero_113_combo_39_loc0 ': (i==0 and int(my_lab)==39),
        'Hero_113_combo_40_loc0 ': (i==0 and int(my_lab)==40),
        'Hero_113_combo_41_loc0 ': (i==0 and int(my_lab)==41),
        'Hero_113_combo_42_loc0 ': (i==0 and int(my_lab)==42),
        'Hero_113_combo_43_loc0 ': (i==0 and int(my_lab)==43),
        'Hero_113_combo_44_loc0 ': (i==0 and int(my_lab)==44),
        'Hero_113_combo_45_loc0 ': (i==0 and int(my_lab)==45),
        'Hero_113_combo_46_loc0 ': (i==0 and int(my_lab)==46),
        'Hero_113_combo_47_loc0 ': (i==0 and int(my_lab)==47),
        'Hero_113_combo_48_loc0 ': (i==0 and int(my_lab)==48),
        'Hero_113_combo_49_loc0 ': (i==0 and int(my_lab)==49),
        'Hero_113_combo_50_loc0 ': (i==0 and int(my_lab)==50),
        'Hero_113_combo_51_loc0 ': (i==0 and int(my_lab)==51),
        'Hero_113_combo_52_loc0 ': (i==0 and int(my_lab)==52),
        'Hero_113_combo_53_loc0 ': (i==0 and int(my_lab)==53),
        'Hero_113_combo_54_loc0 ': (i==0 and int(my_lab)==54),
        'Hero_113_combo_55_loc0 ': (i==0 and int(my_lab)==55),
        'Hero_113_combo_56_loc0 ': (i==0 and int(my_lab)==56),
        'Hero_113_combo_57_loc0 ': (i==0 and int(my_lab)==57),
        'Hero_113_combo_58_loc0 ': (i==0 and int(my_lab)==58),
        'Hero_113_combo_59_loc0 ': (i==0 and int(my_lab)==59),
        'Hero_113_combo_60_loc0 ': (i==0 and int(my_lab)==60),
        'Hero_113_combo_61_loc0 ': (i==0 and int(my_lab)==61),
        'Hero_113_combo_62_loc0 ': (i==0 and int(my_lab)==62),
        'Hero_113_combo_63_loc0 ': (i==0 and int(my_lab)==63),
        'Hero_113_combo_64_loc0 ': (i==0 and int(my_lab)==64),
        'Hero_113_combo_65_loc0 ': (i==0 and int(my_lab)==65),
        'Hero_113_combo_66_loc0 ': (i==0 and int(my_lab)==66),
        'Hero_113_combo_67_loc0 ': (i==0 and int(my_lab)==67),
        'Hero_113_combo_68_loc0 ': (i==0 and int(my_lab)==68),
        'Hero_113_combo_69_loc0 ': (i==0 and int(my_lab)==69),
        'Hero_113_combo_70_loc0 ': (i==0 and int(my_lab)==70),
        'Hero_113_combo_71_loc0 ': (i==0 and int(my_lab)==71),
        'Hero_113_combo_72_loc0 ': (i==0 and int(my_lab)==72),
        'Hero_113_combo_73_loc0 ': (i==0 and int(my_lab)==73),
        'Hero_113_combo_74_loc0 ': (i==0 and int(my_lab)==74),
        'Hero_113_combo_75_loc0 ': (i==0 and int(my_lab)==75),
        'Hero_113_combo_76_loc0 ': (i==0 and int(my_lab)==76),
        'Hero_113_combo_77_loc0 ': (i==0 and int(my_lab)==77),
        'Hero_113_combo_78_loc0 ': (i==0 and int(my_lab)==78),
        'Hero_113_combo_79_loc0 ': (i==0 and int(my_lab)==79),
        'Hero_113_combo_80_loc0 ': (i==0 and int(my_lab)==80),
        'Hero_113_combo_81_loc0 ': (i==0 and int(my_lab)==81),
        'Hero_113_combo_82_loc0 ': (i==0 and int(my_lab)==82),
        'Hero_113_combo_83_loc0 ': (i==0 and int(my_lab)==83),
        'Hero_113_combo_84_loc0 ': (i==0 and int(my_lab)==84),
        'Hero_113_combo_85_loc0 ': (i==0 and int(my_lab)==85),
        'Hero_113_combo_86_loc0 ': (i==0 and int(my_lab)==86),
        'Hero_113_combo_87_loc0 ': (i==0 and int(my_lab)==87),
        'Hero_113_combo_88_loc0 ': (i==0 and int(my_lab)==88),
        'Hero_113_combo_89_loc0 ': (i==0 and int(my_lab)==89),
        'Hero_113_combo_90_loc0 ': (i==0 and int(my_lab)==90),
        'Hero_113_combo_91_loc0 ': (i==0 and int(my_lab)==91),
        'Hero_113_combo_92_loc0 ': (i==0 and int(my_lab)==92),
        'Hero_113_combo_93_loc0 ': (i==0 and int(my_lab)==93),
        'Hero_113_combo_94_loc0 ': (i==0 and int(my_lab)==94),
        'Hero_113_combo_95_loc0 ': (i==0 and int(my_lab)==95),
        'Hero_113_combo_96_loc0 ': (i==0 and int(my_lab)==96),
        'Hero_113_combo_97_loc0 ': (i==0 and int(my_lab)==97),
        'Hero_113_combo_98_loc0 ': (i==0 and int(my_lab)==98),
        'Hero_113_combo_99_loc0 ': (i==0 and int(my_lab)==99),
        'Hero_113_combo_100_loc0 ': (i==0 and int(my_lab)==100),
        'Hero_113_combo_101_loc0 ': (i==0 and int(my_lab)==101),
        'Hero_113_combo_102_loc0 ': (i==0 and int(my_lab)==102),
        'Hero_113_combo_103_loc0 ': (i==0 and int(my_lab)==103),
        'Hero_113_combo_104_loc0 ': (i==0 and int(my_lab)==104),
        'Hero_113_combo_105_loc0 ': (i==0 and int(my_lab)==105),
        'Hero_113_combo_106_loc0 ': (i==0 and int(my_lab)==106),
        'Hero_113_combo_107_loc0 ': (i==0 and int(my_lab)==107),
        'Hero_113_combo_108_loc0 ': (i==0 and int(my_lab)==108),
        'Hero_113_combo_109_loc0 ': (i==0 and int(my_lab)==109),
        'Hero_113_combo_110_loc0 ': (i==0 and int(my_lab)==110),
        'Hero_113_combo_111_loc0 ': (i==0 and int(my_lab)==111),
        'Hero_113_combo_112_loc0 ': (i==0 and int(my_lab)==112),
        'Hero_113_combo_113_loc0 ': (i==0 and int(my_lab)==113),
        'Hero_113_combo_0_loc1 ': (i==1 and int(my_lab)==0),
        'Hero_113_combo_1_loc1 ': (i==1 and int(my_lab)==1),
        'Hero_113_combo_2_loc1 ': (i==1 and int(my_lab)==2),
        'Hero_113_combo_3_loc1 ': (i==1 and int(my_lab)==3),
        'Hero_113_combo_4_loc1 ': (i==1 and int(my_lab)==4),
        'Hero_113_combo_5_loc1 ': (i==1 and int(my_lab)==5),
        'Hero_113_combo_6_loc1 ': (i==1 and int(my_lab)==6),
        'Hero_113_combo_7_loc1 ': (i==1 and int(my_lab)==7),
        'Hero_113_combo_8_loc1 ': (i==1 and int(my_lab)==8),
        'Hero_113_combo_9_loc1 ': (i==1 and int(my_lab)==9),
        'Hero_113_combo_10_loc1 ': (i==1 and int(my_lab)==10),
        'Hero_113_combo_11_loc1 ': (i==1 and int(my_lab)==11),
        'Hero_113_combo_12_loc1 ': (i==1 and int(my_lab)==12),
        'Hero_113_combo_13_loc1 ': (i==1 and int(my_lab)==13),
        'Hero_113_combo_14_loc1 ': (i==1 and int(my_lab)==14),
        'Hero_113_combo_15_loc1 ': (i==1 and int(my_lab)==15),
        'Hero_113_combo_16_loc1 ': (i==1 and int(my_lab)==16),
        'Hero_113_combo_17_loc1 ': (i==1 and int(my_lab)==17),
        'Hero_113_combo_18_loc1 ': (i==1 and int(my_lab)==18),
        'Hero_113_combo_19_loc1 ': (i==1 and int(my_lab)==19),
        'Hero_113_combo_20_loc1 ': (i==1 and int(my_lab)==20),
        'Hero_113_combo_21_loc1 ': (i==1 and int(my_lab)==21),
        'Hero_113_combo_22_loc1 ': (i==1 and int(my_lab)==22),
        'Hero_113_combo_23_loc1 ': (i==1 and int(my_lab)==23),
        'Hero_113_combo_24_loc1 ': (i==1 and int(my_lab)==24),
        'Hero_113_combo_25_loc1 ': (i==1 and int(my_lab)==25),
        'Hero_113_combo_26_loc1 ': (i==1 and int(my_lab)==26),
        'Hero_113_combo_27_loc1 ': (i==1 and int(my_lab)==27),
        'Hero_113_combo_28_loc1 ': (i==1 and int(my_lab)==28),
        'Hero_113_combo_29_loc1 ': (i==1 and int(my_lab)==29),
        'Hero_113_combo_30_loc1 ': (i==1 and int(my_lab)==30),
        'Hero_113_combo_31_loc1 ': (i==1 and int(my_lab)==31),
        'Hero_113_combo_32_loc1 ': (i==1 and int(my_lab)==32),
        'Hero_113_combo_33_loc1 ': (i==1 and int(my_lab)==33),
        'Hero_113_combo_34_loc1 ': (i==1 and int(my_lab)==34),
        'Hero_113_combo_35_loc1 ': (i==1 and int(my_lab)==35),
        'Hero_113_combo_36_loc1 ': (i==1 and int(my_lab)==36),
        'Hero_113_combo_37_loc1 ': (i==1 and int(my_lab)==37),
        'Hero_113_combo_38_loc1 ': (i==1 and int(my_lab)==38),
        'Hero_113_combo_39_loc1 ': (i==1 and int(my_lab)==39),
        'Hero_113_combo_40_loc1 ': (i==1 and int(my_lab)==40),
        'Hero_113_combo_41_loc1 ': (i==1 and int(my_lab)==41),
        'Hero_113_combo_42_loc1 ': (i==1 and int(my_lab)==42),
        'Hero_113_combo_43_loc1 ': (i==1 and int(my_lab)==43),
        'Hero_113_combo_44_loc1 ': (i==1 and int(my_lab)==44),
        'Hero_113_combo_45_loc1 ': (i==1 and int(my_lab)==45),
        'Hero_113_combo_46_loc1 ': (i==1 and int(my_lab)==46),
        'Hero_113_combo_47_loc1 ': (i==1 and int(my_lab)==47),
        'Hero_113_combo_48_loc1 ': (i==1 and int(my_lab)==48),
        'Hero_113_combo_49_loc1 ': (i==1 and int(my_lab)==49),
        'Hero_113_combo_50_loc1 ': (i==1 and int(my_lab)==50),
        'Hero_113_combo_51_loc1 ': (i==1 and int(my_lab)==51),
        'Hero_113_combo_52_loc1 ': (i==1 and int(my_lab)==52),
        'Hero_113_combo_53_loc1 ': (i==1 and int(my_lab)==53),
        'Hero_113_combo_54_loc1 ': (i==1 and int(my_lab)==54),
        'Hero_113_combo_55_loc1 ': (i==1 and int(my_lab)==55),
        'Hero_113_combo_56_loc1 ': (i==1 and int(my_lab)==56),
        'Hero_113_combo_57_loc1 ': (i==1 and int(my_lab)==57),
        'Hero_113_combo_58_loc1 ': (i==1 and int(my_lab)==58),
        'Hero_113_combo_59_loc1 ': (i==1 and int(my_lab)==59),
        'Hero_113_combo_60_loc1 ': (i==1 and int(my_lab)==60),
        'Hero_113_combo_61_loc1 ': (i==1 and int(my_lab)==61),
        'Hero_113_combo_62_loc1 ': (i==1 and int(my_lab)==62),
        'Hero_113_combo_63_loc1 ': (i==1 and int(my_lab)==63),
        'Hero_113_combo_64_loc1 ': (i==1 and int(my_lab)==64),
        'Hero_113_combo_65_loc1 ': (i==1 and int(my_lab)==65),
        'Hero_113_combo_66_loc1 ': (i==1 and int(my_lab)==66),
        'Hero_113_combo_67_loc1 ': (i==1 and int(my_lab)==67),
        'Hero_113_combo_68_loc1 ': (i==1 and int(my_lab)==68),
        'Hero_113_combo_69_loc1 ': (i==1 and int(my_lab)==69),
        'Hero_113_combo_70_loc1 ': (i==1 and int(my_lab)==70),
        'Hero_113_combo_71_loc1 ': (i==1 and int(my_lab)==71),
        'Hero_113_combo_72_loc1 ': (i==1 and int(my_lab)==72),
        'Hero_113_combo_73_loc1 ': (i==1 and int(my_lab)==73),
        'Hero_113_combo_74_loc1 ': (i==1 and int(my_lab)==74),
        'Hero_113_combo_75_loc1 ': (i==1 and int(my_lab)==75),
        'Hero_113_combo_76_loc1 ': (i==1 and int(my_lab)==76),
        'Hero_113_combo_77_loc1 ': (i==1 and int(my_lab)==77),
        'Hero_113_combo_78_loc1 ': (i==1 and int(my_lab)==78),
        'Hero_113_combo_79_loc1 ': (i==1 and int(my_lab)==79),
        'Hero_113_combo_80_loc1 ': (i==1 and int(my_lab)==80),
        'Hero_113_combo_81_loc1 ': (i==1 and int(my_lab)==81),
        'Hero_113_combo_82_loc1 ': (i==1 and int(my_lab)==82),
        'Hero_113_combo_83_loc1 ': (i==1 and int(my_lab)==83),
        'Hero_113_combo_84_loc1 ': (i==1 and int(my_lab)==84),
        'Hero_113_combo_85_loc1 ': (i==1 and int(my_lab)==85),
        'Hero_113_combo_86_loc1 ': (i==1 and int(my_lab)==86),
        'Hero_113_combo_87_loc1 ': (i==1 and int(my_lab)==87),
        'Hero_113_combo_88_loc1 ': (i==1 and int(my_lab)==88),
        'Hero_113_combo_89_loc1 ': (i==1 and int(my_lab)==89),
        'Hero_113_combo_90_loc1 ': (i==1 and int(my_lab)==90),
        'Hero_113_combo_91_loc1 ': (i==1 and int(my_lab)==91),
        'Hero_113_combo_92_loc1 ': (i==1 and int(my_lab)==92),
        'Hero_113_combo_93_loc1 ': (i==1 and int(my_lab)==93),
        'Hero_113_combo_94_loc1 ': (i==1 and int(my_lab)==94),
        'Hero_113_combo_95_loc1 ': (i==1 and int(my_lab)==95),
        'Hero_113_combo_96_loc1 ': (i==1 and int(my_lab)==96),
        'Hero_113_combo_97_loc1 ': (i==1 and int(my_lab)==97),
        'Hero_113_combo_98_loc1 ': (i==1 and int(my_lab)==98),
        'Hero_113_combo_99_loc1 ': (i==1 and int(my_lab)==99),
        'Hero_113_combo_100_loc1 ': (i==1 and int(my_lab)==100),
        'Hero_113_combo_101_loc1 ': (i==1 and int(my_lab)==101),
        'Hero_113_combo_102_loc1 ': (i==1 and int(my_lab)==102),
        'Hero_113_combo_103_loc1 ': (i==1 and int(my_lab)==103),
        'Hero_113_combo_104_loc1 ': (i==1 and int(my_lab)==104),
        'Hero_113_combo_105_loc1 ': (i==1 and int(my_lab)==105),
        'Hero_113_combo_106_loc1 ': (i==1 and int(my_lab)==106),
        'Hero_113_combo_107_loc1 ': (i==1 and int(my_lab)==107),
        'Hero_113_combo_108_loc1 ': (i==1 and int(my_lab)==108),
        'Hero_113_combo_109_loc1 ': (i==1 and int(my_lab)==109),
        'Hero_113_combo_110_loc1 ': (i==1 and int(my_lab)==110),
        'Hero_113_combo_111_loc1 ': (i==1 and int(my_lab)==111),
        'Hero_113_combo_112_loc1 ': (i==1 and int(my_lab)==112),
        'Hero_113_combo_113_loc1 ': (i==1 and int(my_lab)==113),
        'Hero_113_combo_0_loc2 ': (i==2 and int(my_lab)==0),
        'Hero_113_combo_1_loc2 ': (i==2 and int(my_lab)==1),
        'Hero_113_combo_2_loc2 ': (i==2 and int(my_lab)==2),
        'Hero_113_combo_3_loc2 ': (i==2 and int(my_lab)==3),
        'Hero_113_combo_4_loc2 ': (i==2 and int(my_lab)==4),
        'Hero_113_combo_5_loc2 ': (i==2 and int(my_lab)==5),
        'Hero_113_combo_6_loc2 ': (i==2 and int(my_lab)==6),
        'Hero_113_combo_7_loc2 ': (i==2 and int(my_lab)==7),
        'Hero_113_combo_8_loc2 ': (i==2 and int(my_lab)==8),
        'Hero_113_combo_9_loc2 ': (i==2 and int(my_lab)==9),
        'Hero_113_combo_10_loc2 ': (i==2 and int(my_lab)==10),
        'Hero_113_combo_11_loc2 ': (i==2 and int(my_lab)==11),
        'Hero_113_combo_12_loc2 ': (i==2 and int(my_lab)==12),
        'Hero_113_combo_13_loc2 ': (i==2 and int(my_lab)==13),
        'Hero_113_combo_14_loc2 ': (i==2 and int(my_lab)==14),
        'Hero_113_combo_15_loc2 ': (i==2 and int(my_lab)==15),
        'Hero_113_combo_16_loc2 ': (i==2 and int(my_lab)==16),
        'Hero_113_combo_17_loc2 ': (i==2 and int(my_lab)==17),
        'Hero_113_combo_18_loc2 ': (i==2 and int(my_lab)==18),
        'Hero_113_combo_19_loc2 ': (i==2 and int(my_lab)==19),
        'Hero_113_combo_20_loc2 ': (i==2 and int(my_lab)==20),
        'Hero_113_combo_21_loc2 ': (i==2 and int(my_lab)==21),
        'Hero_113_combo_22_loc2 ': (i==2 and int(my_lab)==22),
        'Hero_113_combo_23_loc2 ': (i==2 and int(my_lab)==23),
        'Hero_113_combo_24_loc2 ': (i==2 and int(my_lab)==24),
        'Hero_113_combo_25_loc2 ': (i==2 and int(my_lab)==25),
        'Hero_113_combo_26_loc2 ': (i==2 and int(my_lab)==26),
        'Hero_113_combo_27_loc2 ': (i==2 and int(my_lab)==27),
        'Hero_113_combo_28_loc2 ': (i==2 and int(my_lab)==28),
        'Hero_113_combo_29_loc2 ': (i==2 and int(my_lab)==29),
        'Hero_113_combo_30_loc2 ': (i==2 and int(my_lab)==30),
        'Hero_113_combo_31_loc2 ': (i==2 and int(my_lab)==31),
        'Hero_113_combo_32_loc2 ': (i==2 and int(my_lab)==32),
        'Hero_113_combo_33_loc2 ': (i==2 and int(my_lab)==33),
        'Hero_113_combo_34_loc2 ': (i==2 and int(my_lab)==34),
        'Hero_113_combo_35_loc2 ': (i==2 and int(my_lab)==35),
        'Hero_113_combo_36_loc2 ': (i==2 and int(my_lab)==36),
        'Hero_113_combo_37_loc2 ': (i==2 and int(my_lab)==37),
        'Hero_113_combo_38_loc2 ': (i==2 and int(my_lab)==38),
        'Hero_113_combo_39_loc2 ': (i==2 and int(my_lab)==39),
        'Hero_113_combo_40_loc2 ': (i==2 and int(my_lab)==40),
        'Hero_113_combo_41_loc2 ': (i==2 and int(my_lab)==41),
        'Hero_113_combo_42_loc2 ': (i==2 and int(my_lab)==42),
        'Hero_113_combo_43_loc2 ': (i==2 and int(my_lab)==43),
        'Hero_113_combo_44_loc2 ': (i==2 and int(my_lab)==44),
        'Hero_113_combo_45_loc2 ': (i==2 and int(my_lab)==45),
        'Hero_113_combo_46_loc2 ': (i==2 and int(my_lab)==46),
        'Hero_113_combo_47_loc2 ': (i==2 and int(my_lab)==47),
        'Hero_113_combo_48_loc2 ': (i==2 and int(my_lab)==48),
        'Hero_113_combo_49_loc2 ': (i==2 and int(my_lab)==49),
        'Hero_113_combo_50_loc2 ': (i==2 and int(my_lab)==50),
        'Hero_113_combo_51_loc2 ': (i==2 and int(my_lab)==51),
        'Hero_113_combo_52_loc2 ': (i==2 and int(my_lab)==52),
        'Hero_113_combo_53_loc2 ': (i==2 and int(my_lab)==53),
        'Hero_113_combo_54_loc2 ': (i==2 and int(my_lab)==54),
        'Hero_113_combo_55_loc2 ': (i==2 and int(my_lab)==55),
        'Hero_113_combo_56_loc2 ': (i==2 and int(my_lab)==56),
        'Hero_113_combo_57_loc2 ': (i==2 and int(my_lab)==57),
        'Hero_113_combo_58_loc2 ': (i==2 and int(my_lab)==58),
        'Hero_113_combo_59_loc2 ': (i==2 and int(my_lab)==59),
        'Hero_113_combo_60_loc2 ': (i==2 and int(my_lab)==60),
        'Hero_113_combo_61_loc2 ': (i==2 and int(my_lab)==61),
        'Hero_113_combo_62_loc2 ': (i==2 and int(my_lab)==62),
        'Hero_113_combo_63_loc2 ': (i==2 and int(my_lab)==63),
        'Hero_113_combo_64_loc2 ': (i==2 and int(my_lab)==64),
        'Hero_113_combo_65_loc2 ': (i==2 and int(my_lab)==65),
        'Hero_113_combo_66_loc2 ': (i==2 and int(my_lab)==66),
        'Hero_113_combo_67_loc2 ': (i==2 and int(my_lab)==67),
        'Hero_113_combo_68_loc2 ': (i==2 and int(my_lab)==68),
        'Hero_113_combo_69_loc2 ': (i==2 and int(my_lab)==69),
        'Hero_113_combo_70_loc2 ': (i==2 and int(my_lab)==70),
        'Hero_113_combo_71_loc2 ': (i==2 and int(my_lab)==71),
        'Hero_113_combo_72_loc2 ': (i==2 and int(my_lab)==72),
        'Hero_113_combo_73_loc2 ': (i==2 and int(my_lab)==73),
        'Hero_113_combo_74_loc2 ': (i==2 and int(my_lab)==74),
        'Hero_113_combo_75_loc2 ': (i==2 and int(my_lab)==75),
        'Hero_113_combo_76_loc2 ': (i==2 and int(my_lab)==76),
        'Hero_113_combo_77_loc2 ': (i==2 and int(my_lab)==77),
        'Hero_113_combo_78_loc2 ': (i==2 and int(my_lab)==78),
        'Hero_113_combo_79_loc2 ': (i==2 and int(my_lab)==79),
        'Hero_113_combo_80_loc2 ': (i==2 and int(my_lab)==80),
        'Hero_113_combo_81_loc2 ': (i==2 and int(my_lab)==81),
        'Hero_113_combo_82_loc2 ': (i==2 and int(my_lab)==82),
        'Hero_113_combo_83_loc2 ': (i==2 and int(my_lab)==83),
        'Hero_113_combo_84_loc2 ': (i==2 and int(my_lab)==84),
        'Hero_113_combo_85_loc2 ': (i==2 and int(my_lab)==85),
        'Hero_113_combo_86_loc2 ': (i==2 and int(my_lab)==86),
        'Hero_113_combo_87_loc2 ': (i==2 and int(my_lab)==87),
        'Hero_113_combo_88_loc2 ': (i==2 and int(my_lab)==88),
        'Hero_113_combo_89_loc2 ': (i==2 and int(my_lab)==89),
        'Hero_113_combo_90_loc2 ': (i==2 and int(my_lab)==90),
        'Hero_113_combo_91_loc2 ': (i==2 and int(my_lab)==91),
        'Hero_113_combo_92_loc2 ': (i==2 and int(my_lab)==92),
        'Hero_113_combo_93_loc2 ': (i==2 and int(my_lab)==93),
        'Hero_113_combo_94_loc2 ': (i==2 and int(my_lab)==94),
        'Hero_113_combo_95_loc2 ': (i==2 and int(my_lab)==95),
        'Hero_113_combo_96_loc2 ': (i==2 and int(my_lab)==96),
        'Hero_113_combo_97_loc2 ': (i==2 and int(my_lab)==97),
        'Hero_113_combo_98_loc2 ': (i==2 and int(my_lab)==98),
        'Hero_113_combo_99_loc2 ': (i==2 and int(my_lab)==99),
        'Hero_113_combo_100_loc2 ': (i==2 and int(my_lab)==100),
        'Hero_113_combo_101_loc2 ': (i==2 and int(my_lab)==101),
        'Hero_113_combo_102_loc2 ': (i==2 and int(my_lab)==102),
        'Hero_113_combo_103_loc2 ': (i==2 and int(my_lab)==103),
        'Hero_113_combo_104_loc2 ': (i==2 and int(my_lab)==104),
        'Hero_113_combo_105_loc2 ': (i==2 and int(my_lab)==105),
        'Hero_113_combo_106_loc2 ': (i==2 and int(my_lab)==106),
        'Hero_113_combo_107_loc2 ': (i==2 and int(my_lab)==107),
        'Hero_113_combo_108_loc2 ': (i==2 and int(my_lab)==108),
        'Hero_113_combo_109_loc2 ': (i==2 and int(my_lab)==109),
        'Hero_113_combo_110_loc2 ': (i==2 and int(my_lab)==110),
        'Hero_113_combo_111_loc2 ': (i==2 and int(my_lab)==111),
        'Hero_113_combo_112_loc2 ': (i==2 and int(my_lab)==112),
        'Hero_113_combo_113_loc2 ': (i==2 and int(my_lab)==113),


    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [168]:
sent2features(train_sent[0])
#sent2labels(train_sent[0])
X_train = [sent2features(s) for s in train_sent]
y_train = [sent2labels(s) for s in train_sent]
X_test = [sent2features(s) for s in test_sent]
y_test = [sent2labels(s) for s in test_sent]

In [169]:
crf = sklearn_crfsuite.CRF(
    #algorithm='lbfgs',
    algorithm='l2sgd',
    #c1=0.1,
    #c2=1,
    max_iterations=1000,
    #all_possible_states = True,
    #period=2,
    #all_possible_transitions=True
)
crf.fit(X_train, y_train)


KeyboardInterrupt



In [ ]:
my_labels=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56','57', '58', '59', '60','61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113' ]

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=my_labels)
sklearn_crfsuite.metrics.flat_accuracy_score(y_test, y_pred)

In [138]:
#y_test
my_ytest = np.array(y_test)
#my_ytest_finalhero = my_ytest[:,0]
#my_ytest_finalhero = my_ytest[:,1]
my_ytest_finalhero = my_ytest[:,2]
#my_ytest_finalhero = my_ytest[:,3]
#my_ytest_finalhero = my_ytest[:,4]
#my_ytest_finalhero
my_ypred = np.array(y_pred)
#my_ypred_finalhero = my_ypred[:,0]
#my_ypred_finalhero = my_ypred[:,1]
my_ypred_finalhero = my_ypred[:,2]
#my_ypred_finalhero = my_ypred[:,3]
#my_ypred_finalhero = my_ypred[:,4]
#my_ypred_finalhero
totals = my_ytest.shape[0]
#totals
num_correct = sum(p == t for p, t in zip(my_ypred_finalhero, my_ytest_finalhero))
#num_correct
total = (num_correct/totals)*100
total
print(my_ypred)
print("---------")
print(my_ytest)
print(total)
#y_pred

[['75' '12' '57']
 ['75' '7' '12']
 ['75' '85' '100']
 ['85' '100' '28']
 ['69' '85' '100']
 ['69' '85' '100']
 ['75' '57' '75']
 ['75' '7' '12']
 ['69' '28' '85']
 ['85' '69' '85']
 ['69' '7' '12']
 ['69' '85' '100']
 ['75' '57' '75']
 ['75' '57' '75']
 ['69' '28' '85']
 ['69' '7' '12']
 ['75' '85' '100']
 ['85' '100' '28']
 ['75' '7' '12']
 ['69' '28' '85']
 ['75' '57' '75']
 ['69' '85' '100']
 ['75' '57' '75']
 ['69' '28' '85']
 ['75' '57' '75']
 ['85' '100' '28']
 ['75' '57' '75']
 ['69' '7' '12']
 ['69' '28' '85']
 ['85' '100' '28']
 ['75' '57' '75']
 ['75' '12' '57']
 ['75' '7' '12']
 ['75' '57' '75']
 ['69' '85' '100']
 ['69' '85' '100']
 ['85' '100' '28']
 ['85' '100' '28']
 ['85' '100' '28']
 ['75' '85' '100']
 ['69' '85' '100']
 ['75' '12' '57']
 ['69' '85' '100']
 ['85' '100' '28']
 ['69' '85' '100']
 ['75' '57' '75']
 ['85' '100' '28']
 ['69' '85' '100']
 ['75' '57' '75']
 ['75' '7' '12']
 ['75' '57' '75']
 ['85' '100' '28']
 ['85' '100' '28']
 ['69' '85' '100']
 ['85' '100